# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-616ea09c9e-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series (Tennis explorative search") 

Consider the following exploratory information need:

> investigate tennis and find the main BGPs related to this sport. Compare players per countries, awards of the players and tournament.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q1426`| Roger Federer       | node |
| `wd:Q11459`| Serena Williams       | node |





Also consider

```
wd:Q1426 ?p ?obj .
```

is the BGP to retrieve all **properties of Roger Federer**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for tennis player

2. Identify the BGP for tennis tournament

3. Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) (the result set must be a list of four couples Grand Slam IRI and label).

4. Find the three different type of possible surfaces in a tennis court (the result set must be a list of three couples surface IRI and label).

5. Return the tournaments played in the 2018 Wimbledon Championships edition with the related winners (hint: men and women do not play together, there are two distinct tournament ) (the result set must be a list of elements with tournament IRI and label, and winner IRI and label).

6. Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

7. Consider the country for sport property of the tennis player (NO Country of citizenship). Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

8. Consider the country for sport property of the tennis player (NO Country of citizenship). Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

## Task 1
Identify the BGP for tennis player

In [2]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1426 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('name', 'CONOR.SI ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1284'), ('name', 'Munzinger person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('name', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1352'), ('name', 

#### Found wd:Q10833314 ('tennis player')

In [3]:
# TASK1: Identify the BGP for tennis player
# With this query I'm trying to find Roger Federer's (wd:Q1426) profession (wdt:P106)
queryString = """
SELECT DISTINCT ?profession ?name
WHERE {
   # bind something
   wd:Q1426 wdt:P106 ?profession .
   # get the label
   ?profession sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('profession', 'http://www.wikidata.org/entity/Q10833314'), ('name', 'tennis player')]
1


In [4]:
# Initially, I made a wrong interpretation and thought that I had to find all the tennis players 
# so the following queries are just useless!

In [5]:
# Find all tennis player (wd:Q10833314) 
queryString = """
SELECT DISTINCT ?player ?name
WHERE {
   # bind something
   wd:Q10833314 wdt:P106 ?player .
   # get the label
   ?player sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [6]:
# Find all tennis player (wd:Q10833314) 
queryString = """
SELECT DISTINCT ?player ?name
WHERE {
   # bind something
   wd:Q10833314 ^wdt:P106 ?player .
   # get the label
   ?player sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q1683632'), ('name', 'Jaroslav Navrátil')]
[('player', 'http://www.wikidata.org/entity/Q138094'), ('name', 'Anna Karolína Schmiedlová')]
[('player', 'http://www.wikidata.org/entity/Q15197746'), ('name', 'Wesley Koolhof')]
[('player', 'http://www.wikidata.org/entity/Q1558013'), ('name', 'Pierre Barthès')]
[('player', 'http://www.wikidata.org/entity/Q15849678'), ('name', 'Sue Partridge')]
[('player', 'http://www.wikidata.org/entity/Q15880574'), ('name', 'Paul Oosterbaan')]
[('player', 'http://www.wikidata.org/entity/Q240837'), ('name', 'Rika Fujiwara')]
[('player', 'http://www.wikidata.org/entity/Q4134097'), ('name', 'Margarita Gasparyan')]
[('player', 'http://www.wikidata.org/entity/Q589611'), ('name', 'Philippe Chatrier')]
[('player', 'http://www.wikidata.org/entity/Q16009513'), ('name', 'James Bayley')]
[('player', 'http://www.wikidata.org/entity/Q16023406'), ('name', 'Samuel T. Chase')]
[('player', 'http://www.wikidata.org/entity/Q1

In [7]:
# Find all tennis player (wd:Q10833314), extending limit to 100
queryString = """
SELECT DISTINCT ?player ?name
WHERE {
   # bind something
   wd:Q10833314 ^wdt:P106 ?player .
   # get the label
   ?player sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q1683632'), ('name', 'Jaroslav Navrátil')]
[('player', 'http://www.wikidata.org/entity/Q138094'), ('name', 'Anna Karolína Schmiedlová')]
[('player', 'http://www.wikidata.org/entity/Q15197746'), ('name', 'Wesley Koolhof')]
[('player', 'http://www.wikidata.org/entity/Q1558013'), ('name', 'Pierre Barthès')]
[('player', 'http://www.wikidata.org/entity/Q15849678'), ('name', 'Sue Partridge')]
[('player', 'http://www.wikidata.org/entity/Q15880574'), ('name', 'Paul Oosterbaan')]
[('player', 'http://www.wikidata.org/entity/Q240837'), ('name', 'Rika Fujiwara')]
[('player', 'http://www.wikidata.org/entity/Q4134097'), ('name', 'Margarita Gasparyan')]
[('player', 'http://www.wikidata.org/entity/Q589611'), ('name', 'Philippe Chatrier')]
[('player', 'http://www.wikidata.org/entity/Q16009513'), ('name', 'James Bayley')]
[('player', 'http://www.wikidata.org/entity/Q16023406'), ('name', 'Samuel T. Chase')]
[('player', 'http://www.wikidata.org/entity/Q1

In [8]:
# Find all tennis player (wd:Q10833314), extending limit to 500
queryString = """
SELECT DISTINCT ?player ?name
WHERE {
   # bind something
   wd:Q10833314 ^wdt:P106 ?player .
   # get the label
   ?player sc:name ?name.
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q1683632'), ('name', 'Jaroslav Navrátil')]
[('player', 'http://www.wikidata.org/entity/Q138094'), ('name', 'Anna Karolína Schmiedlová')]
[('player', 'http://www.wikidata.org/entity/Q15197746'), ('name', 'Wesley Koolhof')]
[('player', 'http://www.wikidata.org/entity/Q1558013'), ('name', 'Pierre Barthès')]
[('player', 'http://www.wikidata.org/entity/Q15849678'), ('name', 'Sue Partridge')]
[('player', 'http://www.wikidata.org/entity/Q15880574'), ('name', 'Paul Oosterbaan')]
[('player', 'http://www.wikidata.org/entity/Q240837'), ('name', 'Rika Fujiwara')]
[('player', 'http://www.wikidata.org/entity/Q4134097'), ('name', 'Margarita Gasparyan')]
[('player', 'http://www.wikidata.org/entity/Q589611'), ('name', 'Philippe Chatrier')]
[('player', 'http://www.wikidata.org/entity/Q16009513'), ('name', 'James Bayley')]
[('player', 'http://www.wikidata.org/entity/Q16023406'), ('name', 'Samuel T. Chase')]
[('player', 'http://www.wikidata.org/entity/Q1

In [9]:
# Counting all triplets who have tennis player (wd:Q10833314) as profession (wdt:P106)
queryString = """
SELECT (COUNT(*) AS ?totTennisPlayers)
WHERE {
   # bind something
   wd:Q10833314 ^wdt:P106 ?player .
   # get the label
   ?player sc:name ?name.
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('totTennisPlayers', '11139')]
1


In [10]:
# Counting all triplets who have tennis player (wd:Q10833314) as profession (wdt:P106) and checking their type. Goal only humans.
queryString = """
SELECT ?instanceOf ?instanceName (COUNT(?player) AS ?howMany)
WHERE {
   # bind something
   wd:Q10833314 ^wdt:P106 ?player .
   ?player wdt:P31 ?instanceOf .
   # get the label
   ?instanceOf sc:name ?instanceName .
}
GROUP BY ?instanceOf ?instanceName
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('instanceOf', 'http://www.wikidata.org/entity/Q5'), ('instanceName', 'human'), ('howMany', '11202')]
[('instanceOf', 'http://www.wikidata.org/entity/Q15773317'), ('instanceName', 'television character'), ('howMany', '1')]
[('instanceOf', 'http://www.wikidata.org/entity/Q1114461'), ('instanceName', 'comics character'), ('howMany', '1')]
[('instanceOf', 'http://www.wikidata.org/entity/Q15632617'), ('instanceName', 'fictional human'), ('howMany', '2')]
[('instanceOf', 'http://www.wikidata.org/entity/Q95074'), ('instanceName', 'fictional character'), ('howMany', '1')]
[('instanceOf', 'http://www.wikidata.org/entity/Q4359230'), ('instanceName', 'characters from James Bond series'), ('howMany', '1')]
[('instanceOf', 'http://www.wikidata.org/entity/Q22074477'), ('instanceName', 'cyborg in a work of fiction'), ('howMany', '1')]
[('instanceOf', 'http://www.wikidata.org/entity/Q17362920'), ('instanceName', 'Wikimedia duplicated page'), ('howMany', '1')]
[('instanceOf', 'http://www.wiki

In [11]:
# [RE] Counting all triplets who have tennis player (wd:Q10833314) as profession (wdt:P106)
# TODO: problem totTennisPlayers count is 11139 meanwhile the counting all instanceOf the players is over 11202 -->ARE NOT DISTINCT!!!!
#       NOT excluding mutually
queryString = """
SELECT (COUNT(*) AS ?totTennisPlayers)
WHERE {
   # bind something
   wd:Q10833314 ^wdt:P106 ?player .
   # get the label
   ?player sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('totTennisPlayers', '11139')]
1


In [12]:
# Find all tennis players (wd:Q10833314), and only those who are instanceOf (wdt:P31) of HUMANS (wd:Q5)
queryString = """
SELECT DISTINCT ?player ?name
WHERE {
   # bind something
   wd:Q10833314 ^wdt:P106 ?player .
   ?player wdt:P31 wd:Q5 .
   # get the label
   ?player sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q1683632'), ('name', 'Jaroslav Navrátil')]
[('player', 'http://www.wikidata.org/entity/Q138094'), ('name', 'Anna Karolína Schmiedlová')]
[('player', 'http://www.wikidata.org/entity/Q15197746'), ('name', 'Wesley Koolhof')]
[('player', 'http://www.wikidata.org/entity/Q1558013'), ('name', 'Pierre Barthès')]
[('player', 'http://www.wikidata.org/entity/Q15849678'), ('name', 'Sue Partridge')]
[('player', 'http://www.wikidata.org/entity/Q15880574'), ('name', 'Paul Oosterbaan')]
[('player', 'http://www.wikidata.org/entity/Q240837'), ('name', 'Rika Fujiwara')]
[('player', 'http://www.wikidata.org/entity/Q4134097'), ('name', 'Margarita Gasparyan')]
[('player', 'http://www.wikidata.org/entity/Q589611'), ('name', 'Philippe Chatrier')]
[('player', 'http://www.wikidata.org/entity/Q16009513'), ('name', 'James Bayley')]
[('player', 'http://www.wikidata.org/entity/Q16023406'), ('name', 'Samuel T. Chase')]
[('player', 'http://www.wikidata.org/entity/Q1

In [13]:
# COUNTING all tennis players (wd:Q10833314), and only those who are instanceOf (wdt:P31) of HUMANS (wd:Q5)
queryString = """
SELECT (COUNT(*) AS ?totTennisPlayers)
WHERE {
   # bind something
   wd:Q10833314 ^wdt:P106 ?player .
   ?player wdt:P31 wd:Q5 .
   # get the label
   ?player sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('totTennisPlayers', '11136')]
1


In [14]:
# COUNTING all tennis players (wd:Q10833314), and only those who are instanceOf (wdt:P31) of HUMANS (wd:Q5)
# found that 3 were not HUMANs but something else, lets find who they are
queryString = """
SELECT DISTINCT ?player ?name ?instanceOf
WHERE {
   # bind something
   wd:Q10833314 ^wdt:P106 ?player .
   ?player wdt:P31 ?instanceOf .
   # get the label
   ?player sc:name ?name.
   
   #Excluding HUMANs
   FILTER (?instanceOf != wd:Q5) .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q6123929'), ('name', 'Jaime Sommers'), ('instanceOf', 'http://www.wikidata.org/entity/Q95074')]
[('player', 'http://www.wikidata.org/entity/Q6123929'), ('name', 'Jaime Sommers'), ('instanceOf', 'http://www.wikidata.org/entity/Q22074477')]
[('player', 'http://www.wikidata.org/entity/Q6123929'), ('name', 'Jaime Sommers'), ('instanceOf', 'http://www.wikidata.org/entity/Q15773317')]
[('player', 'http://www.wikidata.org/entity/Q2721145'), ('name', 'Bette Kane'), ('instanceOf', 'http://www.wikidata.org/entity/Q15711870')]
[('player', 'http://www.wikidata.org/entity/Q2721145'), ('name', 'Bette Kane'), ('instanceOf', 'http://www.wikidata.org/entity/Q1114461')]
[('player', 'http://www.wikidata.org/entity/Q2721145'), ('name', 'Bette Kane'), ('instanceOf', 'http://www.wikidata.org/entity/Q15632617')]
[('player', 'http://www.wikidata.org/entity/Q10390567'), ('name', 'Vijay'), ('instanceOf', 'http://www.wikidata.org/entity/Q15632617')]
[('player',

In [15]:
# COUNTING all tennis players (wd:Q10833314), and only those who are instanceOf (wdt:P31) of HUMANS (wd:Q5)
# found that 3 were not HUMANs but something else, lets find who they are WITH instanceName
queryString = """
SELECT DISTINCT ?player ?name ?instanceOf ?instanceName
WHERE {
   # bind something
   wd:Q10833314 ^wdt:P106 ?player .
   ?player wdt:P31 ?instanceOf .
   # get the label
   ?player sc:name ?name.
   ?instanceOf sc:name ?instanceName .
   
   #Excluding HUMANs
   FILTER (?instanceOf != wd:Q5) .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q6123929'), ('name', 'Jaime Sommers'), ('instanceOf', 'http://www.wikidata.org/entity/Q95074'), ('instanceName', 'fictional character')]
[('player', 'http://www.wikidata.org/entity/Q6123929'), ('name', 'Jaime Sommers'), ('instanceOf', 'http://www.wikidata.org/entity/Q22074477'), ('instanceName', 'cyborg in a work of fiction')]
[('player', 'http://www.wikidata.org/entity/Q6123929'), ('name', 'Jaime Sommers'), ('instanceOf', 'http://www.wikidata.org/entity/Q15773317'), ('instanceName', 'television character')]
[('player', 'http://www.wikidata.org/entity/Q2721145'), ('name', 'Bette Kane'), ('instanceOf', 'http://www.wikidata.org/entity/Q15632617'), ('instanceName', 'fictional human')]
[('player', 'http://www.wikidata.org/entity/Q2721145'), ('name', 'Bette Kane'), ('instanceOf', 'http://www.wikidata.org/entity/Q1114461'), ('instanceName', 'comics character')]
[('player', 'http://www.wikidata.org/entity/Q2721145'), ('name', 'Bette Kane'), 

In [16]:
# BGP for *tennis player* (wd:Q10833314)
queryString = """
SELECT DISTINCT ?p ?o
WHERE {
   # bind something
   wd:Q10833314 ?p ?o.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('o', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1014'), ('o', 'http://vocab.getty.edu/aat/300389804')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P227'), ('o', 'https://d-nb.info/gnd/4184716-7')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P244'), ('o', 'https://id.loc.gov/authorities/names/sh85133910')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P268'), ('o', 'http://data.bnf.fr/ark:/12148/cb119355211#about')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P508'), ('o', 'http://purl.org/bncf/tid/4011')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P646'), ('o', 'http://g.co/kg/m/02xhr73')]
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('o', '300389804')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('o', '0145573')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('o', '0275837')]
[('p', 'http://www

In [17]:
# BGP for *tennis player* (wd:Q10833314) extending LIMIT to 50
queryString = """
SELECT DISTINCT ?p ?o
WHERE {
   # bind something
   wd:Q10833314 ?p ?o.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('o', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1014'), ('o', 'http://vocab.getty.edu/aat/300389804')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P227'), ('o', 'https://d-nb.info/gnd/4184716-7')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P244'), ('o', 'https://id.loc.gov/authorities/names/sh85133910')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P268'), ('o', 'http://data.bnf.fr/ark:/12148/cb119355211#about')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P508'), ('o', 'http://purl.org/bncf/tid/4011')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P646'), ('o', 'http://g.co/kg/m/02xhr73')]
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('o', '300389804')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('o', '0145573')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('o', '0275837')]
[('p', 'http://www

#### NOW I realised that the task was not asking all the players but only the BGP for *tennis player*.

In [19]:
# TASK1: Identify the BGP for *tennis player*
# With this query I'm trying to find Roger Federer's (wd:Q1426) profession (wdt:P106) -> tennis player
queryString = """
SELECT DISTINCT ?profession ?name
WHERE {
   # bind something
   wd:Q1426 wdt:P106 ?profession .
   # get the label
   ?profession sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('profession', 'http://www.wikidata.org/entity/Q10833314'), ('name', 'tennis player')]
1


### Final query for this task

In [20]:
# write your final query

# TASK1: Identify the BGP for *tennis player* (wd:Q10833314)
# With this query I'm finding Roger Federer's (wd:Q1426) profession (wdt:P106), which is tennis player (wd:Q10833314).

queryString = """
# retrieving the BGP of the profession of Roger Federer
SELECT DISTINCT ?profession ?name
WHERE {
   # Roger Federer's (wd:Q1426) --profession (wdt:P106)--> ?profession
   wd:Q1426 wdt:P106 ?profession .
   # get the label of the ?profession
   ?profession sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('profession', 'http://www.wikidata.org/entity/Q10833314'), ('name', 'tennis player')]
1


## Task 2
Identify the BGP for tennis tournament

In [21]:
# Identify the BGP for tennis tournament
# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)

# trying to find any traces of a tennis tournament
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1426 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('name', 'CONOR.SI ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1284'), ('name', 'Munzinger person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('name', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1352'), ('name', 

In [22]:
# Identify the BGP for tennis tournament
# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)

# trying to find any traces of a tennis tournament with ?obj & ?objName
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName
WHERE {
   # bind something
   wd:Q1426 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?objName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('name', 'victory'), ('obj', 'http://www.wikidata.org/entity/Q22956224'), ('objName', "2017 Australian Open – Men's Singles")]
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('name', 'victory'), ('obj', 'http://www.wikidata.org/entity/Q612597'), ('objName', '2007 Tennis Masters Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P3342'), ('name', 'significant person'), ('obj', 'http://www.wikidata.org/entity/Q54584'), ('objName', 'Andy Roddick')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received'), ('obj', 'http://www.wikidata.org/entity/Q56509472'), ('objName', 'honorary doctor of the University of Basel')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in'), ('obj', 'http://www.wikidata.org/entity/Q589310'), ('objName', "tennis at the 2000 Summer Olympics – men's singles")]
[('p', 'http://www.wikidata.org/prop/direct/P6758'), ('name', 'supported sports team'), ('ob

In [23]:
# Identify the BGP for tennis tournament
# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)

# trying to find any traces of a tennis tournament with two degrees of depth
queryString = """
SELECT DISTINCT ?obj ?objName ?p ?name ?obj2 ?objName2
WHERE {
   # bind something
   wd:Q1426 wdt:P1344 ?obj .
   ?obj ?p ?obj2.
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?objName.
   ?obj2 sc:name ?objName2.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q8544'), ('objName', '2000 Summer Olympics'), ('p', 'http://www.wikidata.org/prop/direct/P115'), ('name', 'home venue'), ('obj2', 'http://www.wikidata.org/entity/Q295886'), ('objName2', 'Stadium Australia')]
[('obj', 'http://www.wikidata.org/entity/Q8558'), ('objName', '2004 Summer Olympics'), ('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source'), ('obj2', 'http://www.wikidata.org/entity/Q99413897'), ('objName2', 'Medvik')]
[('obj', 'http://www.wikidata.org/entity/Q757054'), ('objName', "tennis at the 2008 Summer Olympics – men's doubles"), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner'), ('obj2', 'http://www.wikidata.org/entity/Q193661'), ('objName2', 'Stan Wawrinka')]
[('obj', 'http://www.wikidata.org/entity/Q757054'), ('objName', "tennis at the 2008 Summer Olympics – men's doubles"), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner'), ('obj2', 'http://www.wikidat

In [24]:
# Identify the BGP for tennis tournament
# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)
# winner (wdt:P1346)

# trying to find Roger Federer's achievements
queryString = """
SELECT DISTINCT ?obj ?objName 
WHERE {
   # bind something
   wd:Q1426 wdt:P1344 ?obj .
   # get the label
   ?obj sc:name ?objName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q589310'), ('objName', "tennis at the 2000 Summer Olympics – men's singles")]
[('obj', 'http://www.wikidata.org/entity/Q8544'), ('objName', '2000 Summer Olympics')]
[('obj', 'http://www.wikidata.org/entity/Q8558'), ('objName', '2004 Summer Olympics')]
[('obj', 'http://www.wikidata.org/entity/Q581788'), ('objName', "tennis at the 2012 Summer Olympics – men's singles")]
[('obj', 'http://www.wikidata.org/entity/Q757054'), ('objName', "tennis at the 2008 Summer Olympics – men's doubles")]
5


In [25]:
# Identify the BGP for tennis tournament
# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)
# winner (wdt:P1346)
# "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310)

# trying to see where I can get from "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310)
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q589310 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('name', 'competition class')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P585'), ('name', 'point in time')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9055'), ('name', 'Olympedia event ID')]
11


In [26]:
# Identify the BGP for tennis tournament
# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)
# winner (wdt:P1346)
# "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310)
# sport (wdt:P641)

# trying to see where I can get from "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310), with sport relation

queryString = """

SELECT DISTINCT ?obj ?objName 
WHERE {
   # bind something
   wd:Q589310 wdt:P641 ?obj .
   # get the label
   ?obj sc:name ?objName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q847'), ('objName', 'tennis')]
1


In [27]:
# Identify the BGP for tennis tournament
# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)
# winner (wdt:P1346)
# "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310)
# sport (wdt:P641)
# part of (wdt:P361) 
# instance of (wdt:P31)

# trying to see where I can get from "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310), 
# with sport|part of|instance of relation

queryString = """
SELECT DISTINCT ?obj ?objName 
WHERE {
   # bind something
   wd:Q589310 wdt:P641|wdt:P361|wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?objName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q18536594'), ('objName', 'Olympic sporting event')]
[('obj', 'http://www.wikidata.org/entity/Q847'), ('objName', 'tennis')]
[('obj', 'http://www.wikidata.org/entity/Q46190676'), ('objName', 'tennis event')]
[('obj', 'http://www.wikidata.org/entity/Q661494'), ('objName', 'tennis at the 2000 Summer Olympics')]
4


In [28]:
# Identify the BGP for tennis tournament
# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)
# winner (wdt:P1346)
# "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310)
# sport (wdt:P641)
# part of (wdt:P361) 
# instance of (wdt:P31)

# tennis event (wd:Q46190676)

# trying to see where I can get from "tennis event" (wd:Q46190676)

queryString = """
SELECT DISTINCT ?obj ?objName 
WHERE {
   # bind something
   wd:Q46190676 ?p ?obj .
   # get the label
   ?obj sc:name ?objName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q21170330'), ('objName', 'tennis qualification event')]
[('obj', 'http://www.wikidata.org/entity/P361'), ('objName', 'part of')]
[('obj', 'http://www.wikidata.org/entity/Q16510064'), ('objName', 'sporting event')]
[('obj', 'http://www.wikidata.org/entity/Q847'), ('objName', 'tennis')]
[('obj', 'http://www.wikidata.org/entity/P641'), ('objName', 'sport')]
[('obj', 'http://www.wikidata.org/entity/Q47345468'), ('objName', 'tennis tournament edition')]
[('obj', 'http://www.wikidata.org/entity/P1346'), ('objName', 'winner')]
[('obj', 'http://www.wikidata.org/entity/P710'), ('objName', 'participant')]
[('obj', 'http://www.wikidata.org/entity/P2094'), ('objName', 'competition class')]
[('obj', 'http://www.wikidata.org/entity/Q47459169'), ('objName', 'tennis match')]
[('obj', 'http://www.wikidata.org/entity/Q1967745'), ('objName', 'type of tennis match')]
11


In [29]:
# Identify the BGP for tennis tournament
# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)
# winner (wdt:P1346)
# "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310)
# sport (wdt:P641)
# part of (wdt:P361) 
# instance of (wdt:P31)

# tennis event (wd:Q46190676)

# trying to see where I can get from "tennis event" (wd:Q46190676), check also property

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName 
WHERE {
   # bind something
   wd:Q46190676 ?p ?obj .
   # get the label
   ?obj sc:name ?objName.
   ?p sc:name ?pName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('pName', 'has quality'), ('obj', 'http://www.wikidata.org/entity/Q1967745'), ('objName', 'type of tennis match')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P710'), ('objName', 'participant')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P641'), ('objName', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P361'), ('objName', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P2094'), ('objName', 'competition class')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P1346'), ('objName', 'w

In [30]:
# Identify the BGP for tennis tournament
# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)
# winner (wdt:P1346)
# "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310)
# sport (wdt:P641)
# part of (wdt:P361) 
# instance of (wdt:P31)
# tennis event (wd:Q46190676)

# wd:Q47459169 tennis match
# wd:Q21170330 tennis qualification event
# wd:Q16510064 sporting event
# wd:Q47345468 tennis tournament edition
# wd:Q847 tennis

# trying to see where I can get from "tennis match" (wd:Q47459169)

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName 
WHERE {
   # bind something
   wd:Q47459169 ?p ?obj .
   # get the label
   ?obj sc:name ?objName.
   ?p sc:name ?pName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('pName', 'has list'), ('obj', 'http://www.wikidata.org/entity/Q7502163'), ('objName', 'Shortest tennis match records')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('pName', 'has parts of the class'), ('obj', 'http://www.wikidata.org/entity/Q47537675'), ('objName', 'set')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q13406554'), ('objName', 'sports competition')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q847'), ('objName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q8843092'), ('objName', 'Category:Tennis matches')]
5


In [31]:
# Identify the BGP for tennis tournament
# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)
# winner (wdt:P1346)
# "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310)
# sport (wdt:P641)
# part of (wdt:P361) 
# instance of (wdt:P31)
# tennis event (wd:Q46190676)

# wd:Q47459169 tennis match
# wd:Q21170330 tennis qualification event
# wd:Q16510064 sporting event
# wd:Q47345468 tennis tournament edition
# wd:Q847 tennis

# trying to see where I can get from "tennis qualification event" (wd:Q21170330), "sporting event" (wd:Q16510064) or "tennis" (wd:Q847)

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName 
WHERE {
   # bind something
   ?s ?p ?obj .
   # get the label
   ?obj sc:name ?objName.
   ?p sc:name ?pName.
   VALUES ?s { wd:Q21170330 wd:Q16510064 wd:Q847 }
}
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('pName', "topic's main Wikimedia portal"), ('obj', 'http://www.wikidata.org/entity/Q8288752'), ('objName', 'Portal:Tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of'), ('obj', 'http://www.wikidata.org/entity/Q349'), ('objName', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q106727050'), ('objName', 'Metropolitan Museum of Art Tagging Vocabulary')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('obj', 'http://www.wikidata.org/entity/Q10943918'), ('objName', 'Template:Tennis box')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('obj', 'http://www.wikidata.org/entity/Q8087332'), ('objName', 'Template:Tennis-stub')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('obj', 'http://www

In [32]:
# Identify the BGP for tennis tournament
# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)
# winner (wdt:P1346)
# "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310)
# sport (wdt:P641)
# part of (wdt:P361) 
# instance of (wdt:P31)
# tennis event (wd:Q46190676)

# wd:Q47459169 tennis match
# wd:Q21170330 tennis qualification event
# wd:Q16510064 sporting event
# wd:Q47345468 tennis tournament edition
# wd:Q847 tennis

# trying to see where I can get from "tennis qualification event" (wd:Q21170330), "sporting event" (wd:Q16510064) or "tennis" (wd:Q847)
# with the subject
queryString = """
SELECT DISTINCT ?s ?p ?pName ?obj ?objName 
WHERE {
   # bind something
   ?s ?p ?obj .
   # get the label
   ?obj sc:name ?objName.
   ?p sc:name ?pName.
   VALUES ?s { wd:Q21170330 wd:Q16510064 wd:Q847 }
}
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q847'), ('p', 'http://www.wikidata.org/prop/direct/P1151'), ('pName', "topic's main Wikimedia portal"), ('obj', 'http://www.wikidata.org/entity/Q8288752'), ('objName', 'Portal:Tennis')]
[('s', 'http://www.wikidata.org/entity/Q16510064'), ('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of'), ('obj', 'http://www.wikidata.org/entity/Q349'), ('objName', 'sport')]
[('s', 'http://www.wikidata.org/entity/Q847'), ('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q106727050'), ('objName', 'Metropolitan Museum of Art Tagging Vocabulary')]
[('s', 'http://www.wikidata.org/entity/Q847'), ('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('obj', 'http://www.wikidata.org/entity/Q10943918'), ('objName', 'Template:Tennis box')]
[('s', 'http://www.wikidata.org/entity/Q847'), ('p', 'http://www.wikidata.org/prop/direct/P1424')

In [33]:
# Identify the BGP for tennis tournament
# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)
# winner (wdt:P1346)
# "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310)
# sport (wdt:P641)
# part of (wdt:P361) 
# instance of (wdt:P31)
# tennis event (wd:Q46190676)

# wd:Q47459169 tennis match
# wd:Q21170330 tennis qualification event
# wd:Q16510064 sporting event
# wd:Q47345468 tennis tournament edition
# wd:Q847 tennis

# trying to see where I can get from "tennis" (wd:Q847)
# with the subject

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName 
WHERE {
   # bind something
   ?s ?p ?obj .
   # get the label
   ?obj sc:name ?objName.
   ?p sc:name ?pName.
   VALUES ?s { wd:Q847 }
}
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('pName', "topic's main Wikimedia portal"), ('obj', 'http://www.wikidata.org/entity/Q8288752'), ('objName', 'Portal:Tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q106727050'), ('objName', 'Metropolitan Museum of Art Tagging Vocabulary')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('obj', 'http://www.wikidata.org/entity/Q8087332'), ('objName', 'Template:Tennis-stub')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('obj', 'http://www.wikidata.org/entity/Q10943918'), ('objName', 'Template:Tennis box')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from'), ('obj', 'http://www.wikidata.org/entity/Q12565374'), ('objName', 'Tenis')]
[('p', 'http://www.wikidata.org/prop/direct/P2184'), ('pName', 'history of topic'), ('obj', 'http

In [34]:
# Identify the BGP for tennis tournament
# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)
# winner (wdt:P1346)
# "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310)
# sport (wdt:P641)
# part of (wdt:P361) 
# instance of (wdt:P31)
# tennis event (wd:Q46190676)

# wd:Q47459169 tennis match
# wd:Q21170330 tennis qualification event
# wd:Q16510064 sporting event
# wd:Q47345468 tennis tournament edition
# wd:Q847 tennis

# trying to see where I can get from "tennis" (wd:Q847)
# with the subject, and extending LIMIT to 50

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName 
WHERE {
   # bind something
   ?s ?p ?obj .
   # get the label
   ?obj sc:name ?objName.
   ?p sc:name ?pName.
   VALUES ?s { wd:Q847 }
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('pName', "topic's main Wikimedia portal"), ('obj', 'http://www.wikidata.org/entity/Q8288752'), ('objName', 'Portal:Tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q106727050'), ('objName', 'Metropolitan Museum of Art Tagging Vocabulary')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('obj', 'http://www.wikidata.org/entity/Q8087332'), ('objName', 'Template:Tennis-stub')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('obj', 'http://www.wikidata.org/entity/Q10943918'), ('objName', 'Template:Tennis box')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from'), ('obj', 'http://www.wikidata.org/entity/Q12565374'), ('objName', 'Tenis')]
[('p', 'http://www.wikidata.org/prop/direct/P2184'), ('pName', 'history of topic'), ('obj', 'http

In [35]:
# Identify the BGP for tennis tournament
# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)
# winner (wdt:P1346)
# "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310)
# sport (wdt:P641)
# part of (wdt:P361) 
# instance of (wdt:P31)
# tennis event (wd:Q46190676)

# wd:Q47459169 tennis match
# wd:Q21170330 tennis qualification event
# wd:Q16510064 sporting event
# wd:Q47345468 tennis tournament edition
# wd:Q847 tennis
# wd:Q223041 International Tennis Federation

# trying to see where I can get from "International Tennis Federation" (wd:Q223041)
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName 
WHERE {
   # bind something
   ?s ?p ?obj .
   # get the label
   ?obj sc:name ?objName.
   ?p sc:name ?pName.
   VALUES ?s { wd:Q223041 }
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('obj', 'http://www.wikidata.org/entity/Q13233631'), ('objName', 'Template:ITF profile')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('pName', 'headquarters location'), ('obj', 'http://www.wikidata.org/entity/Q84'), ('objName', 'London')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property'), ('obj', 'http://www.wikidata.org/entity/P599'), ('objName', 'ITF player ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property'), ('obj', 'http://www.wikidata.org/entity/P4299'), ('objName', 'ITF wheelchair player ID until 2019')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property'), ('obj', 'http://www.wikidata.org/entity/P6841'), ('objName', 'ITF tournament ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property'), ('obj', 'http://www.wikidata.org/entity/P8618'), ('ob

In [36]:
# Identify the BGP for tennis tournament
# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)
# winner (wdt:P1346)
# "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310)
# sport (wdt:P641)
# part of (wdt:P361) 
# instance of (wdt:P31)
# tennis event (wd:Q46190676)

# wd:Q47459169 tennis match
# wd:Q21170330 tennis qualification event
# wd:Q16510064 sporting event
# wd:Q47345468 tennis tournament edition
# wd:Q847 tennis
# wd:Q223041 International Tennis Federation

# trying to see where I can get from "tennis tournament edition" (wd:Q47345468)
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName 
WHERE {
   # bind something
   ?s ?p ?obj .
   # get the label
   ?obj sc:name ?objName.
   ?p sc:name ?pName.
   VALUES ?s { wd:Q47345468 }
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P527'), ('objName', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P276'), ('objName', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P582'), ('objName', 'end time')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P641'), ('objName', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P393'), ('objName', 'edition number')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P580'), ('objName', 'start time'

In [37]:
# Identify the BGP for tennis tournament
# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)
# winner (wdt:P1346)
# "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310)
# sport (wdt:P641)
# instance of (wdt:P31)
# tennis event (wd:Q46190676)

# wd:Q47459169 tennis match
# wd:Q21170330 tennis qualification event
# wd:Q16510064 sporting event
# wd:Q47345468 tennis tournament edition
# wd:Q847 tennis
# wd:Q223041 International Tennis Federation
# part of (wdt:P361) 

# Found tennis tournament from "tennis tournament edition" (wd:Q47345468) with relation "part of" (wdt:P361)

queryString = """
SELECT DISTINCT ?obj ?objName 
WHERE {
   # bind something
   wd:Q47345468 wdt:P361 ?obj .
   # get the label
   ?obj sc:name ?objName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q13219666'), ('objName', 'tennis tournament')]
[('obj', 'http://www.wikidata.org/entity/Q47358534'), ('objName', 'tennis tour edition')]
2


### Final query for this task

In [38]:
# write your final query
# Identify the BGP for tennis tournament
# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)
# winner (wdt:P1346)
# "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310)
# sport (wdt:P641)
# instance of (wdt:P31)
# tennis event (wd:Q46190676)

# wd:Q47459169 tennis match
# wd:Q21170330 tennis qualification event
# wd:Q16510064 sporting event
# wd:Q47345468 tennis tournament edition
# wd:Q847 tennis
# wd:Q223041 International Tennis Federation
# part of (wdt:P361) 

# Found tennis tournament from "tennis tournament edition" (wd:Q47345468) with relation "part of" (wdt:P361)

# 'tennis tournament edition' (wd:Q47345468) is 'part of' (wdt:P361) 'tennis tournament' (wd:Q13219666)

queryString = """
SELECT DISTINCT ?obj ?objName 
WHERE {
   # bind something
   # 'tennis tournament edition' (wd:Q47345468) is 'part of' (wdt:P361) ?obj
   wd:Q47345468 wdt:P361 ?obj .
   # get the label
   ?obj sc:name ?objName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q13219666'), ('objName', 'tennis tournament')]
[('obj', 'http://www.wikidata.org/entity/Q47358534'), ('objName', 'tennis tour edition')]
2


## Task 3
Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) (the result set must be a list of four couples Grand Slam IRI and label).

In [39]:
# Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) 
#   (the result set must be a list of four couples Grand Slam IRI and label).

# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)
# winner (wdt:P1346)
# "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310)
# sport (wdt:P641)
# instance of (wdt:P31)
# tennis event (wd:Q46190676)

# wd:Q47459169 tennis match
# wd:Q21170330 tennis qualification event
# wd:Q16510064 sporting event
# wd:Q47345468 tennis tournament edition
# wd:Q847 tennis
# wd:Q223041 International Tennis Federation
# part of (wdt:P361) 

# trying to see where I can get from "tennis qualification event" (wd:Q21170330)

queryString = """
SELECT DISTINCT ?obj ?objName 
WHERE {
   # bind something
   wd:Q21170330 ?p ?obj .
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q2122052'), ('objName', 'qualification')]
[('obj', 'http://www.wikidata.org/entity/Q847'), ('objName', 'tennis')]
[('obj', 'http://www.wikidata.org/entity/Q46190676'), ('objName', 'tennis event')]
3


In [40]:
# Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) 
#   (the result set must be a list of four couples Grand Slam IRI and label).

# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)
# winner (wdt:P1346)
# "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310)
# sport (wdt:P641)
# instance of (wdt:P31)
# tennis event (wd:Q46190676)

# wd:Q47459169 tennis match
# wd:Q21170330 tennis qualification event
# wd:Q16510064 sporting event
# wd:Q47345468 tennis tournament edition
# wd:Q847 tennis
# wd:Q223041 International Tennis Federation
# part of (wdt:P361) 

# trying to see where I can get from "tennis event" (wd:Q46190676)

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName 
WHERE {
   # bind something
   wd:Q46190676 ?p ?obj .
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('pName', 'has parts of the class'), ('obj', 'http://www.wikidata.org/entity/Q21170330'), ('objName', 'tennis qualification event')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P361'), ('objName', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q16510064'), ('objName', 'sporting event')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q847'), ('objName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P641'), ('objName', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('obj', 'http://www.wikidata.org/entity/Q47345468'), ('objName', 'tennis tournament edition')]
[('p',

In [41]:
# Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) 
#   (the result set must be a list of four couples Grand Slam IRI and label).

# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)
# winner (wdt:P1346)
# "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310)
# sport (wdt:P641)
# instance of (wdt:P31)
# tennis event (wd:Q46190676)

# wd:Q47459169 tennis match
# wd:Q21170330 tennis qualification event
# wd:Q16510064 sporting event
# wd:Q47345468 tennis tournament edition
# wd:Q847 tennis
# wd:Q223041 International Tennis Federation
# part of (wdt:P361) 

# trying to see where I can get from "tennis tournament edition" (wd:Q47345468)

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName 
WHERE {
   # bind something
   wd:Q47345468 ?p ?obj .
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P393'), ('objName', 'edition number')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('pName', 'has parts of the class'), ('obj', 'http://www.wikidata.org/entity/Q47382471'), ('objName', 'day-by-day summary')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P580'), ('objName', 'start time')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P582'), ('objName', 'end time')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P17'), ('objName', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('obj', 'http://www.wikidata.org/entity/Q13219666'), ('objName', 'tennis t

In [42]:
# Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) 
#   (the result set must be a list of four couples Grand Slam IRI and label).

# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)
# winner (wdt:P1346)
# "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310)
# sport (wdt:P641)
# instance of (wdt:P31)
# tennis event (wd:Q46190676)
# wd:Q47459169 tennis match
# wd:Q21170330 tennis qualification event
# wd:Q16510064 sporting event
# wd:Q47345468 tennis tournament edition
# wd:Q847 tennis
# wd:Q223041 International Tennis Federation
# part of (wdt:P361) 

# wd:Q13219666 'tennis tournament'
# trying to see where I can get from "tennis tournament" (wd:Q13219666)


queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName 
WHERE {
   # bind something
   wd:Q13219666 ?p ?obj .
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q13406554'), ('objName', 'sports competition')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q847'), ('objName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('pName', 'has parts of the class'), ('obj', 'http://www.wikidata.org/entity/Q46190676'), ('objName', 'tennis event')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('obj', 'http://www.wikidata.org/entity/Q6452600'), ('objName', 'Template:ATP Tournament')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('pName', 'has list'), ('obj', 'http://www.wikidata.org/entity/Q6642846'), ('objName', 'list of tennis tournaments')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('obj', 'http://www.wikidata.org/entity/Q7474005'), ('objName', 'Templ

In [43]:
# Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) 
#   (the result set must be a list of four couples Grand Slam IRI and label).

# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)
# winner (wdt:P1346)
# "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310)
# sport (wdt:P641)
# instance of (wdt:P31)
# tennis event (wd:Q46190676)
# wd:Q47459169 tennis match
# wd:Q21170330 tennis qualification event
# wd:Q16510064 sporting event
# wd:Q47345468 tennis tournament edition
# wd:Q847 tennis
# wd:Q223041 International Tennis Federation
# part of (wdt:P361) 

# wd:Q13219666 'tennis tournament'
# Can't find a way to get so lets re-trying to see where I can get from "Roger Federer" (wd:Q1426)

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName 
WHERE {
   # bind something
   wd:Q1426 ?p ?obj .
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pName', 'victory'), ('obj', 'http://www.wikidata.org/entity/Q22956224'), ('objName', "2017 Australian Open – Men's Singles")]
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pName', 'victory'), ('obj', 'http://www.wikidata.org/entity/Q612597'), ('objName', '2007 Tennis Masters Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P3342'), ('pName', 'significant person'), ('obj', 'http://www.wikidata.org/entity/Q54584'), ('objName', 'Andy Roddick')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received'), ('obj', 'http://www.wikidata.org/entity/Q56509472'), ('objName', 'honorary doctor of the University of Basel')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in'), ('obj', 'http://www.wikidata.org/entity/Q589310'), ('objName', "tennis at the 2000 Summer Olympics – men's singles")]
[('p', 'http://www.wikidata.org/prop/direct/P6758'), ('pName', 'supported sports team')

In [44]:
# Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) 
#   (the result set must be a list of four couples Grand Slam IRI and label).

# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)
# winner (wdt:P1346)
# "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310)
# sport (wdt:P641)
# instance of (wdt:P31)
# tennis event (wd:Q46190676)
# wd:Q47459169 tennis match
# wd:Q21170330 tennis qualification event
# wd:Q16510064 sporting event
# wd:Q47345468 tennis tournament edition
# wd:Q847 tennis
# wd:Q223041 International Tennis Federation
# part of (wdt:P361) 
# wd:Q13219666 'tennis tournament'

# wdt:P2522 'victory' 
# wd:Q782589 "2010 Australian Open – Men's Singles"

# Can't find a way to get so lets re-trying to see where I can get from "Roger Federer" (wd:Q1426), 
# his achievements with two degrees of depth

queryString = """
SELECT DISTINCT ?objName ?p ?pName ?obj2 ?objName2
WHERE {
   # bind something
   wd:Q1426 wdt:P2522 ?obj .
   ?obj sc:name ?objName.
   
   ?obj ?p ?obj2.
   ?p sc:name ?pName .
   ?obj2 sc:name ?objName2 .
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('objName', "tennis at the 2008 Summer Olympics – men's doubles"), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('obj2', 'http://www.wikidata.org/entity/Q193661'), ('objName2', 'Stan Wawrinka')]
[('objName', '2014 Davis Cup'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('obj2', 'http://www.wikidata.org/entity/Q674139'), ('objName2', 'Switzerland Davis Cup team')]
[('objName', "2017 Australian Open – Men's Singles"), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('obj2', 'http://www.wikidata.org/entity/Q1426'), ('objName2', 'Roger Federer')]
[('objName', "2010 Australian Open – Men's Singles"), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('obj2', 'http://www.wikidata.org/entity/Q1426'), ('objName2', 'Roger Federer')]
[('objName', "2006 Australian Open – Men's Singles"), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('obj2', 'http://www.wik

In [45]:
# Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) 
#   (the result set must be a list of four couples Grand Slam IRI and label).

# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)
# winner (wdt:P1346)
# "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310)
# sport (wdt:P641)
# instance of (wdt:P31)
# tennis event (wd:Q46190676)
# wd:Q47459169 tennis match
# wd:Q21170330 tennis qualification event
# wd:Q16510064 sporting event
# wd:Q47345468 tennis tournament edition
# wd:Q847 tennis
# wd:Q223041 International Tennis Federation
# part of (wdt:P361) 
# wd:Q13219666 'tennis tournament'

# wdt:P2522 'victory' 
# wd:Q782589 "2010 Australian Open – Men's Singles"

# Can't find a way to get so lets re-trying to see where I can get from "Roger Federer" (wd:Q1426), 
# his achievements with two degrees of depth
# and excluding some relations

queryString = """
SELECT DISTINCT ?objName ?p ?pName ?obj2 ?objName2
WHERE {
   # bind something
   wd:Q1426 wdt:P2522 ?obj .
   ?obj sc:name ?objName.
   
   ?obj ?p ?obj2.
   ?p sc:name ?pName .
   ?obj2 sc:name ?objName2 .
   
   # != from winner, follows , followed by
   FILTER (?p != wdt:P1346 && ?p != wdt:P155 && ?p != wdt:P156)
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('objName', "2017 Australian Open – Men's Singles"), ('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj2', 'http://www.wikidata.org/entity/Q408'), ('objName2', 'Australia')]
[('objName', "2017 Australian Open – Men's Singles"), ('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pName', 'competition class'), ('obj2', 'http://www.wikidata.org/entity/Q16893072'), ('objName2', "men's singles")]
[('objName', "2017 Australian Open – Men's Singles"), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj2', 'http://www.wikidata.org/entity/Q46190676'), ('objName2', 'tennis event')]
[('objName', "2017 Australian Open – Men's Singles"), ('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('obj2', 'http://www.wikidata.org/entity/Q22077628'), ('objName2', '2017 Australian Open')]
[('objName', "2017 Australian Open – Men's Singles"), ('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj2', 

In [46]:
# Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) 
#   (the result set must be a list of four couples Grand Slam IRI and label).

# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)
# winner (wdt:P1346)
# "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310)
# sport (wdt:P641)
# instance of (wdt:P31)
# tennis event (wd:Q46190676)
# wd:Q47459169 tennis match
# wd:Q21170330 tennis qualification event
# wd:Q16510064 sporting event
# wd:Q47345468 tennis tournament edition
# wd:Q847 tennis
# wd:Q223041 International Tennis Federation
# part of (wdt:P361) 
# wd:Q13219666 'tennis tournament'

# wdt:P2522 'victory' 
# wd:Q782589 "2010 Australian Open – Men's Singles"

# ('obj2', 'http://www.wikidata.org/entity/Q270907'), ('objName2', 'ATP Finals')]

# trying to see where I can get from "ATP Finals" (wd:Q270907)

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName 
WHERE {
   # bind something
   wd:Q270907 ?p ?obj .
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q47443726'), ('objName', 'recurring tennis tournament')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q26863790'), ('objName', 'ATP tennis tournament')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('obj', 'http://www.wikidata.org/entity/Q5364202'), ('objName', 'The O2 Arena')]
[('p', 'http://www.wikidata.org/prop/direct/P765'), ('pName', 'surface played on'), ('obj', 'http://www.wikidata.org/entity/Q1330871'), ('objName', 'hardcourt')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q145'), ('objName', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pName', 'organizer'), ('obj', 'http://www.wikidata.org/entity/Q744089'), ('objName', 'Association of Tennis Professionals')]


In [47]:
# Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) 
#   (the result set must be a list of four couples Grand Slam IRI and label).

# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)
# winner (wdt:P1346)
# "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310)
# sport (wdt:P641)
# instance of (wdt:P31)
# tennis event (wd:Q46190676)
# wd:Q47459169 tennis match
# wd:Q21170330 tennis qualification event
# wd:Q16510064 sporting event
# wd:Q47345468 tennis tournament edition
# wd:Q847 tennis
# wd:Q223041 International Tennis Federation
# part of (wdt:P361) 
# wd:Q13219666 'tennis tournament'

# wdt:P2522 'victory' 
# wd:Q782589 "2010 Australian Open – Men's Singles"
# wd:Q270907 'ATP Finals'
# wd:Q47443726 'recurring tennis tournament'

# trying to see where I can get from "recurring tennis tournament" (wd:Q47443726)

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName 
WHERE {
   # bind something
   wd:Q47443726 ?p ?obj .
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q15259995'), ('objName', 'recurring tournament')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P576'), ('objName', 'dissolved, abolished or demolished date')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P17'), ('objName', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q847'), ('objName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P571'), ('objName', 'inception')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P856'), ('objName', 'off

In [48]:
# Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) 
#   (the result set must be a list of four couples Grand Slam IRI and label).

# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)
# winner (wdt:P1346)
# "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310)
# sport (wdt:P641)
# instance of (wdt:P31)
# tennis event (wd:Q46190676)
# wd:Q47459169 tennis match
# wd:Q21170330 tennis qualification event
# wd:Q16510064 sporting event
# wd:Q47345468 tennis tournament edition
# wd:Q847 tennis
# wd:Q223041 International Tennis Federation
# part of (wdt:P361) 
# wd:Q13219666 'tennis tournament'

# wdt:P2522 'victory' 
# wd:Q782589 "2010 Australian Open – Men's Singles"
# wd:Q270907 'ATP Finals'
# wd:Q47443726 'recurring tennis tournament'

# trying to see where I can get from "recurring tennis tournament" (wd:Q47443726)

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName 
WHERE {
   # bind something
   wd:Q47443726 ?p ?obj .
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q15259995'), ('objName', 'recurring tournament')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P576'), ('objName', 'dissolved, abolished or demolished date')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P17'), ('objName', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q847'), ('objName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P571'), ('objName', 'inception')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P856'), ('objName', 'off

In [49]:
## REVERSE QUERY

In [50]:
# Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) 
#   (the result set must be a list of four couples Grand Slam IRI and label).

# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)
# winner (wdt:P1346)
# "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310)
# sport (wdt:P641)
# instance of (wdt:P31)
# tennis event (wd:Q46190676)
# wd:Q47459169 tennis match
# wd:Q21170330 tennis qualification event
# wd:Q16510064 sporting event
# wd:Q47345468 tennis tournament edition
# wd:Q847 tennis
# wd:Q223041 International Tennis Federation
# part of (wdt:P361) 
# wd:Q13219666 'tennis tournament'

# wdt:P2522 'victory' 
# wd:Q782589 "2010 Australian Open – Men's Singles"
# wd:Q270907 'ATP Finals'
# wd:Q47443726 'recurring tennis tournament'

# trying to see where I can get from "recurring tennis tournament" (wd:Q47443726) BACKWARD SEARCH
queryString = """
SELECT ?p ?pName ?obj ?objName
WHERE {
   # bind something
   ?obj ?p wd:Q47443726 .
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q299931'), ('objName', 'Chile Open')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q15729796'), ('objName', 'State Bank of India ATP Challenger Tour')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q17306815'), ('objName', 'USTA Player Development Classic')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q17333223'), ('objName', 'UTF Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q17464028'), ('objName', "ITF Women's Circuit – Wuhan")]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q13528155'), ('objName', "IT

In [51]:
# Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) 
#   (the result set must be a list of four couples Grand Slam IRI and label).

# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)
# winner (wdt:P1346)
# "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310)
# sport (wdt:P641)
# instance of (wdt:P31)
# tennis event (wd:Q46190676)
# wd:Q47459169 tennis match
# wd:Q21170330 tennis qualification event
# wd:Q16510064 sporting event
# wd:Q47345468 tennis tournament edition
# wd:Q847 tennis
# wd:Q223041 International Tennis Federation
# part of (wdt:P361) 
# wd:Q13219666 'tennis tournament'

# wdt:P2522 'victory' 
# wd:Q782589 "2010 Australian Open – Men's Singles"
# wd:Q270907 'ATP Finals'
# wd:Q47443726 'recurring tennis tournament'

# trying to see where I can get from "recurring tennis tournament" (wd:Q47443726) BACKWARD SEARCH
# using GROUPING

queryString = """
SELECT ?obj ?objName (count(*) as ?totCount)
WHERE {
   # bind something
   ?obj ?p wd:Q47443726 .
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
GROUP BY ?obj ?objName
ORDER BY desc (?totCount)
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q270907'), ('objName', 'ATP Finals'), ('totCount', '1')]
[('obj', 'http://www.wikidata.org/entity/Q906441'), ('objName', 'Southern California Open'), ('totCount', '1')]
[('obj', 'http://www.wikidata.org/entity/Q3353485'), ('objName', 'Denver Open'), ('totCount', '1')]
[('obj', 'http://www.wikidata.org/entity/Q15249370'), ('objName', 'PTT İstanbul Cup'), ('totCount', '1')]
[('obj', 'http://www.wikidata.org/entity/Q592389'), ('objName', 'Internazionali Femminili di Palermo'), ('totCount', '1')]
[('obj', 'http://www.wikidata.org/entity/Q299914'), ('objName', 'Valencia Open 250'), ('totCount', '1')]
[('obj', 'http://www.wikidata.org/entity/Q299532'), ('objName', 'Madrid Tennis Grand Prix'), ('totCount', '1')]
[('obj', 'http://www.wikidata.org/entity/Q3843957'), ('objName', 'Mallorca Challenger'), ('totCount', '1')]
[('obj', 'http://www.wikidata.org/entity/Q47487928'), ('objName', 'Punta Open'), ('totCount', '1')]
[('obj', 'http://www.wikidat

In [52]:
# Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) 
#   (the result set must be a list of four couples Grand Slam IRI and label).

# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)
# winner (wdt:P1346)
# "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310)
# sport (wdt:P641)
# instance of (wdt:P31)
# tennis event (wd:Q46190676)
# wd:Q47459169 tennis match
# wd:Q21170330 tennis qualification event
# wd:Q16510064 sporting event
# wd:Q47345468 tennis tournament edition
# wd:Q847 tennis
# wd:Q223041 International Tennis Federation
# part of (wdt:P361) 
# wd:Q13219666 'tennis tournament'

# wdt:P2522 'victory' 
# wd:Q782589 "2010 Australian Open – Men's Singles"
# wd:Q270907 'ATP Finals'
# wd:Q47443726 'recurring tennis tournament'

# trying to see where I can get from "recurring tennis tournament" (wd:Q47443726) BACKWARD SEARCH
# extending LIMIT to 100
queryString = """
SELECT ?p ?pName ?obj ?objName
WHERE {
   # bind something
   ?obj ?p wd:Q47443726 .
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q299931'), ('objName', 'Chile Open')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q15729796'), ('objName', 'State Bank of India ATP Challenger Tour')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q17306815'), ('objName', 'USTA Player Development Classic')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q17333223'), ('objName', 'UTF Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q17464028'), ('objName', "ITF Women's Circuit – Wuhan")]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q13528155'), ('objName', "IT

In [53]:
# Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) 
#   (the result set must be a list of four couples Grand Slam IRI and label).

# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)
# winner (wdt:P1346)
# "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310)
# sport (wdt:P641)
# instance of (wdt:P31)
# tennis event (wd:Q46190676)
# wd:Q47459169 tennis match
# wd:Q21170330 tennis qualification event
# wd:Q16510064 sporting event
# wd:Q47345468 tennis tournament edition
# wd:Q847 tennis
# wd:Q223041 International Tennis Federation
# part of (wdt:P361) 
# wd:Q13219666 'tennis tournament'

# wdt:P2522 'victory' 
# wd:Q782589 "2010 Australian Open – Men's Singles"
# wd:Q270907 'ATP Finals'
# wd:Q47443726 'recurring tennis tournament'

# CAN'T find any solution, lets restart from First Seed point Roger Federer's (wd:Q1426) victories 
# with extended limit

queryString = """
SELECT ?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q1426 wdt:P2522 ?obj.
   ?obj sc:name ?objName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q22956224'), ('objName', "2017 Australian Open – Men's Singles")]
[('obj', 'http://www.wikidata.org/entity/Q612597'), ('objName', '2007 Tennis Masters Cup')]
[('obj', 'http://www.wikidata.org/entity/Q782589'), ('objName', "2010 Australian Open – Men's Singles")]
[('obj', 'http://www.wikidata.org/entity/Q782420'), ('objName', "2006 Australian Open – Men's Singles")]
[('obj', 'http://www.wikidata.org/entity/Q300024'), ('objName', '2011 ATP World Tour Finals')]
[('obj', 'http://www.wikidata.org/entity/Q753300'), ('objName', "2012 Wimbledon Championships – Gentlemen's Singles")]
[('obj', 'http://www.wikidata.org/entity/Q757054'), ('objName', "tennis at the 2008 Summer Olympics – men's doubles")]
[('obj', 'http://www.wikidata.org/entity/Q1074654'), ('objName', "2006 Wimbledon Championships – Men's Singles")]
[('obj', 'http://www.wikidata.org/entity/Q384929'), ('objName', "2005 Wimbledon Championships – Men's Singles")]
[('obj', 'http://www.wi

In [54]:
# Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) 
#   (the result set must be a list of four couples Grand Slam IRI and label).

# Roger Federer (wd:Q1426)
# tennis player (wd:Q10833314)
# winner (wdt:P1346)
# "tennis at the 2000 Summer Olympics – men's singles" (wd:Q589310)
# sport (wdt:P641)
# instance of (wdt:P31)
# tennis event (wd:Q46190676)
# wd:Q47459169 tennis match
# wd:Q21170330 tennis qualification event
# wd:Q16510064 sporting event
# wd:Q47345468 tennis tournament edition
# wd:Q847 tennis
# wd:Q223041 International Tennis Federation
# part of (wdt:P361) 
# wd:Q13219666 'tennis tournament'

# wdt:P2522 'victory' 
# wd:Q782589 "2010 Australian Open – Men's Singles"
# wd:Q270907 'ATP Finals'
# wd:Q47443726 'recurring tennis tournament'

# CAN'T find any solution, lets restart from First Seed point Roger Federer's (wd:Q1426) victories 
# with extended limit

queryString = """
SELECT ?obj ?objName
WHERE {
   # Roger Federer -victory-> ?obj
   wd:Q1426 wdt:P2522 ?obj.
   ?obj sc:name ?objName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q22956224'), ('objName', "2017 Australian Open – Men's Singles")]
[('obj', 'http://www.wikidata.org/entity/Q612597'), ('objName', '2007 Tennis Masters Cup')]
[('obj', 'http://www.wikidata.org/entity/Q782589'), ('objName', "2010 Australian Open – Men's Singles")]
[('obj', 'http://www.wikidata.org/entity/Q782420'), ('objName', "2006 Australian Open – Men's Singles")]
[('obj', 'http://www.wikidata.org/entity/Q300024'), ('objName', '2011 ATP World Tour Finals')]
[('obj', 'http://www.wikidata.org/entity/Q753300'), ('objName', "2012 Wimbledon Championships – Gentlemen's Singles")]
[('obj', 'http://www.wikidata.org/entity/Q757054'), ('objName', "tennis at the 2008 Summer Olympics – men's doubles")]
[('obj', 'http://www.wikidata.org/entity/Q1074654'), ('objName', "2006 Wimbledon Championships – Men's Singles")]
[('obj', 'http://www.wikidata.org/entity/Q384929'), ('objName', "2005 Wimbledon Championships – Men's Singles")]
[('obj', 'http://www.wi

In [55]:
## Roger Federer -victory-> ...

In [56]:
## Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) (the result set must be a list of four couples Grand Slam IRI and label).

# Q1426 = Roger Federer -> P2522 = victory
    # Q782589 = 2010 Australian Open – Men's Singles
    # Q753300 = 2012 Wimbledon Championships
    # Q304072 = 2008 US Open – Men's Singles
    # Q741573 = 2009 French Open – Men's Singles

queryString = """
SELECT ?obj ?objName
WHERE {
   # Roger Federer -victory-> ?obj
   wd:Q1426 wdt:P2522 ?obj.
   ?obj sc:name ?objName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q22956224'), ('objName', "2017 Australian Open – Men's Singles")]
[('obj', 'http://www.wikidata.org/entity/Q612597'), ('objName', '2007 Tennis Masters Cup')]
[('obj', 'http://www.wikidata.org/entity/Q782589'), ('objName', "2010 Australian Open – Men's Singles")]
[('obj', 'http://www.wikidata.org/entity/Q782420'), ('objName', "2006 Australian Open – Men's Singles")]
[('obj', 'http://www.wikidata.org/entity/Q300024'), ('objName', '2011 ATP World Tour Finals')]
[('obj', 'http://www.wikidata.org/entity/Q753300'), ('objName', "2012 Wimbledon Championships – Gentlemen's Singles")]
[('obj', 'http://www.wikidata.org/entity/Q757054'), ('objName', "tennis at the 2008 Summer Olympics – men's doubles")]
[('obj', 'http://www.wikidata.org/entity/Q1074654'), ('objName', "2006 Wimbledon Championships – Men's Singles")]
[('obj', 'http://www.wikidata.org/entity/Q384929'), ('objName', "2005 Wimbledon Championships – Men's Singles")]
[('obj', 'http://www.wi

In [57]:
## Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) (the result set must be a list of four couples Grand Slam IRI and label).

# Q1426 = Roger Federer -> P2522 = victory
    # Q782589 = 2010 Australian Open – Men's Singles
    # Q753300 = 2012 Wimbledon Championships
    # Q304072 = 2008 US Open – Men's Singles
    # Q741573 = 2009 French Open – Men's Singles
    
# trying to exprole backward from Q753300 = 2012 Wimbledon Championships

queryString = """
SELECT ?p ?pName ?obj ?objName
WHERE {
   # BACKWARD
   ?obj ?p wd:Q753300 .
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pName', 'victory'), ('obj', 'http://www.wikidata.org/entity/Q1426'), ('objName', 'Roger Federer')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q289587'), ('objName', '2012 Wimbledon Championships')]
2


In [58]:
## Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) (the result set must be a list of four couples Grand Slam IRI and label).

# Q1426 = Roger Federer -> P2522 = victory
    # Q782589 = 2010 Australian Open – Men's Singles
    # Q753300 = 2012 Wimbledon Championships
    # Q304072 = 2008 US Open – Men's Singles
    # Q741573 = 2009 French Open – Men's Singles
    
# Q289587 = '2012 Wimbledon Championships'
# trying to exprole backward from Q289587 = '2012 Wimbledon Championships'
    
queryString = """
SELECT ?p ?pName ?obj ?objName
WHERE {
   # BACKWARD DISCOVERY
   ?obj ?p wd:Q289587 .
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('obj', 'http://www.wikidata.org/entity/Q15854764'), ('objName', "2012 Wimbledon Championships – Ladies' Invitation Doubles")]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('obj', 'http://www.wikidata.org/entity/Q842282'), ('objName', "2012 Wimbledon Championships – Women's Singles")]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('obj', 'http://www.wikidata.org/entity/Q1064519'), ('objName', "2012 Wimbledon Championships – Men's Doubles")]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('obj', 'http://www.wikidata.org/entity/Q1077803'), ('objName', "2012 Wimbledon Championships – Women's Doubles")]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by'), ('obj', 'http://www.wikidata.org/entity/Q245073'), ('objName', '2011 Wimbledon Championships')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), (

In [59]:
## Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) (the result set must be a list of four couples Grand Slam IRI and label).

# Q1426 = Roger Federer -> P2522 = victory
    # Q782589 = 2010 Australian Open – Men's Singles
    # Q753300 = 2012 Wimbledon Championships
    # Q304072 = 2008 US Open – Men's Singles
    # Q741573 = 2009 French Open – Men's Singles
    
# Q289587 = '2012 Wimbledon Championships'
# trying to exprole forward from Q289587 = '2012 Wimbledon Championships'
    
queryString = """
SELECT ?p ?pName ?obj ?objName
WHERE {
   # FORWARD DISCOVERY
   wd:Q289587 ?p ?obj .
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q145'), ('objName', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q847'), ('objName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q47345468'), ('objName', 'tennis tournament edition')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q842282'), ('objName', "2012 Wimbledon Championships – Women's Singles")]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q1064519'), ('objName', "2012 Wimbledon Championships – Men's Doubles")]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q1077803'), ('objName', "2012 Wim

In [60]:
## Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) (the result set must be a list of four couples Grand Slam IRI and label).

# Q1426 = Roger Federer -> P2522 = victory
    # Q782589 = 2010 Australian Open – Men's Singles
    # Q753300 = 2012 Wimbledon Championships
    # Q304072 = 2008 US Open – Men's Singles
    # Q741573 = 2009 French Open – Men's Singles
             
# Q289587 = '2012 Wimbledon Championships'
# Q41520 = Wimbledon Championships
    
# trying to exprole from Q41520 = Wimbledon Championships
    
queryString = """
SELECT ?p ?pName ?obj ?objName
WHERE {
   # FORWARD DISCOVERY
   wd:Q41520 ?p ?obj .
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pName', 'organizer'), ('obj', 'http://www.wikidata.org/entity/Q815369'), ('objName', 'All England Lawn Tennis and Croquet Club')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('obj', 'http://www.wikidata.org/entity/Q815369'), ('objName', 'All England Lawn Tennis and Croquet Club')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pName', 'organizer'), ('obj', 'http://www.wikidata.org/entity/Q223041'), ('objName', 'International Tennis Federation')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q13219666'), ('objName', 'tennis tournament')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q145'), ('objName', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q18608583'), ('o

In [61]:
## Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) (the result set must be a list of four couples Grand Slam IRI and label).

# Q1426 = Roger Federer -> P2522 = victory
    # Q782589 = 2010 Australian Open – Men's Singles
    # Q753300 = 2012 Wimbledon Championships
    # Q304072 = 2008 US Open – Men's Singles
    # Q741573 = 2009 French Open – Men's Singles
             
# Q289587 = '2012 Wimbledon Championships'
     
# Q41520 = Wimbledon Championships -> P361 (part of) Q102113 Grand Slam of tennis

# trying to exprole from Q102113 =  Grand Slam of tennis
    
queryString = """
SELECT ?p ?pName ?obj ?objName
WHERE {
   # FORWARD DISCOVERY
   wd:Q102113 ?p ?obj .
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q50301185'), ('objName', 'tennis tour with multiple editions')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q7700500'), ('objName', 'tennis tour')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q2223649'), ('objName', 'French cheese')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('obj', 'http://www.wikidata.org/entity/Q5851433'), ('objName', 'Template:Infobox GrandSlamTournaments')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q847'), ('objName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
[('p', '

In [62]:
## Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) (the result set must be a list of four couples Grand Slam IRI and label).

# Q1426 = Roger Federer -> P2522 = victory
    # Q782589 = 2010 Australian Open – Men's Singles
    # Q753300 = 2012 Wimbledon Championships
    # Q304072 = 2008 US Open – Men's Singles
    # Q741573 = 2009 French Open – Men's Singles
             
# Q289587 = '2012 Wimbledon Championships'
     
# Q41520 = Wimbledon Championships -> P361 (part of) Q102113 Grand Slam of tennis

# Found from Q102113 =  Grand Slam of tennis through wdt:P527 'has part'
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   # FORWARD DISCOVERY
   wd:Q102113 wdt:P527 ?obj .
   # get the label
   ?obj sc:name ?objName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q2223649'), ('objName', 'French cheese')]
[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]
4


In [63]:
# WORKING BUT WITH French Cheese instead of French Open

In [64]:
## Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) (the result set must be a list of four couples Grand Slam IRI and label).

# Q1426 = Roger Federer -> P2522 = victory
    # Q782589 = 2010 Australian Open – Men's Singles
    # Q753300 = 2012 Wimbledon Championships
    # Q304072 = 2008 US Open – Men's Singles
    # Q741573 = 2009 French Open – Men's Singles
             
# Q289587 = '2012 Wimbledon Championships'
     
# Q41520 = Wimbledon Championships -> P361 (part of) Q102113 Grand Slam of tennis
# WORKING BUT WITH French Cheese instead of French Open so lets re-try
# to exprole from Q741573 = 2009 French Open – Men's Singles 
# and check if It's a typo

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   # FORWARD DISCOVERY
   wd:Q741573 ?p ?obj .
   # get the label
   ?p sc:name ?pName .
   ?obj sc:name ?objName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('obj', 'http://www.wikidata.org/entity/Q381118'), ('objName', '2009 French Open')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q142'), ('objName', 'France')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q847'), ('objName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q1426'), ('objName', 'Roger Federer')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pName', 'competition class'), ('obj', 'http://www.wikidata.org/entity/Q16893072'), ('objName', "men's singles")]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q46190676'), ('objName', 'tennis event')]
6


In [65]:
## Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) (the result set must be a list of four couples Grand Slam IRI and label).

# Q1426 = Roger Federer -> P2522 = victory
    # Q782589 = 2010 Australian Open – Men's Singles
    # Q753300 = 2012 Wimbledon Championships
    # Q304072 = 2008 US Open – Men's Singles
    # Q741573 = 2009 French Open – Men's Singles
             
# Q289587 = '2012 Wimbledon Championships'
     
# Q41520 = Wimbledon Championships -> P361 (part of) Q102113 Grand Slam of tennis

# Q381118 = '2009 French Open'
# trying to re-explore as done with Wimbledon Championship

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   # FORWARD DISCOVERY
   wd:Q381118 ?p ?obj .
   # get the label
   ?p sc:name ?pName .
   ?obj sc:name ?objName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows'), ('obj', 'http://www.wikidata.org/entity/Q277655'), ('objName', '2008 French Open')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q1752825'), ('objName', "2009 French Open – Girls' Doubles")]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by'), ('obj', 'http://www.wikidata.org/entity/Q248438'), ('objName', '2010 French Open')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q142'), ('objName', 'France')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q847'), ('objName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), 

In [66]:
## Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) (the result set must be a list of four couples Grand Slam IRI and label).

# Q1426 = Roger Federer -> P2522 = victory
    # Q782589 = 2010 Australian Open – Men's Singles
    # Q753300 = 2012 Wimbledon Championships
    # Q304072 = 2008 US Open – Men's Singles
    # Q741573 = 2009 French Open – Men's Singles
             
# Q289587 = '2012 Wimbledon Championships'
     
# Q41520 = Wimbledon Championships -> P361 (part of) Q102113 Grand Slam of tennis

# Q381118 = '2009 French Open' -> P31 -> Q43605 'French Open'

# It's not a typo so I lets find another way to get to this node

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   # FORWARD DISCOVERY
   wd:Q43605 ?p ?obj .
   # get the label
   ?p sc:name ?pName .
   ?obj sc:name ?objName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('obj', 'http://www.wikidata.org/entity/Q1613027'), ('objName', 'Stade Roland-Garros')]
[('p', 'http://www.wikidata.org/prop/direct/P765'), ('pName', 'surface played on'), ('obj', 'http://www.wikidata.org/entity/Q198718'), ('objName', 'clay court')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pName', 'organizer'), ('obj', 'http://www.wikidata.org/entity/Q223041'), ('objName', 'International Tennis Federation')]
[('p', 'http://www.wikidata.org/prop/direct/P4794'), ('pName', 'season starts'), ('obj', 'http://www.wikidata.org/entity/Q119'), ('objName', 'May')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q13219666'), ('objName', 'tennis tournament')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q142'), ('objName', 'France')]
[('p', 'http://www.wikidata.org/pr

In [67]:
## Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) (the result set must be a list of four couples Grand Slam IRI and label).

# Q1426 = Roger Federer -> P2522 = victory
    # Q782589 = 2010 Australian Open – Men's Singles
    # Q753300 = 2012 Wimbledon Championships
    # Q304072 = 2008 US Open – Men's Singles
    # Q741573 = 2009 French Open – Men's Singles
             
# Q289587 = '2012 Wimbledon Championships'
# Q41520 = Wimbledon Championships -> P361 (part of) Q102113 Grand Slam of tennis
# Q381118 = '2009 French Open' -> P31 -> Q43605 'French Open'

# It's not a typo so I lets find another way to get to this node
# going to same point from where I have started Q102113
    
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   # FORWARD DISCOVERY
   wd:Q102113 ?p ?obj .
   # get the label
   ?p sc:name ?pName .
   ?obj sc:name ?objName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q50301185'), ('objName', 'tennis tour with multiple editions')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q7700500'), ('objName', 'tennis tour')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q2223649'), ('objName', 'French cheese')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('obj', 'http://www.wikidata.org/entity/Q5851433'), ('objName', 'Template:Infobox GrandSlamTournaments')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q847'), ('objName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
[('p', '

In [68]:
## Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) (the result set must be a list of four couples Grand Slam IRI and label).

# Q1426 = Roger Federer -> P2522 = victory
    # Q782589 = 2010 Australian Open – Men's Singles
    # Q753300 = 2012 Wimbledon Championships
    # Q304072 = 2008 US Open – Men's Singles
    # Q741573 = 2009 French Open – Men's Singles
             
# Q289587 = '2012 Wimbledon Championships'
# Q41520 = Wimbledon Championships -> P361 (part of) Q102113 Grand Slam of tennis
# Q381118 = '2009 French Open' -> P31 -> Q43605 'French Open'

# It's not a typo so I lets find another way to get to this node
# going to same point from where I have started Q102113
# with backward search
    
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   # BACKWARD DISCOVERY
   ?obj ?p wd:Q102113 .
   # get the label
   ?p sc:name ?pName .
   ?obj sc:name ?objName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pName', 'victory'), ('obj', 'http://www.wikidata.org/entity/Q105868'), ('objName', 'Gabriela Sabatini')]
[('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('obj', 'http://www.wikidata.org/entity/Q5851433'), ('objName', 'Template:Infobox GrandSlamTournaments')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pName', 'victory'), ('obj', 'http://www.wikidata.org/entity/Q105838'), ('objName', 'Marion Bartoli')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
[('p', 'http://www

In [69]:
## Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) (the result set must be a list of four couples Grand Slam IRI and label).

# Q1426 = Roger Federer -> P2522 = victory
    # Q782589 = 2010 Australian Open – Men's Singles
    # Q753300 = 2012 Wimbledon Championships
    # Q304072 = 2008 US Open – Men's Singles
    # Q741573 = 2009 French Open – Men's Singles
             
# Q289587 = '2012 Wimbledon Championships'
# Q41520 = Wimbledon Championships -> P361 (part of) Q102113 Grand Slam of tennis
# Q381118 = '2009 French Open' -> P31 -> Q43605 'French Open'

# Found with backward search from Q102113 Grand Slam of tennis with relation P361 'part of'
    
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   # BACKWARD DISCOVERY
   ?obj wdt:P361 wd:Q102113 .
   # get the label
   ?obj sc:name ?objName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]
4


### Final query for this task

In [70]:
# write your final query
## Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) (the result set must be a list of four couples Grand Slam IRI and label).

# Q1426 = Roger Federer -> P2522 = victory
    # Q782589 = 2010 Australian Open – Men's Singles
    # Q753300 = 2012 Wimbledon Championships
    # Q304072 = 2008 US Open – Men's Singles
    # Q741573 = 2009 French Open – Men's Singles
             
    # Q289587 = '2012 Wimbledon Championships'
    # Q41520 = Wimbledon Championships -> P361 (part of) Q102113 Grand Slam of tennis
    # Q381118 = '2009 French Open' -> P31 -> Q43605 'French Open'
    
# SOL:
    # Q1426 (Roger Federer) 
    #    --P2522 (victory)--> Q741573 '2009 French Open – Men's Singles'
    #    --P361 (part of)--> Q381118 '2009 French Open'  
    #    --P31 (instance of)--> Q43605 'French Open'
#SOL#    --P361 (part of) --> Q102113 'Grand Slam of tennis' <-- wdt:P361(part of)-- RESULT

# getting all objects that are part of  Q102113 'Grand Slam of tennis'
    
queryString = """
# Retrieving distinct BGP of events in Grand Slam of tennis
SELECT DISTINCT ?event ?eventName
WHERE {
   # getting all events that are part of Q102113 'Grand Slam of tennis'
   wd:Q102113 ^wdt:P361 ?event.
   # get the label
   ?event sc:name ?eventName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('event', 'http://www.wikidata.org/entity/Q43605'), ('eventName', 'French Open')]
[('event', 'http://www.wikidata.org/entity/Q123577'), ('eventName', 'US Open')]
[('event', 'http://www.wikidata.org/entity/Q41520'), ('eventName', 'Wimbledon Championships')]
[('event', 'http://www.wikidata.org/entity/Q60874'), ('eventName', 'Australian Open')]
4


## Task 4
Find the three different type of possible surfaces in a tennis court (the result set must be a list of three couples surface IRI and label).

In [71]:
##Find the three different type of possible surfaces in a tennis court (the result set must be a list of three couples surface IRI and label).

# Q1426 (Roger Federer) 
    #    --P2522 (victory)--> Q741573 '2009 French Open – Men's Singles'
    #    --P361 (part of)--> Q381118 '2009 French Open'  
    #    --P31 (instance of)--> Q43605 'French Open'
    #    --P361 (part of) --> Q102113 'Grand Slam of tennis' <-- wdt:P361(part of)-- RESULT

# prev TASK
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   # BACKWARD DISCOVERY
   # getting all objects that are part of  Q102113 'Grand Slam of tennis'
   wd:Q102113 ^wdt:P361 ?obj.
   # get the label
   ?obj sc:name ?objName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]
4


In [72]:
##Find the three different type of possible surfaces in a tennis court (the result set must be a list of three couples surface IRI and label).

# Q1426 (Roger Federer) 
    #    --P2522 (victory)--> Q741573 '2009 French Open – Men's Singles'
    #    --P361 (part of)--> Q381118 '2009 French Open'  
    #    --P31 (instance of)--> Q43605 'French Open'
    #    --P361 (part of) --> Q102113 'Grand Slam of tennis' <-- wdt:P361(part of)-- RESULT

# trying to find if there is any node that represent court from Q102113 with 2 degrees of depth

queryString = """
SELECT DISTINCT ?obj ?objName ?p ?pName ?obj2 ?obj2Name
WHERE {
   # BACKWARD DISCOVERY
   wd:Q102113 ^wdt:P361 ?obj.
   # get the label
   ?obj sc:name ?objName.
   ?obj ?p ?obj2.
   
   ?p sc:name ?pName. 
   ?obj2 sc:name ?obj2Name.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open'), ('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('obj2', 'http://www.wikidata.org/entity/Q90'), ('obj2Name', 'Paris')]
[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships'), ('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('obj2', 'http://www.wikidata.org/entity/Q736742'), ('obj2Name', 'Wimbledon')]
[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open'), ('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after'), ('obj2', 'http://www.wikidata.org/entity/Q207709'), ('obj2Name', 'Roland Garros')]
[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships'), ('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('obj2', 'http://www.wikid

In [73]:
##Find the three different type of possible surfaces in a tennis court (the result set must be a list of three couples surface IRI and label).

# Q1426 (Roger Federer) 
    #    --P2522 (victory)--> Q741573 '2009 French Open – Men's Singles'
    #    --P361 (part of)--> Q381118 '2009 French Open'  
    #    --P31 (instance of)--> Q43605 'French Open'
    #    --P361 (part of) --> Q102113 'Grand Slam of tennis' <-- wdt:P361(part of)-- RESULT

# trying to find if there is any node that represent court from Q102113 with 2 degrees of depth
# excluding P17 country
queryString = """
SELECT DISTINCT ?obj ?objName ?p ?pName ?obj2 ?obj2Name
WHERE {
   # BACKWARD DISCOVERY
   wd:Q102113 ^wdt:P361 ?obj.
   # get the label
   ?obj sc:name ?objName.
   ?obj ?p ?obj2.
   
   ?p sc:name ?pName. 
   ?obj2 sc:name ?obj2Name.
   
   # excluding country
   FILTER (?p != wdt:P17)
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open'), ('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('obj2', 'http://www.wikidata.org/entity/Q90'), ('obj2Name', 'Paris')]
[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships'), ('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('obj2', 'http://www.wikidata.org/entity/Q736742'), ('obj2Name', 'Wimbledon')]
[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open'), ('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after'), ('obj2', 'http://www.wikidata.org/entity/Q207709'), ('obj2Name', 'Roland Garros')]
[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships'), ('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('obj2', 'http://www.wikid

In [74]:
##Find the three different type of possible surfaces in a tennis court (the result set must be a list of three couples surface IRI and label).

# Q1426 (Roger Federer) 
    #    --P2522 (victory)--> Q741573 '2009 French Open – Men's Singles'
    #    --P361 (part of)--> Q381118 '2009 French Open'  
    #    --P31 (instance of)--> Q43605 'French Open'
    #    --P361 (part of) --> Q102113 'Grand Slam of tennis' <-- wdt:P361(part of)-- RESULT

# trying to find if there is any node that represent court from Q43605 with 2 degrees of depth
# excluding P17 country

queryString = """
SELECT DISTINCT ?p ?pName ?obj2 ?obj2Name
WHERE {
   # BACKWARD DISCOVERY
   # get the label
   wd:Q43605 ?p ?obj2.
   ?p sc:name ?pName. 
   ?obj2 sc:name ?obj2Name.
   
   FILTER (?p != wdt:P17)
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('obj2', 'http://www.wikidata.org/entity/Q1613027'), ('obj2Name', 'Stade Roland-Garros')]
[('p', 'http://www.wikidata.org/prop/direct/P765'), ('pName', 'surface played on'), ('obj2', 'http://www.wikidata.org/entity/Q198718'), ('obj2Name', 'clay court')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pName', 'organizer'), ('obj2', 'http://www.wikidata.org/entity/Q223041'), ('obj2Name', 'International Tennis Federation')]
[('p', 'http://www.wikidata.org/prop/direct/P4794'), ('pName', 'season starts'), ('obj2', 'http://www.wikidata.org/entity/Q119'), ('obj2Name', 'May')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj2', 'http://www.wikidata.org/entity/Q13219666'), ('obj2Name', 'tennis tournament')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj2', 'http://www.wikidata.org/entity/Q18608583'), ('obj2Name', 'recurring sporting even

In [75]:
##Find the three different type of possible surfaces in a tennis court (the result set must be a list of three couples surface IRI and label).

# Q1426 (Roger Federer) 
    #    --P2522 (victory)--> Q741573 '2009 French Open – Men's Singles'
    #    --P361 (part of)--> Q381118 '2009 French Open'  
    #    --P31 (instance of)--> Q43605 'French Open'
    #    --P361 (part of) --> Q102113 'Grand Slam of tennis' <-- wdt:P361(part of)-- RESULT

# P765'), ('pName', 'surface played on')
    
# FOUND surface played on from Q43605 with 2 degrees of depth
# now lets found all three from Grand Slam of tennis

queryString = """
SELECT DISTINCT ?obj ?objName ?obj2 ?obj2Name
WHERE {
   # BACKWARD DISCOVERY
   wd:Q102113 ^wdt:P361 ?obj.
   ?obj wdt:P765 ?obj2 .
   
   # get the label
   ?obj sc:name ?objName.
   ?obj2 sc:name ?obj2Name.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open'), ('obj2', 'http://www.wikidata.org/entity/Q198718'), ('obj2Name', 'clay court')]
[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open'), ('obj2', 'http://www.wikidata.org/entity/Q1330871'), ('obj2Name', 'hardcourt')]
[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open'), ('obj2', 'http://www.wikidata.org/entity/Q1330871'), ('obj2Name', 'hardcourt')]
[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open'), ('obj2', 'http://www.wikidata.org/entity/Q1151291'), ('obj2Name', 'grass court')]
[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships'), ('obj2', 'http://www.wikidata.org/entity/Q1151291'), ('obj2Name', 'grass court')]
5


In [76]:
##Find the three different type of possible surfaces in a tennis court (the result set must be a list of three couples surface IRI and label).

# Q1426 (Roger Federer) 
    #    --P2522 (victory)--> Q741573 '2009 French Open – Men's Singles'
    #    --P361 (part of)--> Q381118 '2009 French Open'  
    #    --P31 (instance of)--> Q43605 'French Open'
    #    --P361 (part of) --> Q102113 'Grand Slam of tennis' <-- wdt:P361(part of)-- RESULT

# P765'), ('pName', 'surface played on')
    
# FOUND surface played on from Q43605 with 2 degrees of depth
# now lets found all three from Grand Slam of tennis
# renaming variables

queryString = """
SELECT DISTINCT ?event ?eventName ?surface ?surfaceName
WHERE {
   # BACKWARD DISCOVERY
   wd:Q102113 ^wdt:P361 ?event.
   ?event wdt:P765 ?surface .
   
   # get the label
   ?event sc:name ?eventName.
   ?surface sc:name ?surfaceName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('event', 'http://www.wikidata.org/entity/Q43605'), ('eventName', 'French Open'), ('surface', 'http://www.wikidata.org/entity/Q198718'), ('surfaceName', 'clay court')]
[('event', 'http://www.wikidata.org/entity/Q123577'), ('eventName', 'US Open'), ('surface', 'http://www.wikidata.org/entity/Q1330871'), ('surfaceName', 'hardcourt')]
[('event', 'http://www.wikidata.org/entity/Q60874'), ('eventName', 'Australian Open'), ('surface', 'http://www.wikidata.org/entity/Q1330871'), ('surfaceName', 'hardcourt')]
[('event', 'http://www.wikidata.org/entity/Q60874'), ('eventName', 'Australian Open'), ('surface', 'http://www.wikidata.org/entity/Q1151291'), ('surfaceName', 'grass court')]
[('event', 'http://www.wikidata.org/entity/Q41520'), ('eventName', 'Wimbledon Championships'), ('surface', 'http://www.wikidata.org/entity/Q1151291'), ('surfaceName', 'grass court')]
5


In [77]:
##Find the three different type of possible surfaces in a tennis court (the result set must be a list of three couples surface IRI and label).

# Q1426 (Roger Federer) 
    #    --P2522 (victory)--> Q741573 '2009 French Open – Men's Singles'
    #    --P361 (part of)--> Q381118 '2009 French Open'  
    #    --P31 (instance of)--> Q43605 'French Open'
    #    --P361 (part of) --> Q102113 'Grand Slam of tennis' <-- wdt:P361(part of)-- RESULT

# P765'), ('pName', 'surface played on')
    
# FOUND surface played on from Q43605 with 2 degrees of depth
# now lets found all three from Grand Slam of tennis
# renaming variables
# lets group by surface to remove duplicated
queryString = """

SELECT DISTINCT ?surface ?surfaceName
WHERE {
   # BACKWARD DISCOVERY
   wd:Q102113 ^wdt:P361 ?event.
   ?event wdt:P765 ?surface .
   
   # get the label
   ?event sc:name ?eventName.
   ?surface sc:name ?surfaceName.
}
GROUP BY ?surface ?surfaceName
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('surface', 'http://www.wikidata.org/entity/Q1151291'), ('surfaceName', 'grass court')]
[('surface', 'http://www.wikidata.org/entity/Q1330871'), ('surfaceName', 'hardcourt')]
[('surface', 'http://www.wikidata.org/entity/Q198718'), ('surfaceName', 'clay court')]
3


### Final query for this task

In [78]:
##Find the three different type of possible surfaces in a tennis court (the result set must be a list of three couples surface IRI and label).
#    Q102113 'Grand Slam of tennis' <-- wdt:P361(part of)-- ?event --wdt:P765(surface played on)--> ?surface

queryString = """

# Retrieving distinct BJP of surfaces
SELECT DISTINCT ?surface ?surfaceName
WHERE {
   # RETRIEVING ALL THE SURFACE OF EACH EVENT IN Grand Slam of Tennis
   wd:Q102113 ^wdt:P361 ?event.
   ?event wdt:P765 ?surface .
   
   # getting the labels
   ?event sc:name ?eventName.
   ?surface sc:name ?surfaceName.
}
# Grouping so I can remove duplicates
GROUP BY ?surface ?surfaceName
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('surface', 'http://www.wikidata.org/entity/Q1151291'), ('surfaceName', 'grass court')]
[('surface', 'http://www.wikidata.org/entity/Q1330871'), ('surfaceName', 'hardcourt')]
[('surface', 'http://www.wikidata.org/entity/Q198718'), ('surfaceName', 'clay court')]
3


## Task 5
Return the tournaments played in the 2018 Wimbledon Championships edition with the related winners (hint: men and women do not play together, there are two distinct tournament ) (the result set must be a list of elements with tournament IRI and label, and winner IRI and label).

In [79]:
# write your queries

# Return the tournaments played in the 2018 Wimbledon Championships edition with the related winners 
# (hint: men and women do not play together, there are two distinct tournament ) 
# (the result set must be a list of elements with tournament IRI and label, and winner IRI and label).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]

# Trying to explore more from Wimbledon Championships to get the specific edition in the task
queryString = """

SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q41520 ?p ?o.
   # get the label
   ?p sc:name ?pName.
   ?o sc:name ?oName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pName', 'organizer'), ('o', 'http://www.wikidata.org/entity/Q815369'), ('oName', 'All England Lawn Tennis and Croquet Club')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('o', 'http://www.wikidata.org/entity/Q815369'), ('oName', 'All England Lawn Tennis and Croquet Club')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pName', 'organizer'), ('o', 'http://www.wikidata.org/entity/Q223041'), ('oName', 'International Tennis Federation')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q13219666'), ('oName', 'tennis tournament')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('o', 'http://www.wikidata.org/entity/Q145'), ('oName', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q18608583'), ('oName', 'recurring spor

In [80]:
# Return the tournaments played in the 2018 Wimbledon Championships edition with the related winners 
# (hint: men and women do not play together, there are two distinct tournament ) 
# (the result set must be a list of elements with tournament IRI and label, and winner IRI and label).

# Q41520 ('Wimbledon Championships') --P2354 (has list)-> Q6585893 'list of Wimbledon singles finalists during the open era'

# Trying to explore more from Wimbledon Championships to get the specific edition in the task
# trying to get the list of finalist from Q6585893 'list of Wimbledon singles finalists during the open era'
queryString = """

SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q6585893 ?p ?o.
   # get the label
   ?p sc:name ?pName.
   ?o sc:name ?oName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P360'), ('pName', 'is a list of'), ('o', 'http://www.wikidata.org/entity/Q5'), ('oName', 'human')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q19692233'), ('oName', 'Wikimedia list of persons')]
2


In [81]:
# Return the tournaments played in the 2018 Wimbledon Championships edition with the related winners 
# (hint: men and women do not play together, there are two distinct tournament ) 
# (the result set must be a list of elements with tournament IRI and label, and winner IRI and label).

# Q41520 ('Wimbledon Championships') --P2354 (has list)-> Q6585893 'list of Wimbledon singles finalists during the open era'

# Trying to explore more from Wimbledon Championships to get the specific edition in the task
# trying to get the list of finalist from Q6585893 'list of Wimbledon singles finalists during the open era'
# BACKWARD
queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   # BACKWARD DISCOVERY
   ?s ?p wd:Q6585893.
   # get the label
   ?p sc:name ?pName.
   ?s sc:name ?sName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q41520'), ('sName', 'Wimbledon Championships'), ('p', 'http://www.wikidata.org/prop/direct/P2354'), ('pName', 'has list')]
1


In [82]:
#BACKWARD WIMBLEDON CHAMPIONSHIPS

In [83]:
# Return the tournaments played in the 2018 Wimbledon Championships edition with the related winners 
# (hint: men and women do not play together, there are two distinct tournament ) 
# (the result set must be a list of elements with tournament IRI and label, and winner IRI and label).

# Q41520 ('Wimbledon Championships') --P2354 (has list)-> Q6585893 'list of Wimbledon singles finalists during the open era'

# Trying to explore more from Wimbledon Championships to get the specific edition in the task

queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   # BACKWARD DISCOVERY
   ?s ?p wd:Q41520.
   # get the label
   ?p sc:name ?pName.
   ?s sc:name ?sName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q268871'), ('sName', '2004 Wimbledon Championships'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q173961'), ('sName', '1978 Wimbledon Championships'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q925026'), ('sName', '1971 Wimbledon Championships'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q15854765'), ('sName', '2014 Wimbledon Championships'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q30085309'), ('sName', '2018 Wimbledon Championships'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q437350'), ('sName', 'Doris Metaxa'), ('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'par

In [84]:
# Return the tournaments played in the 2018 Wimbledon Championships edition with the related winners 
# (hint: men and women do not play together, there are two distinct tournament ) 
# (the result set must be a list of elements with tournament IRI and label, and winner IRI and label).

# Q41520 ('Wimbledon Championships') --P2354 (has list)-> Q6585893 'list of Wimbledon singles finalists during the open era'

# Trying to explore more from Wimbledon Championships to get the specific edition in the task
# REGEX for 2018
queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   # BACKWARD DISCOVERY
   ?s ?p wd:Q41520.
   # get the label
   ?p sc:name ?pName.
   ?s sc:name ?sName.
   FILTER REGEX(?sName, ".2018*")
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
Empty


In [85]:
# Return the tournaments played in the 2018 Wimbledon Championships edition with the related winners 
# (hint: men and women do not play together, there are two distinct tournament ) 
# (the result set must be a list of elements with tournament IRI and label, and winner IRI and label).

# Q41520 ('Wimbledon Championships') --P2354 (has list)-> Q6585893 'list of Wimbledon singles finalists during the open era'

# Trying to explore more from Wimbledon Championships to get the specific edition in the task
# REGEX for 2018
queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   # BACKWARD DISCOVERY
   ?s ?p wd:Q41520.
   # get the label
   ?p sc:name ?pName.
   ?s sc:name ?sName.
   FILTER REGEX(?sName, ".*2018*")
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q18410121'), ('sName', '2015 Wimbledon Championships'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q559726'), ('sName', '2013 Wimbledon Championships'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q30085309'), ('sName', '2018 Wimbledon Championships'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q54093953'), ('sName', '2019 Wimbledon Championships'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q245073'), ('sName', '2011 Wimbledon Championships'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q15854765'), ('sName', '2014 Wimbledon Championships'), ('p', 'http://www.wikidata.org/prop/direct/P31

In [86]:
# Return the tournaments played in the 2018 Wimbledon Championships edition with the related winners 
# (hint: men and women do not play together, there are two distinct tournament ) 
# (the result set must be a list of elements with tournament IRI and label, and winner IRI and label).

# Q41520 ('Wimbledon Championships') --P2354 (has list)-> Q6585893 'list of Wimbledon singles finalists during the open era'

# Trying to explore more from Wimbledon Championships to get the specific edition in the task
# REGEX for 2018
queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   # BACKWARD DISCOVERY
   ?s ?p wd:Q41520.
   # get the label
   ?p sc:name ?pName.
   ?s sc:name ?sName.
   FILTER REGEX(?sName, "2018.*")
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q30085309'), ('sName', '2018 Wimbledon Championships'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
1


In [87]:
# Return the tournaments played in the 2018 Wimbledon Championships edition with the related winners 
# (hint: men and women do not play together, there are two distinct tournament ) 
# (the result set must be a list of elements with tournament IRI and label, and winner IRI and label).

# Q41520 ('Wimbledon Championships') <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'

# FOUND THE championships, now lets find the winners
queryString = """

SELECT DISTINCT ?p ?pName ?o ?oName 
WHERE {
   wd:Q30085309 ?p ?o.
   
   # get the label
   ?p sc:name ?pName.
   ?o sc:name ?oName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('o', 'http://www.wikidata.org/entity/Q30098268'), ('oName', "2018 Wimbledon Championships – Women's Singles")]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('o', 'http://www.wikidata.org/entity/Q145'), ('oName', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('o', 'http://www.wikidata.org/entity/Q84'), ('oName', 'London')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('o', 'http://www.wikidata.org/entity/Q847'), ('oName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q47345468'), ('oName', 'tennis tournament edition')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q41520'), ('oName', 'Wimbledon Championships')]
[('p', 'http://www.wikidata.org/prop/direct/P1

In [88]:
# ALL SUBTOURNAMENTS OF 2018 Wimbledon Championships edition

In [89]:
# Return the tournaments played in the 2018 Wimbledon Championships edition with the related winners 
# (hint: men and women do not play together, there are two distinct tournament ) 
# (the result set must be a list of elements with tournament IRI and label, and winner IRI and label).

# Q30085309 '2018 Wimbledon Championships' --P31 instance of --> Q41520 ('Wimbledon Championships');
#                                          --P527 ('has part')-->

# trying to find all subtuornaments of Q30085309 '2018 Wimbledon Championships'

queryString = """

SELECT DISTINCT ?o ?oName 
WHERE {
   wd:Q30085309 wdt:P527 ?o.
   
   # get the label
   ?o sc:name ?oName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q30098268'), ('oName', "2018 Wimbledon Championships – Women's Singles")]
[('o', 'http://www.wikidata.org/entity/Q32358401'), ('oName', '2018 Wimbledon Championships – Mixed Doubles')]
[('o', 'http://www.wikidata.org/entity/Q32357790'), ('oName', "2018 Wimbledon Championships – Men's Doubles")]
[('o', 'http://www.wikidata.org/entity/Q32356704'), ('oName', "2018 Wimbledon Championships – Women's Doubles")]
[('o', 'http://www.wikidata.org/entity/Q55381839'), ('oName', "2018 Wimbledon Championships – Girls' Singles")]
[('o', 'http://www.wikidata.org/entity/Q54961494'), ('oName', "2018 Wimbledon Championships – Men's Singles")]
[('o', 'http://www.wikidata.org/entity/Q55410290'), ('oName', "2018 Wimbledon Championships – Girls' Doubles")]
[('o', 'http://www.wikidata.org/entity/Q55410361'), ('oName', "2018 Wimbledon Championships – Boys' Doubles")]
[('o', 'http://www.wikidata.org/entity/Q55455404'), ('oName', "2018 Wimbledon Championships – Whee

In [90]:
# Return the tournaments played in the 2018 Wimbledon Championships edition with the related winners 
# (hint: men and women do not play together, there are two distinct tournament ) 
# (the result set must be a list of elements with tournament IRI and label, and winner IRI and label).

# Q30085309 '2018 Wimbledon Championships' --P31 instance of --> Q41520 ('Wimbledon Championships');
#                                          --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles"


# trying to find the winner of the subtuornament Q30098268 '2018 Wimbledon Championships – Women's Singles'

queryString = """

SELECT DISTINCT ?p ?pName ?o ?oName 
WHERE {
   wd:Q30098268 ?p ?o.
   
   # get the label
   ?p sc:name ?pName.
   ?o sc:name ?oName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows'), ('o', 'http://www.wikidata.org/entity/Q25344122'), ('oName', "2017 Wimbledon Championships – Women's Singles")]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q30085309'), ('oName', '2018 Wimbledon Championships')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pName', 'competition class'), ('o', 'http://www.wikidata.org/entity/Q16893403'), ('oName', "women's singles")]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('o', 'http://www.wikidata.org/entity/Q847'), ('oName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q46190676'), ('oName', 'tennis event')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('o', 'http://www.wikidata.org/entity/Q77178'), ('oName', 'Angelique Kerber')]
[('p', 'http://www.wikida

In [91]:
# Return the tournaments played in the 2018 Wimbledon Championships edition with the related winners 
# (hint: men and women do not play together, there are two distinct tournament ) 
# (the result set must be a list of elements with tournament IRI and label, and winner IRI and label).

# Q30085309 '2018 Wimbledon Championships' --P31 instance of --> Q41520 ('Wimbledon Championships');
#                                          --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles"
# Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') --> ?winner

# trying to find the winner of the subtuornament Q30098268 '2018 Wimbledon Championships – Women's Singles'


queryString = """

SELECT DISTINCT ?o ?oName 
WHERE {
   wd:Q30098268 wdt:P1346 ?o.
   
   # get the label 
   ?o sc:name ?oName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q77178'), ('oName', 'Angelique Kerber')]
1


In [92]:
# Return the tournaments played in the 2018 Wimbledon Championships edition with the related winners 
# (hint: men and women do not play together, there are two distinct tournament ) 
# (the result set must be a list of elements with tournament IRI and label, and winner IRI and label).

# Q30085309 '2018 Wimbledon Championships' --P31 instance of --> Q41520 ('Wimbledon Championships') 
# Q30085309 --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles"
# Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') --> ?winner

# now lets find the winner of each subtuornament of Q30085309 '2018 Wimbledon Championships'

queryString = """

SELECT DISTINCT ?subTournament ?subTournamentName ?winner ?winnerName
WHERE {
   wd:Q30085309 wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner.
   
   # get the label 
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('subTournament', 'http://www.wikidata.org/entity/Q32357790'), ('subTournamentName', "2018 Wimbledon Championships – Men's Doubles"), ('winner', 'http://www.wikidata.org/entity/Q53342'), ('winnerName', 'Mike Bryan')]
[('subTournament', 'http://www.wikidata.org/entity/Q54961494'), ('subTournamentName', "2018 Wimbledon Championships – Men's Singles"), ('winner', 'http://www.wikidata.org/entity/Q5812'), ('winnerName', 'Novak Djokovic')]
[('subTournament', 'http://www.wikidata.org/entity/Q30098268'), ('subTournamentName', "2018 Wimbledon Championships – Women's Singles"), ('winner', 'http://www.wikidata.org/entity/Q77178'), ('winnerName', 'Angelique Kerber')]
[('subTournament', 'http://www.wikidata.org/entity/Q32356704'), ('subTournamentName', "2018 Wimbledon Championships – Women's Doubles"), ('winner', 'http://www.wikidata.org/entity/Q11052500'), ('winnerName', 'Barbora Krejčíková')]
[('subTournament', 'http://www.wikidata.org/entity/Q55455423'), ('subTournamentName', "2018 Wimb

In [93]:
# Return the tournaments played in the 2018 Wimbledon Championships edition with the related winners 
# (hint: men and women do not play together, there are two distinct tournament ) 
# (the result set must be a list of elements with tournament IRI and label, and winner IRI and label).

# Q30085309 '2018 Wimbledon Championships' --P31 instance of --> Q41520 ('Wimbledon Championships') 
# Q30085309 --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles"
# Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') --> ?winner

# now lets find the winner of each subtuornament of Q30085309 '2018 Wimbledon Championships'
# ORDERING to understand 
queryString = """

SELECT DISTINCT ?subTournament ?subTournamentName ?winner ?winnerName
WHERE {
   wd:Q30085309 wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner.
   
   # get the label 
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
}
ORDER BY ASC (?subTournamentName)
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('subTournament', 'http://www.wikidata.org/entity/Q55410361'), ('subTournamentName', "2018 Wimbledon Championships – Boys' Doubles"), ('winner', 'http://www.wikidata.org/entity/Q55569239'), ('winnerName', 'Yankı Erel')]
[('subTournament', 'http://www.wikidata.org/entity/Q55381833'), ('subTournamentName', "2018 Wimbledon Championships – Boys' Singles"), ('winner', 'http://www.wikidata.org/entity/Q47516746'), ('winnerName', 'Tseng Chun-hsin')]
[('subTournament', 'http://www.wikidata.org/entity/Q55410290'), ('subTournamentName', "2018 Wimbledon Championships – Girls' Doubles"), ('winner', 'http://www.wikidata.org/entity/Q44465805'), ('winnerName', 'Wang Xinyu')]
[('subTournament', 'http://www.wikidata.org/entity/Q55410290'), ('subTournamentName', "2018 Wimbledon Championships – Girls' Doubles"), ('winner', 'http://www.wikidata.org/entity/Q41799108'), ('winnerName', 'Wang Xiyu')]
[('subTournament', 'http://www.wikidata.org/entity/Q55381839'), ('subTournamentName', "2018 Wimbledon 

### Final query for this task

In [94]:
# Return the tournaments played in the 2018 Wimbledon Championships edition with the related winners 
# (hint: men and women do not play together, there are two distinct tournament ) 
# (the result set must be a list of elements with tournament IRI and label, and winner IRI and label).

# Q30085309 '2018 Wimbledon Championships' --P31 instance of --> Q41520 ('Wimbledon Championships') 
# Q30085309 --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles"
# Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') --> ?winner

# Q30085309 '2018 Wimbledon Championships' --P527 ('has part')--> ?tournament --P1346 ('winner')--> ?winner

queryString = """
# retrieving BGP of each sub turnament of Q30085309 with the BGP of its winner 

SELECT DISTINCT ?tournament ?tournamentName ?winner ?winnerName
WHERE {
   # Q30085309 '2018 Wimbledon Championships' --P527 ('has part')--> ?tournament
   wd:Q30085309 wdt:P527 ?tournament.
   
   #?tournament --> P1346 ('winner') --> ?winner
   ?tournament wdt:P1346 ?winner.
   
   # get the label 
   ?tournament sc:name ?tournamentName.
   ?winner sc:name ?winnerName.
}
ORDER BY ASC (?tournamentName)
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('tournament', 'http://www.wikidata.org/entity/Q55410361'), ('tournamentName', "2018 Wimbledon Championships – Boys' Doubles"), ('winner', 'http://www.wikidata.org/entity/Q55569239'), ('winnerName', 'Yankı Erel')]
[('tournament', 'http://www.wikidata.org/entity/Q55381833'), ('tournamentName', "2018 Wimbledon Championships – Boys' Singles"), ('winner', 'http://www.wikidata.org/entity/Q47516746'), ('winnerName', 'Tseng Chun-hsin')]
[('tournament', 'http://www.wikidata.org/entity/Q55410290'), ('tournamentName', "2018 Wimbledon Championships – Girls' Doubles"), ('winner', 'http://www.wikidata.org/entity/Q44465805'), ('winnerName', 'Wang Xinyu')]
[('tournament', 'http://www.wikidata.org/entity/Q55410290'), ('tournamentName', "2018 Wimbledon Championships – Girls' Doubles"), ('winner', 'http://www.wikidata.org/entity/Q41799108'), ('winnerName', 'Wang Xiyu')]
[('tournament', 'http://www.wikidata.org/entity/Q55381839'), ('tournamentName', "2018 Wimbledon Championships – Girls' Singles

## Task 6
Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

#### POSSIBLE SOLUTION
1) get all winners of Grand Slams
2) then check their dob
3) filter them (dob>1975)
4) order them
5) select top 10

In [95]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# 1) getting all winner of yearly grand slam

queryString = """

SELECT DISTINCT ?yearlyWimbledonTournament ?yearlyWimbledonTournamentName
WHERE {
   # BACKWARD DISCOVERY
   ?yearlyWimbledonTournament ?p wd:Q41520.
   
   # get the label 
   ?yearlyWimbledonTournament sc:name ?yearlyWimbledonTournamentName .
}
ORDER BY ASC (?yearlyWimbledonTournamentName)
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q1934049'), ('yearlyWimbledonTournamentName', '1877 Wimbledon Championship')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q1253069'), ('yearlyWimbledonTournamentName', '1878 Wimbledon Championships')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q726549'), ('yearlyWimbledonTournamentName', '1879 Wimbledon Championship')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q2296326'), ('yearlyWimbledonTournamentName', '1880 Wimbledon Championship')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q2296335'), ('yearlyWimbledonTournamentName', '1881 Wimbledon Championship')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q2297185'), ('yearlyWimbledonTournamentName', '1882 Wimbledon Championship')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q522860'), ('yearlyWimbledonTournamentName', '1883 Wimbledon Championship')]
[('yea

In [96]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# 1) getting all winner of yearly grand slam

queryString = """

SELECT DISTINCT ?yearlyWimbledonTournament ?yearlyWimbledonTournamentName
WHERE {
   # BACKWARD DISCOVERY
   ?yearlyWimbledonTournament ?p wd:Q41520.
   
   # get the label 
   ?yearlyWimbledonTournament sc:name ?yearlyWimbledonTournamentName .
}
ORDER BY DESC (?yearlyWimbledonTournamentName)
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/P765'), ('yearlyWimbledonTournamentName', 'surface played on')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/P4503'), ('yearlyWimbledonTournamentName', 'Wimbledon player ID')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q837799'), ('yearlyWimbledonTournamentName', 'Wimbledon Effect')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q29473999'), ('yearlyWimbledonTournamentName', 'Vera Mattar')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q7920608'), ('yearlyWimbledonTournamentName', 'Venus Rosewater Dish')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q43323825'), ('yearlyWimbledonTournamentName', 'Template:Wimbledon player')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q3507351'), ('yearlyWimbledonTournamentName', 'Sylvie Jung')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q965205'), ('yearl

In [97]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# 1) getting all winner of yearly grand slam

queryString = """

SELECT DISTINCT ?p ?pName
WHERE {
   # BACKWARD DISCOVERY
   ?yearlyWimbledonTournament ?p wd:Q41520.
   
   # get the label 
   ?yearlyWimbledonTournament sc:name ?yearlyWimbledonTournamentName .
   ?p sc:name ?pName .
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pName', 'victory')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('pName', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pName', 'sports season of league or competition')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P466'), ('pName', 'occupant')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P710'), ('pName', 'participant')]
[('p', 'http://www.wik

In [98]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# 1) getting all winner of yearly grand slam
queryString = """

SELECT DISTINCT ?yearlyWimbledonTournament ?yearlyWimbledonTournamentName
WHERE {
   # BACKWARD DISCOVERY
   ?yearlyWimbledonTournament wdt:P31 wd:Q41520.
   
   # get the label 
   ?yearlyWimbledonTournament sc:name ?yearlyWimbledonTournamentName .
}
ORDER BY  DESC (?yearlyWimbledonTournamentName)
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q107283026'), ('yearlyWimbledonTournamentName', '2022 Wimbledon Championships')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q92032430'), ('yearlyWimbledonTournamentName', '2021 Wimbledon Championships')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q54093953'), ('yearlyWimbledonTournamentName', '2019 Wimbledon Championships')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q30085309'), ('yearlyWimbledonTournamentName', '2018 Wimbledon Championships')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q25344121'), ('yearlyWimbledonTournamentName', '2017 Wimbledon Championships')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q20127632'), ('yearlyWimbledonTournamentName', '2016 Wimbledon Championships')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q18410121'), ('yearlyWimbledonTournamentName', '2015 Wimbledon Champio

In [99]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# 1) getting all winner of yearly grand slam
queryString = """

SELECT DISTINCT ?yearlyWimbledonTournament ?yearlyWimbledonTournamentName
WHERE {
   # BACKWARD DISCOVERY
   ?yearlyWimbledonTournament wdt:P31 wd:Q41520.
   
   # get the label 
   ?yearlyWimbledonTournament sc:name ?yearlyWimbledonTournamentName .
}
ORDER BY  ASC (?yearlyWimbledonTournamentName)
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q1934049'), ('yearlyWimbledonTournamentName', '1877 Wimbledon Championship')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q1253069'), ('yearlyWimbledonTournamentName', '1878 Wimbledon Championships')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q726549'), ('yearlyWimbledonTournamentName', '1879 Wimbledon Championship')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q2296326'), ('yearlyWimbledonTournamentName', '1880 Wimbledon Championship')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q2296335'), ('yearlyWimbledonTournamentName', '1881 Wimbledon Championship')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q2297185'), ('yearlyWimbledonTournamentName', '1882 Wimbledon Championship')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q522860'), ('yearlyWimbledonTournamentName', '1883 Wimbledon Championship')]
[('yea

In [100]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# 1) getting all winner of yearly grand slam
queryString = """

SELECT DISTINCT (COUNT(?yearlyWimbledonTournament ) AS ?totTournaments)
WHERE {
   # BACKWARD DISCOVERY
   ?yearlyWimbledonTournament wdt:P31 wd:Q41520.
   
   # get the label 
   ?yearlyWimbledonTournament sc:name ?yearlyWimbledonTournamentName .
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('totTournaments', '135')]
1


In [101]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# 1) getting all winner of yearly grand slam
queryString = """

SELECT DISTINCT (COUNT(?yearlyWimbledonTournament ) AS ?totTournaments)
WHERE {
   # BACKWARD DISCOVERY
   ?yearlyWimbledonTournament ?p wd:Q41520.
   ?yearlyWimbledonTournament wdt:P527 ?subTournament.
   
   # get the label 
   ?yearlyWimbledonTournament sc:name ?yearlyWimbledonTournamentName .
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('totTournaments', '320')]
1


In [102]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# 1) getting all winner of yearly grand slam
queryString = """

SELECT (COUNT(?yearlyWimbledonTournament ) AS ?totTournaments){
    SELECT DISTINCT ?yearlyWimbledonTournament
    WHERE {
       # BACKWARD DISCOVERY
       ?yearlyWimbledonTournament ?p wd:Q41520.
       ?yearlyWimbledonTournament wdt:P527 ?subTournament.

       # get the label 
       ?yearlyWimbledonTournament sc:name ?yearlyWimbledonTournamentName .
    }
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('totTournaments', '76')]
1


In [103]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# 1) getting all winner of yearly grand slam
queryString = """

SELECT DISTINCT ?yearlyWimbledonTournament ?yearlyWimbledonTournamentName ?subTournament ?subTournamentName ?winner ?winnerName
WHERE {
   # BACKWARD DISCOVERY
   ?yearlyWimbledonTournament ?p wd:Q41520.
   ?yearlyWimbledonTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner.
   
   # get the label 
   ?yearlyWimbledonTournament sc:name ?yearlyWimbledonTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
}
ORDER BY ASC (?subTournamentName)
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q1934049'), ('yearlyWimbledonTournamentName', '1877 Wimbledon Championship'), ('subTournament', 'http://www.wikidata.org/entity/Q1354887'), ('subTournamentName', '1877 Wimbledon Championship – singles'), ('winner', 'http://www.wikidata.org/entity/Q469884'), ('winnerName', 'Spencer Gore')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q1253069'), ('yearlyWimbledonTournamentName', '1878 Wimbledon Championships'), ('subTournament', 'http://www.wikidata.org/entity/Q1356483'), ('subTournamentName', '1878 Wimbledon Championship – singles'), ('winner', 'http://www.wikidata.org/entity/Q469922'), ('winnerName', 'Frank Hadow')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q726549'), ('yearlyWimbledonTournamentName', '1879 Wimbledon Championship'), ('subTournament', 'http://www.wikidata.org/entity/Q1357074'), ('subTournamentName', '1879 Wimbledon Championship – singles'), ('winner', 'http://w

In [104]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# 1) getting all winner of yearly grand slam
queryString = """

SELECT (count(*) as ?tot){
SELECT DISTINCT ?yearlyWimbledonTournament ?yearlyWimbledonTournamentName ?subTournament ?subTournamentName ?winner ?winnerName
WHERE {
   # BACKWARD DISCOVERY
   ?yearlyWimbledonTournament ?p wd:Q41520.
   ?yearlyWimbledonTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner.
   
   # get the label 
   ?yearlyWimbledonTournament sc:name ?yearlyWimbledonTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
}
ORDER BY ASC (?subTournamentName)
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('tot', '422')]
1


In [105]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# 1) getting all winner of yearly grand slam
queryString = """

SELECT (count(*) as ?tot){
SELECT DISTINCT ?yearlyWimbledonTournament ?yearlyWimbledonTournamentName ?subTournament ?subTournamentName ?winner ?winnerName
WHERE {
   # BACKWARD DISCOVERY
   ?yearlyWimbledonTournament wdt:P31 wd:Q41520.
   ?yearlyWimbledonTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner.
   
   # get the label 
   ?yearlyWimbledonTournament sc:name ?yearlyWimbledonTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
}
ORDER BY ASC (?subTournamentName)
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('tot', '422')]
1


In [106]:
# 2) finding dob of the winner

##### All useless queries to find dob

In [107]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# from query 5 Q5812'), ('winnerName', 'Novak Djokovic')]
# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q5812 ?p ?o .
   # get the label 
   ?p sc:name ?pName.
   ?o sc:name ?oName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P22'), ('pName', 'father'), ('o', 'http://www.wikidata.org/entity/Q17291202'), ('oName', 'Srdjan Djokovic')]
[('p', 'http://www.wikidata.org/prop/direct/P25'), ('pName', 'mother'), ('o', 'http://www.wikidata.org/entity/Q17291209'), ('oName', 'Dijana Djokovic')]
[('p', 'http://www.wikidata.org/prop/direct/P3373'), ('pName', 'sibling'), ('o', 'http://www.wikidata.org/entity/Q17291218'), ('oName', 'Djordje Djokovic')]
[('p', 'http://www.wikidata.org/prop/direct/P734'), ('pName', 'family name'), ('o', 'http://www.wikidata.org/entity/Q21146583'), ('oName', 'Djokovic')]
[('p', 'http://www.wikidata.org/prop/direct/P3373'), ('pName', 'sibling'), ('o', 'http://www.wikidata.org/entity/Q3597288'), ('oName', 'Marko Djokovic')]
[('p', 'http://www.wikidata.org/prop/direct/P1050'), ('pName', 'medical condition'), ('o', 'http://www.wikidata.org/entity/Q11088'), ('oName', 'coeliac disease')]
[('p', 'http://www.wikidata.org/prop/direct/P140'), ('pName'

In [108]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# from query 5 Q5812'), ('winnerName', 'Novak Djokovic')]
# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q5812 ?p ?o .
   # get the label 
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   FILTER (?p != wdt:P1344)
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P22'), ('pName', 'father'), ('o', 'http://www.wikidata.org/entity/Q17291202'), ('oName', 'Srdjan Djokovic')]
[('p', 'http://www.wikidata.org/prop/direct/P25'), ('pName', 'mother'), ('o', 'http://www.wikidata.org/entity/Q17291209'), ('oName', 'Dijana Djokovic')]
[('p', 'http://www.wikidata.org/prop/direct/P3373'), ('pName', 'sibling'), ('o', 'http://www.wikidata.org/entity/Q17291218'), ('oName', 'Djordje Djokovic')]
[('p', 'http://www.wikidata.org/prop/direct/P734'), ('pName', 'family name'), ('o', 'http://www.wikidata.org/entity/Q21146583'), ('oName', 'Djokovic')]
[('p', 'http://www.wikidata.org/prop/direct/P3373'), ('pName', 'sibling'), ('o', 'http://www.wikidata.org/entity/Q3597288'), ('oName', 'Marko Djokovic')]
[('p', 'http://www.wikidata.org/prop/direct/P1050'), ('pName', 'medical condition'), ('o', 'http://www.wikidata.org/entity/Q11088'), ('oName', 'coeliac disease')]
[('p', 'http://www.wikidata.org/prop/direct/P140'), ('pName'

In [109]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# from query 5 Q5812'), ('winnerName', 'Novak Djokovic')]
# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q1426 ?p ?o .
   # get the label 
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   FILTER (?p != wdt:P1344)
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pName', 'victory'), ('o', 'http://www.wikidata.org/entity/Q22956224'), ('oName', "2017 Australian Open – Men's Singles")]
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pName', 'victory'), ('o', 'http://www.wikidata.org/entity/Q612597'), ('oName', '2007 Tennis Masters Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P3342'), ('pName', 'significant person'), ('o', 'http://www.wikidata.org/entity/Q54584'), ('oName', 'Andy Roddick')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received'), ('o', 'http://www.wikidata.org/entity/Q56509472'), ('oName', 'honorary doctor of the University of Basel')]
[('p', 'http://www.wikidata.org/prop/direct/P6758'), ('pName', 'supported sports team'), ('o', 'http://www.wikidata.org/entity/Q189671'), ('oName', 'FC Basel')]
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pName', 'victory'), ('o', 'http://www.wikidata.org/entity/Q782589'), ('oName', "2010 Au

In [110]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# from query 5 Q5812'), ('winnerName', 'Novak Djokovic')]
# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q1426 ?p ?o .
   # get the label 
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   FILTER (?p != wdt:P1344 && ?p != wdt:P2522)
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('pName', 'native language'), ('o', 'http://www.wikidata.org/entity/Q188'), ('oName', 'German')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q10833314'), ('oName', 'tennis player')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pName', 'employer'), ('o', 'http://www.wikidata.org/entity/Q740308'), ('oName', 'UNICEF')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q17290934'), ('oName', 'Lentapedia')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q67311526'), ('oName', 'Obalky knih.cz')]
[('p', 'http://www.wikidata.org/prop/direct/P140'), ('pName', 'religion'), ('o', 'http://www.wikidata.org/entity/Q9592'), ('oName', 'Catholic Church')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('pName

In [111]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# from query 5 Q5812'), ('winnerName', 'Novak Djokovic')]
# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q17291294 ?p ?o .
   # get the label 
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   FILTER (?p != wdt:P1344 && ?p != wdt:P2522)
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('pName', 'sex or gender'), ('o', 'http://www.wikidata.org/entity/Q6581072'), ('oName', 'female')]
[('p', 'http://www.wikidata.org/prop/direct/P22'), ('pName', 'father'), ('o', 'http://www.wikidata.org/entity/Q1426'), ('oName', 'Roger Federer')]
[('p', 'http://www.wikidata.org/prop/direct/P25'), ('pName', 'mother'), ('o', 'http://www.wikidata.org/entity/Q233306'), ('oName', 'Mirka Federer')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q5'), ('oName', 'human')]
[('p', 'http://www.wikidata.org/prop/direct/P3373'), ('pName', 'sibling'), ('o', 'http://www.wikidata.org/entity/Q17291316'), ('oName', 'Lennart Federer')]
[('p', 'http://www.wikidata.org/prop/direct/P3373'), ('pName', 'sibling'), ('o', 'http://www.wikidata.org/entity/Q17291300'), ('oName', 'Charlene Federer')]
[('p', 'http://www.wikidata.org/prop/direct/P3373'), ('pName', 'sibling'), ('o', 'http://www

In [112]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# from query 5 Q5812'), ('winnerName', 'Novak Djokovic')]
# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   ?s ?p wd:Q17291294.
   # get the label 
   ?p sc:name ?pName.
   ?s sc:name ?sName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q1426'), ('sName', 'Roger Federer'), ('p', 'http://www.wikidata.org/prop/direct/P40'), ('pName', 'child')]
[('s', 'http://www.wikidata.org/entity/Q17291300'), ('sName', 'Charlene Federer'), ('p', 'http://www.wikidata.org/prop/direct/P3373'), ('pName', 'sibling')]
[('s', 'http://www.wikidata.org/entity/Q17291308'), ('sName', 'Leo Federer'), ('p', 'http://www.wikidata.org/prop/direct/P3373'), ('pName', 'sibling')]
[('s', 'http://www.wikidata.org/entity/Q17291316'), ('sName', 'Lennart Federer'), ('p', 'http://www.wikidata.org/prop/direct/P3373'), ('pName', 'sibling')]
[('s', 'http://www.wikidata.org/entity/Q233306'), ('sName', 'Mirka Federer'), ('p', 'http://www.wikidata.org/prop/direct/P40'), ('pName', 'child')]
5


In [113]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# from query 5 Q5812'), ('winnerName', 'Novak Djokovic')]
# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   ?s ?p wd:Q1426.
   # get the label 
   ?p sc:name ?pName.
   ?s sc:name ?sName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q22956224'), ('sName', "2017 Australian Open – Men's Singles"), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner')]
[('s', 'http://www.wikidata.org/entity/Q3983549'), ('sName', '2007 Tennis Masters Cup – singles'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner')]
[('s', 'http://www.wikidata.org/entity/Q2537671'), ('sName', '2011 Qatar Open – singles'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner')]
[('s', 'http://www.wikidata.org/entity/Q13076407'), ('sName', "2012 Mutua Madrid Open – Men's Singles"), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner')]
[('s', 'http://www.wikidata.org/entity/Q30311438'), ('sName', '2017 Gerry Weber Open – singles'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner')]
[('s', 'http://www.wikidata.org/entity/Q3633911'), ('sName', '2005 Western & Southern Financial Group Masters – singles'), ('p', 'h

In [114]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# from query 5 Q5812'), ('winnerName', 'Novak Djokovic')]
# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   ?s ?p wd:Q30085309.
   # get the label 
   ?p sc:name ?pName.
   ?s sc:name ?sName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q30098268'), ('sName', "2018 Wimbledon Championships – Women's Singles"), ('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of')]
[('s', 'http://www.wikidata.org/entity/Q25344121'), ('sName', '2017 Wimbledon Championships'), ('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by')]
[('s', 'http://www.wikidata.org/entity/Q32358401'), ('sName', '2018 Wimbledon Championships – Mixed Doubles'), ('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of')]
[('s', 'http://www.wikidata.org/entity/Q55639017'), ('sName', '2018 Wimbledon Championships – Wheelchair Quad Doubles'), ('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of')]
[('s', 'http://www.wikidata.org/entity/Q55455418'), ('sName', "2018 Wimbledon Championships – Wheelchair Men's Doubles"), ('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of')]
[('s', 'http://www.wikidata.org/entity/Q54093953'), ('sName',

In [115]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# from query 5 Q5812'), ('winnerName', 'Novak Djokovic')]
# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   ?s ?p wd:Q5.
   # get the label 
   ?p sc:name ?pName.
   ?s sc:name ?sName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q1002728'), ('sName', 'Fonsi Nieto'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q100343342'), ('sName', 'Lubomír Stoklásek'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q102115260'), ('sName', 'Kojiro Kobayashi'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q102118153'), ('sName', 'Miguel E. M. Herrera'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q105753880'), ('sName', 'Mateo Egea'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q107520918'), ('sName', 'Jerónimo Latorre'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q

In [116]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# from query 5 Q5812'), ('winnerName', 'Novak Djokovic')]
# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q5 ?p ?o .
   # get the label 
   ?p sc:name ?pName.
   ?o sc:name ?oName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1317'), ('oName', 'floruit')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1477'), ('oName', 'birth name')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1559'), ('oName', 'name in native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P509'), ('oName', 'cause of death')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P3373'), ('oName', 'sibling')]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('pName', 'has quality'), ('o', 'http://www.wikidata.org/entity/Q785260'), ('oName', 'phase of human life')]


In [117]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# from query 5 Q5812'), ('winnerName', 'Novak Djokovic')]
# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q5 ?p ?o .
   # get the label 
   ?p sc:name ?pName.
   ?o sc:name ?oName.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1317'), ('oName', 'floruit')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1477'), ('oName', 'birth name')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1559'), ('oName', 'name in native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P509'), ('oName', 'cause of death')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P3373'), ('oName', 'sibling')]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('pName', 'has quality'), ('o', 'http://www.wikidata.org/entity/Q785260'), ('oName', 'phase of human life')]


In [118]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# from query 5 Q5812'), ('winnerName', 'Novak Djokovic')]
# 1) getting winners dob 

#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
queryString = """

SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q5 wdt:P569 ?o .
   # get the label 
   ?p sc:name ?pName.
   ?o sc:name ?oName.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
Empty


In [119]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# from query 5 Q5812'), ('winnerName', 'Novak Djokovic')]
# 1) getting winners dob 

#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
queryString = """

SELECT DISTINCT ?s ?sName ?o ?oName
WHERE {
   ?s wdt:P569 ?o .
   # get the label 
   ?s sc:name ?sName.
   ?o sc:name ?oName.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
Empty


In [120]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# from query 5 Q5812'), ('winnerName', 'Novak Djokovic')]
# 1) getting winners dob 

#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
queryString = """

SELECT DISTINCT ?s ?sName ?o ?oName
WHERE {
   ?s wdt:P527 ?o .
   # get the label 
   ?s sc:name ?sName.
   ?o sc:name ?oName.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q8200834'), ('sName', 'Anthriel'), ('o', 'http://www.wikidata.org/entity/Q11871976'), ('oName', 'Klaus Wirzenius')]
[('s', 'http://www.wikidata.org/entity/Q4570176'), ('sName', '1960s in Hong Kong'), ('o', 'http://www.wikidata.org/entity/Q15907987'), ('oName', '1963 in Hong Kong')]
[('s', 'http://www.wikidata.org/entity/Q4570176'), ('sName', '1960s in Hong Kong'), ('o', 'http://www.wikidata.org/entity/Q15925049'), ('oName', '1962 in Hong Kong')]
[('s', 'http://www.wikidata.org/entity/Q661481'), ('sName', '800s BC'), ('o', 'http://www.wikidata.org/entity/Q1640473'), ('oName', '809 BC')]
[('s', 'http://www.wikidata.org/entity/Q4619163'), ('sName', '2010s in Angola'), ('o', 'http://www.wikidata.org/entity/Q18109238'), ('oName', '2013 in Angola')]
[('s', 'http://www.wikidata.org/entity/Q8231071'), ('sName', 'Category:Alberta provincial highways'), ('o', 'http://www.wikidata.org/entity/Q28043697'), ('oName', 'Category:Alberta provincial highway

In [121]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# from query 5 Q5812'), ('winnerName', 'Novak Djokovic')]
# 1) getting winners dob 

#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
queryString = """

SELECT DISTINCT ?s ?sName ?o ?oName
WHERE {
   ?s wdt:P1963 ?o .
   # get the label 
   ?s sc:name ?sName.
   ?o sc:name ?oName.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q386724'), ('sName', 'work'), ('o', 'http://www.wikidata.org/entity/P747'), ('oName', 'has edition or translation')]
[('s', 'http://www.wikidata.org/entity/Q11513337'), ('sName', 'athletics competitor'), ('o', 'http://www.wikidata.org/entity/P1146'), ('oName', 'World Athletics athlete ID')]
[('s', 'http://www.wikidata.org/entity/Q11513337'), ('sName', 'athletics competitor'), ('o', 'http://www.wikidata.org/entity/P3765'), ('oName', 'All-Athletics.com ID')]
[('s', 'http://www.wikidata.org/entity/Q11513337'), ('sName', 'athletics competitor'), ('o', 'http://www.wikidata.org/entity/P3882'), ('oName', 'Tilastopaja female athlete ID')]
[('s', 'http://www.wikidata.org/entity/Q11513337'), ('sName', 'athletics competitor'), ('o', 'http://www.wikidata.org/entity/P3884'), ('oName', 'Tilastopaja male athlete ID')]
[('s', 'http://www.wikidata.org/entity/Q11513337'), ('sName', 'athletics competitor'), ('o', 'http://www.wikidata.org/entity/P3923'), ('oN

In [122]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# from query 5 Q5812'), ('winnerName', 'Novak Djokovic')]
# 1) getting winners dob 

#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   ?s ?p wd:P569.
   # get the label 
   ?s sc:name ?sName.
   ?p sc:name ?pName.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/P2668'), ('sName', 'stability of property value'), ('p', 'http://www.wikidata.org/prop/direct/P2271'), ('pName', 'Wikidata property example for properties')]
[('s', 'http://www.wikidata.org/entity/P3713'), ('sName', 'category for value not in Wikidata'), ('p', 'http://www.wikidata.org/prop/direct/P2271'), ('pName', 'Wikidata property example for properties')]
[('s', 'http://www.wikidata.org/entity/P2302'), ('sName', 'property constraint'), ('p', 'http://www.wikidata.org/prop/direct/P2271'), ('pName', 'Wikidata property example for properties')]
[('s', 'http://www.wikidata.org/entity/Q2389905'), ('sName', 'date of birth'), ('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property')]
[('s', 'http://www.wikidata.org/entity/Q105731028'), ('sName', 'by year of birth'), ('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property')]
[('s', 'http://www.wikidata.org/entity/Q1294626'), ('sName', 'music art

In [123]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# from query 5 Q5812'), ('winnerName', 'Novak Djokovic')]
# 1) getting winners dob 

#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
queryString = """

SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:P569 ?p ?o .
   # get the label 
   ?p sc:name ?pName.
   ?o sc:name ?oName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3709'), ('pName', 'category for value different from Wikidata'), ('o', 'http://www.wikidata.org/entity/Q19332014'), ('oName', 'Category:Pages with P569 (date of birth) different from Wikidata')]
[('p', 'http://www.wikidata.org/prop/direct/P1659'), ('pName', 'see also'), ('o', 'http://www.wikidata.org/entity/P1317'), ('oName', 'floruit')]
[('p', 'http://www.wikidata.org/prop/direct/P1659'), ('pName', 'see also'), ('o', 'http://www.wikidata.org/entity/P3150'), ('oName', 'birthday')]
[('p', 'http://www.wikidata.org/prop/direct/P1855'), ('pName', 'Wikidata property example'), ('o', 'http://www.wikidata.org/entity/Q160432'), ('oName', 'Jude Law')]
[('p', 'http://www.wikidata.org/prop/direct/P1855'), ('pName', 'Wikidata property example'), ('o', 'http://www.wikidata.org/entity/Q935'), ('oName', 'Isaac Newton')]
[('p', 'http://www.wikidata.org/prop/direct/P1629'), ('pName', 'Wikidata item of this property'), ('o', 'http://www.wikidata.org/e

In [124]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# from query 5 Q5812'), ('winnerName', 'Novak Djokovic')]
# 1) getting winners dob 

#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
queryString = """

SELECT NOW()
WHERE {
   wd:P569 ?p ?o .
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('callret-0', '2022-12-09T14:56:01.185593')]
[('callret-0', '2022-12-09T14:56:01.185593')]
[('callret-0', '2022-12-09T14:56:01.185593')]
[('callret-0', '2022-12-09T14:56:01.185593')]
[('callret-0', '2022-12-09T14:56:01.185593')]
[('callret-0', '2022-12-09T14:56:01.185593')]
[('callret-0', '2022-12-09T14:56:01.185593')]
[('callret-0', '2022-12-09T14:56:01.185593')]
[('callret-0', '2022-12-09T14:56:01.185593')]
[('callret-0', '2022-12-09T14:56:01.185593')]
[('callret-0', '2022-12-09T14:56:01.185593')]
[('callret-0', '2022-12-09T14:56:01.185593')]
[('callret-0', '2022-12-09T14:56:01.185593')]
[('callret-0', '2022-12-09T14:56:01.185593')]
[('callret-0', '2022-12-09T14:56:01.185593')]
[('callret-0', '2022-12-09T14:56:01.185593')]
[('callret-0', '2022-12-09T14:56:01.185593')]
[('callret-0', '2022-12-09T14:56:01.185593')]
[('callret-0', '2022-12-09T14:56:01.185593')]
[('callret-0', '2022-12-09T14:56:01.185593')]
[('callret-0', '2022-12-09T14:56:01.185593')]
[('callret-0', '2022-12-09

In [125]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)

# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q1426 ?p ?o .
   # get the label 
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   FILTER (?p != wdt:P1344 && ?p != wdt:P2522)
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('pName', 'native language'), ('o', 'http://www.wikidata.org/entity/Q188'), ('oName', 'German')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q10833314'), ('oName', 'tennis player')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pName', 'employer'), ('o', 'http://www.wikidata.org/entity/Q740308'), ('oName', 'UNICEF')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q17290934'), ('oName', 'Lentapedia')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q67311526'), ('oName', 'Obalky knih.cz')]
[('p', 'http://www.wikidata.org/prop/direct/P140'), ('pName', 'religion'), ('o', 'http://www.wikidata.org/entity/Q9592'), ('oName', 'Catholic Church')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('pName

In [126]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)

# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?p ?pName
WHERE {
   wd:Q1426 ?p ?o .
   # get the label 
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   # FILTER (?p != wdt:P1344 && ?p != wdt:P2522)
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pName', 'victory')]
[('p', 'http://www.wikidata.org/prop/direct/P3342'), ('pName', 'significant person')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P6758'), ('pName', 'supported sports team')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P741'), ('pName', 'playing hand')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('pName', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('pName', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P39'), ('pName', 'position held')]
[('p', 'http://www.wikidata.org/prop/direct/P552'), ('pName', 'handednes

In [127]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)

# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   ?s ?p wd:Q1426.
   # get the label 
   ?p sc:name ?pName.
   ?s sc:name ?sName.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q22956224'), ('sName', "2017 Australian Open – Men's Singles"), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner')]
[('s', 'http://www.wikidata.org/entity/Q3983549'), ('sName', '2007 Tennis Masters Cup – singles'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner')]
[('s', 'http://www.wikidata.org/entity/Q2537671'), ('sName', '2011 Qatar Open – singles'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner')]
[('s', 'http://www.wikidata.org/entity/Q13076407'), ('sName', "2012 Mutua Madrid Open – Men's Singles"), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner')]
[('s', 'http://www.wikidata.org/entity/Q30311438'), ('sName', '2017 Gerry Weber Open – singles'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner')]
[('s', 'http://www.wikidata.org/entity/Q3633911'), ('sName', '2005 Western & Southern Financial Group Masters – singles'), ('p', 'h

In [128]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)

# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?p ?pName (COUNT(*) AS ?tot)
WHERE {
   ?s ?p wd:Q1426.
   ?p sc:name ?pName.
}
GROUP BY ?p ?pName
LIMIT 200
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of'), ('tot', '24')]
[('p', 'http://www.wikidata.org/prop/direct/P40'), ('pName', 'child'), ('tot', '2')]
[('p', 'http://www.wikidata.org/prop/direct/P26'), ('pName', 'spouse'), ('tot', '1')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('tot', '117')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('pName', "category's main topic"), ('tot', '1')]
[('p', 'http://www.wikidata.org/prop/direct/P1855'), ('pName', 'Wikidata property example'), ('tot', '3')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('tot', '1')]
[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pName', 'sports season of league or competition'), ('tot', '1')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('tot', '1')]
[('p', 'http://www.wikidata.org/prop/direct/P921'), ('pName', 'main subject'), ('tot', '2')]
[('p', 'http://www.wikidata.org/prop/

In [129]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]

# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?sName ?pName ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   FILTER (?s = wd:P569)
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)


Results
[('sName', 'date of birth'), ('pName', 'has list'), ('oName', 'Birthday today')]
[('sName', 'date of birth'), ('pName', 'instance of'), ('oName', 'orderable Wikidata property')]
[('sName', 'date of birth'), ('pName', 'instance of'), ('oName', 'Wikidata property for items about people')]
[('sName', 'date of birth'), ('pName', 'instance of'), ('oName', "Wikidata property with datatype 'time'")]
[('sName', 'date of birth'), ('pName', 'instance of'), ('oName', 'Wikidata property for birth or death')]
[('sName', 'date of birth'), ('pName', 'instance of'), ('oName', 'Wikidata property with a single value, but different precisions and references can lead to multiple values')]
[('sName', 'date of birth'), ('pName', 'instance of'), ('oName', 'Wikidata property encoding a vCard value')]
[('sName', 'date of birth'), ('pName', 'see also'), ('oName', 'birthday')]
[('sName', 'date of birth'), ('pName', 'see also'), ('oName', 'date of baptism in early childhood')]
[('sName', 'date of birth'),

In [130]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]

# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?sName ?pName ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   FILTER (?o = wd:P569)
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)


Results
[('sName', 'Category:Pages with P569 (date of birth) different from Wikidata'), ('pName', 'see also'), ('oName', 'date of birth')]
[('sName', 'floruit'), ('pName', 'see also'), ('oName', 'date of birth')]
[('sName', 'birthday'), ('pName', 'see also'), ('oName', 'date of birth')]
[('sName', 'Extraction of Career Profiles from Wikipedia'), ('pName', 'uses property'), ('oName', 'date of birth')]
[('sName', 'fictional character'), ('pName', 'properties for this type'), ('oName', 'date of birth')]
[('sName', 'organizational founder'), ('pName', 'properties for this type'), ('oName', 'date of birth')]
[('sName', 'category for value not in Wikidata'), ('pName', 'Wikidata property example for properties'), ('oName', 'date of birth')]
[('sName', 'date of birth'), ('pName', 'Wikidata property'), ('oName', 'date of birth')]
[('sName', 'composer'), ('pName', 'properties for this type'), ('oName', 'date of birth')]
[('sName', 'human'), ('pName', 'properties for this type'), ('oName', 'date 

In [131]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]

# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   FILTER (?s = wd:P569)
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)


Results
[('pName', 'has list'), ('o', 'http://www.wikidata.org/entity/Q21540125'), ('oName', 'Birthday today')]
[('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q18668171'), ('oName', 'orderable Wikidata property')]
[('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q18608871'), ('oName', 'Wikidata property for items about people')]
[('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q18636219'), ('oName', "Wikidata property with datatype 'time'")]
[('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q18608756'), ('oName', 'Wikidata property for birth or death')]
[('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q99567026'), ('oName', 'Wikidata property with a single value, but different precisions and references can lead to multiple values')]
[('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q26935994'), ('oName', 'Wikidata property encoding a vCard value')]
[('pName', 'see also'), ('o', 'http://www.w

In [132]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]

# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   FILTER (?s = wd:P3150)
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)


Results
[('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q18608756'), ('oName', 'Wikidata property for birth or death')]
[('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q26935994'), ('oName', 'Wikidata property encoding a vCard value')]
[('pName', 'see also'), ('o', 'http://www.wikidata.org/entity/P837'), ('oName', 'day in year for periodic occurrence')]
[('pName', 'see also'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
[('pName', 'Wikidata item of this property'), ('o', 'http://www.wikidata.org/entity/Q47223'), ('oName', 'birthday')]
[('pName', 'Wikidata property example'), ('o', 'http://www.wikidata.org/entity/Q1188210'), ('oName', 'Haruka Shimotsuki')]
[('pName', 'Wikidata property example'), ('o', 'http://www.wikidata.org/entity/Q931'), ('oName', 'Naruto Uzumaki')]
[('pName', 'property constraint'), ('o', 'http://www.wikidata.org/entity/Q52004125'), ('oName', 'allowed entity types constraint')]
[('pName', 'property con

In [133]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]

# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   FILTER (?s = wd:Q1188210)
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)


Results
[('pName', 'field of work'), ('o', 'http://www.wikidata.org/entity/Q92186587'), ('oName', 'composing')]
[('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q488205'), ('oName', 'singer-songwriter')]
[('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q177220'), ('oName', 'singer')]
[('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q36834'), ('oName', 'composer')]
[('pName', 'instrument'), ('o', 'http://www.wikidata.org/entity/Q17172850'), ('oName', 'voice')]
[('pName', 'genre'), ('o', 'http://www.wikidata.org/entity/Q131578'), ('oName', 'J-pop')]
[('pName', 'languages spoken, written or signed'), ('o', 'http://www.wikidata.org/entity/Q5287'), ('oName', 'Japanese')]
[('pName', 'blood type'), ('o', 'http://www.wikidata.org/entity/Q19831453'), ('oName', 'A')]
[('pName', 'place of birth'), ('o', 'http://www.wikidata.org/entity/Q47896'), ('oName', 'Miyagi Prefecture')]
[('pName', 'sex or gender'), ('o', 'http://www.wikidata.org/entity/Q6581072')

In [134]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]

# 1) getting winners dob 
queryString = """

PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   #it never returns the result if un-comment the filter
   #FILTER (?o > "1975"^^xsd:gYear)
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q95452024'), ('sName', 'Template:Vice Presidents of the Maldives'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q10999435'), ('oName', 'Vice President of the Maldives')]
[('s', 'http://www.wikidata.org/entity/Q25752552'), ('sName', 'Template:2001 in Europe'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q17111750'), ('oName', '2001 in Europe')]
[('s', 'http://www.wikidata.org/entity/Q6675314'), ('sName', 'Template:Alcest'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q1517060'), ('oName', 'Alcest')]
[('s', 'http://www.wikidata.org/entity/Q22785857'), ('sName', 'Template:2006 in Europe'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q17111782'), ('oName', '2006 in Europe')]
[('s', 'http://www.wikidata.org/entity/Q85816744'), ('sName', 'Yodli'), ('pName', 'represents'), ('o', 'http://www.wikidata.org/entity/Q1053559'), ('oName', '2020 Wi

In [135]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]

# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   FILTER (REGEX(?oName, "1975.*"))
   FILTER (?p != wdt:P155 && ?p != wdt:P301)
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q4574296'), ('sName', '1970s in Hong Kong'), ('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('o', 'http://www.wikidata.org/entity/Q15930131'), ('oName', '1975 in Hong Kong')]
[('s', 'http://www.wikidata.org/entity/Q19568414'), ('sName', '1974 in Iran'), ('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by'), ('o', 'http://www.wikidata.org/entity/Q20987712'), ('oName', '1975 in Iran')]
[('s', 'http://www.wikidata.org/entity/Q19568417'), ('sName', '1974 in Kuwait'), ('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by'), ('o', 'http://www.wikidata.org/entity/Q19568426'), ('oName', '1975 in Kuwait')]
[('s', 'http://www.wikidata.org/entity/Q30731465'), ('sName', 'Category:1975 in Yugoslavian sport'), ('p', 'http://www.wikidata.org/prop/direct/P971'), ('pName', 'category combines topics'), ('o', 'http://www.wikidata.org/entity/Q1074111'), ('oName', '1975 in sports')]
[('s', 'h

In [136]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]

# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   #it never returns the result if un-comment the filter
   #FILTER (REGEX(?oName, "1981-08-08.*"))
   FILTER (?p NOT IN (wdt:P155,wdt:P301))
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q10275776'), ('sName', 'Estação Ferroviária de Mafra'), ('p', 'http://www.wikidata.org/prop/direct/P795'), ('pName', 'located on linear feature'), ('o', 'http://www.wikidata.org/entity/Q1160445'), ('oName', 'Linha do Oeste')]
[('s', 'http://www.wikidata.org/entity/Q11235194'), ('sName', 'Tokyo Solamachi'), ('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q11497641'), ('oName', 'Oshiage')]
[('s', 'http://www.wikidata.org/entity/Q15628796'), ('sName', 'Pedro Ponce de Cabrera'), ('p', 'http://www.wikidata.org/prop/direct/P119'), ('pName', 'place of burial'), ('o', 'http://www.wikidata.org/entity/Q16608610'), ('oName', 'Monastery of Santa María de Nogales')]
[('s', 'http://www.wikidata.org/entity/Q18483474'), ('sName', 'Telhada halt'), ('p', 'http://www.wikidata.org/prop/direct/P795'), ('pName', 'located on linear feature'), ('o', 'http://www

In [137]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]

# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   #it never returns the result if un-comment the filter
   #FILTER (REGEX(?oName, "1981-08-0.*"))
}
LIMIT 1
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q95452024'), ('sName', 'Template:Vice Presidents of the Maldives'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q10999435'), ('oName', 'Vice President of the Maldives')]
1


In [138]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]

# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   FILTER (REGEX(?oName, "1981.*"))
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q8165395'), ('sName', 'Category:1982 elections in South America'), ('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows'), ('o', 'http://www.wikidata.org/entity/Q16772598'), ('oName', 'Category:1981 elections in South America')]
[('s', 'http://www.wikidata.org/entity/Q15120431'), ('sName', 'Category:1982 in go'), ('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows'), ('o', 'http://www.wikidata.org/entity/Q15120313'), ('oName', 'Category:1981 in go')]
[('s', 'http://www.wikidata.org/entity/Q8165681'), ('sName', 'Category:1982 in New Zealand sport'), ('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows'), ('o', 'http://www.wikidata.org/entity/Q11721574'), ('oName', 'Category:1981 in New Zealand sport')]
[('s', 'http://www.wikidata.org/entity/Q52179909'), ('sName', '1982 in Finland'), ('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows'), ('o', 'http://www.wikidata.org/enti

In [139]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]

# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   FILTER (?oName = "1981")
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
Empty


In [140]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   FILTER (?oName = "1981-08")
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
Empty


In [141]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   FILTER (?oName = "1981-08-08")
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
Empty


In [142]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   FILTER (REGEX(?oName,"1987-.*"))
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q17027610'), ('sName', 'Remembering Christmas'), ('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows'), ('o', 'http://www.wikidata.org/entity/Q16953722'), ('oName', 'The Best of David Benoit 1987-1995')]
[('s', 'http://www.wikidata.org/entity/Q16466348'), ('sName', 'Shaken Not Stirred'), ('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by'), ('o', 'http://www.wikidata.org/entity/Q16953722'), ('oName', 'The Best of David Benoit 1987-1995')]
[('s', 'http://www.wikidata.org/entity/Q34757794'), ('sName', 'Child labor and musculoskeletal disorders: the Pelotas (Brazil) epidemiological survey.'), ('p', 'http://www.wikidata.org/prop/direct/P2860'), ('pName', 'cites work'), ('o', 'http://www.wikidata.org/entity/Q24169399'), ('oName', 'Work-related injuries to Massachusetts teens, 1987-1990')]
[('s', 'http://www.wikidata.org/entity/Q23910866'), ('sName', "Teen workers' exposures to occupational hazards and us

In [143]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   
   #it never returns the result if un-comment the filter
   #FILTER (REGEX(?oName,"1987-05.*"))
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q95452024'), ('sName', 'Template:Vice Presidents of the Maldives'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q10999435'), ('oName', 'Vice President of the Maldives')]
[('s', 'http://www.wikidata.org/entity/Q25752552'), ('sName', 'Template:2001 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q17111750'), ('oName', '2001 in Europe')]
[('s', 'http://www.wikidata.org/entity/Q6675314'), ('sName', 'Template:Alcest'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q1517060'), ('oName', 'Alcest')]
[('s', 'http://www.wikidata.org/entity/Q22785857'), ('sName', 'Template:2006 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity

In [144]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   FILTER (REGEX(?oName,"May 1987"))
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q9615411'), ('sName', 'Category:May 1987 events'), ('p', 'http://www.wikidata.org/prop/direct/P301'), ('pName', "category's main topic"), ('o', 'http://www.wikidata.org/entity/Q13072296'), ('oName', 'May 1987')]
[('s', 'http://www.wikidata.org/entity/Q69297253'), ('sName', 'May 4, 1987'), ('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q13072296'), ('oName', 'May 1987')]
[('s', 'http://www.wikidata.org/entity/Q69297267'), ('sName', 'May 16, 1987'), ('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q13072296'), ('oName', 'May 1987')]
[('s', 'http://www.wikidata.org/entity/Q69297272'), ('sName', 'May 20, 1987'), ('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q13072296'), ('oName', 'May 1987')]
[('s', 'http://www.wikidata.org/entity/Q69297268'), ('sName', 'M

In [145]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   
   #it never returns the result if un-comment the filter
   #FILTER (REGEX(?oName,"May 22, 1987"))
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q95452024'), ('sName', 'Template:Vice Presidents of the Maldives'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q10999435'), ('oName', 'Vice President of the Maldives')]
[('s', 'http://www.wikidata.org/entity/Q25752552'), ('sName', 'Template:2001 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q17111750'), ('oName', '2001 in Europe')]
[('s', 'http://www.wikidata.org/entity/Q6675314'), ('sName', 'Template:Alcest'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q1517060'), ('oName', 'Alcest')]
[('s', 'http://www.wikidata.org/entity/Q22785857'), ('sName', 'Template:2006 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity

In [146]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q69297274 ?p ?o.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q47150325'), ('oName', 'calendar day of a given year')]
[('p', 'http://www.wikidata.org/prop/direct/P2894'), ('pName', 'day of week'), ('o', 'http://www.wikidata.org/entity/Q130'), ('oName', 'Friday')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q13072296'), ('oName', 'May 1987')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q2581'), ('oName', 'May 22')]
4


In [147]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   ?s ?p wd:Q69297274.
   ?p sc:name ?pName.
   ?s sc:name ?sName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
Empty


In [148]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q13072296 ?p ?o.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q119'), ('oName', 'May')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q2429'), ('oName', '1987')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q47018478'), ('oName', 'calendar month of a given year')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q66010148'), ('oName', 'month starting on Friday')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q9615411'), ('oName', 'Category:May 1987 events')]
5


In [149]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q2429 ?p ?o.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q34644'), ('oName', '1980s')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q12138'), ('oName', 'Gregorian calendar')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q235690'), ('oName', 'common year starting and ending on Thursday')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q25860'), ('oName', '2nd millennium')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q3186692'), ('oName', 'calendar year')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q6927'), ('oName', '20th century')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pNa

In [150]:
#BACKWARD YEAR 1987 = NOVAK DJOKOVIC (22 May 1987)

In [151]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   ?s ?p wd:Q2429.
   ?p sc:name ?pName.
   ?s sc:name ?sName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q1078784'), ('sName', '1987 in sports'), ('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of')]
[('s', 'http://www.wikidata.org/entity/Q1187070'), ('sName', '1987 in science'), ('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of')]
[('s', 'http://www.wikidata.org/entity/Q8168795'), ('sName', 'Category:1987 Big Ten Conference football season'), ('p', 'http://www.wikidata.org/prop/direct/P971'), ('pName', 'category combines topics')]
[('s', 'http://www.wikidata.org/entity/Q8168888'), ('sName', 'Category:1987 controversies'), ('p', 'http://www.wikidata.org/prop/direct/P971'), ('pName', 'category combines topics')]
[('s', 'http://www.wikidata.org/entity/Q8168902'), ('sName', 'Category:1987 crimes'), ('p', 'http://www.wikidata.org/prop/direct/P971'), ('pName', 'category combines topics')]
[('s', 'http://www.wikidata.org/entity/Q8168903'), ('sName', 'Category:1987 disestablishments in Norway'), ('p', 'http

In [152]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   ?s ?p wd:Q1078784.
   ?p sc:name ?pName.
   ?s sc:name ?sName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q1191213'), ('sName', '1986 in sports'), ('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by')]
[('s', 'http://www.wikidata.org/entity/Q8169053'), ('sName', 'Category:1987 in American sports'), ('p', 'http://www.wikidata.org/prop/direct/P971'), ('pName', 'category combines topics')]
[('s', 'http://www.wikidata.org/entity/Q8169446'), ('sName', 'Category:1987 in sports'), ('p', 'http://www.wikidata.org/prop/direct/P301'), ('pName', "category's main topic")]
[('s', 'http://www.wikidata.org/entity/Q8169255'), ('sName', 'Category:1987 in Swiss sport'), ('p', 'http://www.wikidata.org/prop/direct/P971'), ('pName', 'category combines topics')]
[('s', 'http://www.wikidata.org/entity/Q301591'), ('sName', '1988 in sports'), ('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows')]
[('s', 'http://www.wikidata.org/entity/Q8169254'), ('sName', 'Category:1987 in Swedish sport'), ('p', 'http://www.wikidata.org/prop/dir

In [153]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q1078784 ?p ?o.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows'), ('o', 'http://www.wikidata.org/entity/Q1191213'), ('oName', '1986 in sports')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q8169446'), ('oName', 'Category:1987 in sports')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q18340514'), ('oName', 'events in a specific year or time period')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of'), ('o', 'http://www.wikidata.org/entity/Q349'), ('oName', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of'), ('o', 'http://www.wikidata.org/entity/Q2429'), ('oName', '1987')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by'), ('o', 'http://www.wikidata.org/entity/Q301591'), ('oName', '1988 in sports')]
6


In [3]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   # SOMETIMES WORKS, sometimes doesn't return any result
   # FILTER (REGEX(?pName,"[B|b]irth*"))
   #FILTER (?p not in (wdt:P19))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
The operation failed timed out


In [155]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   #it never returns the result if un-comment the filter
   #FILTER (REGEX(?pName,"[B|b]irth*"))
   FILTER (?p not in (wdt:P19))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q10275776'), ('sName', 'Estação Ferroviária de Mafra'), ('p', 'http://www.wikidata.org/prop/direct/P795'), ('pName', 'located on linear feature'), ('o', 'http://www.wikidata.org/entity/Q1160445'), ('oName', 'Linha do Oeste')]
[('s', 'http://www.wikidata.org/entity/Q11235194'), ('sName', 'Tokyo Solamachi'), ('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q11497641'), ('oName', 'Oshiage')]
[('s', 'http://www.wikidata.org/entity/Q15628796'), ('sName', 'Pedro Ponce de Cabrera'), ('p', 'http://www.wikidata.org/prop/direct/P119'), ('pName', 'place of burial'), ('o', 'http://www.wikidata.org/entity/Q16608610'), ('oName', 'Monastery of Santa María de Nogales')]
[('s', 'http://www.wikidata.org/entity/Q18483474'), ('sName', 'Telhada halt'), ('p', 'http://www.wikidata.org/prop/direct/P795'), ('pName', 'located on linear feature'), ('o', 'http://www

In [156]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   #it never returns the result if un-comment the filter
   #FILTER (REGEX(?pName,"[D|d][O|o][B|b]"))
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q95452024'), ('sName', 'Template:Vice Presidents of the Maldives'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q10999435'), ('oName', 'Vice President of the Maldives')]
[('s', 'http://www.wikidata.org/entity/Q25752552'), ('sName', 'Template:2001 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q17111750'), ('oName', '2001 in Europe')]
[('s', 'http://www.wikidata.org/entity/Q6675314'), ('sName', 'Template:Alcest'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q1517060'), ('oName', 'Alcest')]
[('s', 'http://www.wikidata.org/entity/Q22785857'), ('sName', 'Template:2006 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity

In [157]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   # Sometime it returns the result and other it doesn't #comment the filter
   #FILTER (REGEX(?pName,"date.*"))
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q95452024'), ('sName', 'Template:Vice Presidents of the Maldives'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q10999435'), ('oName', 'Vice President of the Maldives')]
[('s', 'http://www.wikidata.org/entity/Q25752552'), ('sName', 'Template:2001 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q17111750'), ('oName', '2001 in Europe')]
[('s', 'http://www.wikidata.org/entity/Q6675314'), ('sName', 'Template:Alcest'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q1517060'), ('oName', 'Alcest')]
[('s', 'http://www.wikidata.org/entity/Q22785857'), ('sName', 'Template:2006 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity

In [158]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   # sometimes filter with regex don't work
   #FILTER (REGEX(?pName,"date","i"))
   #FILTER (?p not in (wdt:P726, wdt:P991))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q95452024'), ('sName', 'Template:Vice Presidents of the Maldives'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q10999435'), ('oName', 'Vice President of the Maldives')]
[('s', 'http://www.wikidata.org/entity/Q25752552'), ('sName', 'Template:2001 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q17111750'), ('oName', '2001 in Europe')]
[('s', 'http://www.wikidata.org/entity/Q6675314'), ('sName', 'Template:Alcest'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q1517060'), ('oName', 'Alcest')]
[('s', 'http://www.wikidata.org/entity/Q22785857'), ('sName', 'Template:2006 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity

In [159]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q1426 ?p ?o.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pName', 'victory'), ('o', 'http://www.wikidata.org/entity/Q22956224'), ('oName', "2017 Australian Open – Men's Singles")]
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pName', 'victory'), ('o', 'http://www.wikidata.org/entity/Q612597'), ('oName', '2007 Tennis Masters Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P3342'), ('pName', 'significant person'), ('o', 'http://www.wikidata.org/entity/Q54584'), ('oName', 'Andy Roddick')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received'), ('o', 'http://www.wikidata.org/entity/Q56509472'), ('oName', 'honorary doctor of the University of Basel')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in'), ('o', 'http://www.wikidata.org/entity/Q589310'), ('oName', "tennis at the 2000 Summer Olympics – men's singles")]
[('p', 'http://www.wikidata.org/prop/direct/P6758'), ('pName', 'supported sports team'), ('o', 'http://www.

In [160]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   #this time filter doesnt work 
   #FILTER (REGEX(?pName,"year","i"))
   #FILTER (?p not in (wdt:P726, wdt:P991))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q95452024'), ('sName', 'Template:Vice Presidents of the Maldives'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q10999435'), ('oName', 'Vice President of the Maldives')]
[('s', 'http://www.wikidata.org/entity/Q25752552'), ('sName', 'Template:2001 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q17111750'), ('oName', '2001 in Europe')]
[('s', 'http://www.wikidata.org/entity/Q6675314'), ('sName', 'Template:Alcest'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q1517060'), ('oName', 'Alcest')]
[('s', 'http://www.wikidata.org/entity/Q22785857'), ('sName', 'Template:2006 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity

In [161]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   
   # today filter with regex not working
   # FILTER (REGEX(?pName,"year","i"))
   
   #it never returns the result if un-comment the filter
   #FILTER (?p not in (wdt:P2922, wdt:P991))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q95452024'), ('sName', 'Template:Vice Presidents of the Maldives'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q10999435'), ('oName', 'Vice President of the Maldives')]
[('s', 'http://www.wikidata.org/entity/Q25752552'), ('sName', 'Template:2001 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q17111750'), ('oName', '2001 in Europe')]
[('s', 'http://www.wikidata.org/entity/Q6675314'), ('sName', 'Template:Alcest'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q1517060'), ('oName', 'Alcest')]
[('s', 'http://www.wikidata.org/entity/Q22785857'), ('sName', 'Template:2006 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity

In [162]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   
   #today filter with regex not working
   #FILTER (REGEX(?pName,"year","i"))
   
   #it never returns the result if un-comment the filter
   #FILTER (?p not in (wdt:P2922, wdt:P837))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q95452024'), ('sName', 'Template:Vice Presidents of the Maldives'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q10999435'), ('oName', 'Vice President of the Maldives')]
[('s', 'http://www.wikidata.org/entity/Q25752552'), ('sName', 'Template:2001 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q17111750'), ('oName', '2001 in Europe')]
[('s', 'http://www.wikidata.org/entity/Q6675314'), ('sName', 'Template:Alcest'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q1517060'), ('oName', 'Alcest')]
[('s', 'http://www.wikidata.org/entity/Q22785857'), ('sName', 'Template:2006 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity

In [163]:
# BIRTHDAY FOUND

In [164]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   # today filter with regex not working
   #FILTER (REGEX(?pName,"birth","i"))
   FILTER (?p not in (wdt:P19, wdt:P837))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q10275776'), ('sName', 'Estação Ferroviária de Mafra'), ('p', 'http://www.wikidata.org/prop/direct/P795'), ('pName', 'located on linear feature'), ('o', 'http://www.wikidata.org/entity/Q1160445'), ('oName', 'Linha do Oeste')]
[('s', 'http://www.wikidata.org/entity/Q11235194'), ('sName', 'Tokyo Solamachi'), ('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q11497641'), ('oName', 'Oshiage')]
[('s', 'http://www.wikidata.org/entity/Q15628796'), ('sName', 'Pedro Ponce de Cabrera'), ('p', 'http://www.wikidata.org/prop/direct/P119'), ('pName', 'place of burial'), ('o', 'http://www.wikidata.org/entity/Q16608610'), ('oName', 'Monastery of Santa María de Nogales')]
[('s', 'http://www.wikidata.org/entity/Q18483474'), ('sName', 'Telhada halt'), ('p', 'http://www.wikidata.org/prop/direct/P795'), ('pName', 'located on linear feature'), ('o', 'http://www

In [165]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?s ?sName "birthday" ?o ?oName
WHERE {
   ?s wdt:P3150 ?o.
   ?s sc:name ?sName.
   ?o sc:name ?oName.
   
   FILTER (REGEX(?oName,"August 8","i"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q42012085'), ('sName', 'Hachiman Hikigaya'), ('callret-2', 'birthday'), ('o', 'http://www.wikidata.org/entity/Q2780'), ('oName', 'August 8')]
[('s', 'http://www.wikidata.org/entity/Q11541253'), ('sName', 'Natsuko Kusuda'), ('callret-2', 'birthday'), ('o', 'http://www.wikidata.org/entity/Q2780'), ('oName', 'August 8')]
[('s', 'http://www.wikidata.org/entity/Q107038831'), ('sName', 'Hoho Kajino'), ('callret-2', 'birthday'), ('o', 'http://www.wikidata.org/entity/Q2780'), ('oName', 'August 8')]
[('s', 'http://www.wikidata.org/entity/Q11256924'), ('sName', 'Ryuryū Akari'), ('callret-2', 'birthday'), ('o', 'http://www.wikidata.org/entity/Q2780'), ('oName', 'August 8')]
[('s', 'http://www.wikidata.org/entity/Q11536787'), ('sName', 'Ryū Katsuragi'), ('callret-2', 'birthday'), ('o', 'http://www.wikidata.org/entity/Q2780'), ('oName', 'August 8')]
[('s', 'http://www.wikidata.org/entity/Q106478679'), ('sName', 'Tatsuya Nakamura'), ('callret-2', 'birth

In [166]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q55539941 ?p ?o.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P3150'), ('pName', 'birthday'), ('o', 'http://www.wikidata.org/entity/Q2460'), ('oName', 'March 30')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q191633'), ('oName', 'mangaka')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('pName', 'country of citizenship'), ('o', 'http://www.wikidata.org/entity/Q17'), ('oName', 'Japan')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q5'), ('oName', 'human')]
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('pName', 'sex or gender'), ('o', 'http://www.wikidata.org/entity/Q6581072'), ('oName', 'female')]
[('p', 'http://www.wikidata.org/prop/direct/P735'), ('pName', 'given name'), ('o', 'http://www.wikidata.org/entity/Q650689'), ('oName', 'Alice')]
6


In [167]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   ?s ?p wd:Q55539941.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
Empty


In [168]:
# no use of 'birthday'

In [169]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer -> August 8, 1981
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   # today filter with regex not working
   #FILTER (REGEX(?pName,"birth","i"))
   
   
   #takes to long
   #FILTER (?p not in (wdt:P19, wdt:P3150))
}
LIMIT 5
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q95452024'), ('sName', 'Template:Vice Presidents of the Maldives'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q10999435'), ('oName', 'Vice President of the Maldives')]
[('s', 'http://www.wikidata.org/entity/Q25752552'), ('sName', 'Template:2001 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q17111750'), ('oName', '2001 in Europe')]
[('s', 'http://www.wikidata.org/entity/Q6675314'), ('sName', 'Template:Alcest'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q1517060'), ('oName', 'Alcest')]
[('s', 'http://www.wikidata.org/entity/Q22785857'), ('sName', 'Template:2006 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity

In [170]:
# write your queries

# some how now found [('p', 'http://www.wikidata.org/prop/direct/P569'), ('name', 'date of birth')]
# 1) getting winners dob 


queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q1426 ?p ?o .
   # get the label
   ?p sc:name ?pName.
   ?o sc:name ?oName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pName', 'victory'), ('o', 'http://www.wikidata.org/entity/Q22956224'), ('oName', "2017 Australian Open – Men's Singles")]
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pName', 'victory'), ('o', 'http://www.wikidata.org/entity/Q612597'), ('oName', '2007 Tennis Masters Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P3342'), ('pName', 'significant person'), ('o', 'http://www.wikidata.org/entity/Q54584'), ('oName', 'Andy Roddick')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received'), ('o', 'http://www.wikidata.org/entity/Q56509472'), ('oName', 'honorary doctor of the University of Basel')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in'), ('o', 'http://www.wikidata.org/entity/Q589310'), ('oName', "tennis at the 2000 Summer Olympics – men's singles")]
[('p', 'http://www.wikidata.org/prop/direct/P6758'), ('pName', 'supported sports team'), ('o', 'http://www.

In [171]:
# write your queries

# some how now found [('p', 'http://www.wikidata.org/prop/direct/P569'), ('name', 'date of birth')]
# 1) getting winners dob 

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q1426 ?p ?o .
   # get the label
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   FILTER (?p not in (wdt:P2522, wdt:P166))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3342'), ('pName', 'significant person'), ('o', 'http://www.wikidata.org/entity/Q54584'), ('oName', 'Andy Roddick')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in'), ('o', 'http://www.wikidata.org/entity/Q589310'), ('oName', "tennis at the 2000 Summer Olympics – men's singles")]
[('p', 'http://www.wikidata.org/prop/direct/P6758'), ('pName', 'supported sports team'), ('o', 'http://www.wikidata.org/entity/Q189671'), ('oName', 'FC Basel')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q10833314'), ('oName', 'tennis player')]
[('p', 'http://www.wikidata.org/prop/direct/P741'), ('pName', 'playing hand'), ('o', 'http://www.wikidata.org/entity/Q14420039'), ('oName', 'one-handed backhand')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('pName', 'languages spoken, written or signed'), ('o', 'http://www.wikidata.org/entity/Q150'), ('oNa

In [172]:
# write your queries

# some how now found [('p', 'http://www.wikidata.org/prop/direct/P569'), ('name', 'date of birth')]
# 1) getting winners dob 

queryString = """
SELECT DISTINCT ?o ?oName
WHERE {
   wd:Q1426 wdt:P569 ?o .
   ?o sc:name ?oName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [173]:
# write your queries

# some how now found [('p', 'http://www.wikidata.org/prop/direct/P569'), ('name', 'date of birth')]
# 1) getting winners dob 

queryString = """
SELECT DISTINCT ?p ?pName
WHERE {
   wd:Q1426 ?p ?o .
   ?p sc:name ?pName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('pName', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('pName', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pName', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('pName', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('pName', 'CONOR.SI ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1284'), ('pName', 'Munzinger person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('pName', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('pName', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1352')

In [174]:
# write your queries

# some how now found [('p', 'http://www.wikidata.org/prop/direct/P569'), ('name', 'date of birth')]
# 1) getting winners dob 

queryString = """
SELECT DISTINCT ?p ?pName
WHERE {
   wd:Q1426 ?p ?o .
   ?p sc:name ?pName.
   
   FILTER (?p in (wdt:P569))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P569'), ('pName', 'date of birth')]
1


In [175]:
# write your queries

# some how now found [('p', 'http://www.wikidata.org/prop/direct/P569'), ('name', 'date of birth')]
# 1) getting winners dob 

queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q1426 ?p ?o .
   ?o sc:name ?name.
   
   FILTER (?p in (wdt:P569))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [176]:
# write your queries

# some how now found [('p', 'http://www.wikidata.org/prop/direct/P569'), ('name', 'date of birth')]
# 1) getting winners dob 

queryString = """
SELECT ?o ?name
WHERE {
   wd:Q1426 ?p ?o .
   ?o sc:name ?name.
   
   FILTER (?p in (wdt:P569))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


##### FINALLYYYYYYY AFTER 300 QUERIES

In [177]:
# write your queries

# some how now found [('p', 'http://www.wikidata.org/prop/direct/P569'), ('name', 'date of birth')]
# 1) getting winners dob 

queryString = """
SELECT ?o
WHERE {
   wd:Q1426 ?p ?o .   
   FILTER (?p in (wdt:P569))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', '1981-08-08T00:00:00Z')]
1


In [178]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# 1) getting all winner of yearly Wimbledon Tournament
queryString = """

SELECT DISTINCT ?yearlyWimbledonTournament ?yearlyWimbledonTournamentName ?subTournament ?subTournamentName ?winner ?winnerName
WHERE {
   # BACKWARD DISCOVERY
   ?yearlyWimbledonTournament wdt:P31 wd:Q41520.
   ?yearlyWimbledonTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner.
   
   # get the label 
   ?yearlyWimbledonTournament sc:name ?yearlyWimbledonTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
}
ORDER BY ASC (?subTournamentName)
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q1934049'), ('yearlyWimbledonTournamentName', '1877 Wimbledon Championship'), ('subTournament', 'http://www.wikidata.org/entity/Q1354887'), ('subTournamentName', '1877 Wimbledon Championship – singles'), ('winner', 'http://www.wikidata.org/entity/Q469884'), ('winnerName', 'Spencer Gore')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q1253069'), ('yearlyWimbledonTournamentName', '1878 Wimbledon Championships'), ('subTournament', 'http://www.wikidata.org/entity/Q1356483'), ('subTournamentName', '1878 Wimbledon Championship – singles'), ('winner', 'http://www.wikidata.org/entity/Q469922'), ('winnerName', 'Frank Hadow')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q726549'), ('yearlyWimbledonTournamentName', '1879 Wimbledon Championship'), ('subTournament', 'http://www.wikidata.org/entity/Q1357074'), ('subTournamentName', '1879 Wimbledon Championship – singles'), ('winner', 'http://w

In [179]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# 1 - getting all winner of yearly Wimbledon Tournament
# 2 - retrieving their dob to select only those who was born after 1975
queryString = """

SELECT DISTINCT ?yearlyWimbledonTournament ?yearlyWimbledonTournamentName ?subTournament ?subTournamentName ?winner ?winnerName ?dob
WHERE {
   # BACKWARD DISCOVERY
   ?yearlyWimbledonTournament ?p wd:Q41520.
   ?yearlyWimbledonTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner.
   
   # get the label 
   ?yearlyWimbledonTournament sc:name ?yearlyWimbledonTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?winner wdt:P569 ?dob
}
ORDER BY ASC (?dob)
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q2296326'), ('yearlyWimbledonTournamentName', '1880 Wimbledon Championship'), ('subTournament', 'http://www.wikidata.org/entity/Q1355471'), ('subTournamentName', '1880 Wimbledon Championship – singles'), ('winner', 'http://www.wikidata.org/entity/Q538191'), ('winnerName', 'John Hartley'), ('dob', '1849-01-09T00:00:00Z')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q726549'), ('yearlyWimbledonTournamentName', '1879 Wimbledon Championship'), ('subTournament', 'http://www.wikidata.org/entity/Q1357074'), ('subTournamentName', '1879 Wimbledon Championship – singles'), ('winner', 'http://www.wikidata.org/entity/Q538191'), ('winnerName', 'John Hartley'), ('dob', '1849-01-09T00:00:00Z')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q1934049'), ('yearlyWimbledonTournamentName', '1877 Wimbledon Championship'), ('subTournament', 'http://www.wikidata.org/entity/Q1354887'), ('subTournamentNam

In [180]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# 1 - getting all winner of yearly Wimbledon Tournament
# 2 - select only those who was born after 1975
queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?yearlyWimbledonTournament ?yearlyWimbledonTournamentName ?subTournament ?subTournamentName ?winner ?winnerName ?dob
WHERE {
   # BACKWARD DISCOVERY
   ?yearlyWimbledonTournament ?p wd:Q41520.
   ?yearlyWimbledonTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner.
   
   # get the label 
   ?yearlyWimbledonTournament sc:name ?yearlyWimbledonTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?winner wdt:P569 ?dob
   FILTER (?dob > "1975-01-01"^^xsd:date)
}
ORDER BY ASC (?dob)
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q746693'), ('yearlyWimbledonTournamentName', '2005 Wimbledon Championships'), ('subTournament', 'http://www.wikidata.org/entity/Q2397886'), ('subTournamentName', '2005 Wimbledon Championships – Mixed Doubles'), ('winner', 'http://www.wikidata.org/entity/Q210695'), ('winnerName', 'Mary Pierce'), ('dob', '1975-01-15T00:00:00Z')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q832484'), ('yearlyWimbledonTournamentName', '2003 Wimbledon Championships'), ('subTournament', 'http://www.wikidata.org/entity/Q1575145'), ('subTournamentName', "2003 Wimbledon Championships – Women's Doubles"), ('winner', 'http://www.wikidata.org/entity/Q229100'), ('winnerName', 'Ai Sugiyama'), ('dob', '1975-07-05T00:00:00Z')]
[('yearlyWimbledonTournament', 'http://www.wikidata.org/entity/Q245073'), ('yearlyWimbledonTournamentName', '2011 Wimbledon Championships'), ('subTournament', 'http://www.wikidata.org/entity/Q607519'), ('su

In [181]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# 1 - getting all winner of yearly Wimbledon Tournament
# 2 - select only those who was born after 1975
# 3 - counting their victories

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winner ?winnerName ?dob (COUNT(*) AS ?victories)
WHERE {
   # BACKWARD DISCOVERY
   ?yearlyWimbledonTournament ?p wd:Q41520.
   ?yearlyWimbledonTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner.
   
   # get the label 
   ?yearlyWimbledonTournament sc:name ?yearlyWimbledonTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?winner wdt:P569 ?dob
   FILTER (?dob > "1975-01-01"^^xsd:date)
}
GROUP BY ?winner ?winnerName ?dob
ORDER BY ASC (?dob)
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q210695'), ('winnerName', 'Mary Pierce'), ('dob', '1975-01-15T00:00:00Z'), ('victories', '1')]
[('winner', 'http://www.wikidata.org/entity/Q229100'), ('winnerName', 'Ai Sugiyama'), ('dob', '1975-07-05T00:00:00Z'), ('victories', '1')]
[('winner', 'http://www.wikidata.org/entity/Q234480'), ('winnerName', 'Květa Peschke'), ('dob', '1975-07-09T00:00:00Z'), ('victories', '1')]
[('winner', 'http://www.wikidata.org/entity/Q233231'), ('winnerName', 'Elena Likhovtseva'), ('dob', '1975-09-08T00:00:00Z'), ('victories', '1')]
[('winner', 'http://www.wikidata.org/entity/Q369885'), ('winnerName', 'Stephen Huss'), ('dob', '1975-12-10T00:00:00Z'), ('victories', '1')]
[('winner', 'http://www.wikidata.org/entity/Q310419'), ('winnerName', 'Nenad Zimonjić'), ('dob', '1976-06-04T00:00:00Z'), ('victories', '1')]
[('winner', 'http://www.wikidata.org/entity/Q11677'), ('winnerName', 'Lindsay Davenport'), ('dob', '1976-06-08T00:00:00Z'), ('victories', '2')]
[(

In [182]:
# write your queries
#CORRECT ORDERING [ONLY WIMBLEDON CHAMPIONSHIPS]

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# 1 - getting all winner of yearly Wimbledon Tournament
# 2 - select only those who was born after 1975
# 3 - counting their victories
# 4 - ordering by victories in descenting order

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winner ?winnerName ?dob (COUNT(*) AS ?victories)
WHERE {
   # BACKWARD DISCOVERY
   ?yearlyWimbledonTournament ?p wd:Q41520.
   ?yearlyWimbledonTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner.
   
   # get the label 
   ?yearlyWimbledonTournament sc:name ?yearlyWimbledonTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?winner wdt:P569 ?dob
   FILTER (?dob > "1975-01-01"^^xsd:date)
}
GROUP BY ?winner ?winnerName ?dob
ORDER BY DESC (?victories)
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q11459'), ('winnerName', 'Serena Williams'), ('dob', '1981-09-26T00:00:00Z'), ('victories', '14')]
[('winner', 'http://www.wikidata.org/entity/Q11578'), ('winnerName', 'Venus Williams'), ('dob', '1980-06-17T00:00:00Z'), ('victories', '11')]
[('winner', 'http://www.wikidata.org/entity/Q1426'), ('winnerName', 'Roger Federer'), ('dob', '1981-08-08T00:00:00Z'), ('victories', '8')]
[('winner', 'http://www.wikidata.org/entity/Q5812'), ('winnerName', 'Novak Djokovic'), ('dob', '1987-05-22T00:00:00Z'), ('victories', '6')]
[('winner', 'http://www.wikidata.org/entity/Q134720'), ('winnerName', 'Martina Hingis'), ('dob', '1980-09-30T00:00:00Z'), ('victories', '6')]
[('winner', 'http://www.wikidata.org/entity/Q229128'), ('winnerName', 'Cara Black'), ('dob', '1979-02-17T00:00:00Z'), ('victories', '5')]
[('winner', 'http://www.wikidata.org/entity/Q24574661'), ('winnerName', 'Diede de Groot'), ('dob', '1996-12-19T00:00:00Z'), ('victories', '4')]
[('w

In [183]:
# write your queries

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->

# 1 - getting all winner of yearly Wimbledon Tournament
# 2 - select only those who was born after 1975
# 3 - counting their victories
# 4 - ordering by victories in descenting order
# 5 - limiting to only top 10 distinct winners

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winner ?winnerName (COUNT(*) AS ?victories)
WHERE {
   # BACKWARD DISCOVERY
   ?yearlyWimbledonTournament ?p wd:Q41520.
   ?yearlyWimbledonTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner.
   
   # get the label 
   ?yearlyWimbledonTournament sc:name ?yearlyWimbledonTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?winner wdt:P569 ?dob
   FILTER (?dob > "1975-01-01"^^xsd:date)
}
GROUP BY ?winner ?winnerName
ORDER BY DESC (?victories)
LIMIT 10

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q11459'), ('winnerName', 'Serena Williams'), ('victories', '14')]
[('winner', 'http://www.wikidata.org/entity/Q11578'), ('winnerName', 'Venus Williams'), ('victories', '11')]
[('winner', 'http://www.wikidata.org/entity/Q1426'), ('winnerName', 'Roger Federer'), ('victories', '8')]
[('winner', 'http://www.wikidata.org/entity/Q134720'), ('winnerName', 'Martina Hingis'), ('victories', '6')]
[('winner', 'http://www.wikidata.org/entity/Q5812'), ('winnerName', 'Novak Djokovic'), ('victories', '6')]
[('winner', 'http://www.wikidata.org/entity/Q229128'), ('winnerName', 'Cara Black'), ('victories', '5')]
[('winner', 'http://www.wikidata.org/entity/Q53340'), ('winnerName', 'Bob Bryan'), ('victories', '4')]
[('winner', 'http://www.wikidata.org/entity/Q24574661'), ('winnerName', 'Diede de Groot'), ('victories', '4')]
[('winner', 'http://www.wikidata.org/entity/Q232566'), ('winnerName', 'Hsieh Su-wei'), ('victories', '3')]
[('winner', 'http://www.w

In [184]:
# write your queries
# CORRECT ORDERING [FRENCH OPEN]

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->
# Results TASK 3
#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# 1* - getting all winner of yearly French Open
# 2 - select only those who was born after 1975
# 3 - counting their victories
# 4 - ordering by victories in descenting order
# 5 - limiting to only top 10 distinct winners


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winner ?winnerName ?dob (COUNT(*) AS ?victories)
WHERE {
   # BACKWARD DISCOVERY
   ?yearlyTournament ?p wd:Q43605.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner.
   
   # get the label 
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?winner wdt:P569 ?dob
   FILTER (?dob > "1975-01-01"^^xsd:date)
}
GROUP BY ?winner ?winnerName ?dob
ORDER BY DESC (?victories)
LIMIT 10

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q10132'), ('winnerName', 'Rafael Nadal'), ('dob', '1986-06-03T00:00:00Z'), ('victories', '13')]
[('winner', 'http://www.wikidata.org/entity/Q11459'), ('winnerName', 'Serena Williams'), ('dob', '1981-09-26T00:00:00Z'), ('victories', '5')]
[('winner', 'http://www.wikidata.org/entity/Q16224602'), ('winnerName', 'Yui Kamiji'), ('dob', '1994-04-24T00:00:00Z'), ('victories', '4')]
[('winner', 'http://www.wikidata.org/entity/Q24574661'), ('winnerName', 'Diede de Groot'), ('dob', '1996-12-19T00:00:00Z'), ('victories', '4')]
[('winner', 'http://www.wikidata.org/entity/Q11682'), ('winnerName', 'Justine Henin'), ('dob', '1982-06-01T00:00:00Z'), ('victories', '4')]
[('winner', 'http://www.wikidata.org/entity/Q11052500'), ('winnerName', 'Barbora Krejčíková'), ('dob', '1995-12-18T00:00:00Z'), ('victories', '4')]
[('winner', 'http://www.wikidata.org/entity/Q190723'), ('winnerName', 'Gustavo Kuerten'), ('dob', '1976-09-10T00:00:00Z'), ('victories', '

In [185]:
# write your queries
# CORRECT ORDERING [US OPEN]

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->
# Results TASK 3
#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# 1* - getting all winner of yearly US Open
# 2 - select only those who was born after 1975
# 3 - counting their victories
# 4 - ordering by victories in descenting order
# 5 - limiting to only top 10 distinct winners

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winner ?winnerName ?dob (COUNT(*) AS ?victories)
WHERE {
   # BACKWARD DISCOVERY
   ?yearlyTournament ?p wd:Q123577.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner.
   
   # get the label 
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?winner wdt:P569 ?dob
   FILTER (?dob > "1975-01-01"^^xsd:date)
}
GROUP BY ?winner ?winnerName ?dob
ORDER BY DESC (?victories)
LIMIT 10

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q11459'), ('winnerName', 'Serena Williams'), ('dob', '1981-09-26T00:00:00Z'), ('victories', '6')]
[('winner', 'http://www.wikidata.org/entity/Q26951148'), ('winnerName', 'Alfie Hewett'), ('dob', '1997-12-06T00:00:00Z'), ('victories', '5')]
[('winner', 'http://www.wikidata.org/entity/Q24574661'), ('winnerName', 'Diede de Groot'), ('dob', '1996-12-19T00:00:00Z'), ('victories', '5')]
[('winner', 'http://www.wikidata.org/entity/Q134720'), ('winnerName', 'Martina Hingis'), ('dob', '1980-09-30T00:00:00Z'), ('victories', '4')]
[('winner', 'http://www.wikidata.org/entity/Q10132'), ('winnerName', 'Rafael Nadal'), ('dob', '1986-06-03T00:00:00Z'), ('victories', '3')]
[('winner', 'http://www.wikidata.org/entity/Q54537'), ('winnerName', 'Bethanie Mattek-Sands'), ('dob', '1985-03-23T00:00:00Z'), ('victories', '3')]
[('winner', 'http://www.wikidata.org/entity/Q11581'), ('winnerName', 'Andrea Sestini Hlaváčková'), ('dob', '1986-08-10T00:00:00Z'), ('v

In [186]:
# write your queries
# CORRECT ORDERING [Australian OPEN]

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->
# Results TASK 3
#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# 1* - getting all winner of yearly Australian Open
# 2 - select only those who was born after 1975
# 3 - counting their victories
# 4 - ordering by victories in descenting order
# 5 - limiting to only top 10 distinct winners

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winner ?winnerName ?dob (COUNT(*) AS ?victories)
WHERE {
   # BACKWARD DISCOVERY
   ?yearlyTournament ?p wd:Q60874.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner.
   
   # get the label 
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?winner wdt:P569 ?dob
   FILTER (?dob > "1975-01-01"^^xsd:date)
}
GROUP BY ?winner ?winnerName ?dob
ORDER BY DESC (?victories)
LIMIT 10

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q11459'), ('winnerName', 'Serena Williams'), ('dob', '1981-09-26T00:00:00Z'), ('victories', '11')]
[('winner', 'http://www.wikidata.org/entity/Q134720'), ('winnerName', 'Martina Hingis'), ('dob', '1980-09-30T00:00:00Z'), ('victories', '10')]
[('winner', 'http://www.wikidata.org/entity/Q5812'), ('winnerName', 'Novak Djokovic'), ('dob', '1987-05-22T00:00:00Z'), ('victories', '9')]
[('winner', 'http://www.wikidata.org/entity/Q53340'), ('winnerName', 'Bob Bryan'), ('dob', '1978-04-29T00:00:00Z'), ('victories', '6')]
[('winner', 'http://www.wikidata.org/entity/Q1426'), ('winnerName', 'Roger Federer'), ('dob', '1981-08-08T00:00:00Z'), ('victories', '6')]
[('winner', 'http://www.wikidata.org/entity/Q53342'), ('winnerName', 'Mike Bryan'), ('dob', '1978-04-29T00:00:00Z'), ('victories', '6')]
[('winner', 'http://www.wikidata.org/entity/Q959958'), ('winnerName', 'Shingo Kunieda'), ('dob', '1984-02-21T00:00:00Z'), ('victories', '6')]
[('winner', 

In [187]:
# write your queries
# ALL

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->
# Results TASK 3
#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]


# 1* - getting all winner of yearly GRAND SLAMS  <--- putting all togheter 
# 2 - select only those who was born after 1975
# 3 - counting their victories
# 4 - ordering by victories in descenting order
# 5 - limiting to only top 10 distinct winners

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winner ?winnerName ?dob (COUNT(*) AS ?victories)
WHERE {
   VALUES (?tournament){
   (wd:Q43605)
   (wd:Q123577)
   (wd:Q41520)
   (wd:Q60874)
   }
   ?yearlyTournament ?p ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner.
   
   # get the label 
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?winner wdt:P569 ?dob
   FILTER (?dob > "1975-01-01"^^xsd:date)
}
GROUP BY ?winner ?winnerName ?dob
ORDER BY DESC (?victories)
LIMIT 10

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q11459'), ('winnerName', 'Serena Williams'), ('dob', '1981-09-26T00:00:00Z'), ('victories', '36')]
[('winner', 'http://www.wikidata.org/entity/Q134720'), ('winnerName', 'Martina Hingis'), ('dob', '1980-09-30T00:00:00Z'), ('victories', '23')]
[('winner', 'http://www.wikidata.org/entity/Q11578'), ('winnerName', 'Venus Williams'), ('dob', '1980-06-17T00:00:00Z'), ('victories', '19')]
[('winner', 'http://www.wikidata.org/entity/Q10132'), ('winnerName', 'Rafael Nadal'), ('dob', '1986-06-03T00:00:00Z'), ('victories', '19')]
[('winner', 'http://www.wikidata.org/entity/Q5812'), ('winnerName', 'Novak Djokovic'), ('dob', '1987-05-22T00:00:00Z'), ('victories', '19')]
[('winner', 'http://www.wikidata.org/entity/Q1426'), ('winnerName', 'Roger Federer'), ('dob', '1981-08-08T00:00:00Z'), ('victories', '17')]
[('winner', 'http://www.wikidata.org/entity/Q24574661'), ('winnerName', 'Diede de Groot'), ('dob', '1996-12-19T00:00:00Z'), ('victories', '16')

In [188]:
# write your queries
# ALL

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->
# Results TASK 3
#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]


# 1* - getting all winner of yearly GRAND SLAMS  <--- putting all togheter 
# 2 - select only those who was born after 1975
# 3 - counting their victories
# 4 - ordering by victories in descenting order
# 5 - limiting to only top 10 distinct winners

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winner ?winnerName (COUNT(*) AS ?victories)
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament ?p ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner.
   
   # get the label 
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?winner wdt:P569 ?dob
   FILTER (?dob > "1975-01-01"^^xsd:date)
}
GROUP BY ?winner ?winnerName
ORDER BY DESC (?victories)
LIMIT 10

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q11459'), ('winnerName', 'Serena Williams'), ('victories', '36')]
[('winner', 'http://www.wikidata.org/entity/Q134720'), ('winnerName', 'Martina Hingis'), ('victories', '23')]
[('winner', 'http://www.wikidata.org/entity/Q11578'), ('winnerName', 'Venus Williams'), ('victories', '19')]
[('winner', 'http://www.wikidata.org/entity/Q10132'), ('winnerName', 'Rafael Nadal'), ('victories', '19')]
[('winner', 'http://www.wikidata.org/entity/Q5812'), ('winnerName', 'Novak Djokovic'), ('victories', '19')]
[('winner', 'http://www.wikidata.org/entity/Q1426'), ('winnerName', 'Roger Federer'), ('victories', '17')]
[('winner', 'http://www.wikidata.org/entity/Q24574661'), ('winnerName', 'Diede de Groot'), ('victories', '16')]
[('winner', 'http://www.wikidata.org/entity/Q53340'), ('winnerName', 'Bob Bryan'), ('victories', '14')]
[('winner', 'http://www.wikidata.org/entity/Q16224602'), ('winnerName', 'Yui Kamiji'), ('victories', '12')]
[('winner', 'http

In [189]:
# write your queries
# ALL

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->
# Results TASK 3
#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]
# P31'), ('pName', 'instance of')]

# 1* - getting all winner of yearly GRAND SLAMS  <--- putting all togheter 
# 2 - select only those who was born after 1975
# 3 - counting their victories
# 4 - ordering by victories in descenting order
# 5 - limiting to only top 10 distinct winners

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winner ?winnerName (COUNT(*) AS ?victories)
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner.
   
   # get the label 
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?winner wdt:P569 ?dob
   FILTER (?dob > "1975-01-01"^^xsd:date)
}
GROUP BY ?winner ?winnerName
ORDER BY DESC (?victories)
LIMIT 10

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q11459'), ('winnerName', 'Serena Williams'), ('victories', '36')]
[('winner', 'http://www.wikidata.org/entity/Q134720'), ('winnerName', 'Martina Hingis'), ('victories', '23')]
[('winner', 'http://www.wikidata.org/entity/Q11578'), ('winnerName', 'Venus Williams'), ('victories', '19')]
[('winner', 'http://www.wikidata.org/entity/Q10132'), ('winnerName', 'Rafael Nadal'), ('victories', '19')]
[('winner', 'http://www.wikidata.org/entity/Q5812'), ('winnerName', 'Novak Djokovic'), ('victories', '19')]
[('winner', 'http://www.wikidata.org/entity/Q1426'), ('winnerName', 'Roger Federer'), ('victories', '17')]
[('winner', 'http://www.wikidata.org/entity/Q24574661'), ('winnerName', 'Diede de Groot'), ('victories', '16')]
[('winner', 'http://www.wikidata.org/entity/Q53340'), ('winnerName', 'Bob Bryan'), ('victories', '14')]
[('winner', 'http://www.wikidata.org/entity/Q16224602'), ('winnerName', 'Yui Kamiji'), ('victories', '12')]
[('winner', 'http

In [2]:
# write your queries
# ALL

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

# from task3 [('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
# <---P31 instance of -- Q30085309 '2018 Wimbledon Championships'
# --P527 ('has part')--> Q30098268 "2018 Wimbledon Championships – Women's Singles" --> P1346 ('winner') -->
# Results TASK 3
#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]
# P31'), ('pName', 'instance of')]

# 1* - getting all winner of yearly GRAND SLAMS  <--- putting all togheter 
# 2 - select only those who was born after 1975
# 3 - counting their victories
# 4 - ordering by victories in descenting order
# 5 - limiting to only top 10 distinct winners

# after means 1975 not included ?

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winner ?winnerName (COUNT(*) AS ?victories)
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner.
   
   # get the label 
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?winner wdt:P569 ?dob
   FILTER (?dob >= "1976-01-01"^^xsd:date)
}
GROUP BY ?winner ?winnerName
ORDER BY DESC (?victories)
LIMIT 10

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q11459'), ('winnerName', 'Serena Williams'), ('victories', '36')]
[('winner', 'http://www.wikidata.org/entity/Q134720'), ('winnerName', 'Martina Hingis'), ('victories', '23')]
[('winner', 'http://www.wikidata.org/entity/Q11578'), ('winnerName', 'Venus Williams'), ('victories', '19')]
[('winner', 'http://www.wikidata.org/entity/Q10132'), ('winnerName', 'Rafael Nadal'), ('victories', '19')]
[('winner', 'http://www.wikidata.org/entity/Q5812'), ('winnerName', 'Novak Djokovic'), ('victories', '19')]
[('winner', 'http://www.wikidata.org/entity/Q1426'), ('winnerName', 'Roger Federer'), ('victories', '17')]
[('winner', 'http://www.wikidata.org/entity/Q24574661'), ('winnerName', 'Diede de Groot'), ('victories', '16')]
[('winner', 'http://www.wikidata.org/entity/Q53340'), ('winnerName', 'Bob Bryan'), ('victories', '14')]
[('winner', 'http://www.wikidata.org/entity/Q16224602'), ('winnerName', 'Yui Kamiji'), ('victories', '12')]
[('winner', 'http

### Final query for this task

In [2]:
# write your final query

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 
# (the result set must be a list of triples player IRI, label and #victories).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winner ?winnerName (COUNT(*) AS ?victories)
WHERE {
    
   VALUES (?tournament){
       (wd:Q43605)   #French Open
       (wd:Q123577)  #US Open
       (wd:Q41520)   #Wimbledon Championships
       (wd:Q60874)   #Australian Open
   }
   #FINDING ALL YEARLY TOURNAMENTS OF EACH TOURNAMENT
   ?yearlyTournament wdt:P31 ?tournament.
   
   #FINDING ALL SUBTURNAMENTS OF EACH yearlyTournament
   ?yearlyTournament wdt:P527 ?subTournament.
   
   #FINDING the winner of the SUBTURNAMENTS
   ?subTournament wdt:P1346 ?winner.
   
   # get the labelS
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   
   # GETTING THE DATE OF BIRTH
   ?winner wdt:P569 ?dob
   # FILTERING the players to select only those who were born after 1975
   FILTER (?dob >= "1975-01-01"^^xsd:date)
}

# GROUPING the evertyting w.r.t the player to count the victories
GROUP BY ?winner ?winnerName

# ORDERING in descending order w.r.t. victories
ORDER BY DESC (?victories)

# SELECTING only top 10 players
LIMIT 10

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q11459'), ('winnerName', 'Serena Williams'), ('victories', '36')]
[('winner', 'http://www.wikidata.org/entity/Q134720'), ('winnerName', 'Martina Hingis'), ('victories', '23')]
[('winner', 'http://www.wikidata.org/entity/Q11578'), ('winnerName', 'Venus Williams'), ('victories', '19')]
[('winner', 'http://www.wikidata.org/entity/Q10132'), ('winnerName', 'Rafael Nadal'), ('victories', '19')]
[('winner', 'http://www.wikidata.org/entity/Q5812'), ('winnerName', 'Novak Djokovic'), ('victories', '19')]
[('winner', 'http://www.wikidata.org/entity/Q1426'), ('winnerName', 'Roger Federer'), ('victories', '17')]
[('winner', 'http://www.wikidata.org/entity/Q24574661'), ('winnerName', 'Diede de Groot'), ('victories', '16')]
[('winner', 'http://www.wikidata.org/entity/Q53340'), ('winnerName', 'Bob Bryan'), ('victories', '14')]
[('winner', 'http://www.wikidata.org/entity/Q16224602'), ('winnerName', 'Yui Kamiji'), ('victories', '12')]
[('winner', 'http

#### The result doesn't change if you include or not 1975

In [3]:
# write your final query

# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 
# (the result set must be a list of triples player IRI, label and #victories).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winner ?winnerName (COUNT(*) AS ?victories)
WHERE {
    
   VALUES (?tournament){
       (wd:Q43605)   #French Open
       (wd:Q123577)  #US Open
       (wd:Q41520)   #Wimbledon Championships
       (wd:Q60874)   #Australian Open
   }
   #FINDING ALL YEARLY TOURNAMENTS OF EACH TOURNAMENT
   ?yearlyTournament wdt:P31 ?tournament.
   
   #FINDING ALL SUBTURNAMENTS OF EACH yearlyTournament
   ?yearlyTournament wdt:P527 ?subTournament.
   
   #FINDING the winner of the SUBTURNAMENTS
   ?subTournament wdt:P1346 ?winner.
   
   # get the labelS
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   
   # GETTING THE DATE OF BIRTH
   ?winner wdt:P569 ?dob
   # FILTERING the players to select only those who were born after 1975
   FILTER (?dob >= "1976-01-01"^^xsd:date)
}

# GROUPING the evertyting w.r.t the player to count the victories
GROUP BY ?winner ?winnerName

# ORDERING in descending order w.r.t. victories
ORDER BY DESC (?victories)

# SELECTING only top 10 players
LIMIT 10

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q11459'), ('winnerName', 'Serena Williams'), ('victories', '36')]
[('winner', 'http://www.wikidata.org/entity/Q134720'), ('winnerName', 'Martina Hingis'), ('victories', '23')]
[('winner', 'http://www.wikidata.org/entity/Q11578'), ('winnerName', 'Venus Williams'), ('victories', '19')]
[('winner', 'http://www.wikidata.org/entity/Q10132'), ('winnerName', 'Rafael Nadal'), ('victories', '19')]
[('winner', 'http://www.wikidata.org/entity/Q5812'), ('winnerName', 'Novak Djokovic'), ('victories', '19')]
[('winner', 'http://www.wikidata.org/entity/Q1426'), ('winnerName', 'Roger Federer'), ('victories', '17')]
[('winner', 'http://www.wikidata.org/entity/Q24574661'), ('winnerName', 'Diede de Groot'), ('victories', '16')]
[('winner', 'http://www.wikidata.org/entity/Q53340'), ('winnerName', 'Bob Bryan'), ('victories', '14')]
[('winner', 'http://www.wikidata.org/entity/Q16224602'), ('winnerName', 'Yui Kamiji'), ('victories', '12')]
[('winner', 'http

## Task 7
Consider the country for sport property of the tennis player (NO Country of citizenship). Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

#### Finding "country for sport" property.

In [191]:
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
#   Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# wd:Q1426 Roger Federer

# finding country for sport property
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1426 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('name', 'CONOR.SI ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1284'), ('name', 'Munzinger person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('name', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1352'), ('name', 

In [192]:
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
#   Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# wd:Q1426 Roger Federer

# finding country for sport property
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1426 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('name', 'CONOR.SI ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1284'), ('name', 'Munzinger person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('name', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1352'), ('name', 

In [193]:
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
#   Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# wd:Q1426 Roger Federer --P1532 ('country for sport')-->

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q1426 wdt:P1532 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q39'), ('name', 'Switzerland')]
1


#### Trying to find a data property over yearly edition of Grand Slam which holds the winner of that edition.

In [194]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P527 ('has part')
# P1346 ('winner')
#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# task 6 
queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winner ?winnerName (COUNT(*) AS ?victories)
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament ?p ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner.
   
   # get the label 
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?winner wdt:P569 ?dob
   FILTER (?dob > "1975-01-01"^^xsd:date)
}
GROUP BY ?winner ?winnerName
ORDER BY DESC (?victories)
LIMIT 10

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q11459'), ('winnerName', 'Serena Williams'), ('victories', '36')]
[('winner', 'http://www.wikidata.org/entity/Q134720'), ('winnerName', 'Martina Hingis'), ('victories', '23')]
[('winner', 'http://www.wikidata.org/entity/Q11578'), ('winnerName', 'Venus Williams'), ('victories', '19')]
[('winner', 'http://www.wikidata.org/entity/Q10132'), ('winnerName', 'Rafael Nadal'), ('victories', '19')]
[('winner', 'http://www.wikidata.org/entity/Q5812'), ('winnerName', 'Novak Djokovic'), ('victories', '19')]
[('winner', 'http://www.wikidata.org/entity/Q1426'), ('winnerName', 'Roger Federer'), ('victories', '17')]
[('winner', 'http://www.wikidata.org/entity/Q24574661'), ('winnerName', 'Diede de Groot'), ('victories', '16')]
[('winner', 'http://www.wikidata.org/entity/Q53340'), ('winnerName', 'Bob Bryan'), ('victories', '14')]
[('winner', 'http://www.wikidata.org/entity/Q16224602'), ('winnerName', 'Yui Kamiji'), ('victories', '12')]
[('winner', 'http

In [195]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P527 ('has part')
# P1346 ('winner')
#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# adapting the previous task by removing date contraint 
# finding ways to group all four tournament by year
queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?yearlyTournament ?yearlyTournamentName
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament ?p ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner.
   
   # get the label 
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
}
GROUP BY ?yearlyTournament ?yearlyTournamentName
ORDER BY DESC (?yearlyTournamentName)
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('yearlyTournament', 'http://www.wikidata.org/entity/Q781434'), ('yearlyTournamentName', 'Australian Goldfields Open')]
[('yearlyTournament', 'http://www.wikidata.org/entity/Q92032430'), ('yearlyTournamentName', '2021 Wimbledon Championships')]
[('yearlyTournament', 'http://www.wikidata.org/entity/Q98833450'), ('yearlyTournamentName', '2021 US Open')]
[('yearlyTournament', 'http://www.wikidata.org/entity/Q99526045'), ('yearlyTournamentName', '2021 French Open')]
[('yearlyTournament', 'http://www.wikidata.org/entity/Q83629820'), ('yearlyTournamentName', '2021 Australian Open')]
[('yearlyTournament', 'http://www.wikidata.org/entity/Q66309826'), ('yearlyTournamentName', '2020 US Open')]
[('yearlyTournament', 'http://www.wikidata.org/entity/Q63367899'), ('yearlyTournamentName', '2020 French Open')]
[('yearlyTournament', 'http://www.wikidata.org/entity/Q61059083'), ('yearlyTournamentName', '2020 Australian Open')]
[('yearlyTournament', 'http://www.wikidata.org/entity/Q54093953'), (

In [196]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P527 ('has part')
# P1346 ('winner')
#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# adapting the previous task by removing date contraint 

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?subTournament ?subTournamentName
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament ?p ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner.
   
   # get the label 
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
}
GROUP BY ?subTournament ?subTournamentName
ORDER BY DESC (?subTournamentName)
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('subTournament', 'http://www.wikidata.org/entity/Q958902'), ('subTournamentName', "US Open 2000 women's doubles")]
[('subTournament', 'http://www.wikidata.org/entity/Q4339659'), ('subTournamentName', 'Australian Open 2007')]
[('subTournament', 'http://www.wikidata.org/entity/Q92032445'), ('subTournamentName', "2021 Wimbledon Championships – Women's Singles")]
[('subTournament', 'http://www.wikidata.org/entity/Q92032438'), ('subTournamentName', "2021 Wimbledon Championships – Women's Doubles")]
[('subTournament', 'http://www.wikidata.org/entity/Q107434675'), ('subTournamentName', "2021 Wimbledon Championships – Wheelchair Women's Singles")]
[('subTournament', 'http://www.wikidata.org/entity/Q107434679'), ('subTournamentName', "2021 Wimbledon Championships – Wheelchair Women's Doubles")]
[('subTournament', 'http://www.wikidata.org/entity/Q107379851'), ('subTournamentName', '2021 Wimbledon Championships – Mixed Doubles')]
[('subTournament', 'http://www.wikidata.org/entity/Q92032

In [197]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P527 ('has part')
# P1346 ('winner')
#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# finding ways to group all four tournament by year

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament ?p ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   
   # get the label 
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?p sc:name ?pName.
}
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part')]
3


In [198]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P527 ('has part')
# P1346 ('winner')
#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# finding ways to group all four tournament by year

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament ?p ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner.
   
   # get the label 
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?p sc:name ?pName.
}
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from')]
2


In [199]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# finding ways to group all four tournament by year

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?obj
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament ?p ?obj.
   
   # get the label 
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?p sc:name ?pName.
}
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('pName', 'number of participants'), ('obj', '128')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of'), ('obj', 'http://www.wikidata.org/entity/Q731265')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q52453')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q134720')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q296282')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q1348791')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q1508952')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q22958938')]
[('p',

In [200]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# finding ways to group all four tournament by year

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament ?p ?obj.
   
   # get the label 
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?p sc:name ?pName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('pName', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pName', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pName', 'competition class')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('pName', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('pName', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P393

In [201]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P393 ('edition number')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# finding ways to group all four tournament by year

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?obj
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P393 ?obj.
   
   # get the label 
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
}
ORDER BY DESC (?obj)
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('obj', '99')]
[('obj', '98')]
[('obj', '97')]
[('obj', '96')]
[('obj', '95')]
[('obj', '94')]
[('obj', '94')]
[('obj', '93')]
[('obj', '92')]
[('obj', '91')]
[('obj', '90')]
[('obj', '90')]
[('obj', '89')]
[('obj', '88')]
[('obj', '88')]
[('obj', '87')]
[('obj', '87')]
[('obj', '86')]
[('obj', '86')]
[('obj', '85')]
[('obj', '84')]
[('obj', '83')]
[('obj', '82')]
[('obj', '81')]
[('obj', '80')]
[('obj', '79')]
[('obj', '78')]
[('obj', '77')]
[('obj', '76')]
[('obj', '75')]
[('obj', '74')]
[('obj', '73')]
[('obj', '72')]
[('obj', '71')]
[('obj', '70')]
[('obj', '67')]
[('obj', '67')]
[('obj', '66')]
[('obj', '65')]
[('obj', '62')]
[('obj', '62')]
[('obj', '61')]
[('obj', '140')]
[('obj', '113')]
[('obj', '112')]
[('obj', '111')]
[('obj', '110')]
[('obj', '109')]
[('obj', '108')]
[('obj', '107')]
50


In [202]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P393 ('edition number')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# finding ways to group all four tournament by year

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?subTournament ?obj
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P393 ?obj.
   
   # get the label 
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
}
ORDER BY DESC (xsd:integer(?obj))
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('subTournament', 'http://www.wikidata.org/entity/Q98754894'), ('obj', '140')]
[('subTournament', 'http://www.wikidata.org/entity/Q63367916'), ('obj', '113')]
[('subTournament', 'http://www.wikidata.org/entity/Q54197647'), ('obj', '112')]
[('subTournament', 'http://www.wikidata.org/entity/Q30098117'), ('obj', '111')]
[('subTournament', 'http://www.wikidata.org/entity/Q24387157'), ('obj', '110')]
[('subTournament', 'http://www.wikidata.org/entity/Q23892528'), ('obj', '109')]
[('subTournament', 'http://www.wikidata.org/entity/Q19868605'), ('obj', '108')]
[('subTournament', 'http://www.wikidata.org/entity/Q16744720'), ('obj', '107')]
[('subTournament', 'http://www.wikidata.org/entity/Q4629371'), ('obj', '106')]
[('subTournament', 'http://www.wikidata.org/entity/Q369095'), ('obj', '105')]
[('subTournament', 'http://www.wikidata.org/entity/Q734804'), ('obj', '104')]
[('subTournament', 'http://www.wikidata.org/entity/Q832169'), ('obj', '103')]
[('subTournament', 'http://www.wikidata

In [203]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P393 ('edition number')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# finding ways to group all four tournament by year

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?subTournament ?subTournamentName ?obj
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P393 ?obj.
   
   # get the label 
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
}
ORDER BY DESC (xsd:integer(?obj))
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('subTournament', 'http://www.wikidata.org/entity/Q98754894'), ('subTournamentName', "2020 US Open – Men's Doubles"), ('obj', '140')]
[('subTournament', 'http://www.wikidata.org/entity/Q63367916'), ('subTournamentName', "2020 French Open – Women's Singles"), ('obj', '113')]
[('subTournament', 'http://www.wikidata.org/entity/Q54197647'), ('subTournamentName', "2019 French Open – Women's Singles"), ('obj', '112')]
[('subTournament', 'http://www.wikidata.org/entity/Q30098117'), ('subTournamentName', "2018 French Open – Women's Singles"), ('obj', '111')]
[('subTournament', 'http://www.wikidata.org/entity/Q24387157'), ('subTournamentName', "2017 French Open – Women's Singles"), ('obj', '110')]
[('subTournament', 'http://www.wikidata.org/entity/Q23892528'), ('subTournamentName', "2016 French Open – Women's Singles"), ('obj', '109')]
[('subTournament', 'http://www.wikidata.org/entity/Q19868605'), ('subTournamentName', "2015 French Open – Women's Singles"), ('obj', '108')]
[('subTourn

In [204]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P393 ('edition number')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# finding ways to group all four tournament by year

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT (COUNT(*) as ?totWithEditionNumber)
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P393 ?obj.
   
   # get the label 
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
}
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('totWithEditionNumber', '57')]
1


In [205]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P393 ('edition number')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# finding ways to group all four tournament by year

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT (COUNT(*) as ?totWithoutEditionNumber)
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?obj.
   
   # get the label 
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
}
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('totWithoutEditionNumber', '1809')]
1


In [206]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P393 ('edition number')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# finding ways to group all four tournament by year

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?subTournament ?subTournamentName ?obj
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P393 ?obj.
   
   # get the label 
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
}
ORDER BY DESC (xsd:integer(?obj))
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('subTournament', 'http://www.wikidata.org/entity/Q98754894'), ('subTournamentName', "2020 US Open – Men's Doubles"), ('obj', '140')]
[('subTournament', 'http://www.wikidata.org/entity/Q63367916'), ('subTournamentName', "2020 French Open – Women's Singles"), ('obj', '113')]
[('subTournament', 'http://www.wikidata.org/entity/Q54197647'), ('subTournamentName', "2019 French Open – Women's Singles"), ('obj', '112')]
[('subTournament', 'http://www.wikidata.org/entity/Q30098117'), ('subTournamentName', "2018 French Open – Women's Singles"), ('obj', '111')]
[('subTournament', 'http://www.wikidata.org/entity/Q24387157'), ('subTournamentName', "2017 French Open – Women's Singles"), ('obj', '110')]
[('subTournament', 'http://www.wikidata.org/entity/Q23892528'), ('subTournamentName', "2016 French Open – Women's Singles"), ('obj', '109')]
[('subTournament', 'http://www.wikidata.org/entity/Q19868605'), ('subTournamentName', "2015 French Open – Women's Singles"), ('obj', '108')]
[('subTourn

##### Can't use edition number P393, only 57 has this field

In [207]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# finding ways to group all four tournament by year

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?s ?p ?tournament.
   #?tournament ?direct ?o.  
   ?p sc:name ?pName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pName', 'victory')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('pName', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pName', 'sports season of league or competition')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P466'), ('pName', 'occupant')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part')]
[('p', 'http://www

In [208]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# finding ways to group all four tournament by year

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   #?s ?p ?tournament.
   ?tournament ?p ?o.  
   ?p sc:name ?pName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('pName', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('pName', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('pName', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pName', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1814'), ('pName', 'name in kana')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/

In [209]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# finding ways to group all four tournament by year

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?instance ?name
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?instance wdt:P31 ?tournament.
   ?instance sc:name ?name.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('instance', 'http://www.wikidata.org/entity/Q268871'), ('name', '2004 Wimbledon Championships')]
[('instance', 'http://www.wikidata.org/entity/Q602116'), ('name', '1969 Australian Open')]
[('instance', 'http://www.wikidata.org/entity/Q643498'), ('name', '2000 US Open')]
[('instance', 'http://www.wikidata.org/entity/Q660527'), ('name', '1983 French Open')]
[('instance', 'http://www.wikidata.org/entity/Q840907'), ('name', '2004 US Open')]
[('instance', 'http://www.wikidata.org/entity/Q846695'), ('name', '2000 French Open')]
[('instance', 'http://www.wikidata.org/entity/Q173961'), ('name', '1978 Wimbledon Championships')]
[('instance', 'http://www.wikidata.org/entity/Q925026'), ('name', '1971 Wimbledon Championships')]
[('instance', 'http://www.wikidata.org/entity/Q776766'), ('name', '1928 French Championships')]
[('instance', 'http://www.wikidata.org/entity/Q15854765'), ('name', '2014 Wimbledon Championships')]
[('instance', 'http://www.wikidata.org/entity/Q15851911'), ('name',

In [210]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# finding ways to group all four tournament by year

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   #?yearlyTournament wdt:P527 ?subTournament.
   ?yearlyTournament ?p ?o.
   
   ?yearlyTournament sc:name ?yearlyTournamentName.
   ?p sc:name ?pName.
   
}
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P6880'), ('pName', 'ATP tennis tournament edition ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('pName', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pName', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pName', 'competition class')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('pName', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('pName', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('pName', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('pName', 'Google Knowledge

In [211]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# finding ways to group all four tournament by year

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   VALUES (?p){
       (wdt:P6880)
       #(wdt:P393)
       (wd:P361)
       (wd:P31)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   #?yearlyTournament wdt:P527 ?subTournament.
   ?yearlyTournament ?p ?o.
   
   ?yearlyTournament sc:name ?yearlyTournamentName.
   ?p sc:name ?pName.
   
}
LIMIT 200

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P6880'), ('pName', 'ATP tennis tournament edition ID'), ('o', 'wimbledon/540/2004/results')]
[('p', 'http://www.wikidata.org/prop/direct/P6880'), ('pName', 'ATP tennis tournament edition ID'), ('o', 'australian-open/580/1969/results')]
[('p', 'http://www.wikidata.org/prop/direct/P6880'), ('pName', 'ATP tennis tournament edition ID'), ('o', 'us-open/560/2000/results')]
[('p', 'http://www.wikidata.org/prop/direct/P6880'), ('pName', 'ATP tennis tournament edition ID'), ('o', 'roland-garros/520/1983/results')]
[('p', 'http://www.wikidata.org/prop/direct/P6880'), ('pName', 'ATP tennis tournament edition ID'), ('o', 'roland-garros/520/2000/results')]
[('p', 'http://www.wikidata.org/prop/direct/P6880'), ('pName', 'ATP tennis tournament edition ID'), ('o', 'wimbledon/540/1978/results')]
[('p', 'http://www.wikidata.org/prop/direct/P6880'), ('pName', 'ATP tennis tournament edition ID'), ('o', 'wimbledon/540/1971/results')]
[('p', 'http://www.wi

In [212]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# finding ways to group all four tournament by year

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   VALUES (?p){
       (wdt:P6880)
       #(wdt:P393)
       (wdt:P1346)#1346winner
       (wdt:P582)#, ('pName', 'end time')]
   }
   ?yearlyTournament wdt:P31 ?tournament.
   #?yearlyTournament wdt:P527 ?subTournament.
   ?yearlyTournament ?p ?o.
   
   ?yearlyTournament sc:name ?yearlyTournamentName.
   ?p sc:name ?pName.
   
}
LIMIT 200

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P6880'), ('pName', 'ATP tennis tournament edition ID'), ('o', 'wimbledon/540/2004/results')]
[('p', 'http://www.wikidata.org/prop/direct/P582'), ('pName', 'end time'), ('o', '2004-07-04T00:00:00Z')]
[('p', 'http://www.wikidata.org/prop/direct/P6880'), ('pName', 'ATP tennis tournament edition ID'), ('o', 'australian-open/580/1969/results')]
[('p', 'http://www.wikidata.org/prop/direct/P6880'), ('pName', 'ATP tennis tournament edition ID'), ('o', 'us-open/560/2000/results')]
[('p', 'http://www.wikidata.org/prop/direct/P6880'), ('pName', 'ATP tennis tournament edition ID'), ('o', 'roland-garros/520/1983/results')]
[('p', 'http://www.wikidata.org/prop/direct/P6880'), ('pName', 'ATP tennis tournament edition ID'), ('o', 'roland-garros/520/2000/results')]
[('p', 'http://www.wikidata.org/prop/direct/P6880'), ('pName', 'ATP tennis tournament edition ID'), ('o', 'wimbledon/540/1978/results')]
[('p', 'http://www.wikidata.org/prop/direct/P582'), 

In [213]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# finding ways to group all four tournament by year

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   VALUES (?p){
       #(wdt:P6880)
       #(wdt:P393)
       (wdt:P1346)#1346winner
       #(wdt:P582)#, ('pName', 'end time')]
   }
   ?yearlyTournament wdt:P31 ?tournament.
   #?yearlyTournament wdt:P527 ?subTournament.
   ?yearlyTournament ?p ?o.
   
   ?yearlyTournament sc:name ?yearlyTournamentName.
   ?p sc:name ?pName.
   
}
LIMIT 200

"""

print("Results")
x=run_query(queryString)

Results
Empty


In [214]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# finding ways to group all four tournament by year

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament ?p ?o.
   
   ?p sc:name ?pName.
   
}
LIMIT 200

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('pName', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pName', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pName', 'competition class')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('pName', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('pName', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P393

In [215]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# finding ways to group all four tournament by year

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?subTournament ?p ?pName ?o ?oName
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?o.
   
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   FILTER (?p = wdt:P1346 )
}
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('subTournament', 'http://www.wikidata.org/entity/Q16024356'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('o', 'http://www.wikidata.org/entity/Q299959'), ('oName', 'Eileen Bennett Whittingstall')]
[('subTournament', 'http://www.wikidata.org/entity/Q18888358'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('o', 'http://www.wikidata.org/entity/Q18918709'), ('oName', 'Markéta Vondroušová')]
[('subTournament', 'http://www.wikidata.org/entity/Q26155846'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('o', 'http://www.wikidata.org/entity/Q193661'), ('oName', 'Stan Wawrinka')]
[('subTournament', 'http://www.wikidata.org/entity/Q4630192'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('o', 'http://www.wikidata.org/entity/Q193661'), ('oName', 'Stan Wawrinka')]
[('subTournament', 'http://www.wikidata.org/entity/Q19868596'), ('p', 'http://www.wikidata.org/prop/direct/P1346')

In [216]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# tennis player (wd:Q10833314)
# Q589310 = "tennis at the 2000 Summer Olympics – men's singles"
# P641 = 'sport'
# Q46190676 = tennis event

queryString = """

SELECT DISTINCT ?p ?pName ?obj ?objName 
WHERE {
   # bind something
   wd:Q46190676 ?p ?obj .
   # get the label
   ?p sc:name ?pName .
   ?obj sc:name ?objName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('pName', 'has parts of the class'), ('obj', 'http://www.wikidata.org/entity/Q21170330'), ('objName', 'tennis qualification event')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P361'), ('objName', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q16510064'), ('objName', 'sporting event')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q847'), ('objName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P641'), ('objName', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('obj', 'http://www.wikidata.org/entity/Q47345468'), ('objName', 'tennis tournament edition')]
[('p',

In [217]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# tennis player (wd:Q10833314)
# Q589310 = "tennis at the 2000 Summer Olympics – men's singles"
# P641 = 'sport'
# Q46190676 = tennis event

queryString = """

SELECT DISTINCT ?p ?pName ?obj ?objName 
WHERE {
   # bind something
   wd:Q47345468 ?p ?obj .
   # get the label
   ?p sc:name ?pName .
   ?obj sc:name ?objName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P393'), ('objName', 'edition number')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('pName', 'has parts of the class'), ('obj', 'http://www.wikidata.org/entity/Q47382471'), ('objName', 'day-by-day summary')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P580'), ('objName', 'start time')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P582'), ('objName', 'end time')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P17'), ('objName', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('obj', 'http://www.wikidata.org/entity/Q13219666'), ('objName', 'tennis t

#### Cannot find any property (over yearly edition of Grand Slam which holds the winner of that edition)

#### Possible Solution (SOL 1)
   
```
as we know each Grand Slam should have exaclty 4 tournaments each year,
a solution could be:
```
```
group by (yearly grand slam, winner) -> if there is a winner with exaclty 4 tournament won -> find his country -> return sol
```

In [218]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# 1) group by tournament and winner of the tournament

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?tournament ?winner ?winnerName (COUNT(*) as ?tot)
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }   
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner.
   
   # get the label 
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?winner wdt:P569 ?dob   
}
GROUP BY ?tournament ?winner ?winnerName
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('tournament', 'http://www.wikidata.org/entity/Q60874'), ('winner', 'http://www.wikidata.org/entity/Q233234'), ('winnerName', 'Virginia Wade'), ('tot', '1')]
[('tournament', 'http://www.wikidata.org/entity/Q41520'), ('winner', 'http://www.wikidata.org/entity/Q54544'), ('winnerName', 'Gardnar Mulloy'), ('tot', '1')]
[('tournament', 'http://www.wikidata.org/entity/Q41520'), ('winner', 'http://www.wikidata.org/entity/Q913948'), ('winnerName', 'Mervyn Rose'), ('tot', '1')]
[('tournament', 'http://www.wikidata.org/entity/Q41520'), ('winner', 'http://www.wikidata.org/entity/Q229005'), ('winnerName', 'Sara Errani'), ('tot', '1')]
[('tournament', 'http://www.wikidata.org/entity/Q41520'), ('winner', 'http://www.wikidata.org/entity/Q11459'), ('winnerName', 'Serena Williams'), ('tot', '14')]
[('tournament', 'http://www.wikidata.org/entity/Q41520'), ('winner', 'http://www.wikidata.org/entity/Q53390'), ('winnerName', 'Maureen Connolly'), ('tot', '3')]
[('tournament', 'http://www.wikidata.o

In [219]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# 1) group by tournament and winner of the tournament

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?tournament ?tournamentName ?winner ?winnerName (COUNT(*) as ?tot)
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }   
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?winner wdt:P569 ?dob   
}
GROUP BY ?winner ?winnerName ?tournament ?tournamentName
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('tournament', 'http://www.wikidata.org/entity/Q41520'), ('tournamentName', 'Wimbledon Championships'), ('winner', 'http://www.wikidata.org/entity/Q104506'), ('winnerName', 'Björn Borg'), ('tot', '5')]
[('tournament', 'http://www.wikidata.org/entity/Q41520'), ('tournamentName', 'Wimbledon Championships'), ('winner', 'http://www.wikidata.org/entity/Q310419'), ('winnerName', 'Nenad Zimonjić'), ('tot', '1')]
[('tournament', 'http://www.wikidata.org/entity/Q60874'), ('tournamentName', 'Australian Open'), ('winner', 'http://www.wikidata.org/entity/Q228989'), ('winnerName', 'Lucie Šafářová'), ('tot', '2')]
[('tournament', 'http://www.wikidata.org/entity/Q41520'), ('tournamentName', 'Wimbledon Championships'), ('winner', 'http://www.wikidata.org/entity/Q457476'), ('winnerName', 'Alexander Peya'), ('tot', '1')]
[('tournament', 'http://www.wikidata.org/entity/Q41520'), ('tournamentName', 'Wimbledon Championships'), ('winner', 'http://www.wikidata.org/entity/Q26840238'), ('winnerName', 

In [220]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# 1) group by yearlyTournament and winner of the yearlyTournament

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?yearlyTournament ?yearlyTournamentName ?winner ?winnerName (COUNT(*) as ?tot)
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }   
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?winner wdt:P569 ?dob   
}
GROUP BY ?winner ?winnerName ?yearlyTournament ?yearlyTournamentName
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('yearlyTournament', 'http://www.wikidata.org/entity/Q99526045'), ('yearlyTournamentName', '2021 French Open'), ('winner', 'http://www.wikidata.org/entity/Q5812'), ('winnerName', 'Novak Djokovic'), ('tot', '1')]
[('yearlyTournament', 'http://www.wikidata.org/entity/Q1576241'), ('yearlyTournamentName', '1984 US Open'), ('winner', 'http://www.wikidata.org/entity/Q54654'), ('winnerName', 'Pam Shriver'), ('tot', '1')]
[('yearlyTournament', 'http://www.wikidata.org/entity/Q911810'), ('yearlyTournamentName', '1968 French Open'), ('winner', 'http://www.wikidata.org/entity/Q263134'), ('winnerName', 'Françoise Dürr'), ('tot', '1')]
[('yearlyTournament', 'http://www.wikidata.org/entity/Q1508020'), ('yearlyTournamentName', '1982 US Open'), ('winner', 'http://www.wikidata.org/entity/Q53400'), ('winnerName', 'Kevin Curren'), ('tot', '2')]
[('yearlyTournament', 'http://www.wikidata.org/entity/Q2620232'), ('yearlyTournamentName', '1896 U.S. National Championships'), ('winner', 'http://www.wi

In [221]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# 1) group by yearlyTournament and winner of the yearlyTournament
# cheching subtournaments

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?subTournament ?subTournamentName
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?winner wdt:P569 ?dob   
}
#GROUP BY ?winner ?winnerName ?tournament ?tournamentName
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('subTournament', 'http://www.wikidata.org/entity/Q248965'), ('subTournamentName', "2004 Wimbledon Championships – Men's Singles")]
[('subTournament', 'http://www.wikidata.org/entity/Q3994898'), ('subTournamentName', "2004 Wimbledon Championships – Men's Doubles")]
[('subTournament', 'http://www.wikidata.org/entity/Q1575133'), ('subTournamentName', "2004 Wimbledon Championships – Women's Doubles")]
[('subTournament', 'http://www.wikidata.org/entity/Q378546'), ('subTournamentName', "2004 Wimbledon Championships – Women's Singles")]
[('subTournament', 'http://www.wikidata.org/entity/Q388810'), ('subTournamentName', '2004 Wimbledon Championships - Mixed Doubles')]
[('subTournament', 'http://www.wikidata.org/entity/Q781612'), ('subTournamentName', "1969 Australian Open – Women's Doubles")]
[('subTournament', 'http://www.wikidata.org/entity/Q781629'), ('subTournamentName', "1969 Australian Open – Women's Singles")]
[('subTournament', 'http://www.wikidata.org/entity/Q781620'), ('sub

In [222]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# 1) group by yearlyTournament and winner of the yearlyTournament
# cheching subtournaments
# added ordering

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?subTournament ?subTournamentName
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?winner wdt:P569 ?dob   
}
#GROUP BY ?winner ?winnerName ?tournament ?tournamentName
ORDER BY DESC (?subTournamentName)
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('subTournament', 'http://www.wikidata.org/entity/Q958902'), ('subTournamentName', "US Open 2000 women's doubles")]
[('subTournament', 'http://www.wikidata.org/entity/Q4339659'), ('subTournamentName', 'Australian Open 2007')]
[('subTournament', 'http://www.wikidata.org/entity/Q92032445'), ('subTournamentName', "2021 Wimbledon Championships – Women's Singles")]
[('subTournament', 'http://www.wikidata.org/entity/Q92032438'), ('subTournamentName', "2021 Wimbledon Championships – Women's Doubles")]
[('subTournament', 'http://www.wikidata.org/entity/Q107434675'), ('subTournamentName', "2021 Wimbledon Championships – Wheelchair Women's Singles")]
[('subTournament', 'http://www.wikidata.org/entity/Q107434679'), ('subTournamentName', "2021 Wimbledon Championships – Wheelchair Women's Doubles")]
[('subTournament', 'http://www.wikidata.org/entity/Q107379851'), ('subTournamentName', '2021 Wimbledon Championships – Mixed Doubles')]
[('subTournament', 'http://www.wikidata.org/entity/Q92032

#####  Problem found
Problem found with initial solution, if I group by winner & only yearlGrandSlam a winner can have more than 4 winning in a year, due to the subTournaments.
```
Goal is to have at most 4 repetition respectively of 4 tournaments of Grand Slam```

In [223]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# Q99526059 - 2021 French Open – Women's Singles

# trying to find about 'Women's Singles' from Q99526059

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o
WHERE {
   wd:Q99526059 ?p ?o .
   # get the label 
   ?p sc:name ?pName.
   #?o sc:name ?oName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('o', 'http://www.wikidata.org/entity/Q11052500')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows'), ('o', 'http://www.wikidata.org/entity/Q63367916')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by'), ('o', 'http://www.wikidata.org/entity/Q107126330')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pName', 'competition class'), ('o', 'http://www.wikidata.org/entity/Q16893403')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q46190676')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q99526045')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('o', 'http://www.wikidata.org/entity/Q106956794')]
[('p', 'http://www.wikidata.org/prop/direct/P585'), ('pName', 'point in time'), ('o', '2021-01-0

In [224]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# Q99526059 - 2021 French Open – Women's Singles
# Q99526059 --P2094 ('competition class')-> wd:Q16893403 ("women's singles")

# trying to find about 'Women's Singles' from Q99526059

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?o ?oName
WHERE {
   wd:Q99526059 wdt:P2094 ?o .
   # get the label 
   ?o sc:name ?oName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q16893403'), ('oName', "women's singles")]
1


In [225]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# Q99526059 - 2021 French Open – Women's Singles

# from above:
#[('subTournament', 'http://www.wikidata.org/entity/Q99526059'), ('subTournamentName', "2021 French Open – Women's Singles")]
#[('subTournament', 'http://www.wikidata.org/entity/Q99756845'), ('subTournamentName', "2021 French Open – Women's Doubles")]
#[('subTournament', 'http://www.wikidata.org/entity/Q99756902'), ('subTournamentName', '2021 French Open – Mixed Doubles')]
#[('subTournament', 'http://www.wikidata.org/entity/Q99526058'), ('subTournamentName', "2021 French Open – Men's Singles")]
#[('subTournament', 'http://www.wikidata.org/entity/Q99756869'), ('subTournamentName', "2021 French Open – Men's Doubles")]
#[('subTournament', 'http://www.wikidata.org/entity/Q107111218'), ('subTournamentName', "2021 French Open – Girls' Singles")]
#[('subTournament', 'http://www.wikidata.org/entity/Q107125108'), ('subTournamentName', "2021 French Open – Girls' Doubles")]
#[('subTournament', 'http://www.wikidata.org/entity/Q107111575'), ('subTournamentName', "2021 French Open – Boys' Singles")]
#[('subTournament', 'http://www.wikidata.org/entity/Q107125095'), ('subTournamentName', "2021 French Open – Boys' Doubles")]

# lets check if they all have --P2094 ('competition class')-> relationship

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
    VALUES(?s){
        (wd:Q99526059)
        (wd:Q99756845)
        (wd:Q99756902)
        (wd:Q99526058)
        (wd:Q99756869)
        (wd:Q107111218)
        (wd:Q107125108)
        (wd:Q107111575)
        (wd:Q107125095)
    }
   ?s ?p ?o .
   # get the label 
   ?p sc:name ?pName.
   ?o sc:name ?oName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pName', 'competition class'), ('o', 'http://www.wikidata.org/entity/Q27907702'), ('oName', "boys' singles")]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pName', 'competition class'), ('o', 'http://www.wikidata.org/entity/Q27907692'), ('oName', "girls' doubles")]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pName', 'competition class'), ('o', 'http://www.wikidata.org/entity/Q27907716'), ('oName', "boys' doubles")]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pName', 'competition class'), ('o', 'http://www.wikidata.org/entity/Q16629853'), ('oName', 'mixed doubles')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pName', 'competition class'), ('o', 'http://www.wikidata.org/entity/Q16893403'), ('oName', "women's singles")]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pName', 'competition class'), ('o', 'http://www.wikidata.org/entity/Q17299348'), ('oName', "women's doubles")]
[('

In [226]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# Q99526059 - 2021 French Open – Women's Singles
# P2094 'competition class'

# lets check if they all have --P2094 ('competition class')-> relationship

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?o ?oName
WHERE {
    VALUES(?s){
        (wd:Q99526059)
        (wd:Q99756845)
        (wd:Q99756902)
        (wd:Q99526058)
        (wd:Q99756869)
        (wd:Q107111218)
        (wd:Q107125108)
        (wd:Q107111575)
        (wd:Q107125095)
    }
   ?s wdt:P2094 ?o .
   ?o sc:name ?oName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q27907702'), ('oName', "boys' singles")]
[('o', 'http://www.wikidata.org/entity/Q27907692'), ('oName', "girls' doubles")]
[('o', 'http://www.wikidata.org/entity/Q27907716'), ('oName', "boys' doubles")]
[('o', 'http://www.wikidata.org/entity/Q16629853'), ('oName', 'mixed doubles')]
[('o', 'http://www.wikidata.org/entity/Q16893403'), ('oName', "women's singles")]
[('o', 'http://www.wikidata.org/entity/Q17299348'), ('oName', "women's doubles")]
[('o', 'http://www.wikidata.org/entity/Q27907674'), ('oName', "girls' singles")]
[('o', 'http://www.wikidata.org/entity/Q16893072'), ('oName', "men's singles")]
[('o', 'http://www.wikidata.org/entity/Q17299700'), ('oName', "men's doubles")]
9


##### So now lets try grouping by competition class, 
```
then possible solution could be to check if someone has won in all 4 tournament in the same competition class```

In [227]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# Q99526059 - 2021 French Open – Women's Singles

#P2094' competition class'

# checking if in 2021 someone has won all edition of each Grand Slam in the same category 

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?o ?oName
WHERE {
    VALUES(?s){
        (wd:Q92032445) #subTournamentName', "2021 Wimbledon Championships – Women's Singles")]
        (wd:Q92032438) #subTournamentName', "2021 Wimbledon Championships – Women's Doubles")]
        (wd:Q107434675) #subTournamentName', "2021 Wimbledon Championships – Wheelchair Women's Singles")]
        (wd:Q107434679) #subTournamentName', "2021 Wimbledon Championships – Wheelchair Women's Doubles")]
        (wd:Q107379851) #subTournamentName', '2021 Wimbledon Championships – Mixed Doubles')]
        (wd:Q92032464) #subTournamentName', "2021 Wimbledon Championships – Men's Singles")]
        (wd:Q107345665) #subTournamentName', "2021 Wimbledon Championships – Men's Doubles")]
        (wd:Q98811676) #subTournamentName', "2021 US Open – Women's Singles")]
        (wd:Q108436150) #subTournamentName', "2021 US Open – Girls' Singles")]
        (wd:Q108437590) #subTournamentName', "2021 US Open – Girls' Doubles")]
        (wd:Q99526059) #subTournamentName', "2021 French Open – Women's Singles")]
        (wd:Q99756845) #subTournamentName', "2021 French Open – Women's Doubles")]
        (wd:Q99756902) #subTournamentName', '2021 French Open – Mixed Doubles')]
        (wd:Q99526058) #subTournamentName', "2021 French Open – Men's Singles")]
        (wd:Q99756869) #subTournamentName', "2021 French Open – Men's Doubles")]
        (wd:Q107111218) #subTournamentName', "2021 French Open – Girls' Singles")]
        (wd:Q107125108) #subTournamentName', "2021 French Open – Girls' Doubles")]
        (wd:Q107111575) #subTournamentName', "2021 French Open – Boys' Singles")]
        (wd:Q107125095) #subTournamentName', "2021 French Open – Boys' Doubles")]
        (wd:Q83629825) #subTournamentName', "2021 Australian Open – Women's Singles")]
        (wd:Q105353897) #subTournamentName', "2021 Australian Open – Women's Doubles")]
        (wd:Q105433973) #subTournamentName', '2021 Australian Open – Mixed Doubles')]
        (wd:Q83629829) #subTournamentName', "2021 Australian Open – Men's Singles")]
        (wd:Q105357561) #subTournamentName', "2021 Australian Open – Men's Doubles")]
    }
   ?s wdt:P2094 ?o .
   ?o sc:name ?oName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q27907702'), ('oName', "boys' singles")]
[('o', 'http://www.wikidata.org/entity/Q27907692'), ('oName', "girls' doubles")]
[('o', 'http://www.wikidata.org/entity/Q27907692'), ('oName', "girls' doubles")]
[('o', 'http://www.wikidata.org/entity/Q27907716'), ('oName', "boys' doubles")]
[('o', 'http://www.wikidata.org/entity/Q16629853'), ('oName', 'mixed doubles')]
[('o', 'http://www.wikidata.org/entity/Q16629853'), ('oName', 'mixed doubles')]
[('o', 'http://www.wikidata.org/entity/Q16629853'), ('oName', 'mixed doubles')]
[('o', 'http://www.wikidata.org/entity/Q16893403'), ('oName', "women's singles")]
[('o', 'http://www.wikidata.org/entity/Q16893403'), ('oName', "women's singles")]
[('o', 'http://www.wikidata.org/entity/Q16893403'), ('oName', "women's singles")]
[('o', 'http://www.wikidata.org/entity/Q16893403'), ('oName', "women's singles")]
[('o', 'http://www.wikidata.org/entity/Q17299348'), ('oName', "women's doubles")]
[('o', 'http://www.w

In [228]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# Q99526059 - 2021 French Open – Women's Singles

#P2094' competition class'

# 1) checking if in 2021 someone has won all edition of each Grand Slam in the same category 
# 1.1) checking categories of 2021 GS tournament 

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?o ?oName (COUNT(*) AS ?tot)
WHERE {
    VALUES(?s){
        (wd:Q92032445) #subTournamentName', "2021 Wimbledon Championships – Women's Singles")]
        (wd:Q92032438) #subTournamentName', "2021 Wimbledon Championships – Women's Doubles")]
        (wd:Q107434675) #subTournamentName', "2021 Wimbledon Championships – Wheelchair Women's Singles")]
        (wd:Q107434679) #subTournamentName', "2021 Wimbledon Championships – Wheelchair Women's Doubles")]
        (wd:Q107379851) #subTournamentName', '2021 Wimbledon Championships – Mixed Doubles')]
        (wd:Q92032464) #subTournamentName', "2021 Wimbledon Championships – Men's Singles")]
        (wd:Q107345665) #subTournamentName', "2021 Wimbledon Championships – Men's Doubles")]
        (wd:Q98811676) #subTournamentName', "2021 US Open – Women's Singles")]
        (wd:Q108436150) #subTournamentName', "2021 US Open – Girls' Singles")]
        (wd:Q108437590) #subTournamentName', "2021 US Open – Girls' Doubles")]
        (wd:Q99526059) #subTournamentName', "2021 French Open – Women's Singles")]
        (wd:Q99756845) #subTournamentName', "2021 French Open – Women's Doubles")]
        (wd:Q99756902) #subTournamentName', '2021 French Open – Mixed Doubles')]
        (wd:Q99526058) #subTournamentName', "2021 French Open – Men's Singles")]
        (wd:Q99756869) #subTournamentName', "2021 French Open – Men's Doubles")]
        (wd:Q107111218) #subTournamentName', "2021 French Open – Girls' Singles")]
        (wd:Q107125108) #subTournamentName', "2021 French Open – Girls' Doubles")]
        (wd:Q107111575) #subTournamentName', "2021 French Open – Boys' Singles")]
        (wd:Q107125095) #subTournamentName', "2021 French Open – Boys' Doubles")]
        (wd:Q83629825) #subTournamentName', "2021 Australian Open – Women's Singles")]
        (wd:Q105353897) #subTournamentName', "2021 Australian Open – Women's Doubles")]
        (wd:Q105433973) #subTournamentName', '2021 Australian Open – Mixed Doubles')]
        (wd:Q83629829) #subTournamentName', "2021 Australian Open – Men's Singles")]
        (wd:Q105357561) #subTournamentName', "2021 Australian Open – Men's Doubles")]
    }
   ?s wdt:P2094 ?o .
   ?o sc:name ?oName.
}
GROUP BY ?o ?oName
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q27907692'), ('oName', "girls' doubles"), ('tot', '2')]
[('o', 'http://www.wikidata.org/entity/Q16629853'), ('oName', 'mixed doubles'), ('tot', '3')]
[('o', 'http://www.wikidata.org/entity/Q16893403'), ('oName', "women's singles"), ('tot', '4')]
[('o', 'http://www.wikidata.org/entity/Q27907674'), ('oName', "girls' singles"), ('tot', '2')]
[('o', 'http://www.wikidata.org/entity/Q16893072'), ('oName', "men's singles"), ('tot', '3')]
[('o', 'http://www.wikidata.org/entity/Q27907702'), ('oName', "boys' singles"), ('tot', '1')]
[('o', 'http://www.wikidata.org/entity/Q17299348'), ('oName', "women's doubles"), ('tot', '3')]
[('o', 'http://www.wikidata.org/entity/Q50211929'), ('oName', "wheelchair tennis women's singles"), ('tot', '1')]
[('o', 'http://www.wikidata.org/entity/Q50211939'), ('oName', "wheelchair tennis women's doubles"), ('tot', '1')]
[('o', 'http://www.wikidata.org/entity/Q27907716'), ('oName', "boys' doubles"), ('tot', '1')]
[('o',

In [229]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# Q99526059 - 2021 French Open – Women's Singles

#P2094' competition class'

# 1) checking if in 2021 someone has won all edition of each Grand Slam in the same category 
# 1.1) checking categories of 2021 GS tournament 
queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?o ?oName (COUNT(*) AS ?tot)
WHERE {
    VALUES(?s){
        (wd:Q92032445) #subTournamentName', "2021 Wimbledon Championships – Women's Singles")]
        (wd:Q92032438) #subTournamentName', "2021 Wimbledon Championships – Women's Doubles")]
        (wd:Q107434675) #subTournamentName', "2021 Wimbledon Championships – Wheelchair Women's Singles")]
        (wd:Q107434679) #subTournamentName', "2021 Wimbledon Championships – Wheelchair Women's Doubles")]
        (wd:Q107379851) #subTournamentName', '2021 Wimbledon Championships – Mixed Doubles')]
        (wd:Q92032464) #subTournamentName', "2021 Wimbledon Championships – Men's Singles")]
        (wd:Q107345665) #subTournamentName', "2021 Wimbledon Championships – Men's Doubles")]
        (wd:Q98811676) #subTournamentName', "2021 US Open – Women's Singles")]
        (wd:Q108436150) #subTournamentName', "2021 US Open – Girls' Singles")]
        (wd:Q108437590) #subTournamentName', "2021 US Open – Girls' Doubles")]
        (wd:Q99526059) #subTournamentName', "2021 French Open – Women's Singles")]
        (wd:Q99756845) #subTournamentName', "2021 French Open – Women's Doubles")]
        (wd:Q99756902) #subTournamentName', '2021 French Open – Mixed Doubles')]
        (wd:Q99526058) #subTournamentName', "2021 French Open – Men's Singles")]
        (wd:Q99756869) #subTournamentName', "2021 French Open – Men's Doubles")]
        (wd:Q107111218) #subTournamentName', "2021 French Open – Girls' Singles")]
        (wd:Q107125108) #subTournamentName', "2021 French Open – Girls' Doubles")]
        (wd:Q107111575) #subTournamentName', "2021 French Open – Boys' Singles")]
        (wd:Q107125095) #subTournamentName', "2021 French Open – Boys' Doubles")]
        (wd:Q83629825) #subTournamentName', "2021 Australian Open – Women's Singles")]
        (wd:Q105353897) #subTournamentName', "2021 Australian Open – Women's Doubles")]
        (wd:Q105433973) #subTournamentName', '2021 Australian Open – Mixed Doubles')]
        (wd:Q83629829) #subTournamentName', "2021 Australian Open – Men's Singles")]
        (wd:Q105357561) #subTournamentName', "2021 Australian Open – Men's Doubles")]
    }
   ?s wdt:P2094 ?o .
   ?o sc:name ?oName.
}
GROUP BY ?o ?oName
HAVING (COUNT(*) >= 4)
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q16893403'), ('oName', "women's singles"), ('tot', '4')]
1


##### Attention: only 'wommen's singles' competition has been played in all four tournaments of Grand Slam 2021

In [230]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# Q99526059 - 2021 French Open – Women's Singles

#P2094' competition class'

# 1) checking if in 2021 someone has won all edition of each Grand Slam in the same category 
# 1.1) checking categories of 2021 GS tournament 
# 1.2) finding the winners of 2021 GS tournament in each category

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?sName ?o ?oName ?winner ?winnerName
WHERE {
    VALUES(?s){
        (wd:Q92032445) #subTournamentName', "2021 Wimbledon Championships – Women's Singles")]
        (wd:Q92032438) #subTournamentName', "2021 Wimbledon Championships – Women's Doubles")]
        (wd:Q107434675) #subTournamentName', "2021 Wimbledon Championships – Wheelchair Women's Singles")]
        (wd:Q107434679) #subTournamentName', "2021 Wimbledon Championships – Wheelchair Women's Doubles")]
        (wd:Q107379851) #subTournamentName', '2021 Wimbledon Championships – Mixed Doubles')]
        (wd:Q92032464) #subTournamentName', "2021 Wimbledon Championships – Men's Singles")]
        (wd:Q107345665) #subTournamentName', "2021 Wimbledon Championships – Men's Doubles")]
        (wd:Q98811676) #subTournamentName', "2021 US Open – Women's Singles")]
        (wd:Q108436150) #subTournamentName', "2021 US Open – Girls' Singles")]
        (wd:Q108437590) #subTournamentName', "2021 US Open – Girls' Doubles")]
        (wd:Q99526059) #subTournamentName', "2021 French Open – Women's Singles")]
        (wd:Q99756845) #subTournamentName', "2021 French Open – Women's Doubles")]
        (wd:Q99756902) #subTournamentName', '2021 French Open – Mixed Doubles')]
        (wd:Q99526058) #subTournamentName', "2021 French Open – Men's Singles")]
        (wd:Q99756869) #subTournamentName', "2021 French Open – Men's Doubles")]
        (wd:Q107111218) #subTournamentName', "2021 French Open – Girls' Singles")]
        (wd:Q107125108) #subTournamentName', "2021 French Open – Girls' Doubles")]
        (wd:Q107111575) #subTournamentName', "2021 French Open – Boys' Singles")]
        (wd:Q107125095) #subTournamentName', "2021 French Open – Boys' Doubles")]
        (wd:Q83629825) #subTournamentName', "2021 Australian Open – Women's Singles")]
        (wd:Q105353897) #subTournamentName', "2021 Australian Open – Women's Doubles")]
        (wd:Q105433973) #subTournamentName', '2021 Australian Open – Mixed Doubles')]
        (wd:Q83629829) #subTournamentName', "2021 Australian Open – Men's Singles")]
        (wd:Q105357561) #subTournamentName', "2021 Australian Open – Men's Doubles")]
    }
   ?s wdt:P2094 ?o;
      wdt:P1346 ?winner.
   
   ?s sc:name ?sName.  
   ?o sc:name ?oName.  
   ?winner sc:name ?winnerName.
}
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('sName', "2021 French Open – Boys' Singles"), ('o', 'http://www.wikidata.org/entity/Q27907702'), ('oName', "boys' singles"), ('winner', 'http://www.wikidata.org/entity/Q107205896'), ('winnerName', 'Luca Van Assche')]
[('sName', "2021 US Open – Girls' Doubles"), ('o', 'http://www.wikidata.org/entity/Q27907692'), ('oName', "girls' doubles"), ('winner', 'http://www.wikidata.org/entity/Q66309318'), ('winnerName', 'Robin Montgomery')]
[('sName', "2021 French Open – Girls' Doubles"), ('o', 'http://www.wikidata.org/entity/Q27907692'), ('oName', "girls' doubles"), ('winner', 'http://www.wikidata.org/entity/Q67088022'), ('winnerName', 'Alex Eala')]
[('sName', "2021 French Open – Boys' Doubles"), ('o', 'http://www.wikidata.org/entity/Q27907716'), ('oName', "boys' doubles"), ('winner', 'http://www.wikidata.org/entity/Q107207693'), ('winnerName', 'Arthur Fils')]
[('sName', '2021 French Open – Mixed Doubles'), ('o', 'http://www.wikidata.org/entity/Q16629853'), ('oName', 'mixed doubles'), 

In [231]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# 1) checking if in all seasons if someone has won all edition of each Grand Slam in the same category 
# 1.2) finding the winners of each GS tournament in each category
queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winner ?winnerName ?competitionClass ?yearlyTournamentName (COUNT(*) as ?tot)
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner;
                   wdt:P2094 ?competitionClass.
   
   ?winner wdt:P569 ?dob.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?competitionClass sc:name ?competitionClassName.
}
#GROUP BY ?winner ?winnerName ?competitionClassName ?yearlyTournamentName
ORDER BY DESC (?tot)
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q725525'), ('winnerName', 'Ken Fletcher'), ('competitionClass', 'http://www.wikidata.org/entity/Q16629853'), ('yearlyTournamentName', '1963 Australian Championships'), ('tot', '2')]
[('winner', 'http://www.wikidata.org/entity/Q261983'), ('winnerName', 'Nancye Wynne Bolton'), ('competitionClass', 'http://www.wikidata.org/entity/Q17299348'), ('yearlyTournamentName', '1951 Australian Championships'), ('tot', '2')]
[('winner', 'http://www.wikidata.org/entity/Q506606'), ('winnerName', 'Mark Edmondson'), ('competitionClass', 'http://www.wikidata.org/entity/Q16893072'), ('yearlyTournamentName', '1976 Australian Open'), ('tot', '2')]
[('winner', 'http://www.wikidata.org/entity/Q261983'), ('winnerName', 'Nancye Wynne Bolton'), ('competitionClass', 'http://www.wikidata.org/entity/Q16629853'), ('yearlyTournamentName', '1946 Australian Championships'), ('tot', '2')]
[('winner', 'http://www.wikidata.org/entity/Q374047'), ('winnerName', 'Adrian Qui

In [232]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# 1) checking if in all seasons if someone has won all edition of each Grand Slam in the same category 
# 1.2) finding the winners of each GS tournament in each category
#    + competitionClassName
#    + LIMIT 20

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winnerName ?competitionClassName ?yearlyTournamentName (COUNT(*) as ?tot)
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner;
                   wdt:P2094 ?competitionClass.
   
   ?winner wdt:P569 ?dob.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?competitionClass sc:name ?competitionClassName.
}
#GROUP BY ?winner ?winnerName ?competitionClassName ?yearlyTournamentName
ORDER BY DESC (?tot)
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('winnerName', 'Mark Edmondson'), ('competitionClassName', "men's doubles"), ('yearlyTournamentName', '1980 Australian Open'), ('tot', '2')]
[('winnerName', 'Mark Edmondson'), ('competitionClassName', "men's doubles"), ('yearlyTournamentName', '1985 French Open'), ('tot', '2')]
[('winnerName', 'Nancye Wynne Bolton'), ('competitionClassName', "women's singles"), ('yearlyTournamentName', '1946 Australian Championships'), ('tot', '2')]
[('winnerName', 'Nancye Wynne Bolton'), ('competitionClassName', "women's singles"), ('yearlyTournamentName', '1940 Australian Championships'), ('tot', '2')]
[('winnerName', 'Nancye Wynne Bolton'), ('competitionClassName', "women's doubles"), ('yearlyTournamentName', '1948 Australian Championships'), ('tot', '2')]
[('winnerName', 'Nancye Wynne Bolton'), ('competitionClassName', 'mixed doubles'), ('yearlyTournamentName', '1948 Australian Championships'), ('tot', '2')]
[('winnerName', 'Adrian Quist'), ('competitionClassName', "men's doubles"), ('year

In [233]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# 1) checking if in all seasons if someone has won all edition of each Grand Slam in the same category 
# 1.2) finding the winners of each GS tournament in each category
#    + competitionClassName
#    + LIMIT 20
#    + year

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winnerName ?competitionClassName ?year (COUNT(*) as ?tot)
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner;
                   wdt:P2094 ?competitionClass.
   
   ?winner wdt:P569 ?dob.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?competitionClass sc:name ?competitionClassName.
   
   BIND(SUBSTR(?yearlyTournamentName,0,5) as ?year)
}
#GROUP BY ?winner ?winnerName ?competitionClassName ?year
ORDER BY DESC (?tot)
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('winnerName', 'Steffi Graf'), ('competitionClassName', "women's singles"), ('year', '1988 '), ('tot', '4')]
[('winnerName', 'Diede de Groot'), ('competitionClassName', "wheelchair tennis women's doubles"), ('year', '2019 '), ('tot', '4')]
[('winnerName', 'Bob Bryan'), ('competitionClassName', "men's doubles"), ('year', '2013 '), ('tot', '3')]
[('winnerName', 'Martina Navratilova'), ('competitionClassName', "women's singles"), ('year', '1983 '), ('tot', '3')]
[('winnerName', 'Martina Hingis'), ('competitionClassName', "women's doubles"), ('year', '1998 '), ('tot', '3')]
[('winnerName', 'Jacco Eltingh'), ('competitionClassName', "men's doubles"), ('year', '1998 '), ('tot', '3')]
[('winnerName', 'Steffi Graf'), ('competitionClassName', "women's singles"), ('year', '1995 '), ('tot', '3')]
[('winnerName', 'Serena Williams'), ('competitionClassName', "women's doubles"), ('year', '2009 '), ('tot', '3')]
[('winnerName', 'Leander Paes'), ('competitionClassName', 'mixed doubles'), ('ye

In [234]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# 1) checking if in all seasons if someone has won all edition of each Grand Slam in the same category 
# 1.2) finding the winners of each GS tournament in each category
#    + competitionClassName
#    + LIMIT 20
#    ! checking if year is correct

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winnerName ?competitionClassName ?year (COUNT(*) as ?tot)
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner;
                   wdt:P2094 ?competitionClass.
   
   ?winner wdt:P569 ?dob.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?competitionClass sc:name ?competitionClassName.
   
   BIND(xsd:gYear(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
}
#GROUP BY ?winner ?winnerName ?competitionClassName ?year
ORDER BY DESC (?tot)
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('winnerName', 'Steffi Graf'), ('competitionClassName', "women's singles"), ('year', '1988-01-01'), ('tot', '4')]
[('winnerName', 'Diede de Groot'), ('competitionClassName', "wheelchair tennis women's doubles"), ('year', '2019-01-01'), ('tot', '4')]
[('winnerName', 'Rod Laver'), ('competitionClassName', "men's singles"), ('year', '1962-01-01'), ('tot', '3')]
[('winnerName', 'Martina Navratilova'), ('competitionClassName', 'mixed doubles'), ('year', '1985-01-01'), ('tot', '3')]
[('winnerName', 'Maureen Connolly'), ('competitionClassName', "women's singles"), ('year', '1953-01-01'), ('tot', '3')]
[('winnerName', 'Natasha Zvereva'), ('competitionClassName', "women's doubles"), ('year', '1994-01-01'), ('tot', '3')]
[('winnerName', 'Gigi Fernández'), ('competitionClassName', "women's doubles"), ('year', '1994-01-01'), ('tot', '3')]
[('winnerName', 'Diede de Groot'), ('competitionClassName', "wheelchair tennis women's singles"), ('year', '2019-01-01'), ('tot', '3')]
[('winnerName', 

In [235]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# 1) checking if in all seasons if someone has won all edition of each Grand Slam in the same category 
# 1.2) finding the winners of each GS tournament in each category
#    + competitionClassName
#    + LIMIT 20
#    + ok year is correct
queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winnerName ?competitionClassName ?year (COUNT(*) as ?tot)
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner;
                   wdt:P2094 ?competitionClass.
   
   ?winner wdt:P569 ?dob.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?competitionClass sc:name ?competitionClassName.
   
   BIND(xsd:integer(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
}
#GROUP BY ?winner ?winnerName ?competitionClassName ?year
ORDER BY DESC (?tot)
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('winnerName', 'Steffi Graf'), ('competitionClassName', "women's singles"), ('year', '1988'), ('tot', '4')]
[('winnerName', 'Diede de Groot'), ('competitionClassName', "wheelchair tennis women's doubles"), ('year', '2019'), ('tot', '4')]
[('winnerName', 'Serena Williams'), ('competitionClassName', "women's doubles"), ('year', '2009'), ('tot', '3')]
[('winnerName', 'Martina Hingis'), ('competitionClassName', 'mixed doubles'), ('year', '2015'), ('tot', '3')]
[('winnerName', 'Leander Paes'), ('competitionClassName', 'mixed doubles'), ('year', '2015'), ('tot', '3')]
[('winnerName', 'Virginia Ruano Pascual'), ('competitionClassName', "women's doubles"), ('year', '2004'), ('tot', '3')]
[('winnerName', 'Bob Hewitt'), ('competitionClassName', 'mixed doubles'), ('year', '1979'), ('tot', '3')]
[('winnerName', 'Martina Navratilova'), ('competitionClassName', "women's doubles"), ('year', '1982'), ('tot', '3')]
[('winnerName', 'Natasha Zvereva'), ('competitionClassName', "women's doubles")

In [236]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# 1) checking if in all seasons if someone has won all edition of each Grand Slam in the same category 
# 1.2) finding the winners of each GS tournament in each category
#    + competitionClassName
#    + !LIMIT 50
#    + ok year is correct
#    + GROUPING & ORDER BY year
#    + HAVING 

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winnerName ?competitionClassName ?year (COUNT(*) as ?tot)
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner;
                   wdt:P2094 ?competitionClass.
   
   ?winner wdt:P569 ?dob.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?competitionClass sc:name ?competitionClassName.
   
   BIND(xsd:integer(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
}
GROUP BY ?winner ?winnerName ?competitionClassName ?year
HAVING (count(*) >2)
ORDER BY DESC (?year)
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('winnerName', 'Novak Djokovic'), ('competitionClassName', "men's singles"), ('year', '2021'), ('tot', '3')]
[('winnerName', 'Diede de Groot'), ('competitionClassName', "wheelchair tennis women's singles"), ('year', '2019'), ('tot', '3')]
[('winnerName', 'Diede de Groot'), ('competitionClassName', "wheelchair tennis women's doubles"), ('year', '2019'), ('tot', '4')]
[('winnerName', 'Diede de Groot'), ('competitionClassName', "wheelchair tennis women's doubles"), ('year', '2018'), ('tot', '3')]
[('winnerName', 'Diede de Groot'), ('competitionClassName', "wheelchair tennis women's singles"), ('year', '2018'), ('tot', '3')]
[('winnerName', 'Yui Kamiji'), ('competitionClassName', "wheelchair tennis women's singles"), ('year', '2017'), ('tot', '3')]
[('winnerName', 'Martina Hingis'), ('competitionClassName', 'mixed doubles'), ('year', '2015'), ('tot', '3')]
[('winnerName', 'Serena Williams'), ('competitionClassName', "women's singles"), ('year', '2015'), ('tot', '3')]
[('winnerName

##### Almost done, except some result on wikipedia for eg. all the following has won all 4 in the same year
```
    for male: 1969 Rod Laver,1962 Rod Laver,1938 Don Budge
```
```
    for female: 1953 Maureen Connolly,1970 Margaret Smith Court
```

Data is missing

In [237]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# 1) checking if in all seasons if someone has won all edition of each Grand Slam in the same category 
# 2) finding the winners of each GS tournament in each category
# 3) selecting only those who have 2 or more wins in the same category 

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winnerName ?competitionClassName ?year (COUNT(*) as ?tot)
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner;
                   wdt:P2094 ?competitionClass.
   
   ?winner wdt:P569 ?dob.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?competitionClass sc:name ?competitionClassName.
   
   BIND(xsd:integer(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
}
GROUP BY ?winner ?winnerName ?competitionClassName ?year
HAVING (count(*) >2)
ORDER BY DESC (?year)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('winnerName', 'Novak Djokovic'), ('competitionClassName', "men's singles"), ('year', '2021'), ('tot', '3')]
[('winnerName', 'Diede de Groot'), ('competitionClassName', "wheelchair tennis women's singles"), ('year', '2019'), ('tot', '3')]
[('winnerName', 'Diede de Groot'), ('competitionClassName', "wheelchair tennis women's doubles"), ('year', '2019'), ('tot', '4')]
[('winnerName', 'Diede de Groot'), ('competitionClassName', "wheelchair tennis women's doubles"), ('year', '2018'), ('tot', '3')]
[('winnerName', 'Diede de Groot'), ('competitionClassName', "wheelchair tennis women's singles"), ('year', '2018'), ('tot', '3')]
[('winnerName', 'Yui Kamiji'), ('competitionClassName', "wheelchair tennis women's singles"), ('year', '2017'), ('tot', '3')]
[('winnerName', 'Martina Hingis'), ('competitionClassName', 'mixed doubles'), ('year', '2015'), ('tot', '3')]
[('winnerName', 'Serena Williams'), ('competitionClassName', "women's singles"), ('year', '2015'), ('tot', '3')]
[('winnerName

In [238]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# 1) checking if in all seasons if someone has won all edition of each Grand Slam in the same category 
# 2) finding the winners of each GS tournament in each category
# 3) selecting only those who have 4 or more wins in the same category

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winnerName ?competitionClassName ?year (COUNT(*) as ?tot)
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner;
                   wdt:P2094 ?competitionClass.
   
   ?winner wdt:P569 ?dob.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?competitionClass sc:name ?competitionClassName.
   
   BIND(xsd:integer(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
}
GROUP BY ?winner ?winnerName ?competitionClassName ?year
HAVING (count(*) >=4)
ORDER BY DESC (?year)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('winnerName', 'Diede de Groot'), ('competitionClassName', "wheelchair tennis women's doubles"), ('year', '2019'), ('tot', '4')]
[('winnerName', 'Steffi Graf'), ('competitionClassName', "women's singles"), ('year', '1988'), ('tot', '4')]
2


In [239]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# 1) checking if in all seasons if someone has won all edition of each Grand Slam in the same category 
# 2) finding the winners of each GS tournament in each category
# 3) selecting only those who have 4 or more wins in the same category
# 4) getting the country for sport of the players
queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winnerName ?countryForSport ?competitionClassName ?year (COUNT(*) as ?tot)
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner;
                   wdt:P2094 ?competitionClass.
   
   ?winner wdt:P1532 ?countryForSport.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?competitionClass sc:name ?competitionClassName.
   
   BIND(xsd:integer(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
}
GROUP BY ?winner ?winnerName ?countryForSport ?competitionClassName ?year
HAVING (count(*) >=4)
ORDER BY DESC (?year)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('winnerName', 'Steffi Graf'), ('countryForSport', 'http://www.wikidata.org/entity/Q183'), ('competitionClassName', "women's singles"), ('year', '1988'), ('tot', '4')]
1


##### Attention One record is withoud P1532 (coutry for sport) relationship ! SEE ABOVE

##### Now trying to finding sol for data missing for 
```
    for male: 1969 Rod Laver,1962 Rod Laver,1938 Don Budge
```
```
    for female: 1953 Maureen Connolly,1970 Margaret Smith Court
```

In [240]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# getting all games in 1969

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?subTournamentName ?winnerName ?year ?competitionClassName ?subTournament ?yearlyTournament
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner;
                   wdt:P2094 ?competitionClass.
      
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?competitionClass sc:name ?competitionClassName.
   
   BIND(xsd:integer(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
   FILTER(?year = "1969"^^xsd:integer)
}
#GROUP BY ?winner ?winnerName ?competitionClassName ?year
#HAVING (count(*) >=4)
#ORDER BY DESC (?year)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('subTournamentName', '1969 French Open – Mixed Doubles'), ('winnerName', 'Marty Riessen'), ('year', '1969'), ('competitionClassName', 'mixed doubles'), ('subTournament', 'http://www.wikidata.org/entity/Q3037679'), ('yearlyTournament', 'http://www.wikidata.org/entity/Q202226')]
[('subTournamentName', "1969 French Open – Men's Doubles"), ('winnerName', 'John Newcombe'), ('year', '1969'), ('competitionClassName', "men's doubles"), ('subTournament', 'http://www.wikidata.org/entity/Q12412317'), ('yearlyTournament', 'http://www.wikidata.org/entity/Q202226')]
[('subTournamentName', "1969 French Open – Women's Doubles"), ('winnerName', 'Françoise Dürr'), ('year', '1969'), ('competitionClassName', "women's doubles"), ('subTournament', 'http://www.wikidata.org/entity/Q1455320'), ('yearlyTournament', 'http://www.wikidata.org/entity/Q202226')]
[('subTournamentName', "1969 French Open – Women's Singles"), ('winnerName', 'Margaret Court'), ('year', '1969'), ('competitionClassName', "women'

In [241]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# prob: missing Men's Single in US open and Wimbledon Chapionshiops in 1969

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?subTournamentName ?winnerName ?year ?subTournament ?yearlyTournament
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner.
      
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   
   BIND(xsd:integer(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
   FILTER(?year = "1969"^^xsd:integer)
}
#GROUP BY ?winner ?winnerName ?competitionClassName ?year
#HAVING (count(*) >=4)
#ORDER BY DESC (?year)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('subTournamentName', '1969 French Open – Mixed Doubles'), ('winnerName', 'Marty Riessen'), ('year', '1969'), ('subTournament', 'http://www.wikidata.org/entity/Q3037679'), ('yearlyTournament', 'http://www.wikidata.org/entity/Q202226')]
[('subTournamentName', "1969 French Open – Men's Doubles"), ('winnerName', 'John Newcombe'), ('year', '1969'), ('subTournament', 'http://www.wikidata.org/entity/Q12412317'), ('yearlyTournament', 'http://www.wikidata.org/entity/Q202226')]
[('subTournamentName', "1969 French Open – Women's Doubles"), ('winnerName', 'Françoise Dürr'), ('year', '1969'), ('subTournament', 'http://www.wikidata.org/entity/Q1455320'), ('yearlyTournament', 'http://www.wikidata.org/entity/Q202226')]
[('subTournamentName', "1969 French Open – Women's Singles"), ('winnerName', 'Margaret Court'), ('year', '1969'), ('subTournament', 'http://www.wikidata.org/entity/Q594977'), ('yearlyTournament', 'http://www.wikidata.org/entity/Q202226')]
[('subTournamentName', "1969 French Op

In [242]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# checking if removing filter solves the porblem

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?subTournamentName ?winnerName ?competitionClassName ?yearlyTournamentName
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner;
                   wdt:P2094 ?competitionClass.
      
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?competitionClass sc:name ?competitionClassName.
}
ORDER BY ASC (?yearlyTournamentName)
LIMIT 200

"""

print("Results")
x=run_query(queryString)

Results
[('subTournamentName', '1877 Wimbledon Championship – singles'), ('winnerName', 'Spencer Gore'), ('competitionClassName', "men's singles"), ('yearlyTournamentName', '1877 Wimbledon Championship')]
[('subTournamentName', '1878 Wimbledon Championship – singles'), ('winnerName', 'Frank Hadow'), ('competitionClassName', "men's singles"), ('yearlyTournamentName', '1878 Wimbledon Championships')]
[('subTournamentName', '1879 Wimbledon Championship – singles'), ('winnerName', 'John Hartley'), ('competitionClassName', "men's singles"), ('yearlyTournamentName', '1879 Wimbledon Championship')]
[('subTournamentName', '1880 Wimbledon Championship – singles'), ('winnerName', 'John Hartley'), ('competitionClassName', "men's singles"), ('yearlyTournamentName', '1880 Wimbledon Championship')]
[('subTournamentName', '1881 Wimbledon Championship – singles'), ('winnerName', 'William Renshaw'), ('competitionClassName', "men's singles"), ('yearlyTournamentName', '1881 Wimbledon Championship')]
[('s

##### The data is incomplete, even 1938 has no Wimbledon neither US open ...

##### Now trying to solve countryForSport problem that escludes one matching, is the same problem: Data Missing?

In [243]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# final query with country for sport 

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winner ?winnerName ?countryForSport ?competitionClassName ?year (COUNT(*) as ?tot)
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner;
                   wdt:P2094 ?competitionClass.
   
   ?winner wdt:P1532 ?countryForSport.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?competitionClass sc:name ?competitionClassName.
   
   BIND(xsd:integer(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
}
GROUP BY ?winner ?winnerName ?countryForSport ?competitionClassName ?year
HAVING (count(*) >=4)
ORDER BY DESC (?year)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q11662'), ('winnerName', 'Steffi Graf'), ('countryForSport', 'http://www.wikidata.org/entity/Q183'), ('competitionClassName', "women's singles"), ('year', '1988'), ('tot', '4')]
1


In [244]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# final query without country for sport 

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winner ?winnerName ?competitionClassName ?year (COUNT(*) as ?tot)
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner;
                   wdt:P2094 ?competitionClass.
   
   #?winner wdt:P1532 ?countryForSport.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?competitionClass sc:name ?competitionClassName.
   
   BIND(xsd:integer(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
}
GROUP BY ?winner ?winnerName ?competitionClassName ?year
HAVING (count(*) >=4)
ORDER BY DESC (?year)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q24574661'), ('winnerName', 'Diede de Groot'), ('competitionClassName', "wheelchair tennis women's doubles"), ('year', '2019'), ('tot', '4')]
[('winner', 'http://www.wikidata.org/entity/Q11662'), ('winnerName', 'Steffi Graf'), ('competitionClassName', "women's singles"), ('year', '1988'), ('tot', '4')]
2


In [245]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]
# ('winner', 'http://www.wikidata.org/entity/Q24574661'), ('winnerName', 'Diede de Groot'), 

# checking Diede de Groot's Q24574661 properties
queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q24574661 ?p ?o .
   # get the label 
   ?p sc:name ?pName.
   ?o sc:name ?oName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for'), ('o', 'http://www.wikidata.org/entity/Q6501370'), ('oName', 'Laureus World Sports Award for Sportsperson of the Year with a Disability')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('pName', 'country of citizenship'), ('o', 'http://www.wikidata.org/entity/Q29999'), ('oName', 'Kingdom of the Netherlands')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q5'), ('oName', 'human')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pName', 'place of birth'), ('o', 'http://www.wikidata.org/entity/Q5128'), ('oName', 'Woerden')]
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('pName', 'sex or gender'), ('o', 'http://www.wikidata.org/entity/Q6581072'), ('oName', 'female')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('o', 'http://www.wikidata.org/entity/Q1153025'), ('oName', 'wheelchair tennis')]
[(

In [246]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]
# ('winner', 'http://www.wikidata.org/entity/Q24574661'), ('winnerName', 'Diede de Groot'), 

# checking Diede de Groot's Q24574661 properties
# without ?o

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName
WHERE {
   wd:Q24574661 ?p ?o .
   # get the label 
   ?p sc:name ?pName.
   #?o sc:name ?oName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1352'), ('pName', 'ranking')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pName', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pName', 'place of birth')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('pName', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P2003'), ('pName', 'Instagram username')]
[('p', 'http://www.wikidata.org/prop/direct/P2013'), ('pName', 'Facebook ID')]
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('pName', 'sex or gender')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('pName', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('pName', 'country of citizenship')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('p', 'http://www.wik

##### No problem, there is no property countryForSport in KB for Q24574661 ('Diede de Groot')

In [247]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]


# 1) checking if in all seasons if someone has won all edition of each Grand Slam in the same category 
# 2) finding the winners of each GS tournament in each category
# 3) selecting only those who have 4 or more wins in the same category
# -> 4) grouping and then getting distinct countryForSportName 

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?countryForSport ?countryForSportName
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner;
                   wdt:P2094 ?competitionClass.
   
   ?winner wdt:P1532 ?countryForSport.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?competitionClass sc:name ?competitionClassName.
   
   ?countryForSport sc:name ?countryForSportName
   
   BIND(xsd:integer(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
}
GROUP BY ?countryForSport ?countryForSportName
HAVING (count(*) >=4)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('countryForSport', 'http://www.wikidata.org/entity/Q215'), ('countryForSportName', 'Slovenia')]
[('countryForSport', 'http://www.wikidata.org/entity/Q218'), ('countryForSportName', 'Romania')]
[('countryForSport', 'http://www.wikidata.org/entity/Q28'), ('countryForSportName', 'Hungary')]
[('countryForSport', 'http://www.wikidata.org/entity/Q38'), ('countryForSportName', 'Italy')]
[('countryForSport', 'http://www.wikidata.org/entity/Q16'), ('countryForSportName', 'Canada')]
[('countryForSport', 'http://www.wikidata.org/entity/Q664'), ('countryForSportName', 'New Zealand')]
[('countryForSport', 'http://www.wikidata.org/entity/Q183'), ('countryForSportName', 'Germany')]
[('countryForSport', 'http://www.wikidata.org/entity/Q216923'), ('countryForSportName', 'Chinese Taipei')]
[('countryForSport', 'http://www.wikidata.org/entity/Q83286'), ('countryForSportName', 'Socialist Federal Republic of Yugoslavia')]
[('countryForSport', 'http://www.wikidata.org/entity/Q34'), ('countryForSpo

In [248]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# 1) checking if in all seasons if someone has won all edition of each Grand Slam in the same category 
# 2) finding the winners of each GS tournament in each category
# 3) selecting only those who have 4 or more wins in the same category
# -> 4) grouping and then getting distinct countryForSportName 
#       GROUP BY corrected

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?countryForSport ?countryForSportName ?year (COUNT(*) as ?tot)
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner;
                   wdt:P2094 ?competitionClass.
   
   ?winner wdt:P1532 ?countryForSport.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?competitionClass sc:name ?competitionClassName.
   
   ?countryForSport sc:name ?countryForSportName
   
   BIND(xsd:integer(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
}
GROUP BY ?winner ?winnerName ?countryForSport ?countryForSportName ?competitionClassName ?year
HAVING (count(*) >=4)
ORDER BY DESC (?year)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('countryForSport', 'http://www.wikidata.org/entity/Q183'), ('countryForSportName', 'Germany'), ('year', '1988'), ('tot', '4')]
1


In [249]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]

# 1) checking if in all seasons if someone has won all edition of each Grand Slam in the same category 
# 2) finding the winners of each GS tournament in each category
# 3) selecting only those who have 4 or more wins in the same category
# 4) grouping and then getting distinct countryForSportName 
#    GROUP BY corrected
# -> 5) Cleaning result

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?countryForSport ?countryForSportName
WHERE {
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner;
                   wdt:P2094 ?competitionClass.
   
   ?winner wdt:P1532 ?countryForSport.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?competitionClass sc:name ?competitionClassName.
   
   ?countryForSport sc:name ?countryForSportName
   
   BIND(xsd:integer(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
}
GROUP BY ?winner ?winnerName ?countryForSport ?countryForSportName ?competitionClassName ?year
HAVING (count(*) >=4)
ORDER BY DESC (?year)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('countryForSport', 'http://www.wikidata.org/entity/Q183'), ('countryForSportName', 'Germany')]
1


In [250]:
# write your final query
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')

#[('obj', 'http://www.wikidata.org/entity/Q43605'), ('objName', 'French Open')]
#[('obj', 'http://www.wikidata.org/entity/Q123577'), ('objName', 'US Open')]
#[('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
#[('obj', 'http://www.wikidata.org/entity/Q60874'), ('objName', 'Australian Open')]


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?countryForSport ?countryForSportName
WHERE {
    # All four tourmamets in Grand Slam
   VALUES (?tournament){
       (wd:Q43605)
       (wd:Q123577)
       (wd:Q41520)
       (wd:Q60874)
   }
   ?yearlyTournament wdt:P31 ?tournament;
                    wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner;
                   wdt:P2094 ?competitionClass.
   
   ?winner wdt:P1532 ?countryForSport.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?competitionClass sc:name ?competitionClassName.
   
   ?countryForSport sc:name ?countryForSportName
   
   BIND(xsd:integer(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
}
GROUP BY ?winner ?winnerName ?countryForSport ?countryForSportName ?competitionClassName ?year
HAVING (count(*) >=4)
ORDER BY DESC (?year)
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('countryForSport', 'http://www.wikidata.org/entity/Q183'), ('countryForSportName', 'Germany')]
1


#### SOL 1: all tournaments in the same year, a player is considere if and only if he/she wins in the same category/competitionClass

In [251]:
# write your final query
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).


# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?countryForSport ?countryForSportName
WHERE {
    # All four tourmamets in Grand Slam
   VALUES (?tournament){
       (wd:Q43605)  # French Open
       (wd:Q123577) # US Open
       (wd:Q41520)  # Wimbledon Championships
       (wd:Q60874)  # Australian Open
   }
   ?yearlyTournament wdt:P31 ?tournament;
                    wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner;
                   wdt:P2094 ?competitionClass.
   
   ?winner wdt:P1532 ?countryForSport.
   
   # get the label 
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?countryForSport sc:name ?countryForSportName.
   
   BIND(xsd:integer(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
}
GROUP BY ?winner ?countryForSport ?countryForSportName ?competitionClass ?year
HAVING (count(*) >=4)
ORDER BY DESC (?year)
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('countryForSport', 'http://www.wikidata.org/entity/Q183'), ('countryForSportName', 'Germany')]
1


#### Another interpretation of: Return the countries with at least one player who won at least one edition of each Grand Slam (SOL 2)

In [252]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# REMOVED ?year

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winner ?winnerName ?competitionClassName (COUNT(*) as ?tot) #?year ?countryForSport
WHERE {
   # All four tourmamets in Grand Slam
   VALUES (?tournament){
       (wd:Q43605)  # French Open
       (wd:Q123577) # US Open
       (wd:Q41520)  # Wimbledon Championships
       (wd:Q60874)  # Australian Open
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner;
                   wdt:P2094 ?competitionClass.
   
   ?winner wdt:P1532 ?countryForSport.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?competitionClass sc:name ?competitionClassName.
   
   #BIND(xsd:integer(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
}
GROUP BY ?winner ?winnerName ?competitionClassName #?countryForSport  ?year
HAVING (count(*) >=4)
#ORDER BY DESC (?year)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q104506'), ('winnerName', 'Björn Borg'), ('competitionClassName', "men's singles"), ('tot', '9')]
[('winner', 'http://www.wikidata.org/entity/Q16224602'), ('winnerName', 'Yui Kamiji'), ('competitionClassName', "wheelchair tennis women's singles"), ('tot', '6')]
[('winner', 'http://www.wikidata.org/entity/Q14286'), ('winnerName', 'Chris Evert'), ('competitionClassName', "women's singles"), ('tot', '13')]
[('winner', 'http://www.wikidata.org/entity/Q312554'), ('winnerName', 'Mahesh Bhupathi'), ('competitionClassName', 'mixed doubles'), ('tot', '4')]
[('winner', 'http://www.wikidata.org/entity/Q240695'), ('winnerName', 'Larisa Neiland'), ('competitionClassName', 'mixed doubles'), ('tot', '8')]
[('winner', 'http://www.wikidata.org/entity/Q54654'), ('winnerName', 'Pam Shriver'), ('competitionClassName', "women's doubles"), ('tot', '15')]
[('winner', 'http://www.wikidata.org/entity/Q232566'), ('winnerName', 'Hsieh Su-wei'), ('competitionCla

In [253]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# REMOVED ?year
# COUNTING the wins of winner, who has atleas 4 total wins in his/her whole carrier, GROUPED BY competitionClass

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winner ?winnerName ?competitionClassName (COUNT(*) as ?tot) #?year ?countryForSport
WHERE {
   # All four tourmamets in Grand Slam
   VALUES (?tournament){
       (wd:Q43605)  # French Open
       (wd:Q123577) # US Open
       (wd:Q41520)  # Wimbledon Championships
       (wd:Q60874)  # Australian Open
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner;
                   wdt:P2094 ?competitionClass.
   
   ?winner wdt:P1532 ?countryForSport.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?competitionClass sc:name ?competitionClassName.
   
   #BIND(xsd:integer(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
}
GROUP BY ?winner ?winnerName ?competitionClassName #?countryForSport  ?year
HAVING (count(*) >=4)
#ORDER BY DESC (?year)
ORDER BY DESC (?winnerName)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q53546'), ('winnerName', 'Zina Garrison'), ('competitionClassName', 'mixed doubles'), ('tot', '4')]
[('winner', 'http://www.wikidata.org/entity/Q16224602'), ('winnerName', 'Yui Kamiji'), ('competitionClassName', "wheelchair tennis women's singles"), ('tot', '6')]
[('winner', 'http://www.wikidata.org/entity/Q16224602'), ('winnerName', 'Yui Kamiji'), ('competitionClassName', "wheelchair tennis women's doubles"), ('tot', '5')]
[('winner', 'http://www.wikidata.org/entity/Q365139'), ('winnerName', 'William Renshaw'), ('competitionClassName', "men's singles"), ('tot', '6')]
[('winner', 'http://www.wikidata.org/entity/Q229093'), ('winnerName', 'Virginia Ruano Pascual'), ('competitionClassName', "women's doubles"), ('tot', '6')]
[('winner', 'http://www.wikidata.org/entity/Q11578'), ('winnerName', 'Venus Williams'), ('competitionClassName', "women's doubles"), ('tot', '11')]
[('winner', 'http://www.wikidata.org/entity/Q11578'), ('winnerName', 

In [254]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# REMOVED ?year
# COUNTING the wins of winner, who has atleas 4 total wins in his/her whole carrier, 
# REMOVING GROUPED by to check in depth if the results are correct

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winner ?winnerName ?competitionClassName ?subTournamentName #(COUNT(*) as ?tot) #?year ?countryForSport
WHERE {
   # All four tourmamets in Grand Slam
   VALUES (?tournament){
       (wd:Q43605)  # French Open
       (wd:Q123577) # US Open
       (wd:Q41520)  # Wimbledon Championships
       (wd:Q60874)  # Australian Open
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner;
                   wdt:P2094 ?competitionClass.
   
   ?winner wdt:P1532 ?countryForSport.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?competitionClass sc:name ?competitionClassName.
   
   #BIND(xsd:integer(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
}
#GROUP BY ?winner ?winnerName ?competitionClassName #?countryForSport  ?year
#HAVING (count(*) >=4)
#ORDER BY DESC (?year)
ORDER BY DESC (?winnerName)
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q345626'), ('winnerName', 'Łukasz Kubot'), ('competitionClassName', "men's doubles"), ('subTournamentName', "2014 Australian Open – Men's Doubles")]
[('winner', 'http://www.wikidata.org/entity/Q345626'), ('winnerName', 'Łukasz Kubot'), ('competitionClassName', "men's doubles"), ('subTournamentName', "2017 Wimbledon Championships – Men's Doubles")]
[('winner', 'http://www.wikidata.org/entity/Q8080164'), ('winnerName', 'İpek Soylu'), ('competitionClassName', "girls' doubles"), ('subTournamentName', "2014 US Open – Girls' Doubles")]
[('winner', 'http://www.wikidata.org/entity/Q227389'), ('winnerName', 'Zsuzsa Körmöczy'), ('competitionClassName', "women's singles"), ('subTournamentName', "1958 French Championships – Women's Singles")]
[('winner', 'http://www.wikidata.org/entity/Q53546'), ('winnerName', 'Zina Garrison'), ('competitionClassName', 'mixed doubles'), ('subTournamentName', '1988 Wimbledon Championships – Mixed Doubles')]
[('win

In [255]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# REMOVED ?year
# COUNTING the wins of winner, who has atleas 4 total wins in his/her whole carrier, 
# REMOVING GROUPED by to check in depth if the results are correct


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winner ?winnerName ?competitionClassName ?yearlyTournamentName ?subTournamentName #(COUNT(*) as ?tot) #?year ?countryForSport
WHERE {
   # All four tourmamets in Grand Slam
   VALUES (?tournament){
       (wd:Q43605)  # French Open
       (wd:Q123577) # US Open
       (wd:Q41520)  # Wimbledon Championships
       (wd:Q60874)  # Australian Open
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner;
                   wdt:P2094 ?competitionClass.
   
   ?winner wdt:P1532 ?countryForSport.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?competitionClass sc:name ?competitionClassName.
   
   #BIND(xsd:integer(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
   #BIND(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
}
#GROUP BY ?winner ?winnerName ?competitionClassName #?countryForSport  ?year
#HAVING (count(*) >=4)
#ORDER BY DESC (?year)
ORDER BY DESC (?winnerName)
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q345626'), ('winnerName', 'Łukasz Kubot'), ('competitionClassName', "men's doubles"), ('yearlyTournamentName', '2014 Australian Open'), ('subTournamentName', "2014 Australian Open – Men's Doubles")]
[('winner', 'http://www.wikidata.org/entity/Q345626'), ('winnerName', 'Łukasz Kubot'), ('competitionClassName', "men's doubles"), ('yearlyTournamentName', '2017 Wimbledon Championships'), ('subTournamentName', "2017 Wimbledon Championships – Men's Doubles")]
[('winner', 'http://www.wikidata.org/entity/Q8080164'), ('winnerName', 'İpek Soylu'), ('competitionClassName', "girls' doubles"), ('yearlyTournamentName', '2014 US Open'), ('subTournamentName', "2014 US Open – Girls' Doubles")]
[('winner', 'http://www.wikidata.org/entity/Q227389'), ('winnerName', 'Zsuzsa Körmöczy'), ('competitionClassName', "women's singles"), ('yearlyTournamentName', '1958 French Championships'), ('subTournamentName', "1958 French Championships – Women's Singles")]
[(

In [256]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# FINDING shor way to GROUP something above yearlyTournament 


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName #(COUNT(*) as ?tot) #?year ?countryForSport
WHERE {
   # All four tourmamets in Grand Slam
   VALUES (?tournament){
       (wd:Q43605)  # French Open
       (wd:Q123577) # US Open
       (wd:Q41520)  # Wimbledon Championships
       (wd:Q60874)  # Australian Open
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament ?p ?o.
   #?subTournament wdt:P1346 ?winner;
   #                wdt:P2094 ?competitionClass.
   
   #?winner wdt:P1532 ?countryForSport.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?p sc:name ?pName.
   #?o sc:name ?oName.
   #?subTournament sc:name ?subTournamentName.
   #?winner sc:name ?winnerName.
   #?competitionClass sc:name ?competitionClassName.
   
   #BIND(xsd:integer(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
   #BIND(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
}
#GROUP BY ?winner ?winnerName ?competitionClassName #?countryForSport  ?year
#HAVING (count(*) >=4)
#ORDER BY DESC (?year)
#ORDER BY DESC (?winnerName)
#ORDER BY DESC (?winnerName)
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P6880'), ('pName', 'ATP tennis tournament edition ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('pName', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pName', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pName', 'competition class')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('pName', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('pName', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('pName', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('pName', 'Google Knowledge

In [257]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# FINDING shor way to GROUP something above yearlyTournament
# added FILTER


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName #(COUNT(*) as ?tot) #?year ?countryForSport
WHERE {
   # All four tourmamets in Grand Slam
   VALUES (?tournament){
       (wd:Q43605)  # French Open
       (wd:Q123577) # US Open
       (wd:Q41520)  # Wimbledon Championships
       (wd:Q60874)  # Australian Open
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament ?p ?o.
   #?subTournament wdt:P1346 ?winner;
   #                wdt:P2094 ?competitionClass.
   
   #?winner wdt:P1532 ?countryForSport.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   #?subTournament sc:name ?subTournamentName.
   #?winner sc:name ?winnerName.
   #?competitionClass sc:name ?competitionClassName.
   
   #BIND(xsd:integer(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
   #BIND(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
   
   FILTER (?p not in (wdt:P155, wdt:P156))
}
#GROUP BY ?winner ?winnerName ?competitionClassName #?countryForSport  ?year
#HAVING (count(*) >=4)
#ORDER BY DESC (?year)
#ORDER BY DESC (?winnerName)
#ORDER BY DESC (?winnerName)
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q3130'), ('oName', 'Sydney')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q3141'), ('oName', 'Melbourne')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q60'), ('oName', 'New York City')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q90'), ('oName', 'Paris')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q5112'), ('oName', 'Adelaide')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located

In [258]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# FINDING shor way to GROUP something above yearlyTournament
# added FILTER and extended LIMIT

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName #(COUNT(*) as ?tot) #?year ?countryForSport
WHERE {
   # All four tourmamets in Grand Slam
   VALUES (?tournament){
       (wd:Q43605)  # French Open
       (wd:Q123577) # US Open
       (wd:Q41520)  # Wimbledon Championships
       (wd:Q60874)  # Australian Open
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament ?p ?o.
   #?subTournament wdt:P1346 ?winner;
   #                wdt:P2094 ?competitionClass.
   
   #?winner wdt:P1532 ?countryForSport.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   #?subTournament sc:name ?subTournamentName.
   #?winner sc:name ?winnerName.
   #?competitionClass sc:name ?competitionClassName.
   
   #BIND(xsd:integer(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
   #BIND(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
   
   FILTER (?p not in (wdt:P155, wdt:P156))
}
#GROUP BY ?winner ?winnerName ?competitionClassName #?countryForSport  ?year
#HAVING (count(*) >=4)
#ORDER BY DESC (?year)
#ORDER BY DESC (?winnerName)
#ORDER BY DESC (?winnerName)
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q3130'), ('oName', 'Sydney')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q3141'), ('oName', 'Melbourne')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q60'), ('oName', 'New York City')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q90'), ('oName', 'Paris')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q5112'), ('oName', 'Adelaide')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located

In [259]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# FINDING shor way to GROUP something above yearlyTournament
# added more stuff FILTER and extended LIMIT


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName #(COUNT(*) as ?tot) #?year ?countryForSport
WHERE {
   # All four tourmamets in Grand Slam
   VALUES (?tournament){
       (wd:Q43605)  # French Open
       (wd:Q123577) # US Open
       (wd:Q41520)  # Wimbledon Championships
       (wd:Q60874)  # Australian Open
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament ?p ?o.
   #?subTournament wdt:P1346 ?winner;
   #                wdt:P2094 ?competitionClass.
   
   #?winner wdt:P1532 ?countryForSport.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   #?subTournament sc:name ?subTournamentName.
   #?winner sc:name ?winnerName.
   #?competitionClass sc:name ?competitionClassName.
   
   #BIND(xsd:integer(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
   #BIND(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
   
   FILTER (?p not in (wdt:P155, wdt:P156, wdt:P527,wdt:P910, wdt:P276))
}
#GROUP BY ?winner ?winnerName ?competitionClassName #?countryForSport  ?year
#HAVING (count(*) >=4)
#ORDER BY DESC (?year)
#ORDER BY DESC (?winnerName)
#ORDER BY DESC (?winnerName)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q3130'), ('oName', 'Sydney')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q3141'), ('oName', 'Melbourne')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q60'), ('oName', 'New York City')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q90'), ('oName', 'Paris')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q5112'), ('oName', 'Adelaide')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located

In [260]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# FINDING shor way to GROUP something above yearlyTournament 
# added more stuff (x2) FILTER and extended LIMIT


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName #(COUNT(*) as ?tot) #?year ?countryForSport
WHERE {
   # All four tourmamets in Grand Slam
   VALUES (?tournament){
       (wd:Q43605)  # French Open
       (wd:Q123577) # US Open
       (wd:Q41520)  # Wimbledon Championships
       (wd:Q60874)  # Australian Open
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament ?p ?o.
   #?subTournament wdt:P1346 ?winner;
   #                wdt:P2094 ?competitionClass.
   
   #?winner wdt:P1532 ?countryForSport.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   #?subTournament sc:name ?subTournamentName.
   #?winner sc:name ?winnerName.
   #?competitionClass sc:name ?competitionClassName.
   
   #BIND(xsd:integer(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
   #BIND(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
   
   FILTER (?p not in (wdt:P155, wdt:P156, wdt:P527,wdt:P910, wdt:P276, wdt:P361))
}
#GROUP BY ?winner ?winnerName ?competitionClassName #?countryForSport  ?year
#HAVING (count(*) >=4)
#ORDER BY DESC (?year)
#ORDER BY DESC (?winnerName)
#ORDER BY DESC (?winnerName)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q3130'), ('oName', 'Sydney')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q3141'), ('oName', 'Melbourne')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q60'), ('oName', 'New York City')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q90'), ('oName', 'Paris')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q5112'), ('oName', 'Adelaide')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located

In [261]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# FINDING shor way to GROUP something above yearlyTournament 
# FILTER ?p to P3450 ('sports season of league or competition')

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName #(COUNT(*) as ?tot) #?year ?countryForSport
WHERE {
   # All four tourmamets in Grand Slam
   VALUES (?tournament){
       (wd:Q43605)  # French Open
       (wd:Q123577) # US Open
       (wd:Q41520)  # Wimbledon Championships
       (wd:Q60874)  # Australian Open
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament ?p ?o.
   #?subTournament wdt:P1346 ?winner;
   #                wdt:P2094 ?competitionClass.
   
   #?winner wdt:P1532 ?countryForSport.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   #?subTournament sc:name ?subTournamentName.
   #?winner sc:name ?winnerName.
   #?competitionClass sc:name ?competitionClassName.
   
   #BIND(xsd:integer(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
   #BIND(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
   
   #FILTER (?p not in (wdt:P155, wdt:P156, wdt:P527,wdt:P910, wdt:P276, wdt:P361))
   FILTER (?p in (wdt:P3450))
}
#GROUP BY ?winner ?winnerName ?competitionClassName #?countryForSport  ?year
#HAVING (count(*) >=4)
#ORDER BY DESC (?year)
#ORDER BY DESC (?winnerName)
#ORDER BY DESC (?winnerName)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pName', 'sports season of league or competition'), ('o', 'http://www.wikidata.org/entity/Q41520'), ('oName', 'Wimbledon Championships')]
1


In [262]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# FINDING shor way to GROUP something above yearlyTournament 
# FILTER ?p to P31, P3450 ('sports season of league or competition') has only 1 result

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName #(COUNT(*) as ?tot) #?year ?countryForSport
WHERE {
   # All four tourmamets in Grand Slam
   VALUES (?tournament){
       (wd:Q43605)  # French Open
       (wd:Q123577) # US Open
       (wd:Q41520)  # Wimbledon Championships
       (wd:Q60874)  # Australian Open
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament ?p ?o.
   #?subTournament wdt:P1346 ?winner;
   #                wdt:P2094 ?competitionClass.
   
   #?winner wdt:P1532 ?countryForSport.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   #?subTournament sc:name ?subTournamentName.
   #?winner sc:name ?winnerName.
   #?competitionClass sc:name ?competitionClassName.
   
   #BIND(xsd:integer(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
   #BIND(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
   
   #FILTER (?p not in (wdt:P155, wdt:P156, wdt:P527,wdt:P910, wdt:P276, wdt:P361))
   FILTER (?p in (wdt:P31))
}
#GROUP BY ?winner ?winnerName ?competitionClassName #?countryForSport  ?year
#HAVING (count(*) >=4)
#ORDER BY DESC (?year)
#ORDER BY DESC (?winnerName)
#ORDER BY DESC (?winnerName)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q43605'), ('oName', 'French Open')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q47345468'), ('oName', 'tennis tournament edition')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q123577'), ('oName', 'US Open')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q41520'), ('oName', 'Wimbledon Championships')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q60874'), ('oName', 'Australian Open')]
5


In [263]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# FINDING shor way to group by instead of doing inner queries
# FILTER ?p to P31 --> is what I was already doing = ?tournament
# RECOVER initial query

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winner ?winnerName ?tournamentName ?subTournamentName
WHERE {
   # All four tourmamets in Grand Slam
   VALUES (?tournament){
       (wd:Q43605)  # French Open
       (wd:Q123577) # US Open
       (wd:Q41520)  # Wimbledon Championships
       (wd:Q60874)  # Australian Open
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner;
                   wdt:P2094 ?competitionClass.
   
   ?winner wdt:P1532 ?countryForSport.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?competitionClass sc:name ?competitionClassName.
}
ORDER BY asc (?winnerName)
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q54668'), ('winnerName', 'Abigail Spears'), ('tournamentName', 'Australian Open'), ('subTournamentName', '2017 Australian Open – Mixed Doubles')]
[('winner', 'http://www.wikidata.org/entity/Q207352'), ('winnerName', 'Agnieszka Radwańska'), ('tournamentName', 'French Open'), ('subTournamentName', "2006 French Open – Girls' Singles")]
[('winner', 'http://www.wikidata.org/entity/Q229100'), ('winnerName', 'Ai Sugiyama'), ('tournamentName', 'US Open'), ('subTournamentName', '1999 US Open – Mixed Doubles')]
[('winner', 'http://www.wikidata.org/entity/Q229100'), ('winnerName', 'Ai Sugiyama'), ('tournamentName', 'Wimbledon Championships'), ('subTournamentName', "2003 Wimbledon Championships – Women's Doubles")]
[('winner', 'http://www.wikidata.org/entity/Q2712407'), ('winnerName', 'Ajla Tomljanović'), ('tournamentName', 'Australian Open'), ('subTournamentName', "2009 Australian Open – Girls' Doubles")]
[('winner', 'http://www.wikidata.org/ent

##### Partial sol found: got all wins of each person in each tournament: 

In [264]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# GROUP BY ?winner & ?tournament


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winner ?winnerName ?tournamentName (COUNT(*) AS ?tot)
WHERE {
   # All four tourmamets in Grand Slam
   VALUES (?tournament){
       (wd:Q43605)  # French Open
       (wd:Q123577) # US Open
       (wd:Q41520)  # Wimbledon Championships
       (wd:Q60874)  # Australian Open
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner;
                   wdt:P2094 ?competitionClass.
   
   ?winner wdt:P1532 ?countryForSport.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?competitionClass sc:name ?competitionClassName.
}
GROUP BY ?winner ?winnerName ?tournamentName
ORDER BY DESC (?tot)
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q54545'), ('winnerName', 'Martina Navratilova'), ('tournamentName', 'Wimbledon Championships'), ('tot', '51')]
[('winner', 'http://www.wikidata.org/entity/Q54545'), ('winnerName', 'Martina Navratilova'), ('tournamentName', 'Australian Open'), ('tot', '33')]
[('winner', 'http://www.wikidata.org/entity/Q54545'), ('winnerName', 'Martina Navratilova'), ('tournamentName', 'US Open'), ('tot', '33')]
[('winner', 'http://www.wikidata.org/entity/Q54545'), ('winnerName', 'Martina Navratilova'), ('tournamentName', 'French Open'), ('tot', '30')]
[('winner', 'http://www.wikidata.org/entity/Q11459'), ('winnerName', 'Serena Williams'), ('tournamentName', 'Wimbledon Championships'), ('tot', '14')]
[('winner', 'http://www.wikidata.org/entity/Q10132'), ('winnerName', 'Rafael Nadal'), ('tournamentName', 'French Open'), ('tot', '13')]
[('winner', 'http://www.wikidata.org/entity/Q229121'), ('winnerName', 'Jana Novotná'), ('tournamentName', 'Wimbledon Cham

In [265]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# GROUP BY ?winner & ?tournament
# -> ORDERING & EXTENDING LIMIT TO 50


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winner ?winnerName ?tournamentName (COUNT(*) AS ?tot)
WHERE {
   # All four tourmamets in Grand Slam
   VALUES (?tournament){
       (wd:Q43605)  # French Open
       (wd:Q123577) # US Open
       (wd:Q41520)  # Wimbledon Championships
       (wd:Q60874)  # Australian Open
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner;
                   wdt:P2094 ?competitionClass.
   
   ?winner wdt:P1532 ?countryForSport.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?competitionClass sc:name ?competitionClassName.
}
GROUP BY ?winner ?winnerName ?tournamentName
ORDER BY ?winnerName ?tot
LIMIT 50

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q54668'), ('winnerName', 'Abigail Spears'), ('tournamentName', 'Australian Open'), ('tot', '1')]
[('winner', 'http://www.wikidata.org/entity/Q207352'), ('winnerName', 'Agnieszka Radwańska'), ('tournamentName', 'French Open'), ('tot', '1')]
[('winner', 'http://www.wikidata.org/entity/Q229100'), ('winnerName', 'Ai Sugiyama'), ('tournamentName', 'US Open'), ('tot', '1')]
[('winner', 'http://www.wikidata.org/entity/Q229100'), ('winnerName', 'Ai Sugiyama'), ('tournamentName', 'Wimbledon Championships'), ('tot', '1')]
[('winner', 'http://www.wikidata.org/entity/Q2712407'), ('winnerName', 'Ajla Tomljanović'), ('tournamentName', 'Australian Open'), ('tot', '2')]
[('winner', 'http://www.wikidata.org/entity/Q317828'), ('winnerName', 'Albert Costa'), ('tournamentName', 'French Open'), ('tot', '1')]
[('winner', 'http://www.wikidata.org/entity/Q737425'), ('winnerName', "Alex O'Brien"), ('tournamentName', 'US Open'), ('tot', '1')]
[('winner', 'http

In [266]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# GROUP BY ?winner & ?tournament
# ORDERING
# -> LIMIT extended to 100


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?winner ?winnerName ?tournamentName (COUNT(*) AS ?tot)
WHERE {
   # All four tourmamets in Grand Slam
   VALUES (?tournament){
       (wd:Q43605)  # French Open
       (wd:Q123577) # US Open
       (wd:Q41520)  # Wimbledon Championships
       (wd:Q60874)  # Australian Open
   }
   ?yearlyTournament wdt:P31 ?tournament.
   ?yearlyTournament wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner;
                   wdt:P2094 ?competitionClass.
   
   ?winner wdt:P1532 ?countryForSport.
   
   # get the label 
   ?tournament sc:name ?tournamentName .
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?subTournament sc:name ?subTournamentName.
   ?winner sc:name ?winnerName.
   ?competitionClass sc:name ?competitionClassName.
}
GROUP BY ?winner ?winnerName ?tournamentName
ORDER BY ?winnerName ?tot
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q54668'), ('winnerName', 'Abigail Spears'), ('tournamentName', 'Australian Open'), ('tot', '1')]
[('winner', 'http://www.wikidata.org/entity/Q207352'), ('winnerName', 'Agnieszka Radwańska'), ('tournamentName', 'French Open'), ('tot', '1')]
[('winner', 'http://www.wikidata.org/entity/Q229100'), ('winnerName', 'Ai Sugiyama'), ('tournamentName', 'US Open'), ('tot', '1')]
[('winner', 'http://www.wikidata.org/entity/Q229100'), ('winnerName', 'Ai Sugiyama'), ('tournamentName', 'Wimbledon Championships'), ('tot', '1')]
[('winner', 'http://www.wikidata.org/entity/Q2712407'), ('winnerName', 'Ajla Tomljanović'), ('tournamentName', 'Australian Open'), ('tot', '2')]
[('winner', 'http://www.wikidata.org/entity/Q317828'), ('winnerName', 'Albert Costa'), ('tournamentName', 'French Open'), ('tot', '1')]
[('winner', 'http://www.wikidata.org/entity/Q737425'), ('winnerName', "Alex O'Brien"), ('tournamentName', 'US Open'), ('tot', '1')]
[('winner', 'http

In [267]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# GROUP BY ?winner & ?tournament
# ORDERING
# LIMIT extended to 100

# CREATING outer query


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT * 
WHERE {
    SELECT DISTINCT ?winner ?winnerName ?tournamentName (COUNT(*) AS ?tot)
    WHERE {
       # All four tourmamets in Grand Slam
       VALUES (?tournament){
           (wd:Q43605)  # French Open
           (wd:Q123577) # US Open
           (wd:Q41520)  # Wimbledon Championships
           (wd:Q60874)  # Australian Open
       }
       ?yearlyTournament wdt:P31 ?tournament.
       ?yearlyTournament wdt:P527 ?subTournament.
       ?subTournament wdt:P1346 ?winner;
                       wdt:P2094 ?competitionClass.

       ?winner wdt:P1532 ?countryForSport.

       # get the label 
       ?tournament sc:name ?tournamentName .
       ?yearlyTournament sc:name ?yearlyTournamentName .
       ?subTournament sc:name ?subTournamentName.
       ?winner sc:name ?winnerName.
       ?competitionClass sc:name ?competitionClassName.
    }
    GROUP BY ?winner ?winnerName ?tournamentName
    ORDER BY ?winnerName ?tot
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q54668'), ('winnerName', 'Abigail Spears'), ('tournamentName', 'Australian Open'), ('tot', '1')]
[('winner', 'http://www.wikidata.org/entity/Q207352'), ('winnerName', 'Agnieszka Radwańska'), ('tournamentName', 'French Open'), ('tot', '1')]
[('winner', 'http://www.wikidata.org/entity/Q229100'), ('winnerName', 'Ai Sugiyama'), ('tournamentName', 'US Open'), ('tot', '1')]
[('winner', 'http://www.wikidata.org/entity/Q229100'), ('winnerName', 'Ai Sugiyama'), ('tournamentName', 'Wimbledon Championships'), ('tot', '1')]
[('winner', 'http://www.wikidata.org/entity/Q2712407'), ('winnerName', 'Ajla Tomljanović'), ('tournamentName', 'Australian Open'), ('tot', '2')]
[('winner', 'http://www.wikidata.org/entity/Q317828'), ('winnerName', 'Albert Costa'), ('tournamentName', 'French Open'), ('tot', '1')]
[('winner', 'http://www.wikidata.org/entity/Q737425'), ('winnerName', "Alex O'Brien"), ('tournamentName', 'US Open'), ('tot', '1')]
[('winner', 'http

In [268]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# GROUP BY ?winner & ?tournament
# ORDERING
# LIMIT extended to 100

# CREATING outer query


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?winner ?winnerName ?tot
WHERE {
    SELECT DISTINCT ?winner ?winnerName ?tournamentName (COUNT(*) AS ?tot)
    WHERE {
       # All four tourmamets in Grand Slam
       VALUES (?tournament){
           (wd:Q43605)  # French Open
           (wd:Q123577) # US Open
           (wd:Q41520)  # Wimbledon Championships
           (wd:Q60874)  # Australian Open
       }
       ?yearlyTournament wdt:P31 ?tournament.
       ?yearlyTournament wdt:P527 ?subTournament.
       ?subTournament wdt:P1346 ?winner;
                       wdt:P2094 ?competitionClass.

       ?winner wdt:P1532 ?countryForSport.

       # get the label 
       ?tournament sc:name ?tournamentName .
       ?yearlyTournament sc:name ?yearlyTournamentName .
       ?subTournament sc:name ?subTournamentName.
       ?winner sc:name ?winnerName.
       ?competitionClass sc:name ?competitionClassName.
    }
    GROUP BY ?winner ?winnerName ?tournamentName
    ORDER BY ?winnerName ?tot
}
GROUP BY ?winner ?winnerName
#ORDER BY DESC ?tot
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q30758'), ('winnerName', 'Ana Ivanovic'), ('tot', '1')]
[('winner', 'http://www.wikidata.org/entity/Q2365460'), ('winnerName', 'Ana Konjuh'), ('tot', '1')]
[('winner', 'http://www.wikidata.org/entity/Q77178'), ('winnerName', 'Angelique Kerber'), ('tot', '1')]
[('winner', 'http://www.wikidata.org/entity/Q107207693'), ('winnerName', 'Arthur Fils'), ('tot', '1')]
[('winner', 'http://www.wikidata.org/entity/Q53570'), ('winnerName', 'Barbara Jordan'), ('tot', '1')]
[('winner', 'http://www.wikidata.org/entity/Q259282'), ('winnerName', 'Betty Stöve'), ('tot', '3')]
[('winner', 'http://www.wikidata.org/entity/Q54527'), ('winnerName', 'Billie Jean King'), ('tot', '3')]
[('winner', 'http://www.wikidata.org/entity/Q104506'), ('winnerName', 'Björn Borg'), ('tot', '5')]
[('winner', 'http://www.wikidata.org/entity/Q888085'), ('winnerName', 'Bob Lutz'), ('tot', '3')]
[('winner', 'http://www.wikidata.org/entity/Q14905846'), ('winnerName', 'Borna Ćori

In [269]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# GROUP BY ?winner & ?tournament
# ORDERING
# LIMIT extended to 100

# CREATING outer query -> prob result is incorrect, it should have atmost 4 as there are only 4 tournaments possible but 
#                         it is keeping only one row for each player from inner query

# TRYING to debug with one player specifically ('Natasha Zvereva')


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT *
WHERE {
    {
        SELECT DISTINCT ?winner ?winnerName ?tournamentName (COUNT(*) AS ?tot)
        WHERE {
           # All four tourmamets in Grand Slam
           VALUES (?tournament){
               (wd:Q43605)  # French Open
               (wd:Q123577) # US Open
               (wd:Q41520)  # Wimbledon Championships
               (wd:Q60874)  # Australian Open
           }
           ?yearlyTournament wdt:P31 ?tournament.
           ?yearlyTournament wdt:P527 ?subTournament.
           ?subTournament wdt:P1346 ?winner;
                           wdt:P2094 ?competitionClass.

           ?winner wdt:P1532 ?countryForSport.

           # get the label 
           ?tournament sc:name ?tournamentName .
           ?yearlyTournament sc:name ?yearlyTournamentName .
           ?subTournament sc:name ?subTournamentName.
           ?winner sc:name ?winnerName.
           ?competitionClass sc:name ?competitionClassName.
        }
        GROUP BY ?winner ?winnerName ?tournamentName
        ORDER BY ?winnerName ?tot
    }
    FILTER (?winnerName = "Natasha Zvereva")
}
#GROUP BY ?winner ?winnerName
#ORDER BY DESC ?tot
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
Empty


##### Cool query: founds all winning of a player in each tournament

In [270]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# GROUP BY ?winner & ?tournament
# ORDERING
# LIMIT extended to 100

# CREATING outer query -> prob result is incorrect, it should have atmost 4 as there are only 4 tournaments possible but 
#                         it is keeping only one row for each player from inner query

# TRYING to debug with one player specifically ('Natasha Zvereva')
# FILTER with wrong parametes, added regex


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT *
WHERE {
    {
        SELECT DISTINCT ?winner ?winnerName ?tournamentName (COUNT(*) AS ?tot)
        WHERE {
           # All four tourmamets in Grand Slam
           VALUES (?tournament){
               (wd:Q43605)  # French Open
               (wd:Q123577) # US Open
               (wd:Q41520)  # Wimbledon Championships
               (wd:Q60874)  # Australian Open
           }
           ?yearlyTournament wdt:P31 ?tournament.
           ?yearlyTournament wdt:P527 ?subTournament.
           ?subTournament wdt:P1346 ?winner;
                           wdt:P2094 ?competitionClass.

           ?winner wdt:P1532 ?countryForSport.

           # get the label 
           ?tournament sc:name ?tournamentName .
           ?yearlyTournament sc:name ?yearlyTournamentName .
           ?subTournament sc:name ?subTournamentName.
           ?winner sc:name ?winnerName.
           ?competitionClass sc:name ?competitionClassName.
        }
        GROUP BY ?winner ?winnerName ?tournamentName
        ORDER BY ?winnerName ?tot
    }
    FILTER (regex(?winnerName,"Natasha Zvereva"))
}
#GROUP BY ?winner ?winnerName
#ORDER BY DESC ?tot
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q233243'), ('winnerName', 'Natasha Zvereva'), ('tournamentName', 'US Open'), ('tot', '2')]
[('winner', 'http://www.wikidata.org/entity/Q233243'), ('winnerName', 'Natasha Zvereva'), ('tournamentName', 'French Open'), ('tot', '4')]
[('winner', 'http://www.wikidata.org/entity/Q233243'), ('winnerName', 'Natasha Zvereva'), ('tournamentName', 'Australian Open'), ('tot', '5')]
[('winner', 'http://www.wikidata.org/entity/Q233243'), ('winnerName', 'Natasha Zvereva'), ('tournamentName', 'Wimbledon Championships'), ('tot', '5')]
4


In [271]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# GROUP BY ?winner & ?tournament
# ORDERING
# LIMIT extended to 100

# CREATING outer query -> prob result is incorrect, it should have atmost 4 as there are only 4 tournaments possible but 
#                         it is keeping only one row for each player from inner query

# TRYING to debug with one player specifically ('Natasha Zvereva')
# FILTER with wrong parametes, added regex

# FIND, I need to group also in the outer select


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?winner ?winnerName (count(*) as ?all)
WHERE {
    {
        SELECT DISTINCT ?winner ?winnerName ?tournamentName (COUNT(*) AS ?tot)
        WHERE {
           # All four tourmamets in Grand Slam
           VALUES (?tournament){
               (wd:Q43605)  # French Open
               (wd:Q123577) # US Open
               (wd:Q41520)  # Wimbledon Championships
               (wd:Q60874)  # Australian Open
           }
           ?yearlyTournament wdt:P31 ?tournament.
           ?yearlyTournament wdt:P527 ?subTournament.
           ?subTournament wdt:P1346 ?winner;
                           wdt:P2094 ?competitionClass.

           ?winner wdt:P1532 ?countryForSport.

           # get the label 
           ?tournament sc:name ?tournamentName .
           ?yearlyTournament sc:name ?yearlyTournamentName .
           ?subTournament sc:name ?subTournamentName.
           ?winner sc:name ?winnerName.
           ?competitionClass sc:name ?competitionClassName.
        }
        GROUP BY ?winner ?winnerName ?tournamentName
        ORDER BY ?winnerName ?tot
    }
    FILTER (regex(?winnerName,"Natasha Zvereva"))
}
GROUP BY ?winner ?winnerName
ORDER BY desc (?all)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q233243'), ('winnerName', 'Natasha Zvereva'), ('all', '4')]
1


In [272]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# GROUP BY ?winner & ?tournament
# ORDERING
# LIMIT extended to 100

# CREATING outer query -> prob result is incorrect, it should have atmost 4 as there are only 4 tournaments possible but 
#                         it is keeping only one row for each player from inner query

# TRYING to debug with one player specifically ('Natasha Zvereva')
# FILTER with wrong parametes, added regex

# FIND, I need to group also in the outer select


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?winner ?winnerName (count(*) as ?all)
WHERE {
    {
        SELECT DISTINCT ?winner ?winnerName ?tournamentName (COUNT(*) AS ?tot)
        WHERE {
           # All four tourmamets in Grand Slam
           VALUES (?tournament){
               (wd:Q43605)  # French Open
               (wd:Q123577) # US Open
               (wd:Q41520)  # Wimbledon Championships
               (wd:Q60874)  # Australian Open
           }
           ?yearlyTournament wdt:P31 ?tournament.
           ?yearlyTournament wdt:P527 ?subTournament.
           ?subTournament wdt:P1346 ?winner;
                           wdt:P2094 ?competitionClass.

           ?winner wdt:P1532 ?countryForSport.

           # get the label 
           ?tournament sc:name ?tournamentName .
           ?yearlyTournament sc:name ?yearlyTournamentName .
           ?subTournament sc:name ?subTournamentName.
           ?winner sc:name ?winnerName.
           ?competitionClass sc:name ?competitionClassName.
        }
        GROUP BY ?winner ?winnerName ?tournamentName
        ORDER BY ?winnerName ?tot
    }
    #FILTER (regex(?winnerName,"Natasha Zvereva"))
}
GROUP BY ?winner ?winnerName
ORDER BY desc (?all)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q188080'), ('winnerName', 'Arantxa Sánchez Vicario'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q264493'), ('winnerName', 'Kathy Jordan'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q233243'), ('winnerName', 'Natasha Zvereva'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q132147'), ('winnerName', 'Sania Mirza'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q233332'), ('winnerName', 'Gigi Fernández'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q233247'), ('winnerName', 'Maria Bueno'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q14286'), ('winnerName', 'Chris Evert'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q11685'), ('winnerName', 'Kim Clijsters'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q11578'), ('winnerName', 'Venus Williams'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q10132'), ('winnerName', '

In [273]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# GROUP BY ?winner & ?tournament
# ORDERING
# LIMIT extended to 100

# CREATING outer query -> prob result is incorrect, it should have atmost 4 as there are only 4 tournaments possible but 
#                         it is keeping only one row for each player from inner query
# TRYING to debug with one player specifically ('Natasha Zvereva')
# FILTER with wrong parametes, added regex
# FIND, I need to group also in the outer select

# Checking correctness, ORDER BY 


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?winner ?winnerName (count(*) as ?all)
WHERE {
    {
        SELECT DISTINCT ?winner ?winnerName ?tournamentName (COUNT(*) AS ?tot)
        WHERE {
           # All four tourmamets in Grand Slam
           VALUES (?tournament){
               (wd:Q43605)  # French Open
               (wd:Q123577) # US Open
               (wd:Q41520)  # Wimbledon Championships
               (wd:Q60874)  # Australian Open
           }
           ?yearlyTournament wdt:P31 ?tournament.
           ?yearlyTournament wdt:P527 ?subTournament.
           ?subTournament wdt:P1346 ?winner;
                           wdt:P2094 ?competitionClass.

           ?winner wdt:P1532 ?countryForSport.

           # get the label 
           ?tournament sc:name ?tournamentName .
           ?yearlyTournament sc:name ?yearlyTournamentName .
           ?subTournament sc:name ?subTournamentName.
           ?winner sc:name ?winnerName.
           ?competitionClass sc:name ?competitionClassName.
        }
        GROUP BY ?winner ?winnerName ?tournamentName
        ORDER BY ?winnerName ?tot
    }
    #FILTER (regex(?winnerName,"Natasha Zvereva"))
}
GROUP BY ?winner ?winnerName
ORDER BY asc (?all)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q207352'), ('winnerName', 'Agnieszka Radwańska'), ('all', '1')]
[('winner', 'http://www.wikidata.org/entity/Q457476'), ('winnerName', 'Alexander Peya'), ('all', '1')]
[('winner', 'http://www.wikidata.org/entity/Q219724'), ('winnerName', 'Anabel Medina Garrigues'), ('all', '1')]
[('winner', 'http://www.wikidata.org/entity/Q458438'), ('winnerName', 'Ann Kiyomura'), ('all', '1')]
[('winner', 'http://www.wikidata.org/entity/Q273468'), ('winnerName', 'Blanche Bingley'), ('all', '1')]
[('winner', 'http://www.wikidata.org/entity/Q27537235'), ('winnerName', 'Caty McNally'), ('all', '1')]
[('winner', 'http://www.wikidata.org/entity/Q237894'), ('winnerName', 'Chanda Rubin'), ('all', '1')]
[('winner', 'http://www.wikidata.org/entity/Q54688'), ('winnerName', 'Coco Vandeweghe'), ('all', '1')]
[('winner', 'http://www.wikidata.org/entity/Q5363900'), ('winnerName', 'Elizaveta Kulichkova'), ('all', '1')]
[('winner', 'http://www.wikidata.org/entity/Q31

In [274]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# GROUP BY ?winner & ?tournament
# ORDERING
# LIMIT extended to 100

# CREATING outer query -> prob result is incorrect, it should have atmost 4 as there are only 4 tournaments possible but 
#                         it is keeping only one row for each player from inner query
# TRYING to debug with one player specifically ('Natasha Zvereva')
# FILTER with wrong parametes, added regex
# FIND, I need to group also in the outer select

# Checking correctness, ORDER BY + HAVING


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?winner ?winnerName (count(*) as ?all)
WHERE {
    {
        SELECT DISTINCT ?winner ?winnerName ?tournamentName (COUNT(*) AS ?tot)
        WHERE {
           # All four tourmamets in Grand Slam
           VALUES (?tournament){
               (wd:Q43605)  # French Open
               (wd:Q123577) # US Open
               (wd:Q41520)  # Wimbledon Championships
               (wd:Q60874)  # Australian Open
           }
           ?yearlyTournament wdt:P31 ?tournament.
           ?yearlyTournament wdt:P527 ?subTournament.
           ?subTournament wdt:P1346 ?winner;
                           wdt:P2094 ?competitionClass.

           ?winner wdt:P1532 ?countryForSport.

           # get the label 
           ?tournament sc:name ?tournamentName .
           ?yearlyTournament sc:name ?yearlyTournamentName .
           ?subTournament sc:name ?subTournamentName.
           ?winner sc:name ?winnerName.
           ?competitionClass sc:name ?competitionClassName.
        }
        GROUP BY ?winner ?winnerName ?tournamentName
        ORDER BY ?winnerName ?tot
    }
    #FILTER (regex(?winnerName,"Natasha Zvereva"))
}
GROUP BY ?winner ?winnerName
HAVING (count(*) >=3)
ORDER BY asc (?all)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q255011'), ('winnerName', 'Hana Mandlíková'), ('all', '3')]
[('winner', 'http://www.wikidata.org/entity/Q228989'), ('winnerName', 'Lucie Šafářová'), ('all', '3')]
[('winner', 'http://www.wikidata.org/entity/Q16475'), ('winnerName', 'Mats Wilander'), ('all', '3')]
[('winner', 'http://www.wikidata.org/entity/Q54537'), ('winnerName', 'Bethanie Mattek-Sands'), ('all', '3')]
[('winner', 'http://www.wikidata.org/entity/Q317820'), ('winnerName', 'Jamie Murray'), ('all', '3')]
[('winner', 'http://www.wikidata.org/entity/Q1289530'), ('winnerName', 'Marty Riessen'), ('all', '3')]
[('winner', 'http://www.wikidata.org/entity/Q241727'), ('winnerName', 'Angela Mortimer'), ('all', '3')]
[('winner', 'http://www.wikidata.org/entity/Q228559'), ('winnerName', 'Evonne Goolagong Cawley'), ('all', '3')]
[('winner', 'http://www.wikidata.org/entity/Q60980388'), ('winnerName', 'Jonáš Forejtek'), ('all', '3')]
[('winner', 'http://www.wikidata.org/entity/Q62773

In [275]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# GROUP BY ?winner & ?tournament
# ORDERING
# LIMIT extended to 100

# CREATING outer query -> prob result is incorrect, it should have atmost 4 as there are only 4 tournaments possible but 
#                         it is keeping only one row for each player from inner query
# TRYING to debug with one player specifically ('Natasha Zvereva')
# FILTER with wrong parametes, added regex
# FIND, I need to group also in the outer select

# Checking correctness, ORDER BY + HAVING, OK
# Selecting players those who won all four tournaments in GS


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?winner ?winnerName (count(*) as ?all)
WHERE {
    {
        SELECT DISTINCT ?winner ?winnerName ?tournamentName (COUNT(*) AS ?tot)
        WHERE {
           # All four tourmamets in Grand Slam
           VALUES (?tournament){
               (wd:Q43605)  # French Open
               (wd:Q123577) # US Open
               (wd:Q41520)  # Wimbledon Championships
               (wd:Q60874)  # Australian Open
           }
           ?yearlyTournament wdt:P31 ?tournament.
           ?yearlyTournament wdt:P527 ?subTournament.
           ?subTournament wdt:P1346 ?winner;
                           wdt:P2094 ?competitionClass.

           ?winner wdt:P1532 ?countryForSport.

           # get the label 
           ?tournament sc:name ?tournamentName .
           ?yearlyTournament sc:name ?yearlyTournamentName .
           ?subTournament sc:name ?subTournamentName.
           ?winner sc:name ?winnerName.
           ?competitionClass sc:name ?competitionClassName.
        }
        GROUP BY ?winner ?winnerName ?tournamentName
        ORDER BY ?winnerName ?tot
    }
    #FILTER (regex(?winnerName,"Natasha Zvereva"))
}
GROUP BY ?winner ?winnerName
HAVING (count(*) >=4)
ORDER BY asc (?all)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q188080'), ('winnerName', 'Arantxa Sánchez Vicario'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q264493'), ('winnerName', 'Kathy Jordan'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q233243'), ('winnerName', 'Natasha Zvereva'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q132147'), ('winnerName', 'Sania Mirza'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q233332'), ('winnerName', 'Gigi Fernández'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q233247'), ('winnerName', 'Maria Bueno'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q14286'), ('winnerName', 'Chris Evert'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q11685'), ('winnerName', 'Kim Clijsters'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q11578'), ('winnerName', 'Venus Williams'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q10132'), ('winnerName', '

In [276]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# GROUP BY ?winner & ?tournament
# ORDERING
# LIMIT extended to 100

# CREATING outer query -> prob result is incorrect, it should have atmost 4 as there are only 4 tournaments possible but 
#                         it is keeping only one row for each player from inner query
# TRYING to debug with one player specifically ('Natasha Zvereva')
# FILTER with wrong parametes, added regex
# FIND, I need to group also in the outer select

# Checking correctness, ORDER BY + HAVING, OK
# Selecting players those who won all four tournaments in GS

# removing ?winner wdt:P1532 ?countryForSport.


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?winner ?winnerName (count(*) as ?all)
WHERE {
    {
        SELECT DISTINCT ?winner ?winnerName ?tournamentName (COUNT(*) AS ?tot)
        WHERE {
           # All four tourmamets in Grand Slam
           VALUES (?tournament){
               (wd:Q43605)  # French Open
               (wd:Q123577) # US Open
               (wd:Q41520)  # Wimbledon Championships
               (wd:Q60874)  # Australian Open
           }
           ?yearlyTournament wdt:P31 ?tournament.
           ?yearlyTournament wdt:P527 ?subTournament.
           ?subTournament wdt:P1346 ?winner.

           # get the label 
           ?tournament sc:name ?tournamentName .
           ?winner sc:name ?winnerName.
        }
        GROUP BY ?winner ?winnerName ?tournamentName
        ORDER BY ?winnerName ?tot
    }
    #FILTER (regex(?winnerName,"Natasha Zvereva"))
}
GROUP BY ?winner ?winnerName
HAVING (count(*) >=4)
ORDER BY asc (?all)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q7407'), ('winnerName', 'Andre Agassi'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q188080'), ('winnerName', 'Arantxa Sánchez Vicario'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q52453'), ('winnerName', 'Bob Hewitt'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q264493'), ('winnerName', 'Kathy Jordan'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q233243'), ('winnerName', 'Natasha Zvereva'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q132147'), ('winnerName', 'Sania Mirza'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q233332'), ('winnerName', 'Gigi Fernández'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q233247'), ('winnerName', 'Maria Bueno'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q14286'), ('winnerName', 'Chris Evert'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q11685'), ('winnerName', 'Kim Cl

##### 10 rows without country for sport ! DEBUGING STARTED:

In [277]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# GROUP BY ?winner & ?tournament
# ORDERING
# LIMIT extended to 100

# CREATING outer query -> prob result is incorrect, it should have atmost 4 as there are only 4 tournaments possible but 
#                         it is keeping only one row for each player from inner query
# TRYING to debug with one player specifically ('Natasha Zvereva')
# FILTER with wrong parametes, added regex
# FIND, I need to group also in the outer select

# Checking correctness, ORDER BY + HAVING, OK
# Selecting players those who won all four tournaments in GS

# DEBUGING


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?winner ?winnerName (count(*) as ?all)
WHERE {
    {
        SELECT DISTINCT ?winner ?winnerName ?tournamentName (COUNT(*) AS ?tot)
        WHERE {
           # All four tourmamets in Grand Slam
           VALUES (?tournament){
               (wd:Q43605)  # French Open
               (wd:Q123577) # US Open
               (wd:Q41520)  # Wimbledon Championships
               (wd:Q60874)  # Australian Open
           }
           ?yearlyTournament wdt:P31 ?tournament.
           ?yearlyTournament wdt:P527 ?subTournament.
           ?subTournament wdt:P1346 ?winner.

           # get the label 
           ?tournament sc:name ?tournamentName .
           ?winner sc:name ?winnerName.
        }
        GROUP BY ?winner ?winnerName ?tournamentName
        ORDER BY ?winnerName ?tot
    }
    #FILTER (regex(?winnerName,"Natasha Zvereva"))
}
GROUP BY ?winner ?winnerName
HAVING (count(*) >=4)
ORDER BY asc (?all)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q7407'), ('winnerName', 'Andre Agassi'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q188080'), ('winnerName', 'Arantxa Sánchez Vicario'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q52453'), ('winnerName', 'Bob Hewitt'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q264493'), ('winnerName', 'Kathy Jordan'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q233243'), ('winnerName', 'Natasha Zvereva'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q132147'), ('winnerName', 'Sania Mirza'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q233332'), ('winnerName', 'Gigi Fernández'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q233247'), ('winnerName', 'Maria Bueno'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q14286'), ('winnerName', 'Chris Evert'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q11685'), ('winnerName', 'Kim Cl

In [278]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# GROUP BY ?winner & ?tournament
# ORDERING
# LIMIT extended to 100

# CREATING outer query -> prob result is incorrect, it should have atmost 4 as there are only 4 tournaments possible but 
#                         it is keeping only one row for each player from inner query
# TRYING to debug with one player specifically ('Natasha Zvereva')
# FILTER with wrong parametes, added regex
# FIND, I need to group also in the outer select

# Checking correctness, ORDER BY + HAVING, OK
# Selecting players those who won all four tournaments in GS

# DEBUGING

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?winner ?winnerName (count(*) as ?all)
WHERE {
    {
        SELECT DISTINCT ?winner ?winnerName ?tournamentName (COUNT(*) AS ?tot)
        WHERE {
           # All four tourmamets in Grand Slam
           VALUES (?tournament){
               (wd:Q43605)  # French Open
               (wd:Q123577) # US Open
               (wd:Q41520)  # Wimbledon Championships
               (wd:Q60874)  # Australian Open
           }
           ?yearlyTournament wdt:P31 ?tournament.
           ?yearlyTournament wdt:P527 ?subTournament.
           ?subTournament wdt:P1346 ?winner.
           ?winner wdt:P1532 ?countryForSport.

           # get the label 
           ?tournament sc:name ?tournamentName .
           ?winner sc:name ?winnerName.
        }
        GROUP BY ?winner ?winnerName ?tournamentName
        ORDER BY ?winnerName ?tot
    }
    #FILTER (regex(?winnerName,"Natasha Zvereva"))
}
GROUP BY ?winner ?winnerName
HAVING (count(*) >=4)
ORDER BY asc (?all)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q188080'), ('winnerName', 'Arantxa Sánchez Vicario'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q264493'), ('winnerName', 'Kathy Jordan'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q233243'), ('winnerName', 'Natasha Zvereva'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q132147'), ('winnerName', 'Sania Mirza'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q233332'), ('winnerName', 'Gigi Fernández'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q233247'), ('winnerName', 'Maria Bueno'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q14286'), ('winnerName', 'Chris Evert'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q11685'), ('winnerName', 'Kim Clijsters'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q11578'), ('winnerName', 'Venus Williams'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q10132'), ('winnerName', '

In [279]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# GROUP BY ?winner & ?tournament
# ORDERING
# LIMIT extended to 100

# CREATING outer query -> prob result is incorrect, it should have atmost 4 as there are only 4 tournaments possible but 
#                         it is keeping only one row for each player from inner query
# TRYING to debug with one player specifically ('Natasha Zvereva')
# FILTER with wrong parametes, added regex
# FIND, I need to group also in the outer select

# Checking correctness, ORDER BY + HAVING, OK
# Selecting players those who won all four tournaments in GS

# DEBUGING


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?winner ?winnerName (count(*) as ?all)
WHERE {
    {
        SELECT DISTINCT ?winner ?winnerName ?tournamentName (COUNT(*) AS ?tot)
        WHERE {
           # All four tourmamets in Grand Slam
           VALUES (?tournament){
               (wd:Q43605)  # French Open
               (wd:Q123577) # US Open
               (wd:Q41520)  # Wimbledon Championships
               (wd:Q60874)  # Australian Open
           }
           ?yearlyTournament wdt:P31 ?tournament.
           ?yearlyTournament wdt:P527 ?subTournament.
           ?subTournament wdt:P1346 ?winner.

           # get the label 
           ?tournament sc:name ?tournamentName .
           ?winner sc:name ?winnerName.
        }
        GROUP BY ?winner ?winnerName ?tournamentName
        ORDER BY ?winnerName ?tot
    }
    #FILTER (regex(?winnerName,"Natasha Zvereva"))
}
GROUP BY ?winner ?winnerName
HAVING (count(*) >=4)
ORDER BY asc (?winnerName)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q7407'), ('winnerName', 'Andre Agassi'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q270414'), ('winnerName', 'Anne Smith'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q188080'), ('winnerName', 'Arantxa Sánchez Vicario'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q11052500'), ('winnerName', 'Barbora Krejčíková'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q54527'), ('winnerName', 'Billie Jean King'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q53340'), ('winnerName', 'Bob Bryan'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q52453'), ('winnerName', 'Bob Hewitt'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q229128'), ('winnerName', 'Cara Black'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q14286'), ('winnerName', 'Chris Evert'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q24574661'), ('winnerName', 'Die

In [280]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# GROUP BY ?winner & ?tournament
# ORDERING
# LIMIT extended to 100

# CREATING outer query -> prob result is incorrect, it should have atmost 4 as there are only 4 tournaments possible but 
#                         it is keeping only one row for each player from inner query
# TRYING to debug with one player specifically ('Natasha Zvereva')
# FILTER with wrong parametes, added regex
# FIND, I need to group also in the outer select

# Checking correctness, ORDER BY + HAVING, OK
# Selecting players those who won all four tournaments in GS

# DEBUGING


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?winner ?winnerName (count(*) as ?all)
WHERE {
    {
        SELECT DISTINCT ?winner ?winnerName ?tournamentName (COUNT(*) AS ?tot)
        WHERE {
           # All four tourmamets in Grand Slam
           VALUES (?tournament){
               (wd:Q43605)  # French Open
               (wd:Q123577) # US Open
               (wd:Q41520)  # Wimbledon Championships
               (wd:Q60874)  # Australian Open
           }
           ?yearlyTournament wdt:P31 ?tournament.
           ?yearlyTournament wdt:P527 ?subTournament.
           ?subTournament wdt:P1346 ?winner.
           ?winner wdt:P1532 ?countryForSport.

           # get the label 
           ?tournament sc:name ?tournamentName .
           ?winner sc:name ?winnerName.
        }
        GROUP BY ?winner ?winnerName ?tournamentName
        ORDER BY ?winnerName ?tot
    }
    #FILTER (regex(?winnerName,"Natasha Zvereva"))
}
GROUP BY ?winner ?winnerName
HAVING (count(*) >=4)
ORDER BY asc (?winnerName)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q270414'), ('winnerName', 'Anne Smith'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q188080'), ('winnerName', 'Arantxa Sánchez Vicario'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q11052500'), ('winnerName', 'Barbora Krejčíková'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q54527'), ('winnerName', 'Billie Jean King'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q14286'), ('winnerName', 'Chris Evert'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q634844'), ('winnerName', 'Elena Vesnina'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q233332'), ('winnerName', 'Gigi Fernández'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q229151'), ('winnerName', 'Helena Suková'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q229121'), ('winnerName', 'Jana Novotná'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q426049'), ('winn

In [281]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# GROUP BY ?winner & ?tournament
# ORDERING
# LIMIT extended to 100

# CREATING outer query -> prob result is incorrect, it should have atmost 4 as there are only 4 tournaments possible but 
#                         it is keeping only one row for each player from inner query
# TRYING to debug with one player specifically ('Natasha Zvereva')
# FILTER with wrong parametes, added regex
# FIND, I need to group also in the outer select

# Checking correctness, ORDER BY + HAVING, OK
# Selecting players those who won all four tournaments in GS

# DEBUGING


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?winner ?winnerName (count(*) as ?all)
WHERE {
    {
        SELECT DISTINCT ?winner ?winnerName ?tournamentName (COUNT(*) AS ?tot)
        WHERE {
           # All four tourmamets in Grand Slam
           VALUES (?tournament){
               (wd:Q43605)  # French Open
               (wd:Q123577) # US Open
               (wd:Q41520)  # Wimbledon Championships
               (wd:Q60874)  # Australian Open
           }
           ?yearlyTournament wdt:P31 ?tournament.
           ?yearlyTournament wdt:P527 ?subTournament.
           ?subTournament wdt:P1346 ?winner.

           # get the label 
           ?tournament sc:name ?tournamentName .
           ?winner sc:name ?winnerName.
        }
        GROUP BY ?winner ?winnerName ?tournamentName
        ORDER BY ?winnerName ?tot
    }
    #FILTER (regex(?winnerName,"Natasha Zvereva"))
}
GROUP BY ?winner ?winnerName
HAVING (count(*) >=4)
ORDER BY asc (?winnerName)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q7407'), ('winnerName', 'Andre Agassi'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q270414'), ('winnerName', 'Anne Smith'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q188080'), ('winnerName', 'Arantxa Sánchez Vicario'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q11052500'), ('winnerName', 'Barbora Krejčíková'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q54527'), ('winnerName', 'Billie Jean King'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q53340'), ('winnerName', 'Bob Bryan'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q52453'), ('winnerName', 'Bob Hewitt'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q229128'), ('winnerName', 'Cara Black'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q14286'), ('winnerName', 'Chris Evert'), ('all', '4')]
[('winner', 'http://www.wikidata.org/entity/Q24574661'), ('winnerName', 'Die

In [282]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# GROUP BY ?winner & ?tournament
# ORDERING
# LIMIT extended to 100

# CREATING outer query -> prob result is incorrect, it should have atmost 4 as there are only 4 tournaments possible but 
#                         it is keeping only one row for each player from inner query
# TRYING to debug with one player specifically ('Natasha Zvereva')
# FILTER with wrong parametes, added regex
# FIND, I need to group also in the outer select

# Checking correctness, ORDER BY + HAVING, OK
# Selecting players those who won all four tournaments in GS

# DEBUGING


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
        SELECT DISTINCT ?winner ?winnerName ?p ?pName
        WHERE {
           VALUES (?winner){
               (wd:Q7407)# 'winnerName', 'Andre Agassi'), ('all', '4')]
                (wd:Q270414)# 'winnerName', 'Anne Smith'), ('all', '4')]
                (wd:Q188080)# 'winnerName', 'Arantxa Sánchez Vicario'), ('all', '4')]
                (wd:Q11052500)# 'winnerName', 'Barbora Krejčíková'), ('all', '4')]
                (wd:Q54527)# 'winnerName', 'Billie Jean King'), ('all', '4')]
                (wd:Q53340)# 'winnerName', 'Bob Bryan'), ('all', '4')]
                (wd:Q52453)# 'winnerName', 'Bob Hewitt'), ('all', '4')]
                (wd:Q229128)# 'winnerName', 'Cara Black'), ('all', '4')]
                (wd:Q14286)# 'winnerName', 'Chris Evert'), ('all', '4')]
                (wd:Q24574661)# 'winnerName', 'Diede de Groot'), ('all', '4')]
                (wd:Q634844)# 'winnerName', 'Elena Vesnina'), ('all', '4')]
                (wd:Q233332)# 'winnerName', 'Gigi Fernández'), ('all', '4')]
                (wd:Q229151)# 'winnerName', 'Helena Suková'), ('all', '4')]
                (wd:Q560151)# 'winnerName', 'Jacco Eltingh'), ('all', '4')]
                (wd:Q229121)# 'winnerName', 'Jana Novotná'), ('all', '4')]
                (wd:Q426049)# 'winnerName', 'John Fitzgerald'), ('all', '4')]
                (wd:Q264493)# 'winnerName', 'Kathy Jordan'), ('all', '4')]
                (wd:Q11685)# 'winnerName', 'Kim Clijsters'), ('all', '4')]
                (wd:Q296282)# 'winnerName', 'Leander Paes'), ('all', '4')]
                (wd:Q11677)# 'winnerName', 'Lindsay Davenport'), ('all', '4')]
                (wd:Q233247)# 'winnerName', 'Maria Bueno'), ('all', '4')]
                (wd:Q335333)# 'winnerName', 'Mark Knowles'), ('all', '4')]
                (wd:Q360630)# 'winnerName', 'Mark Woodforde'), ('all', '4')]
                (wd:Q134720)# 'winnerName', 'Martina Hingis'), ('all', '4')]
                (wd:Q54545)# 'winnerName', 'Martina Navratilova'), ('all', '4')]
                (wd:Q53342)# 'winnerName', 'Mike Bryan'), ('all', '4')]
                (wd:Q233243)# 'winnerName', 'Natasha Zvereva'), ('all', '4')]
                (wd:Q5812)# 'winnerName', 'Novak Djokovic'), ('all', '4')]
                (wd:Q54654)# 'winnerName', 'Pam Shriver'), ('all', '4')]
                (wd:Q2093033)# 'winnerName', 'Pierre-Hugues Herbert'), ('all', '4')]
                (wd:Q10132)# 'winnerName', 'Rafael Nadal'), ('all', '4')]
                (wd:Q1426)# 'winnerName', 'Roger Federer'), ('all', '4')]
                (wd:Q132147)# 'winnerName', 'Sania Mirza'), ('all', '4')]
                (wd:Q11459)# 'winnerName', 'Serena Williams'), ('all', '4')]
                (wd:Q11662)# 'winnerName', 'Steffi Graf'), ('all', '4')]
                (wd:Q335312)# 'winnerName', 'Todd Woodbridge'), ('all', '4')]
                (wd:Q11578)# 'winnerName', 'Venus Williams'), ('all', '4')]
                (wd:Q16224602)# 'winnerName', 'Yui Kamiji'), ('all', '4')]
           }
           ?winner ?p ?o.
           # get the label 
           ?winner sc:name ?winnerName.
           ?p sc:name ?pName.
           }
ORDER BY asc (?winnerName)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q7407'), ('winnerName', 'Andre Agassi'), ('p', 'http://www.wikidata.org/prop/direct/P3363'), ('pName', 'Tennis Hall of Fame player ID')]
[('winner', 'http://www.wikidata.org/entity/Q7407'), ('winnerName', 'Andre Agassi'), ('p', 'http://www.wikidata.org/prop/direct/P1006'), ('pName', 'Nationale Thesaurus voor Auteurs ID')]
[('winner', 'http://www.wikidata.org/entity/Q7407'), ('winnerName', 'Andre Agassi'), ('p', 'http://www.wikidata.org/prop/direct/P102'), ('pName', 'member of political party')]
[('winner', 'http://www.wikidata.org/entity/Q7407'), ('winnerName', 'Andre Agassi'), ('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation')]
[('winner', 'http://www.wikidata.org/entity/Q7407'), ('winnerName', 'Andre Agassi'), ('p', 'http://www.wikidata.org/prop/direct/P1207'), ('pName', 'NUKAT ID')]
[('winner', 'http://www.wikidata.org/entity/Q7407'), ('winnerName', 'Andre Agassi'), ('p', 'http://www.wikidata.org/prop/direct

In [283]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# GROUP BY ?winner & ?tournament
# ORDERING
# LIMIT extended to 100

# CREATING outer query -> prob result is incorrect, it should have atmost 4 as there are only 4 tournaments possible but 
#                         it is keeping only one row for each player from inner query
# TRYING to debug with one player specifically ('Natasha Zvereva')
# FILTER with wrong parametes, added regex
# FIND, I need to group also in the outer select

# Checking correctness, ORDER BY + HAVING, OK
# Selecting players those who won all four tournaments in GS

# DEBUGING


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
        SELECT DISTINCT ?winner ?winnerName ?p ?pName
        WHERE {
           VALUES (?winner){
               (wd:Q7407)# 'winnerName', 'Andre Agassi'), ('all', '4')]
                (wd:Q270414)# 'winnerName', 'Anne Smith'), ('all', '4')]
                (wd:Q188080)# 'winnerName', 'Arantxa Sánchez Vicario'), ('all', '4')]
                (wd:Q11052500)# 'winnerName', 'Barbora Krejčíková'), ('all', '4')]
                (wd:Q54527)# 'winnerName', 'Billie Jean King'), ('all', '4')]
                (wd:Q53340)# 'winnerName', 'Bob Bryan'), ('all', '4')]
                (wd:Q52453)# 'winnerName', 'Bob Hewitt'), ('all', '4')]
                (wd:Q229128)# 'winnerName', 'Cara Black'), ('all', '4')]
                (wd:Q14286)# 'winnerName', 'Chris Evert'), ('all', '4')]
                (wd:Q24574661)# 'winnerName', 'Diede de Groot'), ('all', '4')]
                (wd:Q634844)# 'winnerName', 'Elena Vesnina'), ('all', '4')]
                (wd:Q233332)# 'winnerName', 'Gigi Fernández'), ('all', '4')]
                (wd:Q229151)# 'winnerName', 'Helena Suková'), ('all', '4')]
                (wd:Q560151)# 'winnerName', 'Jacco Eltingh'), ('all', '4')]
                (wd:Q229121)# 'winnerName', 'Jana Novotná'), ('all', '4')]
                (wd:Q426049)# 'winnerName', 'John Fitzgerald'), ('all', '4')]
                (wd:Q264493)# 'winnerName', 'Kathy Jordan'), ('all', '4')]
                (wd:Q11685)# 'winnerName', 'Kim Clijsters'), ('all', '4')]
                (wd:Q296282)# 'winnerName', 'Leander Paes'), ('all', '4')]
                (wd:Q11677)# 'winnerName', 'Lindsay Davenport'), ('all', '4')]
                (wd:Q233247)# 'winnerName', 'Maria Bueno'), ('all', '4')]
                (wd:Q335333)# 'winnerName', 'Mark Knowles'), ('all', '4')]
                (wd:Q360630)# 'winnerName', 'Mark Woodforde'), ('all', '4')]
                (wd:Q134720)# 'winnerName', 'Martina Hingis'), ('all', '4')]
                (wd:Q54545)# 'winnerName', 'Martina Navratilova'), ('all', '4')]
                (wd:Q53342)# 'winnerName', 'Mike Bryan'), ('all', '4')]
                (wd:Q233243)# 'winnerName', 'Natasha Zvereva'), ('all', '4')]
                (wd:Q5812)# 'winnerName', 'Novak Djokovic'), ('all', '4')]
                (wd:Q54654)# 'winnerName', 'Pam Shriver'), ('all', '4')]
                (wd:Q2093033)# 'winnerName', 'Pierre-Hugues Herbert'), ('all', '4')]
                (wd:Q10132)# 'winnerName', 'Rafael Nadal'), ('all', '4')]
                (wd:Q1426)# 'winnerName', 'Roger Federer'), ('all', '4')]
                (wd:Q132147)# 'winnerName', 'Sania Mirza'), ('all', '4')]
                (wd:Q11459)# 'winnerName', 'Serena Williams'), ('all', '4')]
                (wd:Q11662)# 'winnerName', 'Steffi Graf'), ('all', '4')]
                (wd:Q335312)# 'winnerName', 'Todd Woodbridge'), ('all', '4')]
                (wd:Q11578)# 'winnerName', 'Venus Williams'), ('all', '4')]
                (wd:Q16224602)# 'winnerName', 'Yui Kamiji'), ('all', '4')]
           }
           VALUES(?p){
           (wdt:P1532)
           }
           ?winner ?p ?o.
           # get the label 
           ?winner sc:name ?winnerName.
           ?p sc:name ?pName.
           }
ORDER BY asc (?winnerName)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q270414'), ('winnerName', 'Anne Smith'), ('p', 'http://www.wikidata.org/prop/direct/P1532'), ('pName', 'country for sport')]
[('winner', 'http://www.wikidata.org/entity/Q188080'), ('winnerName', 'Arantxa Sánchez Vicario'), ('p', 'http://www.wikidata.org/prop/direct/P1532'), ('pName', 'country for sport')]
[('winner', 'http://www.wikidata.org/entity/Q11052500'), ('winnerName', 'Barbora Krejčíková'), ('p', 'http://www.wikidata.org/prop/direct/P1532'), ('pName', 'country for sport')]
[('winner', 'http://www.wikidata.org/entity/Q54527'), ('winnerName', 'Billie Jean King'), ('p', 'http://www.wikidata.org/prop/direct/P1532'), ('pName', 'country for sport')]
[('winner', 'http://www.wikidata.org/entity/Q14286'), ('winnerName', 'Chris Evert'), ('p', 'http://www.wikidata.org/prop/direct/P1532'), ('pName', 'country for sport')]
[('winner', 'http://www.wikidata.org/entity/Q634844'), ('winnerName', 'Elena Vesnina'), ('p', 'http://www.wikidata.org/

In [284]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# GROUP BY ?winner & ?tournament
# ORDERING
# LIMIT extended to 100

# CREATING outer query -> prob result is incorrect, it should have atmost 4 as there are only 4 tournaments possible but 
#                         it is keeping only one row for each player from inner query
# TRYING to debug with one player specifically ('Natasha Zvereva')
# FILTER with wrong parametes, added regex
# FIND, I need to group also in the outer select

# Checking correctness, ORDER BY + HAVING, OK
# Selecting players those who won all four tournaments in GS

# DEBUGING


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
        SELECT DISTINCT ?winner ?winnerName ?p ?pName
        WHERE {
           VALUES (?winner){
               (wd:Q7407)# 'winnerName', 'Andre Agassi'), ('all', '4')]
                (wd:Q270414)# 'winnerName', 'Anne Smith'), ('all', '4')]
                (wd:Q188080)# 'winnerName', 'Arantxa Sánchez Vicario'), ('all', '4')]
                (wd:Q11052500)# 'winnerName', 'Barbora Krejčíková'), ('all', '4')]
                (wd:Q54527)# 'winnerName', 'Billie Jean King'), ('all', '4')]
                (wd:Q53340)# 'winnerName', 'Bob Bryan'), ('all', '4')]
                (wd:Q52453)# 'winnerName', 'Bob Hewitt'), ('all', '4')]
                (wd:Q229128)# 'winnerName', 'Cara Black'), ('all', '4')]
                (wd:Q14286)# 'winnerName', 'Chris Evert'), ('all', '4')]
                (wd:Q24574661)# 'winnerName', 'Diede de Groot'), ('all', '4')]
                (wd:Q634844)# 'winnerName', 'Elena Vesnina'), ('all', '4')]
                (wd:Q233332)# 'winnerName', 'Gigi Fernández'), ('all', '4')]
                (wd:Q229151)# 'winnerName', 'Helena Suková'), ('all', '4')]
                (wd:Q560151)# 'winnerName', 'Jacco Eltingh'), ('all', '4')]
                (wd:Q229121)# 'winnerName', 'Jana Novotná'), ('all', '4')]
                (wd:Q426049)# 'winnerName', 'John Fitzgerald'), ('all', '4')]
                (wd:Q264493)# 'winnerName', 'Kathy Jordan'), ('all', '4')]
                (wd:Q11685)# 'winnerName', 'Kim Clijsters'), ('all', '4')]
                (wd:Q296282)# 'winnerName', 'Leander Paes'), ('all', '4')]
                (wd:Q11677)# 'winnerName', 'Lindsay Davenport'), ('all', '4')]
                (wd:Q233247)# 'winnerName', 'Maria Bueno'), ('all', '4')]
                (wd:Q335333)# 'winnerName', 'Mark Knowles'), ('all', '4')]
                (wd:Q360630)# 'winnerName', 'Mark Woodforde'), ('all', '4')]
                (wd:Q134720)# 'winnerName', 'Martina Hingis'), ('all', '4')]
                (wd:Q54545)# 'winnerName', 'Martina Navratilova'), ('all', '4')]
                (wd:Q53342)# 'winnerName', 'Mike Bryan'), ('all', '4')]
                (wd:Q233243)# 'winnerName', 'Natasha Zvereva'), ('all', '4')]
                (wd:Q5812)# 'winnerName', 'Novak Djokovic'), ('all', '4')]
                (wd:Q54654)# 'winnerName', 'Pam Shriver'), ('all', '4')]
                (wd:Q2093033)# 'winnerName', 'Pierre-Hugues Herbert'), ('all', '4')]
                (wd:Q10132)# 'winnerName', 'Rafael Nadal'), ('all', '4')]
                (wd:Q1426)# 'winnerName', 'Roger Federer'), ('all', '4')]
                (wd:Q132147)# 'winnerName', 'Sania Mirza'), ('all', '4')]
                (wd:Q11459)# 'winnerName', 'Serena Williams'), ('all', '4')]
                (wd:Q11662)# 'winnerName', 'Steffi Graf'), ('all', '4')]
                (wd:Q335312)# 'winnerName', 'Todd Woodbridge'), ('all', '4')]
                (wd:Q11578)# 'winnerName', 'Venus Williams'), ('all', '4')]
                (wd:Q16224602)# 'winnerName', 'Yui Kamiji'), ('all', '4')]
           }
           #VALUES(?p){
           #(wdt:P1532)
           #}
           ?winner ?p ?o.
           # get the label 
           ?winner sc:name ?winnerName.
           ?p sc:name ?pName.
           FILTER (regex(?pName, "country"))
           }
ORDER BY asc (?winnerName)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q7407'), ('winnerName', 'Andre Agassi'), ('p', 'http://www.wikidata.org/prop/direct/P27'), ('pName', 'country of citizenship')]
[('winner', 'http://www.wikidata.org/entity/Q270414'), ('winnerName', 'Anne Smith'), ('p', 'http://www.wikidata.org/prop/direct/P1532'), ('pName', 'country for sport')]
[('winner', 'http://www.wikidata.org/entity/Q270414'), ('winnerName', 'Anne Smith'), ('p', 'http://www.wikidata.org/prop/direct/P27'), ('pName', 'country of citizenship')]
[('winner', 'http://www.wikidata.org/entity/Q188080'), ('winnerName', 'Arantxa Sánchez Vicario'), ('p', 'http://www.wikidata.org/prop/direct/P1532'), ('pName', 'country for sport')]
[('winner', 'http://www.wikidata.org/entity/Q188080'), ('winnerName', 'Arantxa Sánchez Vicario'), ('p', 'http://www.wikidata.org/prop/direct/P27'), ('pName', 'country of citizenship')]
[('winner', 'http://www.wikidata.org/entity/Q11052500'), ('winnerName', 'Barbora Krejčíková'), ('p', 'http://www

##### End DEBUGGING: the 10 missing rows, they don't have P1532 (country for sport)

In [285]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# GROUP BY ?winner & ?tournament
# ORDERING
# LIMIT extended to 100
# CREATING outer query -> prob result is incorrect, it should have atmost 4 as there are only 4 tournaments possible but 
#                         it is keeping only one row for each player from inner query
# TRYING to debug with one player specifically ('Natasha Zvereva')
# FILTER with wrong parametes, added regex
# FIND, I need to group also in the outer select
# Checking correctness, ORDER BY + HAVING, OK
# Selecting players those who won all four tournaments in GS

# GETTING countryForSport for the winners



queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?player ?playerName ?winner ?winnerName ?countryForSport (count(*) as ?all)
WHERE {
    ?player wdt:P1532 ?countryForSport;
            sc:name ?playerName.
    FILTER (?player = ?winner)
    {
        SELECT DISTINCT ?winner ?winnerName ?tournamentName (COUNT(*) AS ?tot)
        WHERE {
           # All four tourmamets in Grand Slam
           VALUES (?tournament){
               (wd:Q43605)  # French Open
               (wd:Q123577) # US Open
               (wd:Q41520)  # Wimbledon Championships
               (wd:Q60874)  # Australian Open
           }
           ?yearlyTournament wdt:P31 ?tournament.
           ?yearlyTournament wdt:P527 ?subTournament.
           ?subTournament wdt:P1346 ?winner.

           # get the label 
           ?tournament sc:name ?tournamentName .
           ?winner sc:name ?winnerName.
        }
        GROUP BY ?winner ?winnerName ?tournamentName
        ORDER BY ?winnerName ?tot
    }
}
GROUP BY ?winner ?winnerName ?player ?playerName ?countryForSport
HAVING (count(*) >=4)
ORDER BY asc (?all)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q17381675'), ('playerName', 'Sania Mirza'), ('winner', 'http://www.wikidata.org/entity/Q17381675'), ('winnerName', 'Sania Mirza'), ('countryForSport', 'http://www.wikidata.org/entity/Q668'), ('all', '4')]
[('player', 'http://www.wikidata.org/entity/Q17381675'), ('playerName', 'Anne Smith'), ('winner', 'http://www.wikidata.org/entity/Q17381675'), ('winnerName', 'Anne Smith'), ('countryForSport', 'http://www.wikidata.org/entity/Q30'), ('all', '4')]
[('player', 'http://www.wikidata.org/entity/Q95162'), ('playerName', 'Venus Williams'), ('winner', 'http://www.wikidata.org/entity/Q95162'), ('winnerName', 'Venus Williams'), ('countryForSport', 'http://www.wikidata.org/entity/Q30'), ('all', '4')]
[('player', 'http://www.wikidata.org/entity/Q231962'), ('playerName', 'Yui Kamiji'), ('winner', 'http://www.wikidata.org/entity/Q231962'), ('winnerName', 'Yui Kamiji'), ('countryForSport', 'http://www.wikidata.org/entity/Q17'), ('all', '4')]
[('play

In [286]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# GROUP BY ?winner & ?tournament
# ORDERING
# LIMIT extended to 100
# CREATING outer query -> prob result is incorrect, it should have atmost 4 as there are only 4 tournaments possible but 
#                         it is keeping only one row for each player from inner query
# TRYING to debug with one player specifically ('Natasha Zvereva')
# FILTER with wrong parametes, added regex
# FIND, I need to group also in the outer select
# Checking correctness, ORDER BY + HAVING, OK
# Selecting players those who won all four tournaments in GS

# GETTING countryForSport for the winners, Trying to change GROUP BY have more than 28 results


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?player ?playerName ?countryForSport ?countryForSportName (count(*) as ?all)
WHERE {
    ?player wdt:P1532 ?countryForSport;
            sc:name ?playerName.
    ?countryForSport sc:name ?countryForSportName.
    FILTER (?player = ?winner)
    {
        SELECT DISTINCT ?winner ?winnerName ?tournamentName (COUNT(*) AS ?tot)
        WHERE {
           # All four tourmamets in Grand Slam
           VALUES (?tournament){
               (wd:Q43605)  # French Open
               (wd:Q123577) # US Open
               (wd:Q41520)  # Wimbledon Championships
               (wd:Q60874)  # Australian Open
           }
           ?yearlyTournament wdt:P31 ?tournament.
           ?yearlyTournament wdt:P527 ?subTournament.
           ?subTournament wdt:P1346 ?winner.

           # get the label 
           ?tournament sc:name ?tournamentName .
           ?winner sc:name ?winnerName.
        }
        GROUP BY ?winner ?winnerName ?tournamentName
        ORDER BY ?winnerName ?tot
    }
}
GROUP BY ?player ?playerName ?countryForSport ?countryForSportName
HAVING (count(*) >=4)
ORDER BY asc (?all)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q134720'), ('playerName', 'Martina Hingis'), ('countryForSport', 'http://www.wikidata.org/entity/Q39'), ('countryForSportName', 'Switzerland'), ('all', '4')]
[('player', 'http://www.wikidata.org/entity/Q296282'), ('playerName', 'Leander Paes'), ('countryForSport', 'http://www.wikidata.org/entity/Q668'), ('countryForSportName', 'India'), ('all', '4')]
[('player', 'http://www.wikidata.org/entity/Q5812'), ('playerName', 'Novak Djokovic'), ('countryForSport', 'http://www.wikidata.org/entity/Q403'), ('countryForSportName', 'Serbia'), ('all', '4')]
[('player', 'http://www.wikidata.org/entity/Q132147'), ('playerName', 'Sania Mirza'), ('countryForSport', 'http://www.wikidata.org/entity/Q668'), ('countryForSportName', 'India'), ('all', '4')]
[('player', 'http://www.wikidata.org/entity/Q634844'), ('playerName', 'Elena Vesnina'), ('countryForSport', 'http://www.wikidata.org/entity/Q159'), ('countryForSportName', 'Russia'), ('all', '4')]
[('playe

In [287]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# GROUP BY ?winner & ?tournament
# ORDERING
# LIMIT extended to 100
# CREATING outer query -> prob result is incorrect, it should have atmost 4 as there are only 4 tournaments possible but 
#                         it is keeping only one row for each player from inner query
# TRYING to debug with one player specifically ('Natasha Zvereva')
# FILTER with wrong parametes, added regex
# FIND, I need to group also in the outer select
# Checking correctness, ORDER BY + HAVING, OK
# Selecting players those who won all four tournaments in GS

# GETTING countryForSport for the winners, some players has more than one countryForSport
# GETTING END result


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?countryForSport ?countryForSportName (count(*) as ?all)
WHERE {
    ?player wdt:P1532 ?countryForSport;
            sc:name ?playerName.
    ?countryForSport sc:name ?countryForSportName.
    FILTER (?player = ?winner)
    {
        SELECT DISTINCT ?winner ?winnerName ?tournamentName (COUNT(*) AS ?tot)
        WHERE {
           # All four tourmamets in Grand Slam
           VALUES (?tournament){
               (wd:Q43605)  # French Open
               (wd:Q123577) # US Open
               (wd:Q41520)  # Wimbledon Championships
               (wd:Q60874)  # Australian Open
           }
           ?yearlyTournament wdt:P31 ?tournament.
           ?yearlyTournament wdt:P527 ?subTournament.
           ?subTournament wdt:P1346 ?winner.

           # get the label 
           ?tournament sc:name ?tournamentName .
           ?winner sc:name ?winnerName.
        }
        GROUP BY ?winner ?winnerName ?tournamentName
        ORDER BY ?winnerName ?tot
    }
}
GROUP BY ?player ?playerName ?countryForSport ?countryForSportName
HAVING (count(*) >=4)
ORDER BY asc (?all)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('countryForSport', 'http://www.wikidata.org/entity/Q39'), ('countryForSportName', 'Switzerland'), ('all', '4')]
[('countryForSport', 'http://www.wikidata.org/entity/Q668'), ('countryForSportName', 'India'), ('all', '4')]
[('countryForSport', 'http://www.wikidata.org/entity/Q403'), ('countryForSportName', 'Serbia'), ('all', '4')]
[('countryForSport', 'http://www.wikidata.org/entity/Q159'), ('countryForSportName', 'Russia'), ('all', '4')]
[('countryForSport', 'http://www.wikidata.org/entity/Q30'), ('countryForSportName', 'United States of America'), ('all', '4')]
[('countryForSport', 'http://www.wikidata.org/entity/Q183'), ('countryForSportName', 'Germany'), ('all', '4')]
[('countryForSport', 'http://www.wikidata.org/entity/Q155'), ('countryForSportName', 'Brazil'), ('all', '4')]
[('countryForSport', 'http://www.wikidata.org/entity/Q31'), ('countryForSportName', 'Belgium'), ('all', '4')]
[('countryForSport', 'http://www.wikidata.org/entity/Q142'), ('countryForSportName', 'Franc

#### SOL 2: any subTournament is eligible count as a win in a tournament, any of 4 tournaments can be won in different year

In [288]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

# GROUP BY ?winner & ?tournament
# ORDERING
# LIMIT extended to 100
# CREATING outer query -> prob result is incorrect, it should have atmost 4 as there are only 4 tournaments possible but 
#                         it is keeping only one row for each player from inner query
# TRYING to debug with one player specifically ('Natasha Zvereva')
# FILTER with wrong parametes, added regex
# FIND, I need to group also in the outer select
# Checking correctness, ORDER BY + HAVING, OK
# Selecting players those who won all four tournaments in GS

# GETTING countryForSport for the winners, some players has more than one countryForSport
# GETTING END result, removing wrong all


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?countryForSport ?countryForSportName
WHERE {
    ?player wdt:P1532 ?countryForSport;
            sc:name ?playerName.
    ?countryForSport sc:name ?countryForSportName.
    FILTER (?player = ?winner)
    {
        SELECT DISTINCT ?winner ?winnerName ?tournamentName (COUNT(*) AS ?tot)
        WHERE {
           # All four tourmamets in Grand Slam
           VALUES (?tournament){
               (wd:Q43605)  # French Open
               (wd:Q123577) # US Open
               (wd:Q41520)  # Wimbledon Championships
               (wd:Q60874)  # Australian Open
           }
           ?yearlyTournament wdt:P31 ?tournament.
           ?yearlyTournament wdt:P527 ?subTournament.
           ?subTournament wdt:P1346 ?winner.

           # get the label 
           ?tournament sc:name ?tournamentName .
           ?winner sc:name ?winnerName.
        }
        GROUP BY ?winner ?winnerName ?tournamentName
        ORDER BY ?winnerName ?tot
    }
}
GROUP BY ?player ?playerName ?countryForSport ?countryForSportName
HAVING (count(*) >=4)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('countryForSport', 'http://www.wikidata.org/entity/Q39'), ('countryForSportName', 'Switzerland')]
[('countryForSport', 'http://www.wikidata.org/entity/Q668'), ('countryForSportName', 'India')]
[('countryForSport', 'http://www.wikidata.org/entity/Q403'), ('countryForSportName', 'Serbia')]
[('countryForSport', 'http://www.wikidata.org/entity/Q159'), ('countryForSportName', 'Russia')]
[('countryForSport', 'http://www.wikidata.org/entity/Q30'), ('countryForSportName', 'United States of America')]
[('countryForSport', 'http://www.wikidata.org/entity/Q183'), ('countryForSportName', 'Germany')]
[('countryForSport', 'http://www.wikidata.org/entity/Q155'), ('countryForSportName', 'Brazil')]
[('countryForSport', 'http://www.wikidata.org/entity/Q31'), ('countryForSportName', 'Belgium')]
[('countryForSport', 'http://www.wikidata.org/entity/Q142'), ('countryForSportName', 'France')]
[('countryForSport', 'http://www.wikidata.org/entity/Q213'), ('countryForSportName', 'Czech Republic')]
[('

### Final query for this task:
#### 2 different interpretation of: Return the countries with at least one player who won at least one edition of each Grand Slam

#### SOL 1: 
    A) All tournaments has to be won in the same year,
    B) A player is considere if and only if he/she wins in the same category/competitionClass of every tournament

In [289]:
# write your final query
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).


# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')


queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?countryForSport ?countryForSportName
WHERE {
    # All four tourmamets in Grand Slam
   VALUES (?tournament){
       (wd:Q43605)  # French Open
       (wd:Q123577) # US Open
       (wd:Q41520)  # Wimbledon Championships
       (wd:Q60874)  # Australian Open
   }
   ?yearlyTournament wdt:P31 ?tournament;
                    wdt:P527 ?subTournament.
   ?subTournament wdt:P1346 ?winner;
                   wdt:P2094 ?competitionClass.
   
   ?winner wdt:P1532 ?countryForSport.
   
   # get the label 
   ?yearlyTournament sc:name ?yearlyTournamentName .
   ?countryForSport sc:name ?countryForSportName.
   
   BIND(xsd:integer(SUBSTR(?yearlyTournamentName,0,4)) as ?year)
}
GROUP BY ?winner ?countryForSport ?countryForSportName ?competitionClass ?year
HAVING (count(*) >=4)
ORDER BY DESC (?year)
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('countryForSport', 'http://www.wikidata.org/entity/Q183'), ('countryForSportName', 'Germany')]
1


#### SOL 2:
    A) Tournaments can be won in different year
    B) Any win in any subTournament is eligible to count as a win in its respective tournament

In [290]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

# P31 ('instance of')
# P527 ('has part')
# P1346 ('winner')
# P1532 ('country for sport')

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
# OUTER QUERY to get countryForSport of the players who won all four tournaments in GS (HAVING), no matters when
SELECT DISTINCT ?countryForSport ?countryForSportName
WHERE {
    ?player wdt:P1532 ?countryForSport;
            sc:name ?playerName.
    ?countryForSport sc:name ?countryForSportName.
    
    # JOINING OUTER TO INNER QUERY
    FILTER (?player = ?winner)
    {
        # INNER QUERY to get players and their wins in each tournament
        SELECT DISTINCT ?winner ?winnerName ?tournamentName (COUNT(*) AS ?tot)
        WHERE {
           # All four tourmamets in Grand Slam
           VALUES (?tournament){
               (wd:Q43605)  # French Open
               (wd:Q123577) # US Open
               (wd:Q41520)  # Wimbledon Championships
               (wd:Q60874)  # Australian Open
           }
           ?yearlyTournament wdt:P31 ?tournament.
           ?yearlyTournament wdt:P527 ?subTournament.
           ?subTournament wdt:P1346 ?winner.

           # get the label 
           ?tournament sc:name ?tournamentName .
           ?winner sc:name ?winnerName.
        }
        GROUP BY ?winner ?winnerName ?tournamentName
        ORDER BY ?winnerName ?tot
    }
}
GROUP BY ?player ?playerName ?countryForSport ?countryForSportName
HAVING (count(*) >=4)
ORDER BY ASC (?countryForSportName)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('countryForSport', 'http://www.wikidata.org/entity/Q408'), ('countryForSportName', 'Australia')]
[('countryForSport', 'http://www.wikidata.org/entity/Q184'), ('countryForSportName', 'Belarus')]
[('countryForSport', 'http://www.wikidata.org/entity/Q31'), ('countryForSportName', 'Belgium')]
[('countryForSport', 'http://www.wikidata.org/entity/Q155'), ('countryForSportName', 'Brazil')]
[('countryForSport', 'http://www.wikidata.org/entity/Q213'), ('countryForSportName', 'Czech Republic')]
[('countryForSport', 'http://www.wikidata.org/entity/Q33946'), ('countryForSportName', 'Czechoslovakia')]
[('countryForSport', 'http://www.wikidata.org/entity/Q142'), ('countryForSportName', 'France')]
[('countryForSport', 'http://www.wikidata.org/entity/Q183'), ('countryForSportName', 'Germany')]
[('countryForSport', 'http://www.wikidata.org/entity/Q668'), ('countryForSportName', 'India')]
[('countryForSport', 'http://www.wikidata.org/entity/Q17'), ('countryForSportName', 'Japan')]
[('countryFo

## Task 8
Consider the country for sport property of the tennis player (NO Country of citizenship). Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

In [291]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

# [('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]

# STARTING FROM Q1426, Roger Federer to find the ranking

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName
WHERE {
   wd:Q1426 ?p ?o .
   # get the label 
   ?p sc:name ?pName.
   #?o sc:name ?oName.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('pName', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('pName', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pName', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('pName', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('pName', 'CONOR.SI ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1284'), ('pName', 'Munzinger person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('pName', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('pName', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1352')

In [292]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

# P1532 ('country for sport')

# STARTING FROM Q1426, Roger Federer to find the ranking

# found: P1352 ('ranking') ,P166 ('award received'), P2522 ('victory')

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?o
WHERE {
   wd:Q1426 wdt:P1352 ?o .
   # get the label 
   #?p sc:name ?pName.
   #?o sc:name ?oName.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('o', '1')]
[('o', '24')]
2


In [293]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

# P1532 ('country for sport')

# STARTING FROM Q1426, Roger Federer to find the ranking

# found: P1352 ('ranking') ,P166 ('award received'), P2522 ('victory')

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?o ?oName
WHERE {
   wd:Q1426 wdt:P166 ?o .
   # get the label 
   #?p sc:name ?pName.
   ?o sc:name ?oName.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q56509472'), ('oName', 'honorary doctor of the University of Basel')]
[('o', 'http://www.wikidata.org/entity/Q1061233'), ('oName', "L'Équipe Champion of Champions")]
[('o', 'http://www.wikidata.org/entity/Q338737'), ('oName', 'Swiss Sports Personality of the Year')]
[('o', 'http://www.wikidata.org/entity/Q3494048'), ('oName', 'BBC Overseas Sports Personality of the Year')]
[('o', 'http://www.wikidata.org/entity/Q4896696'), ('oName', 'Best International Athlete ESPY Award')]
[('o', 'http://www.wikidata.org/entity/Q693708'), ('oName', 'ITF World Champions')]
6


In [294]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

# P1532 ('country for sport')

# STARTING FROM Q1426, Roger Federer to find the ranking

# found: P1352 ('ranking') ,P166 ('award received'), P2522 ('victory')

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?o ?oName
WHERE {
   wd:Q1426 wdt:P2522 ?o .
   # get the label 
   #?p sc:name ?pName.
   ?o sc:name ?oName.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q22956224'), ('oName', "2017 Australian Open – Men's Singles")]
[('o', 'http://www.wikidata.org/entity/Q612597'), ('oName', '2007 Tennis Masters Cup')]
[('o', 'http://www.wikidata.org/entity/Q782589'), ('oName', "2010 Australian Open – Men's Singles")]
[('o', 'http://www.wikidata.org/entity/Q782420'), ('oName', "2006 Australian Open – Men's Singles")]
[('o', 'http://www.wikidata.org/entity/Q300024'), ('oName', '2011 ATP World Tour Finals')]
[('o', 'http://www.wikidata.org/entity/Q753300'), ('oName', "2012 Wimbledon Championships – Gentlemen's Singles")]
[('o', 'http://www.wikidata.org/entity/Q757054'), ('oName', "tennis at the 2008 Summer Olympics – men's doubles")]
[('o', 'http://www.wikidata.org/entity/Q1074654'), ('oName', "2006 Wimbledon Championships – Men's Singles")]
[('o', 'http://www.wikidata.org/entity/Q384929'), ('oName', "2005 Wimbledon Championships – Men's Singles")]
[('o', 'http://www.wikidata.org/entity/Q14916861'), ('oName

#### Finding overall ranking of each tournament

In [295]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

# finding overall ranking, in other ways

# Q41520 wimbledon
queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName
WHERE {
   wd:Q41520 ?p ?o.
   # get the label 
   ?p sc:name ?pName.
   #?o sc:name ?oName.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('pName', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pName', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('pName', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P2003'), ('pName', 'Instagram username')]
[('p', 'http://www.wikidata.org/prop/direct/P2013'), ('pName', 'Facebook ID')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('pName', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2257'), 

In [296]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

#finding overall ranking

# Q41520 wimbledon
queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q41520 ?p ?o.
   # get the label 
   ?p sc:name ?pName.
   ?o sc:name ?oName.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pName', 'organizer'), ('o', 'http://www.wikidata.org/entity/Q815369'), ('oName', 'All England Lawn Tennis and Croquet Club')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('o', 'http://www.wikidata.org/entity/Q815369'), ('oName', 'All England Lawn Tennis and Croquet Club')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pName', 'organizer'), ('o', 'http://www.wikidata.org/entity/Q223041'), ('oName', 'International Tennis Federation')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q13219666'), ('oName', 'tennis tournament')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('o', 'http://www.wikidata.org/entity/Q145'), ('oName', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q18608583'), ('oName', 'recurring spor

In [297]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

#finding overall ranking

# Q41520 wimbledon
# [('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q102113'), ('oName', 'Grand Slam of tennis')]
queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName #?o ?oName
WHERE {
   wd:Q102113 ?p ?o.
   # get the label 
   ?p sc:name ?pName.
   #?o sc:name ?oName.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('pName', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pName', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('pName', 'has list')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P2924'), ('pName', 'Great Russian Encyclopedia Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3222'), ('pName', 'NE.se ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('pName', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('pName', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport')]
[('p', '

In [298]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

#finding overall ranking

# Q41520 wimbledon
# [('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q102113'), ('oName', 'Grand Slam of tennis')]
queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q102113 ?p ?o.
   # get the label 
   ?p sc:name ?pName.
   ?o sc:name ?oName.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q50301185'), ('oName', 'tennis tour with multiple editions')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q7700500'), ('oName', 'tennis tour')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('o', 'http://www.wikidata.org/entity/Q2223649'), ('oName', 'French cheese')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('o', 'http://www.wikidata.org/entity/Q5851433'), ('oName', 'Template:Infobox GrandSlamTournaments')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('o', 'http://www.wikidata.org/entity/Q847'), ('oName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('o', 'http://www.wikidata.org/entity/Q123577'), ('oName', 'US Open')]
[('p', 'http://www.wikidata.org/

In [299]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

#finding overall ranking

# Q41520 wimbledon
# [('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q102113'), ('oName', 'Grand Slam of tennis')]
queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?list ?listName ?p ?pName ?o ?oName
WHERE {
   wd:Q102113 wdt:P2354 ?list.
   ?list ?p ?o.
   # get the label 
   ?p sc:name ?pName.
   ?o sc:name ?oName.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('list', 'http://www.wikidata.org/entity/Q645455'), ('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pName', 'competition class'), ('o', 'http://www.wikidata.org/entity/Q16893403'), ('oName', "women's singles")]
[('list', 'http://www.wikidata.org/entity/Q1426439'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q13406463'), ('oName', 'Wikimedia list article')]
[('list', 'http://www.wikidata.org/entity/Q645455'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q13406463'), ('oName', 'Wikimedia list article')]
[('list', 'http://www.wikidata.org/entity/Q645455'), ('p', 'http://www.wikidata.org/prop/direct/P360'), ('pName', 'is a list of'), ('o', 'http://www.wikidata.org/entity/Q5'), ('oName', 'human')]
[('list', 'http://www.wikidata.org/entity/Q1426439'), ('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('o', 'http://www.w

In [300]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

#finding overall ranking

# Q41520 wimbledon
# [('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q102113'), ('oName', 'Grand Slam of tennis')]
queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT  ?s ?sName ?p ?pName ?list ?listName 
WHERE {
   wd:Q102113 wdt:P2354 ?list.
   # BACKWARD
   ?s ?p ?list.
   # get the label 
   ?p sc:name ?pName.
   ?s sc:name ?sName.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q102113'), ('sName', 'Grand Slam of tennis'), ('p', 'http://www.wikidata.org/prop/direct/P2354'), ('pName', 'has list'), ('list', 'http://www.wikidata.org/entity/Q645455')]
[('s', 'http://www.wikidata.org/entity/Q102113'), ('sName', 'Grand Slam of tennis'), ('p', 'http://www.wikidata.org/prop/direct/P2354'), ('pName', 'has list'), ('list', 'http://www.wikidata.org/entity/Q1426439')]
[('s', 'http://www.wikidata.org/entity/Q8498722'), ('sName', "Category:Grand Slam (tennis) champions in women's singles"), ('p', 'http://www.wikidata.org/prop/direct/P1753'), ('pName', 'list related to category'), ('list', 'http://www.wikidata.org/entity/Q645455')]
[('s', 'http://www.wikidata.org/entity/Q8498719'), ('sName', "Category:Grand Slam (tennis) champions in men's singles"), ('p', 'http://www.wikidata.org/prop/direct/P1753'), ('pName', 'list related to category'), ('list', 'http://www.wikidata.org/entity/Q1426439')]
4


In [301]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

#finding overall ranking

# Q41520 wimbledon
# [('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q102113'), ('oName', 'Grand Slam of tennis')]
queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT  ?p ?pName ?o
WHERE {
   wd:Q1426439 ?p ?o.
   # get the label 
   ?p sc:name ?pName.
   #?s sc:name ?sName.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1754'), ('pName', 'category related to list'), ('o', 'http://www.wikidata.org/entity/Q8498719')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pName', 'competition class'), ('o', 'http://www.wikidata.org/entity/Q16893072')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q13406463')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('o', 'http://www.wikidata.org/entity/Q847')]
4


In [302]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

#finding overall ranking

# Q41520 wimbledon
# [('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q102113'), ('oName', 'Grand Slam of tennis')]
queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT  ?p ?pName ?o ?oName 
WHERE {
   wd:Q1426439 ?p ?o.
   # get the label 
   ?p sc:name ?pName.
   ?o sc:name ?oName.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q13406463'), ('oName', 'Wikimedia list article')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('o', 'http://www.wikidata.org/entity/Q847'), ('oName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pName', 'competition class'), ('o', 'http://www.wikidata.org/entity/Q16893072'), ('oName', "men's singles")]
[('p', 'http://www.wikidata.org/prop/direct/P1754'), ('pName', 'category related to list'), ('o', 'http://www.wikidata.org/entity/Q8498719'), ('oName', "Category:Grand Slam (tennis) champions in men's singles")]
4


In [303]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

#finding overall ranking

# Q41520 wimbledon
queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT  ?p ?pName ?o ?oName 
WHERE {
   wd:Q41520 ?p ?o.
   # get the label 
   ?p sc:name ?pName.
   OPTIONAL { ?o sc:name ?oName }.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pName', 'organizer'), ('o', 'http://www.wikidata.org/entity/Q815369'), ('oName', 'All England Lawn Tennis and Croquet Club')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('o', 'http://www.wikidata.org/entity/Q815369'), ('oName', 'All England Lawn Tennis and Croquet Club')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pName', 'organizer'), ('o', 'http://www.wikidata.org/entity/Q223041'), ('oName', 'International Tennis Federation')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q13219666'), ('oName', 'tennis tournament')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('o', 'http://www.wikidata.org/entity/Q145'), ('oName', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q18608583'), ('oName', 'recurring spor

In [304]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

#finding overall ranking

# Q41520 wimbledon
queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT  ?p ?pName ?o ?oName 
WHERE {
   wd:Q13219666 ?p ?o.
   # get the label 
   ?p sc:name ?pName.
   OPTIONAL { ?o sc:name ?oName }.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q13406554'), ('oName', 'sports competition')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('o', 'http://www.wikidata.org/entity/Q847'), ('oName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('pName', 'has parts of the class'), ('o', 'http://www.wikidata.org/entity/Q46190676'), ('oName', 'tennis event')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('o', 'http://www.wikidata.org/entity/Q6452600'), ('oName', 'Template:ATP Tournament')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('pName', 'has list'), ('o', 'http://www.wikidata.org/entity/Q6642846'), ('oName', 'list of tennis tournaments')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('o', 'http://www.wikidata.org/entity/Q7474005'), ('oName', 'Template:Infobox tennis tourn

In [305]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

#finding overall ranking

# Q41520 wimbledon
queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT  ?p ?pName ?o ?oName 
WHERE {
   wd:Q6642846 ?p ?o.
   # get the label 
   ?p sc:name ?pName.
   OPTIONAL { ?o sc:name ?oName }.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P360'), ('pName', 'is a list of'), ('o', 'http://www.wikidata.org/entity/Q13219666'), ('oName', 'tennis tournament')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q13406463'), ('oName', 'Wikimedia list article')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('o', 'http://www.wikidata.org/entity/Q847'), ('oName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('pName', 'Commons category'), ('o', 'Tennis tournaments')]
4


In [306]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

#finding overall ranking

# Q41520 wimbledon
queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT  ?p ?pName ?o ?oName 
WHERE {
   wd:Q6452600 ?p ?o.
   # get the label 
   ?p sc:name ?pName.
   OPTIONAL { ?o sc:name ?oName }.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q11266439'), ('oName', 'Wikimedia template')]
[('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q13219666'), ('oName', 'tennis tournament')]
[('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q744089'), ('oName', 'Association of Tennis Professionals')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property'), ('o', 'http://www.wikidata.org/entity/P3456'), ('oName', 'ATP tennis tournament ID')]
4


In [307]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

#finding overall ranking

# Q41520 wimbledon
queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT  ?p ?pName ?o ?oName 
WHERE {
   wd:Q13219666 ?p ?o.
   # get the label 
   ?p sc:name ?pName.
   OPTIONAL { ?o sc:name ?oName }.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q13406554'), ('oName', 'sports competition')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('o', 'http://www.wikidata.org/entity/Q847'), ('oName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('pName', 'has parts of the class'), ('o', 'http://www.wikidata.org/entity/Q46190676'), ('oName', 'tennis event')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('o', 'http://www.wikidata.org/entity/Q6452600'), ('oName', 'Template:ATP Tournament')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('pName', 'has list'), ('o', 'http://www.wikidata.org/entity/Q6642846'), ('oName', 'list of tennis tournaments')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('o', 'http://www.wikidata.org/entity/Q7474005'), ('oName', 'Template:Infobox tennis tourn

In [308]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

#finding overall ranking

# Q41520 wimbledon
queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT  ?p ?pName ?o ?oName 
WHERE {
   wd:Q6642846 ?p ?o.
   # get the label 
   ?p sc:name ?pName.
   OPTIONAL { ?o sc:name ?oName }.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P360'), ('pName', 'is a list of'), ('o', 'http://www.wikidata.org/entity/Q13219666'), ('oName', 'tennis tournament')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q13406463'), ('oName', 'Wikimedia list article')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('o', 'http://www.wikidata.org/entity/Q847'), ('oName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('pName', 'Commons category'), ('o', 'Tennis tournaments')]
4


In [309]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

#finding overall ranking

# Q41520 wimbledon
queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT  ?p ?pName ?o ?oName 
WHERE {
   ?o ?p wd:Q13219666 .
   # get the label 
   ?p sc:name ?pName.
   OPTIONAL { ?o sc:name ?oName }.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q299931'), ('oName', 'Chile Open')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q299384'), ('oName', 'Delray Beach International Tennis Championships')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q1961762'), ('oName', 'Mersin Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q2415348'), ('oName', 'Masters France')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q3935780'), ('oName', 'Rijeka Open')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q3954502'), ('oName', 'Seguros Bolívar Open Cali')]
[('p', 'http://www.w

In [310]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

#finding overall ranking

# Q41520 wimbledon
queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT  ?p ?pName ?o ?oName 
WHERE {
   ?o ?p wd:Q26863790 .
   # get the label 
   ?p sc:name ?pName.
   OPTIONAL { ?o sc:name ?oName }.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q13222012'), ('oName', 'Kremlin Cup (men)')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q17485677'), ('oName', 'Washington Open (men)')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q270907'), ('oName', 'ATP Finals')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q16465941'), ('oName', 'ATP Dubai Tennis Championships')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q299406'), ('oName', 'Power Horse Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q1993182'), ('oName', 'Open de la Réunion')]
[('p', 'http://www.wik

In [311]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

#finding overall ranking

# Q41520 wimbledon
queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT  ?p ?pName ?o ?oName 
WHERE {
   wd:Q26863790 ?p ?o .
   # get the label 
   ?p sc:name ?pName.
   OPTIONAL { ?o sc:name ?oName }.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pName', 'competition class'), ('o', 'http://www.wikidata.org/entity/Q56054951'), ('oName', "men's tennis")]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q13219666'), ('oName', 'tennis tournament')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pName', 'organizer'), ('o', 'http://www.wikidata.org/entity/Q744089'), ('oName', 'Association of Tennis Professionals')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('o', 'http://www.wikidata.org/entity/Q847'), ('oName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('o', 'http://www.wikidata.org/entity/Q6453914'), ('oName', 'Template:Infobox ATP Tennis Tournament')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('pName', 'KBpedia ID'), ('o', 'ATPTennisTournament')]
6


In [312]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

#finding overall ranking

# Q41520 wimbledon
queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT  ?p ?pName ?o ?oName 
WHERE {
   wd:Q744089 ?p ?o .
   # get the label 
   ?p sc:name ?pName.
   OPTIONAL { ?o sc:name ?oName }.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('pName', 'headquarters location'), ('o', 'http://www.wikidata.org/entity/Q84'), ('oName', 'London')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('o', 'http://www.wikidata.org/entity/Q847'), ('oName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q4438121'), ('oName', 'sports organization')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('o', 'http://www.wikidata.org/entity/Q6452600'), ('oName', 'Template:ATP Tournament')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property'), ('o', 'http://www.wikidata.org/entity/P536'), ('oName', 'ATP player ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property'), ('o', 'http://www.wikidata.org/entity/P6880'), ('oName', 'ATP tennis tournament edition ID')]
[('p', 'http://www.wiki

In [313]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

#finding overall ranking

# Q41520 wimbledon

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q41520 ?p ?o.
   # get the label 
   OPTIONAL { ?p sc:name ?pName }.
   OPTIONAL { ?o sc:name ?oName }.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q736742'), ('oName', 'Wimbledon')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('o', 'http://www.wikidata.org/entity/Q43323825'), ('oName', 'Template:Wimbledon player')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property'), ('o', 'http://www.wikidata.org/entity/P4503'), ('oName', 'Wimbledon player ID')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('o', 'http://www.wikidata.org/entity/Q145'), ('oName', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from'), ('o', 'http://www.wikidata.org/entity/Q206375'), ('oName', 'Wimbledon')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('pName', 'has list'), ('o', 'http://www.wikidata.org/entity/Q6585893'), ('oName', 'list of Wimbledon 

In [314]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

#finding overall ranking

# Q41520 wimbledon

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q223041 ?p ?o.
   # get the label 
   OPTIONAL { ?p sc:name ?pName }.
   OPTIONAL { ?o sc:name ?oName }.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('o', 'http://www.wikidata.org/entity/Q13233631'), ('oName', 'Template:ITF profile')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('pName', 'headquarters location'), ('o', 'http://www.wikidata.org/entity/Q84'), ('oName', 'London')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property'), ('o', 'http://www.wikidata.org/entity/P599'), ('oName', 'ITF player ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property'), ('o', 'http://www.wikidata.org/entity/P4299'), ('oName', 'ITF wheelchair player ID until 2019')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property'), ('o', 'http://www.wikidata.org/entity/P6841'), ('oName', 'ITF tournament ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property'), ('o', 'http://www.wikidata.org/entity/P8618'), ('oName', 'ITF player ID 2

In [315]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

#finding overall ranking

# Q41520 wimbledon

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q102113 ?p ?o.
   # get the label 
   OPTIONAL { ?p sc:name ?pName }.
   OPTIONAL { ?o sc:name ?oName }.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('o', 'http://www.wikidata.org/entity/Q5851433'), ('oName', 'Template:Infobox GrandSlamTournaments')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('pName', 'has list'), ('o', 'http://www.wikidata.org/entity/Q645455'), ('oName', "list of Grand Slam women's singles champions")]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('pName', 'has list'), ('o', 'http://www.wikidata.org/entity/Q1426439'), ('oName', "list of Grand Slam men's singles champions")]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q7700500'), ('oName', 'tennis tour')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q50301185'), ('oName', 'tennis tour with multiple editions')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('o', 'http://www.wikidata.o

In [316]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

#finding overall ranking

# Q41520 wimbledon

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   ?o ?p wd:Q102113.
   # get the label 
   OPTIONAL { ?p sc:name ?pName }.
   OPTIONAL { ?o sc:name ?oName }.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q5851433'), ('oName', 'Template:Infobox GrandSlamTournaments')]
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pName', 'victory'), ('o', 'http://www.wikidata.org/entity/Q105868'), ('oName', 'Gabriela Sabatini')]
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pName', 'victory'), ('o', 'http://www.wikidata.org/entity/Q105838'), ('oName', 'Marion Bartoli')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q11300305'), ('oName', 'Grand Slam')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('pName', "category's main topic"), ('o', 'http://www.wikidata.org/entity/Q18156144'), ('oName', 'Category:Grand Slam (tennis)')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q41520'), ('oName', 'Wimbledon Championships')]

In [317]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

#finding overall ranking

# Q41520 wimbledon

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q11300305 ?p ?o.
   # get the label 
   OPTIONAL { ?p sc:name ?pName }.
   OPTIONAL { ?o sc:name ?oName }.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q102113'), ('oName', 'Grand Slam of tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('o', 'http://www.wikidata.org/entity/Q1153025'), ('oName', 'wheelchair tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('pName', 'Google Knowledge Graph ID'), ('o', '/g/121xqb7v')]
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('o', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P2671'), ('o', 'http://g.co/kg/g/121xqb7v')]
[('p', 'http://schema.org/name'), ('o', 'Grand Slam')]
6


In [318]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

#finding overall ranking

# Q41520 wimbledon

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   ?o ?p wd:Q11300305.
   # get the label 
   OPTIONAL { ?p sc:name ?pName }.
   OPTIONAL { ?o sc:name ?oName }.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://schema.org/about'), ('o', 'https://www.wikidata.org/wiki/Special:EntityData/Q11300305')]
1


#### Can't find overlaal ranking, back to solution 1 with prop wdt:P1352 ('ranking')

In [319]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

#finding overall ranking

# Q41520 wimbledon
queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT  ?p ?pName ?o ?oName 
WHERE {
   wd:Q1426 ?p ?o .
   # get the label 
   ?p sc:name ?pName.
   OPTIONAL { ?o sc:name ?oName }.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pName', 'victory'), ('o', 'http://www.wikidata.org/entity/Q22956224'), ('oName', "2017 Australian Open – Men's Singles")]
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pName', 'victory'), ('o', 'http://www.wikidata.org/entity/Q612597'), ('oName', '2007 Tennis Masters Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P3342'), ('pName', 'significant person'), ('o', 'http://www.wikidata.org/entity/Q54584'), ('oName', 'Andy Roddick')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received'), ('o', 'http://www.wikidata.org/entity/Q56509472'), ('oName', 'honorary doctor of the University of Basel')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in'), ('o', 'http://www.wikidata.org/entity/Q589310'), ('oName', "tennis at the 2000 Summer Olympics – men's singles")]
[('p', 'http://www.wikidata.org/prop/direct/P6758'), ('pName', 'supported sports team'), ('o', 'http://www.

In [320]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

# Q41520 wimbledon

# P106('occupation'),
# Q10833314 ('tennis player')

# GETTING ALL TENNIS PLAYER
queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?o ?oName 
WHERE {
   ?o wdt:P106 wd:Q10833314 .
   # get the label 
   OPTIONAL { ?o sc:name ?oName }.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1683632'), ('oName', 'Jaroslav Navrátil')]
[('o', 'http://www.wikidata.org/entity/Q138094'), ('oName', 'Anna Karolína Schmiedlová')]
[('o', 'http://www.wikidata.org/entity/Q15197746'), ('oName', 'Wesley Koolhof')]
[('o', 'http://www.wikidata.org/entity/Q1558013'), ('oName', 'Pierre Barthès')]
[('o', 'http://www.wikidata.org/entity/Q15849678'), ('oName', 'Sue Partridge')]
[('o', 'http://www.wikidata.org/entity/Q15880574'), ('oName', 'Paul Oosterbaan')]
[('o', 'http://www.wikidata.org/entity/Q240837'), ('oName', 'Rika Fujiwara')]
[('o', 'http://www.wikidata.org/entity/Q4134097'), ('oName', 'Margarita Gasparyan')]
[('o', 'http://www.wikidata.org/entity/Q589611'), ('oName', 'Philippe Chatrier')]
[('o', 'http://www.wikidata.org/entity/Q16009513'), ('oName', 'James Bayley')]
[('o', 'http://www.wikidata.org/entity/Q16023406'), ('oName', 'Samuel T. Chase')]
[('o', 'http://www.wikidata.org/entity/Q16063202'), ('oName', 'Agnete Goldschmidt')]
[('o'

In [321]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

# Q41520 wimbledon

# P106('occupation'),
# Q10833314 ('tennis player')
# --P1352 ('ranking')--> ('o', '1')

# GETTING ALL TENNIS PLAYER
# -> GETTING their ranking

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?o ?oName ?ranking
WHERE {
   ?o wdt:P106 wd:Q10833314;
        wdt:P1352 ?ranking
   # get the label 
   OPTIONAL { ?o sc:name ?oName }.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q138094'), ('oName', 'Anna Karolína Schmiedlová'), ('ranking', '26')]
[('o', 'http://www.wikidata.org/entity/Q15197746'), ('oName', 'Wesley Koolhof'), ('ranking', '5')]
[('o', 'http://www.wikidata.org/entity/Q15197746'), ('oName', 'Wesley Koolhof'), ('ranking', '462')]
[('o', 'http://www.wikidata.org/entity/Q240837'), ('oName', 'Rika Fujiwara'), ('ranking', '13')]
[('o', 'http://www.wikidata.org/entity/Q240837'), ('oName', 'Rika Fujiwara'), ('ranking', '84')]
[('o', 'http://www.wikidata.org/entity/Q4134097'), ('oName', 'Margarita Gasparyan'), ('ranking', '25')]
[('o', 'http://www.wikidata.org/entity/Q7281705'), ('oName', 'Radu Albot'), ('ranking', '56')]
[('o', 'http://www.wikidata.org/entity/Q7281705'), ('oName', 'Radu Albot'), ('ranking', '108')]
[('o', 'http://www.wikidata.org/entity/Q18918709'), ('oName', 'Markéta Vondroušová'), ('ranking', '14')]
[('o', 'http://www.wikidata.org/entity/Q6408932'), ('oName', 'Kim Jones'), ('ranking', '2

In [322]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

# Q41520 wimbledon

# P106('occupation'),
# Q10833314 ('tennis player')
# --P1352 ('ranking')--> ('o', '1')

# GETTING ALL TENNIS PLAYER
# GETTING their ranking
# -> FILTERING, only who was ranked 1

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?o ?oName ?ranking
WHERE {
   ?o wdt:P106 wd:Q10833314;
        wdt:P1352 ?ranking
   # get the label 
   OPTIONAL { ?o sc:name ?oName }.
   FILTER (xsd:integer(?ranking) = 1)
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q207352'), ('oName', 'Agnieszka Radwańska'), ('ranking', '1')]
[('o', 'http://www.wikidata.org/entity/Q455136'), ('oName', 'Anders Järryd'), ('ranking', '1')]
[('o', 'http://www.wikidata.org/entity/Q455411'), ('oName', 'Tom Okker'), ('ranking', '1')]
[('o', 'http://www.wikidata.org/entity/Q310419'), ('oName', 'Nenad Zimonjić'), ('ranking', '1')]
[('o', 'http://www.wikidata.org/entity/Q53393'), ('oName', 'Jimmy Connors'), ('ranking', '1')]
[('o', 'http://www.wikidata.org/entity/Q10118'), ('oName', 'Victoria Azarenka'), ('ranking', '1')]
[('o', 'http://www.wikidata.org/entity/Q13538'), ('oName', 'Henri Kontinen'), ('ranking', '1')]
[('o', 'http://www.wikidata.org/entity/Q1743002'), ('oName', 'Robert Seguso'), ('ranking', '1')]
[('o', 'http://www.wikidata.org/entity/Q189542'), ('oName', 'Stefan Edberg'), ('ranking', '1')]
[('o', 'http://www.wikidata.org/entity/Q232566'), ('oName', 'Hsieh Su-wei'), ('ranking', '1')]
[('o', 'http://www.wikidata

In [323]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

# Q41520 wimbledon

# P106('occupation'),
# Q10833314 ('tennis player')
# --P1352 ('ranking')--> ('o', '1')

# GETTING ALL TENNIS PLAYER
# GETTING their ranking
# FILTERING, only who was ranked 1
# -> getting their countryForSport

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?o ?oName ?ranking ?countryForSport ?countryForSportName
WHERE {
   ?o wdt:P106 wd:Q10833314;
        wdt:P1352 ?ranking;
        wdt:P1532 ?countryForSport.
   # get the label 
   ?countryForSport sc:name ?countryForSportName
   OPTIONAL { ?o sc:name ?oName }.
   FILTER (xsd:integer(?ranking) = 1)
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q232566'), ('oName', 'Hsieh Su-wei'), ('ranking', '1'), ('countryForSport', 'http://www.wikidata.org/entity/Q216923'), ('countryForSportName', 'Chinese Taipei')]
[('o', 'http://www.wikidata.org/entity/Q11662'), ('oName', 'Steffi Graf'), ('ranking', '1'), ('countryForSport', 'http://www.wikidata.org/entity/Q183'), ('countryForSportName', 'Germany')]
[('o', 'http://www.wikidata.org/entity/Q76334'), ('oName', 'Boris Becker'), ('ranking', '1'), ('countryForSport', 'http://www.wikidata.org/entity/Q183'), ('countryForSportName', 'Germany')]
[('o', 'http://www.wikidata.org/entity/Q10118'), ('oName', 'Victoria Azarenka'), ('ranking', '1'), ('countryForSport', 'http://www.wikidata.org/entity/Q184'), ('countryForSportName', 'Belarus')]
[('o', 'http://www.wikidata.org/entity/Q289365'), ('oName', 'Karolína Plíšková'), ('ranking', '1'), ('countryForSport', 'http://www.wikidata.org/entity/Q213'), ('countryForSportName', 'Czech Republic')]
[('o', 'http:/

In [324]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

# Q41520 wimbledon

# P106('occupation'),
# Q10833314 ('tennis player')
# --P1352 ('ranking')--> ('o', '1')

# GETTING ALL TENNIS PLAYER
# GETTING their ranking
# FILTERING, only who was ranked 1
# getting their countryForSport
# RETURNing distinc countries

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?countryForSport ?countryForSportName (COUNT(*) as ?tot)
WHERE {
   ?o wdt:P106 wd:Q10833314;
        wdt:P1352 ?ranking;
        wdt:P1532 ?countryForSport.
   # get the label 
   ?countryForSport sc:name ?countryForSportName.
   OPTIONAL { ?o sc:name ?oName }.
   FILTER (xsd:integer(?ranking) = 1)
}
GROUP BY ?countryForSport ?countryForSportName
ORDER BY desc (?tot)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('countryForSport', 'http://www.wikidata.org/entity/Q30'), ('countryForSportName', 'United States of America'), ('tot', '21')]
[('countryForSport', 'http://www.wikidata.org/entity/Q159'), ('countryForSportName', 'Russia'), ('tot', '9')]
[('countryForSport', 'http://www.wikidata.org/entity/Q213'), ('countryForSportName', 'Czech Republic'), ('tot', '8')]
[('countryForSport', 'http://www.wikidata.org/entity/Q29'), ('countryForSportName', 'Spain'), ('tot', '7')]
[('countryForSport', 'http://www.wikidata.org/entity/Q142'), ('countryForSportName', 'France'), ('tot', '5')]
[('countryForSport', 'http://www.wikidata.org/entity/Q408'), ('countryForSportName', 'Australia'), ('tot', '4')]
[('countryForSport', 'http://www.wikidata.org/entity/Q183'), ('countryForSportName', 'Germany'), ('tot', '3')]
[('countryForSport', 'http://www.wikidata.org/entity/Q34'), ('countryForSportName', 'Sweden'), ('tot', '3')]
[('countryForSport', 'http://www.wikidata.org/entity/Q184'), ('countryForSportName', 

In [325]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

# Q41520 wimbledon

# P106('occupation'),
# Q10833314 ('tennis player')
# --P1352 ('ranking')--> ('o', '1')

# GETTING ALL TENNIS PLAYER
# GETTING their ranking
# FILTERING, only who was ranked 1
# getting their countryForSport
# RETURNing distinc countries 
# REMOVED counting
queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?countryForSport ?countryForSportName
WHERE {
   ?o wdt:P106 wd:Q10833314;
        wdt:P1352 ?ranking;
        wdt:P1532 ?countryForSport.
   # get the label 
   ?countryForSport sc:name ?countryForSportName.
   OPTIONAL { ?o sc:name ?oName }.
   FILTER (xsd:integer(?ranking) = 1)
}
GROUP BY ?countryForSport ?countryForSportName
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('countryForSport', 'http://www.wikidata.org/entity/Q215'), ('countryForSportName', 'Slovenia')]
[('countryForSport', 'http://www.wikidata.org/entity/Q218'), ('countryForSportName', 'Romania')]
[('countryForSport', 'http://www.wikidata.org/entity/Q16'), ('countryForSportName', 'Canada')]
[('countryForSport', 'http://www.wikidata.org/entity/Q38'), ('countryForSportName', 'Italy')]
[('countryForSport', 'http://www.wikidata.org/entity/Q216923'), ('countryForSportName', 'Chinese Taipei')]
[('countryForSport', 'http://www.wikidata.org/entity/Q183'), ('countryForSportName', 'Germany')]
[('countryForSport', 'http://www.wikidata.org/entity/Q83286'), ('countryForSportName', 'Socialist Federal Republic of Yugoslavia')]
[('countryForSport', 'http://www.wikidata.org/entity/Q34'), ('countryForSportName', 'Sweden')]
[('countryForSport', 'http://www.wikidata.org/entity/Q31'), ('countryForSportName', 'Belgium')]
[('countryForSport', 'http://www.wikidata.org/entity/Q403'), ('countryForSportNam

In [326]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

# Q41520 wimbledon

# P106('occupation'),
# Q10833314 ('tennis player')
# --P1352 ('ranking')--> ('o', '1')

# GETTING ALL TENNIS PLAYER
# GETTING their ranking
# FILTERING, only who was ranked 1
# getting their countryForSport
# RETURNing distinct countries

# checking winner from Italy

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?o ?oName ?ranking ?countryForSport ?countryForSportName
WHERE {
   ?o wdt:P106 wd:Q10833314;
        wdt:P1352 ?ranking;
        wdt:P1532 ?countryForSport.
   # get the label 
   ?countryForSport sc:name ?countryForSportName
   OPTIONAL { ?o sc:name ?oName }.
   FILTER (xsd:integer(?ranking) = 1)
   FILTER (regex(?countryForSportName,"Italy"))
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q95162'), ('oName', 'Roberta Vinci'), ('ranking', '1'), ('countryForSport', 'http://www.wikidata.org/entity/Q38'), ('countryForSportName', 'Italy')]
[('o', 'http://www.wikidata.org/entity/Q207528'), ('oName', 'Flavia Pennetta'), ('ranking', '1'), ('countryForSport', 'http://www.wikidata.org/entity/Q38'), ('countryForSportName', 'Italy')]
2


In [327]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

# Q41520 wimbledon

# P106('occupation'),
# Q10833314 ('tennis player')
# --P1352 ('ranking')--> ('o', '1')

# GETTING ALL TENNIS PLAYER
# GETTING their ranking
# FILTERING, only who was ranked 1
# getting their countryForSport
# RETURNing distinct countries

# checking winner from Italy

# RESULT with ordering

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?countryForSport ?countryForSportName
WHERE {
   ?o wdt:P106 wd:Q10833314;
        wdt:P1352 ?ranking;
        wdt:P1532 ?countryForSport.
   # get the label 
   ?countryForSport sc:name ?countryForSportName.
   OPTIONAL { ?o sc:name ?oName }.
   FILTER (xsd:integer(?ranking) = 1)
}
GROUP BY ?countryForSport ?countryForSportName
ORDER BY ?countryForSportName
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('countryForSport', 'http://www.wikidata.org/entity/Q408'), ('countryForSportName', 'Australia')]
[('countryForSport', 'http://www.wikidata.org/entity/Q184'), ('countryForSportName', 'Belarus')]
[('countryForSport', 'http://www.wikidata.org/entity/Q31'), ('countryForSportName', 'Belgium')]
[('countryForSport', 'http://www.wikidata.org/entity/Q155'), ('countryForSportName', 'Brazil')]
[('countryForSport', 'http://www.wikidata.org/entity/Q16'), ('countryForSportName', 'Canada')]
[('countryForSport', 'http://www.wikidata.org/entity/Q216923'), ('countryForSportName', 'Chinese Taipei')]
[('countryForSport', 'http://www.wikidata.org/entity/Q739'), ('countryForSportName', 'Colombia')]
[('countryForSport', 'http://www.wikidata.org/entity/Q224'), ('countryForSportName', 'Croatia')]
[('countryForSport', 'http://www.wikidata.org/entity/Q213'), ('countryForSportName', 'Czech Republic')]
[('countryForSport', 'http://www.wikidata.org/entity/Q33946'), ('countryForSportName', 'Czechoslovakia'

### Final query for this task

In [328]:
# write your queries
# Consider the country for sport property of the tennis player (NO Country of citizenship). 
# Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

# P106('occupation'),
# Q10833314 ('tennis player')
# --P1352 ('ranking')--> ('o', '1')
# --P1532 ('country for sport')

queryString = """

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?countryForSport ?countryForSportName
WHERE {
   # getting all tennis player
   ?o wdt:P106 wd:Q10833314;
        wdt:P1352 ?ranking; # their rankings
        wdt:P1532 ?countryForSport. # their country for sport
   
   # get the label 
   ?countryForSport sc:name ?countryForSportName.
   
   # Filtering player who ranked as first
   FILTER (xsd:integer(?ranking) = 1)
}
GROUP BY ?countryForSport ?countryForSportName
ORDER BY ?countryForSportName
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('countryForSport', 'http://www.wikidata.org/entity/Q408'), ('countryForSportName', 'Australia')]
[('countryForSport', 'http://www.wikidata.org/entity/Q184'), ('countryForSportName', 'Belarus')]
[('countryForSport', 'http://www.wikidata.org/entity/Q31'), ('countryForSportName', 'Belgium')]
[('countryForSport', 'http://www.wikidata.org/entity/Q155'), ('countryForSportName', 'Brazil')]
[('countryForSport', 'http://www.wikidata.org/entity/Q16'), ('countryForSportName', 'Canada')]
[('countryForSport', 'http://www.wikidata.org/entity/Q216923'), ('countryForSportName', 'Chinese Taipei')]
[('countryForSport', 'http://www.wikidata.org/entity/Q739'), ('countryForSportName', 'Colombia')]
[('countryForSport', 'http://www.wikidata.org/entity/Q224'), ('countryForSportName', 'Croatia')]
[('countryForSport', 'http://www.wikidata.org/entity/Q213'), ('countryForSportName', 'Czech Republic')]
[('countryForSport', 'http://www.wikidata.org/entity/Q33946'), ('countryForSportName', 'Czechoslovakia'

## ALERT! DO NOT RUN the following queries: 
The following queries are my old and BAD queries in TASK 6 (I have commented them above). Even though they did not help me find what I was looking for, the professor asked me to put them anyway in the notebook. 
Please do not run them as they are very computationally heavy, and they may break the server.

### All useless queries to find dob

In [134]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]

# 1) getting winners dob 
queryString = """

PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   #it never returns the result if un-comment the filter
   #FILTER (?o > "1975"^^xsd:gYear)
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q95452024'), ('sName', 'Template:Vice Presidents of the Maldives'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q10999435'), ('oName', 'Vice President of the Maldives')]
[('s', 'http://www.wikidata.org/entity/Q25752552'), ('sName', 'Template:2001 in Europe'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q17111750'), ('oName', '2001 in Europe')]
[('s', 'http://www.wikidata.org/entity/Q6675314'), ('sName', 'Template:Alcest'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q1517060'), ('oName', 'Alcest')]
[('s', 'http://www.wikidata.org/entity/Q22785857'), ('sName', 'Template:2006 in Europe'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q17111782'), ('oName', '2006 in Europe')]
[('s', 'http://www.wikidata.org/entity/Q85816744'), ('sName', 'Yodli'), ('pName', 'represents'), ('o', 'http://www.wikidata.org/entity/Q1053559'), ('oName', '2020 Wi

In [135]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]

# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   FILTER (REGEX(?oName, "1975.*"))
   FILTER (?p != wdt:P155 && ?p != wdt:P301)
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q4574296'), ('sName', '1970s in Hong Kong'), ('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('o', 'http://www.wikidata.org/entity/Q15930131'), ('oName', '1975 in Hong Kong')]
[('s', 'http://www.wikidata.org/entity/Q19568414'), ('sName', '1974 in Iran'), ('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by'), ('o', 'http://www.wikidata.org/entity/Q20987712'), ('oName', '1975 in Iran')]
[('s', 'http://www.wikidata.org/entity/Q19568417'), ('sName', '1974 in Kuwait'), ('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by'), ('o', 'http://www.wikidata.org/entity/Q19568426'), ('oName', '1975 in Kuwait')]
[('s', 'http://www.wikidata.org/entity/Q30731465'), ('sName', 'Category:1975 in Yugoslavian sport'), ('p', 'http://www.wikidata.org/prop/direct/P971'), ('pName', 'category combines topics'), ('o', 'http://www.wikidata.org/entity/Q1074111'), ('oName', '1975 in sports')]
[('s', 'h

In [136]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]

# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   #it never returns the result if un-comment the filter
   FILTER (REGEX(?oName, "1981-08-08.*"))
   FILTER (?p NOT IN (wdt:P155,wdt:P301))
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q10275776'), ('sName', 'Estação Ferroviária de Mafra'), ('p', 'http://www.wikidata.org/prop/direct/P795'), ('pName', 'located on linear feature'), ('o', 'http://www.wikidata.org/entity/Q1160445'), ('oName', 'Linha do Oeste')]
[('s', 'http://www.wikidata.org/entity/Q11235194'), ('sName', 'Tokyo Solamachi'), ('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q11497641'), ('oName', 'Oshiage')]
[('s', 'http://www.wikidata.org/entity/Q15628796'), ('sName', 'Pedro Ponce de Cabrera'), ('p', 'http://www.wikidata.org/prop/direct/P119'), ('pName', 'place of burial'), ('o', 'http://www.wikidata.org/entity/Q16608610'), ('oName', 'Monastery of Santa María de Nogales')]
[('s', 'http://www.wikidata.org/entity/Q18483474'), ('sName', 'Telhada halt'), ('p', 'http://www.wikidata.org/prop/direct/P795'), ('pName', 'located on linear feature'), ('o', 'http://www

In [137]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]

# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   #it never returns the result if un-comment the filter
   FILTER (REGEX(?oName, "1981-08-0.*"))
}
LIMIT 1
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q95452024'), ('sName', 'Template:Vice Presidents of the Maldives'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q10999435'), ('oName', 'Vice President of the Maldives')]
1


In [138]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]

# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   FILTER (REGEX(?oName, "1981.*"))
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q8165395'), ('sName', 'Category:1982 elections in South America'), ('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows'), ('o', 'http://www.wikidata.org/entity/Q16772598'), ('oName', 'Category:1981 elections in South America')]
[('s', 'http://www.wikidata.org/entity/Q15120431'), ('sName', 'Category:1982 in go'), ('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows'), ('o', 'http://www.wikidata.org/entity/Q15120313'), ('oName', 'Category:1981 in go')]
[('s', 'http://www.wikidata.org/entity/Q8165681'), ('sName', 'Category:1982 in New Zealand sport'), ('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows'), ('o', 'http://www.wikidata.org/entity/Q11721574'), ('oName', 'Category:1981 in New Zealand sport')]
[('s', 'http://www.wikidata.org/entity/Q52179909'), ('sName', '1982 in Finland'), ('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows'), ('o', 'http://www.wikidata.org/enti

In [139]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]

# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   FILTER (?oName = "1981")
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
Empty


In [140]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   FILTER (?oName = "1981-08")
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
Empty


In [141]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   FILTER (?oName = "1981-08-08")
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
Empty


In [142]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   FILTER (REGEX(?oName,"1987-.*"))
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q17027610'), ('sName', 'Remembering Christmas'), ('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows'), ('o', 'http://www.wikidata.org/entity/Q16953722'), ('oName', 'The Best of David Benoit 1987-1995')]
[('s', 'http://www.wikidata.org/entity/Q16466348'), ('sName', 'Shaken Not Stirred'), ('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by'), ('o', 'http://www.wikidata.org/entity/Q16953722'), ('oName', 'The Best of David Benoit 1987-1995')]
[('s', 'http://www.wikidata.org/entity/Q34757794'), ('sName', 'Child labor and musculoskeletal disorders: the Pelotas (Brazil) epidemiological survey.'), ('p', 'http://www.wikidata.org/prop/direct/P2860'), ('pName', 'cites work'), ('o', 'http://www.wikidata.org/entity/Q24169399'), ('oName', 'Work-related injuries to Massachusetts teens, 1987-1990')]
[('s', 'http://www.wikidata.org/entity/Q23910866'), ('sName', "Teen workers' exposures to occupational hazards and us

In [143]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 
queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   
   #it never returns the result if un-comment the filter
   FILTER (REGEX(?oName,"1987-05.*"))
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q95452024'), ('sName', 'Template:Vice Presidents of the Maldives'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q10999435'), ('oName', 'Vice President of the Maldives')]
[('s', 'http://www.wikidata.org/entity/Q25752552'), ('sName', 'Template:2001 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q17111750'), ('oName', '2001 in Europe')]
[('s', 'http://www.wikidata.org/entity/Q6675314'), ('sName', 'Template:Alcest'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q1517060'), ('oName', 'Alcest')]
[('s', 'http://www.wikidata.org/entity/Q22785857'), ('sName', 'Template:2006 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity

In [144]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   FILTER (REGEX(?oName,"May 1987"))
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q9615411'), ('sName', 'Category:May 1987 events'), ('p', 'http://www.wikidata.org/prop/direct/P301'), ('pName', "category's main topic"), ('o', 'http://www.wikidata.org/entity/Q13072296'), ('oName', 'May 1987')]
[('s', 'http://www.wikidata.org/entity/Q69297253'), ('sName', 'May 4, 1987'), ('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q13072296'), ('oName', 'May 1987')]
[('s', 'http://www.wikidata.org/entity/Q69297267'), ('sName', 'May 16, 1987'), ('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q13072296'), ('oName', 'May 1987')]
[('s', 'http://www.wikidata.org/entity/Q69297272'), ('sName', 'May 20, 1987'), ('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q13072296'), ('oName', 'May 1987')]
[('s', 'http://www.wikidata.org/entity/Q69297268'), ('sName', 'M

In [145]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   
   #it never returns the result if un-comment the filter
   FILTER (REGEX(?oName,"May 22, 1987"))
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q95452024'), ('sName', 'Template:Vice Presidents of the Maldives'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q10999435'), ('oName', 'Vice President of the Maldives')]
[('s', 'http://www.wikidata.org/entity/Q25752552'), ('sName', 'Template:2001 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q17111750'), ('oName', '2001 in Europe')]
[('s', 'http://www.wikidata.org/entity/Q6675314'), ('sName', 'Template:Alcest'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q1517060'), ('oName', 'Alcest')]
[('s', 'http://www.wikidata.org/entity/Q22785857'), ('sName', 'Template:2006 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity

In [ ]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   # SOMETIMES WORKS, sometimes doesn't return any result
   FILTER (REGEX(?pName,"[B|b]irth*"))
   #FILTER (?p not in (wdt:P19))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


In [155]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   #it never returns the result if un-comment the filter
   FILTER (REGEX(?pName,"[B|b]irth*"))
   FILTER (?p not in (wdt:P19))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q10275776'), ('sName', 'Estação Ferroviária de Mafra'), ('p', 'http://www.wikidata.org/prop/direct/P795'), ('pName', 'located on linear feature'), ('o', 'http://www.wikidata.org/entity/Q1160445'), ('oName', 'Linha do Oeste')]
[('s', 'http://www.wikidata.org/entity/Q11235194'), ('sName', 'Tokyo Solamachi'), ('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q11497641'), ('oName', 'Oshiage')]
[('s', 'http://www.wikidata.org/entity/Q15628796'), ('sName', 'Pedro Ponce de Cabrera'), ('p', 'http://www.wikidata.org/prop/direct/P119'), ('pName', 'place of burial'), ('o', 'http://www.wikidata.org/entity/Q16608610'), ('oName', 'Monastery of Santa María de Nogales')]
[('s', 'http://www.wikidata.org/entity/Q18483474'), ('sName', 'Telhada halt'), ('p', 'http://www.wikidata.org/prop/direct/P795'), ('pName', 'located on linear feature'), ('o', 'http://www

In [156]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   #it never returns the result if un-comment the filter
   FILTER (REGEX(?pName,"[D|d][O|o][B|b]"))
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q95452024'), ('sName', 'Template:Vice Presidents of the Maldives'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q10999435'), ('oName', 'Vice President of the Maldives')]
[('s', 'http://www.wikidata.org/entity/Q25752552'), ('sName', 'Template:2001 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q17111750'), ('oName', '2001 in Europe')]
[('s', 'http://www.wikidata.org/entity/Q6675314'), ('sName', 'Template:Alcest'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q1517060'), ('oName', 'Alcest')]
[('s', 'http://www.wikidata.org/entity/Q22785857'), ('sName', 'Template:2006 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity

In [157]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   # Sometime it returns the result and other it doesn't #comment the filter
   FILTER (REGEX(?pName,"date.*"))
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q95452024'), ('sName', 'Template:Vice Presidents of the Maldives'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q10999435'), ('oName', 'Vice President of the Maldives')]
[('s', 'http://www.wikidata.org/entity/Q25752552'), ('sName', 'Template:2001 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q17111750'), ('oName', '2001 in Europe')]
[('s', 'http://www.wikidata.org/entity/Q6675314'), ('sName', 'Template:Alcest'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q1517060'), ('oName', 'Alcest')]
[('s', 'http://www.wikidata.org/entity/Q22785857'), ('sName', 'Template:2006 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity

In [158]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   # sometimes filter with regex don't work
   FILTER (REGEX(?pName,"date","i"))
   #FILTER (?p not in (wdt:P726, wdt:P991))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q95452024'), ('sName', 'Template:Vice Presidents of the Maldives'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q10999435'), ('oName', 'Vice President of the Maldives')]
[('s', 'http://www.wikidata.org/entity/Q25752552'), ('sName', 'Template:2001 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q17111750'), ('oName', '2001 in Europe')]
[('s', 'http://www.wikidata.org/entity/Q6675314'), ('sName', 'Template:Alcest'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q1517060'), ('oName', 'Alcest')]
[('s', 'http://www.wikidata.org/entity/Q22785857'), ('sName', 'Template:2006 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity

In [160]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   #this time filter doesnt work 
   FILTER (REGEX(?pName,"year","i"))
   #FILTER (?p not in (wdt:P726, wdt:P991))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q95452024'), ('sName', 'Template:Vice Presidents of the Maldives'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q10999435'), ('oName', 'Vice President of the Maldives')]
[('s', 'http://www.wikidata.org/entity/Q25752552'), ('sName', 'Template:2001 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q17111750'), ('oName', '2001 in Europe')]
[('s', 'http://www.wikidata.org/entity/Q6675314'), ('sName', 'Template:Alcest'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q1517060'), ('oName', 'Alcest')]
[('s', 'http://www.wikidata.org/entity/Q22785857'), ('sName', 'Template:2006 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity

In [161]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   
   # today filter with regex not working
   FILTER (REGEX(?pName,"year","i"))
   
   #it never returns the result if un-comment the filter
   #FILTER (?p not in (wdt:P2922, wdt:P991))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q95452024'), ('sName', 'Template:Vice Presidents of the Maldives'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q10999435'), ('oName', 'Vice President of the Maldives')]
[('s', 'http://www.wikidata.org/entity/Q25752552'), ('sName', 'Template:2001 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q17111750'), ('oName', '2001 in Europe')]
[('s', 'http://www.wikidata.org/entity/Q6675314'), ('sName', 'Template:Alcest'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q1517060'), ('oName', 'Alcest')]
[('s', 'http://www.wikidata.org/entity/Q22785857'), ('sName', 'Template:2006 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity

In [162]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   
   #today filter with regex not working
   FILTER (REGEX(?pName,"year","i"))
   
   #it never returns the result if un-comment the filter
   #FILTER (?p not in (wdt:P2922, wdt:P837))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q95452024'), ('sName', 'Template:Vice Presidents of the Maldives'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q10999435'), ('oName', 'Vice President of the Maldives')]
[('s', 'http://www.wikidata.org/entity/Q25752552'), ('sName', 'Template:2001 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q17111750'), ('oName', '2001 in Europe')]
[('s', 'http://www.wikidata.org/entity/Q6675314'), ('sName', 'Template:Alcest'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q1517060'), ('oName', 'Alcest')]
[('s', 'http://www.wikidata.org/entity/Q22785857'), ('sName', 'Template:2006 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity

In [164]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   # today filter with regex not working
   FILTER (REGEX(?pName,"birth","i"))
   FILTER (?p not in (wdt:P19, wdt:P837))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q10275776'), ('sName', 'Estação Ferroviária de Mafra'), ('p', 'http://www.wikidata.org/prop/direct/P795'), ('pName', 'located on linear feature'), ('o', 'http://www.wikidata.org/entity/Q1160445'), ('oName', 'Linha do Oeste')]
[('s', 'http://www.wikidata.org/entity/Q11235194'), ('sName', 'Tokyo Solamachi'), ('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q11497641'), ('oName', 'Oshiage')]
[('s', 'http://www.wikidata.org/entity/Q15628796'), ('sName', 'Pedro Ponce de Cabrera'), ('p', 'http://www.wikidata.org/prop/direct/P119'), ('pName', 'place of burial'), ('o', 'http://www.wikidata.org/entity/Q16608610'), ('oName', 'Monastery of Santa María de Nogales')]
[('s', 'http://www.wikidata.org/entity/Q18483474'), ('sName', 'Telhada halt'), ('p', 'http://www.wikidata.org/prop/direct/P795'), ('pName', 'located on linear feature'), ('o', 'http://www

In [165]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?s ?sName "birthday" ?o ?oName
WHERE {
   ?s wdt:P3150 ?o.
   ?s sc:name ?sName.
   ?o sc:name ?oName.
   
   FILTER (REGEX(?oName,"August 8","i"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q42012085'), ('sName', 'Hachiman Hikigaya'), ('callret-2', 'birthday'), ('o', 'http://www.wikidata.org/entity/Q2780'), ('oName', 'August 8')]
[('s', 'http://www.wikidata.org/entity/Q11541253'), ('sName', 'Natsuko Kusuda'), ('callret-2', 'birthday'), ('o', 'http://www.wikidata.org/entity/Q2780'), ('oName', 'August 8')]
[('s', 'http://www.wikidata.org/entity/Q107038831'), ('sName', 'Hoho Kajino'), ('callret-2', 'birthday'), ('o', 'http://www.wikidata.org/entity/Q2780'), ('oName', 'August 8')]
[('s', 'http://www.wikidata.org/entity/Q11256924'), ('sName', 'Ryuryū Akari'), ('callret-2', 'birthday'), ('o', 'http://www.wikidata.org/entity/Q2780'), ('oName', 'August 8')]
[('s', 'http://www.wikidata.org/entity/Q11536787'), ('sName', 'Ryū Katsuragi'), ('callret-2', 'birthday'), ('o', 'http://www.wikidata.org/entity/Q2780'), ('oName', 'August 8')]
[('s', 'http://www.wikidata.org/entity/Q106478679'), ('sName', 'Tatsuya Nakamura'), ('callret-2', 'birth

In [169]:
# Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).
# Q1426 = Roger Federer -> August 8, 1981
# wdt:P1344 (participant in)
# wdt:P2522 (victory)
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
# 1) getting winners dob 

queryString = """

SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   ?s ?p ?o.
   ?s sc:name ?sName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
   # today filter with regex not working
   FILTER (REGEX(?pName,"birth","i"))
   
   
   #takes to long
   #FILTER (?p not in (wdt:P19, wdt:P3150))
}
LIMIT 5
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q95452024'), ('sName', 'Template:Vice Presidents of the Maldives'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q10999435'), ('oName', 'Vice President of the Maldives')]
[('s', 'http://www.wikidata.org/entity/Q25752552'), ('sName', 'Template:2001 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q17111750'), ('oName', '2001 in Europe')]
[('s', 'http://www.wikidata.org/entity/Q6675314'), ('sName', 'Template:Alcest'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q1517060'), ('oName', 'Alcest')]
[('s', 'http://www.wikidata.org/entity/Q22785857'), ('sName', 'Template:2006 in Europe'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity